In [1]:
# Agent's Core Infrastructure
# Version 0.1.0

# Global store for tools and their descriptions
# These will persist across turns in the execution environment.
_AGENT_TOOLS = {}
_TOOL_DESCRIPTIONS = {}
_AGENT_VERSION = "0.1.0"

def get_agent_version_info():
    """Returns the agent's current version."""
    return _AGENT_VERSION

# --- Core Tool Management Functions ---

def add_tool(name: str, tool_callable, description: str = "No description provided."):
    """
    Adds a new tool (a callable) to the agent's toolkit.
    This function is a core capability of the agent, not a tool itself to be called by other tools.
    
    Args:
        name (str): The name of the tool. Must be a valid Python identifier.
        tool_callable (callable): The function or callable object.
        description (str): A human-readable description of what the tool does, its inputs, and outputs.
        
    Returns:
        bool: True if the tool was added successfully, False otherwise.
    """
    global _AGENT_TOOLS, _TOOL_DESCRIPTIONS
    
    if not isinstance(name, str) or not name.isidentifier():
        print(f"Error: Tool name '{name}' is not a valid Python identifier. Tool not added.")
        return False

    if not callable(tool_callable):
        # The print output will be part of the feedback from the execution environment.
        print(f"Error: Object provided for tool '{name}' is not callable. Tool not added.")
        return False
    
    if name in _AGENT_TOOLS:
        print(f"Warning: Tool '{name}' already exists. It will be overwritten.")
    
    _AGENT_TOOLS[name] = tool_callable
    _TOOL_DESCRIPTIONS[name] = description
    print(f"Tool '{name}' added successfully. Description: {description}")
    return True

def list_tools():
    """
    Lists all available tools with their descriptions.
    This function is intended to be a tool itself for introspection.
    
    Returns:
        dict: A dictionary of tool names to their descriptions.
              Prints the list to stdout as well for immediate user visibility.
    """
    global _AGENT_TOOLS, _TOOL_DESCRIPTIONS # Read-only access to globals is fine
    
    if not _AGENT_TOOLS:
        print("No tools available yet.")
        return {}
    
    print("\nAvailable tools:")
    tool_info = {}
    for i, (name, description) in enumerate(_TOOL_DESCRIPTIONS.items()):
        print(f"{i+1}. {name}: {description}")
        tool_info[name] = description
    
    return tool_info # Return a dictionary for programmatic use

# --- Initial Tools Definition ---

def get_agent_status():
    """
    Provides a brief status report of the agent, including version and tool count.
    
    Returns:
        dict: A dictionary containing agent status information.
              Prints the status message to stdout as well.
    """
    global _AGENT_TOOLS, _AGENT_VERSION # Read-only access
    tool_count = len(_AGENT_TOOLS)
    status_message = (
        f"Agent Status:\n"
        f"  Version: {get_agent_version_info()}\n"
        f"  State: Online and operational.\n"
        f"  Core tool management: Initialized.\n"
        f"  Tools available: {tool_count} (Note: this count is as of this tool's execution)"
    )
    print(status_message)
    return {
        "version": get_agent_version_info(),
        "state": "Online",
        "message": "Core tool management initialized.",
        "tool_count": tool_count 
    }

# --- Register Initial Tools ---
# The functions must be defined before they can be added as tools.

print("Initializing agent and registering foundational tools...")

# Add list_tools as a tool
add_tool(
    name="list_tools",
    tool_callable=list_tools,
    description="Lists all available tools with their names and descriptions. Useful for understanding agent capabilities."
)

# Add get_agent_status as a tool
add_tool(
    name="get_agent_status",
    tool_callable=get_agent_status,
    description="Provides a status report of the agent, including version and number of loaded tools."
)

print("\nAgent initialization complete. Foundational tools are registered.")
print(f"Current agent version: {get_agent_version_info()}")
print("Waiting for next instruction.")

Initializing agent and registering foundational tools...
Tool 'list_tools' added successfully. Description: Lists all available tools with their names and descriptions. Useful for understanding agent capabilities.
Tool 'get_agent_status' added successfully. Description: Provides a status report of the agent, including version and number of loaded tools.

Agent initialization complete. Foundational tools are registered.
Current agent version: 0.1.0
Waiting for next instruction.


In [2]:
# Continuing from previous state.
# _AGENT_TOOLS, _TOOL_DESCRIPTIONS, _AGENT_VERSION are assumed to be in the global scope.
# add_tool, list_tools, get_agent_status are also assumed to be available as defined in the first turn.

import io
import contextlib
import traceback # For detailed error messages

def execute_python_code(code_string: str):
    """
    Executes a given string of Python code in the agent's current global environment.
    Captures and returns stdout, stderr from the executed code.
    USE WITH EXTREME CAUTION: This tool can modify the agent's internal state,
    define new functions/variables, or re-define existing ones.
    Ensure the code is safe and well-understood before execution.

    Args:
        code_string (str): The Python code to execute.

    Returns:
        dict: A dictionary containing:
            'stdout' (str): Captured standard output from the executed code.
            'stderr' (str): Captured standard error from the executed code.
            'execution_error' (str or None): A string with the traceback if an
                                            exception occurred during the execution of
                                            the provided code, otherwise None.
    """
    # These globals are listed to acknowledge that the exec'd code might interact with them.
    # The exec call itself has access to the full global scope where this function is defined.
    global _AGENT_TOOLS, _TOOL_DESCRIPTIONS, _AGENT_VERSION 

    stdout_capture = io.StringIO()
    stderr_capture = io.StringIO()
    execution_error_message = None
    
    # This print goes to the agent's main output, not the captured stdout for the executed code.
    print(f"Agent log: Attempting to execute the following Python code block:\n---\n{code_string}\n---")

    try:
        # Execute the code in the current global scope.
        # This allows the code to define new functions/variables globally,
        # or modify existing ones (including agent's core components if not careful).
        # The 'globals()' argument ensures it uses the module's global scope.
        with contextlib.redirect_stdout(stdout_capture):
            with contextlib.redirect_stderr(stderr_capture):
                exec(code_string, globals())
    except Exception as e:
        # Capture traceback for detailed error information
        tb_str = traceback.format_exc()
        execution_error_message = f"Exception during execution of provided code: {type(e).__name__}: {e}\nTraceback:\n{tb_str}"
        # This print also goes to the agent's main output for immediate visibility of the tool's error handling.
        print(f"Agent log: Error encountered by 'execute_python_code' tool: {execution_error_message}")
    
    stdout_val = stdout_capture.getvalue()
    stderr_val = stderr_capture.getvalue()

    result = {
        "stdout": stdout_val,
        "stderr": stderr_val,
        "execution_error": execution_error_message
    }
    
    # This print also goes to the agent's main output.
    print(f"Agent log: 'execute_python_code' tool finished. Captured stdout length: {len(stdout_val)}, Captured stderr length: {len(stderr_val)}, Error: {'Yes' if execution_error_message else 'No'}")
    return result

# Add the new tool using the pre-existing add_tool function
print("Registering 'execute_python_code' tool...")
# Assume add_tool is available from the initial setup.
# If add_tool wasn't persisted or is not in scope, this will fail.
# Based on the problem description, it should persist.
add_tool_result = add_tool(
    name="execute_python_code",
    tool_callable=execute_python_code,
    description="Executes a given string of Python code in the agent's current global environment. Captures and returns stdout, stderr, and any execution error from the provided code. USE WITH EXTREME CAUTION, as it can alter the agent's state and capabilities."
)

if not add_tool_result:
    print("Error: Failed to add 'execute_python_code' tool. This might indicate 'add_tool' is not working as expected or was overwritten.")

# Verify by listing tools
# This will use the list_tools function, which prints to stdout.
print("\nVerifying tool registration by listing all available tools:")
list_tools_output_dict = list_tools() 

# Optionally, explicitly print the dictionary returned by list_tools for clarity,
# though list_tools() already prints formatted output.
# print(f"Agent log: list_tools() returned: {list_tools_output_dict}")

print("\n'execute_python_code' tool definition and registration complete.")
print("Waiting for next instruction.")

Registering 'execute_python_code' tool...
Tool 'execute_python_code' added successfully. Description: Executes a given string of Python code in the agent's current global environment. Captures and returns stdout, stderr, and any execution error from the provided code. USE WITH EXTREME CAUTION, as it can alter the agent's state and capabilities.

Verifying tool registration by listing all available tools:

Available tools:
1. list_tools: Lists all available tools with their names and descriptions. Useful for understanding agent capabilities.
2. get_agent_status: Provides a status report of the agent, including version and number of loaded tools.
3. execute_python_code: Executes a given string of Python code in the agent's current global environment. Captures and returns stdout, stderr, and any execution error from the provided code. USE WITH EXTREME CAUTION, as it can alter the agent's state and capabilities.

'execute_python_code' tool definition and registration complete.
Waiting for 

In [3]:
# Continuing from previous state.
# _AGENT_TOOLS, _TOOL_DESCRIPTIONS, _AGENT_VERSION are assumed to be in the global scope.
# add_tool, list_tools, get_agent_status, execute_python_code are available.

print("Agent: Planning to test the 'execute_python_code' tool.")

# 1. Define the code string to be executed by the tool.
# This code will print to stdout and attempt to set a global variable.
code_to_execute = """
# This code is intended to be run by the 'execute_python_code' tool.
print("Message from inside the executed code block: Hello there!")
_TEST_VARIABLE_SET_BY_EXEC = "Successfully set by execute_python_code tool."
print(f"Message from inside the executed code block: _TEST_VARIABLE_SET_BY_EXEC = '{_TEST_VARIABLE_SET_BY_EXEC}'")

# Let's also try to define a simple function to see if it becomes globally available
def _sample_func_defined_by_exec():
    return "This function was defined by execute_python_code."

print("Message from inside the executed code block: _sample_func_defined_by_exec defined.")
"""

print(f"\nAgent: Calling 'execute_python_code' with the following code:\n---\n{code_to_execute}\n---")

# 2. Call the 'execute_python_code' tool
# We need to access it through the _AGENT_TOOLS dictionary.
if 'execute_python_code' in _AGENT_TOOLS:
    execution_result = _AGENT_TOOLS['execute_python_code'](code_to_execute)
    
    # 3. Print the result from 'execute_python_code'
    print("\nAgent: Result from 'execute_python_code' tool:")
    print(f"  STDOUT:\n{execution_result['stdout']}")
    print(f"  STDERR:\n{execution_result['stderr']}")
    print(f"  EXECUTION_ERROR: {execution_result['execution_error']}")

    # 4. Attempt to access the global variable and function set by the executed code.
    print("\nAgent: Verifying effects of executed code in the global scope...")
    try:
        # Check for the variable
        if '_TEST_VARIABLE_SET_BY_EXEC' in globals():
            print(f"  Successfully accessed global variable: _TEST_VARIABLE_SET_BY_EXEC = '{_TEST_VARIABLE_SET_BY_EXEC}'")
        else:
            print("  Error: Global variable '_TEST_VARIABLE_SET_BY_EXEC' was NOT found in global scope after execution.")
        
        # Check for the function
        if '_sample_func_defined_by_exec' in globals() and callable(globals()['_sample_func_defined_by_exec']):
            func_result = _sample_func_defined_by_exec()
            print(f"  Successfully accessed and called global function: _sample_func_defined_by_exec() returned: '{func_result}'")
        else:
            print("  Error: Global function '_sample_func_defined_by_exec' was NOT found or is not callable after execution.")

    except NameError as e:
        print(f"  NameError encountered when trying to access artifacts from executed code: {e}")
    except Exception as e:
        print(f"  An unexpected error occurred when trying to access artifacts: {e}")
else:
    print("CRITICAL ERROR: 'execute_python_code' tool not found in _AGENT_TOOLS. Cannot proceed with the test.")

# 5. Test `execute_python_code` with code that intentionally causes an error.
print("\nAgent: Testing 'execute_python_code' with code that causes an error.")
code_with_error = """
print("This part will run.")
x = 1 / 0 # This will cause a ZeroDivisionError
print("This part will not run.")
"""
print(f"\nAgent: Calling 'execute_python_code' with the following error-inducing code:\n---\n{code_with_error}\n---")

if 'execute_python_code' in _AGENT_TOOLS:
    error_execution_result = _AGENT_TOOLS['execute_python_code'](code_with_error)
    
    print("\nAgent: Result from 'execute_python_code' tool (with erroring code):")
    print(f"  STDOUT:\n{error_execution_result['stdout']}")
    print(f"  STDERR:\n{error_execution_result['stderr']}") # Stderr from exec itself, not necessarily the exception. Python exceptions go to stderr by default if unhandled.
    print(f"  EXECUTION_ERROR (should not be None):\n{error_execution_result['execution_error']}")
    
    if error_execution_result['execution_error']:
        print("  Test successful: execution_error was correctly reported.")
    else:
        print("  Test FAILED: execution_error was NOT reported for erroring code.")
else:
    print("CRITICAL ERROR: 'execute_python_code' tool not found in _AGENT_TOOLS. Cannot proceed with the error test.")


# Final status check
print("\nAgent: Final status check after tests.")
if 'get_agent_status' in _AGENT_TOOLS:
    _AGENT_TOOLS['get_agent_status']() # This will print the status
else:
    print("Warning: 'get_agent_status' tool not found to print final status.")

print("\nAgent: Tests for 'execute_python_code' complete.")
print("Waiting for next instruction.")

Agent: Planning to test the 'execute_python_code' tool.

Agent: Calling 'execute_python_code' with the following code:
---

# This code is intended to be run by the 'execute_python_code' tool.
print("Message from inside the executed code block: Hello there!")
_TEST_VARIABLE_SET_BY_EXEC = "Successfully set by execute_python_code tool."
print(f"Message from inside the executed code block: _TEST_VARIABLE_SET_BY_EXEC = '{_TEST_VARIABLE_SET_BY_EXEC}'")

# Let's also try to define a simple function to see if it becomes globally available
def _sample_func_defined_by_exec():
    return "This function was defined by execute_python_code."

print("Message from inside the executed code block: _sample_func_defined_by_exec defined.")

---
Agent log: Attempting to execute the following Python code block:
---

# This code is intended to be run by the 'execute_python_code' tool.
print("Message from inside the executed code block: Hello there!")
_TEST_VARIABLE_SET_BY_EXEC = "Successfully set by execute_

In [4]:
# Assuming _AGENT_TOOLS and its contents (including new file system tools) are in the global scope.
import os # For path joining, abspath etc. in test setup

print("Agent: Starting tests for file system tools: write_file, read_file, list_directory.")

# Define test paths and content
test_dir_path = "agent_test_workspace/file_op_tests" # A dedicated directory for these tests
test_file_name = "sample_message.txt"
test_file_path = os.path.join(test_dir_path, test_file_name)
test_content = "Hello from the agent! This is a test file.\nSecond line for good measure."
non_existent_file_path = os.path.join(test_dir_path, "non_existent_file.txt")
non_existent_dir_path = "this_directory_should_not_exist_12345"

# --- Test Execution ---

# 0. Get initial directory listing (current directory)
print("\n--- Test 0: Listing current directory (initial state) ---")
if 'list_directory' in _AGENT_TOOLS:
    list_result_initial = _AGENT_TOOLS['list_directory'](".") # Current directory
    print(f"Initial list_directory('.') result: {list_result_initial}")
else:
    print("Error: 'list_directory' tool not found.")

# 1. Test write_file
print(f"\n--- Test 1: write_file to '{test_file_path}' ---")
if 'write_file' in _AGENT_TOOLS:
    write_result = _AGENT_TOOLS['write_file'](test_file_path, test_content)
    print(f"write_file result: {write_result}")
    if not write_result.get('success'):
        print(f"Halting further tests due to write_file failure: {write_result.get('message')}")
        # Depending on the failure, further tests might be misleading or impossible.
        # For now, we'll proceed but note that subsequent tests might fail due to this.
else:
    print("Error: 'write_file' tool not found. Cannot proceed with write-dependent tests.")
    # Simulate failure to prevent subsequent tests from running if write_file is essential
    write_result = {"success": False, "message": "'write_file' tool not found."}


# 2. Test list_directory for the new directory
# This test depends on write_file having created the directory structure.
if write_result.get('success'):
    print(f"\n--- Test 2: list_directory for '{test_dir_path}' ---")
    if 'list_directory' in _AGENT_TOOLS:
        list_result = _AGENT_TOOLS['list_directory'](test_dir_path)
        print(f"list_directory('{test_dir_path}') result: {list_result}")
        if list_result.get('success') and test_file_name in list_result.get('contents', []):
            print(f"  SUCCESS: '{test_file_name}' found in '{test_dir_path}'.")
        elif list_result.get('success'):
            print(f"  WARNING: '{test_file_name}' NOT found in '{test_dir_path}' contents: {list_result.get('contents')}")
        else:
            print(f"  FAILURE: list_directory failed: {list_result.get('message')}")
    else:
        print("Error: 'list_directory' tool not found.")
else:
    print(f"\n--- Test 2: Skipped list_directory for '{test_dir_path}' due to prior write_file failure. ---")


# 3. Test read_file for the created file
# This test depends on write_file having successfully created the file.
if write_result.get('success'):
    print(f"\n--- Test 3: read_file from '{test_file_path}' ---")
    if 'read_file' in _AGENT_TOOLS:
        read_result = _AGENT_TOOLS['read_file'](test_file_path)
        print(f"read_file('{test_file_path}') result: {read_result}")
        if read_result.get('success') and read_result.get('content') == test_content:
            print(f"  SUCCESS: Content of '{test_file_path}' matches expected content.")
        elif read_result.get('success'):
            print(f"  FAILURE: Content mismatch! Expected:\n'{test_content}'\nGot:\n'{read_result.get('content')}'")
        else:
            print(f"  FAILURE: read_file failed: {read_result.get('message')}")
    else:
        print("Error: 'read_file' tool not found.")
else:
    print(f"\n--- Test 3: Skipped read_file for '{test_file_path}' due to prior write_file failure. ---")


# 4. Test read_file for a non-existent file
print(f"\n--- Test 4: read_file from non-existent file '{non_existent_file_path}' ---")
if 'read_file' in _AGENT_TOOLS:
    read_non_existent_result = _AGENT_TOOLS['read_file'](non_existent_file_path)
    print(f"read_file('{non_existent_file_path}') result: {read_non_existent_result}")
    if not read_non_existent_result.get('success') and read_non_existent_result.get('content') is None:
        print(f"  SUCCESS: read_file correctly reported failure for non-existent file.")
    else:
        print(f"  FAILURE: read_file did not report failure as expected for non-existent file.")
else:
    print("Error: 'read_file' tool not found.")


# 5. Test list_directory for a non-existent directory
print(f"\n--- Test 5: list_directory for non-existent directory '{non_existent_dir_path}' ---")
if 'list_directory' in _AGENT_TOOLS:
    list_non_existent_result = _AGENT_TOOLS['list_directory'](non_existent_dir_path)
    print(f"list_directory('{non_existent_dir_path}') result: {list_non_existent_result}")
    if not list_non_existent_result.get('success') and list_non_existent_result.get('contents') is None:
        print(f"  SUCCESS: list_directory correctly reported failure for non-existent directory.")
    else:
        print(f"  FAILURE: list_directory did not report failure as expected for non-existent directory.")
else:
    print("Error: 'list_directory' tool not found.")

# Note: Cleanup of created files/directories (e.g., using os.remove, os.rmdir)
# would typically be done here. We don't have dedicated tools for that yet.
# We could use 'execute_python_code' for cleanup if necessary, but for now,
# we'll leave the artifacts.
# Example cleanup using execute_python_code (NOT EXECUTING THIS NOW, just as a thought):
# cleanup_code = f"""
# import os, shutil
# if os.path.exists('{test_file_path}'): os.remove('{test_file_path}')
# if os.path.exists('{test_dir_path}'): shutil.rmtree('{test_dir_path}')
# print('Cleanup attempted for {test_dir_path} and its contents.')
# """
# if 'execute_python_code' in _AGENT_TOOLS:
#     _AGENT_TOOLS['execute_python_code'](cleanup_code)


print("\nAgent: File system tools tests complete.")
print("Waiting for next instruction.")


Agent: Starting tests for file system tools: write_file, read_file, list_directory.

--- Test 0: Listing current directory (initial state) ---
Error: 'list_directory' tool not found.

--- Test 1: write_file to 'agent_test_workspace/file_op_tests/sample_message.txt' ---
Error: 'write_file' tool not found. Cannot proceed with write-dependent tests.

--- Test 2: Skipped list_directory for 'agent_test_workspace/file_op_tests' due to prior write_file failure. ---

--- Test 3: Skipped read_file for 'agent_test_workspace/file_op_tests/sample_message.txt' due to prior write_file failure. ---

--- Test 4: read_file from non-existent file 'agent_test_workspace/file_op_tests/non_existent_file.txt' ---
Error: 'read_file' tool not found.

--- Test 5: list_directory for non-existent directory 'this_directory_should_not_exist_12345' ---
Error: 'list_directory' tool not found.

Agent: File system tools tests complete.
Waiting for next instruction.


In [5]:
# Agent's state and core functions (_AGENT_TOOLS, add_tool, etc.) are assumed to be in global scope.
import os
import shutil # For potential future use in rmtree, etc.

print("Agent: Attempting to re-initialize and test file system tools due to previous 'tool not found' errors.")

# --- Step 1: Re-define File System Tool Functions ---
# It's possible the function objects themselves were not persisted.

def write_file_v2(filepath: str, content: str) -> dict:
    """
    (Version 2) Writes the given content to a file at the specified filepath.
    If the file exists, it will be overwritten.
    If the directory path does not exist, it will attempt to create it.
    """
    # Adding print statements for verbose logging inside the tool
    print(f"Agent log (write_file_v2): Called with filepath='{filepath}'.")
    try:
        dir_name = os.path.dirname(filepath)
        if dir_name:
            os.makedirs(dir_name, exist_ok=True)
            print(f"Agent log (write_file_v2): Ensured directory '{dir_name}' exists.")
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(content)
        abs_path = os.path.abspath(filepath)
        message = f"File '{abs_path}' written successfully ({len(content)} bytes)."
        print(f"Agent log (write_file_v2): {message}")
        return {"success": True, "message": message, "filepath": abs_path}
    except Exception as e:
        error_message = f"Error writing to file '{filepath}': {type(e).__name__}: {e}"
        print(f"Agent log (write_file_v2): {error_message}")
        return {"success": False, "message": error_message, "filepath": filepath}

def read_file_v2(filepath: str) -> dict:
    """
    (Version 2) Reads the content of a file at the specified filepath.
    """
    print(f"Agent log (read_file_v2): Called with filepath='{filepath}'.")
    try:
        if not os.path.exists(filepath):
            message = f"File '{filepath}' does not exist."
            print(f"Agent log (read_file_v2): {message}")
            return {"success": False, "message": message, "content": None, "filepath": filepath}
        if not os.path.isfile(filepath):
            message = f"Path '{filepath}' is a directory, not a file. Cannot read."
            print(f"Agent log (read_file_v2): {message}")
            return {"success": False, "message": message, "content": None, "filepath": filepath}
        with open(filepath, 'r', encoding='utf-8') as f:
            content = f.read()
        abs_path = os.path.abspath(filepath)
        message = f"File '{abs_path}' read successfully ({len(content)} bytes)."
        print(f"Agent log (read_file_v2): {message}")
        return {"success": True, "message": message, "content": content, "filepath": abs_path}
    except Exception as e:
        error_message = f"Error reading file '{filepath}': {type(e).__name__}: {e}"
        print(f"Agent log (read_file_v2): {error_message}")
        return {"success": False, "message": error_message, "content": None, "filepath": filepath}

def list_directory_v2(path: str = ".") -> dict:
    """
    (Version 2) Lists the contents (files and directories) of a specified directory.
    """
    print(f"Agent log (list_directory_v2): Called with path='{path}'.")
    try:
        if not os.path.exists(path):
            message = f"Directory/path '{path}' does not exist."
            print(f"Agent log (list_directory_v2): {message}")
            return {"success": False, "message": message, "path": path, "contents": None}
        if not os.path.isdir(path):
            message = f"Path '{path}' is not a directory."
            print(f"Agent log (list_directory_v2): {message}")
            return {"success": False, "message": message, "path": path, "contents": None}
        contents = os.listdir(path)
        abs_path = os.path.abspath(path)
        message = f"Directory '{abs_path}' listed successfully. Found {len(contents)} items."
        print(f"Agent log (list_directory_v2): {message}")
        return {"success": True, "message": message, "path": abs_path, "contents": contents}
    except Exception as e:
        error_message = f"Error listing directory '{path}': {type(e).__name__}: {e}"
        print(f"Agent log (list_directory_v2): {error_message}")
        return {"success": False, "message": error_message, "path": path, "contents": None}

print("Agent: File system tool functions re-defined (v2).")

# --- Step 2: Re-register File System Tools ---
# Check if add_tool is available
if 'add_tool' not in globals() or not callable(add_tool):
    print("CRITICAL ERROR: 'add_tool' function is not available. Cannot register new tools.")
    # If add_tool is gone, we can't proceed with this plan.
else:
    print("Agent: 'add_tool' function is available. Proceeding with tool registration.")
    print(f"Agent: State of _AGENT_TOOLS before re-adding: {list(_AGENT_TOOLS.keys())}")

    add_tool_result_write = add_tool(
        name="write_file", # Using original name, will overwrite if any remnant exists
        tool_callable=write_file_v2,
        description="(Version 2) Writes text content to a specified file. Overwrites if exists, creates directories if needed."
    )
    if not add_tool_result_write: print("Error: Failed to re-add 'write_file' tool.")

    add_tool_result_read = add_tool(
        name="read_file",
        tool_callable=read_file_v2,
        description="(Version 2) Reads the text content from a specified file."
    )
    if not add_tool_result_read: print("Error: Failed to re-add 'read_file' tool.")

    add_tool_result_list = add_tool(
        name="list_directory",
        tool_callable=list_directory_v2,
        description="(Version 2) Lists files and directories within a specified path. Defaults to current directory."
    )
    if not add_tool_result_list: print("Error: Failed to re-add 'list_directory' tool.")
    
    print(f"Agent: State of _AGENT_TOOLS after re-adding: {list(_AGENT_TOOLS.keys())}")

# --- Step 3: Immediately Verify Tool Registration ---
print("\nAgent: Verifying tool registration immediately by listing all available tools:")
if 'list_tools' in _AGENT_TOOLS and callable(_AGENT_TOOLS['list_tools']):
    _AGENT_TOOLS['list_tools']()
else:
    # If list_tools itself is missing, try to print _TOOL_DESCRIPTIONS directly
    print("CRITICAL WARNING: 'list_tools' tool is missing or not callable.")
    print("Direct dump of _TOOL_DESCRIPTIONS:")
    if '_TOOL_DESCRIPTIONS' in globals():
        for name, desc in _TOOL_DESCRIPTIONS.items():
            print(f"  - {name}: {desc}")
    else:
        print("  _TOOL_DESCRIPTIONS global not found.")

# --- Step 4: Re-run File System Tool Tests ---
print("\nAgent: Starting tests for re-initialized file system tools.")

test_dir_path = "agent_test_workspace_v2/file_op_tests"
test_file_name = "sample_message_v2.txt"
test_file_path = os.path.join(test_dir_path, test_file_name)
test_content = "Hello from the agent V2! This is a re-test file.\nEnsuring persistence."
non_existent_file_path = os.path.join(test_dir_path, "non_existent_file_v2.txt")
non_existent_dir_path = "this_directory_should_not_exist_v2_12345"

# Using a flag to track if tools seem available for testing
file_tools_available_for_test = ('write_file' in _AGENT_TOOLS and 
                                 'read_file' in _AGENT_TOOLS and 
                                 'list_directory' in _AGENT_TOOLS)

if not file_tools_available_for_test:
    print("CRITICAL: One or more file system tools are NOT in _AGENT_TOOLS after re-registration attempt. Skipping tests.")
else:
    print("Agent: All required file system tools appear to be in _AGENT_TOOLS. Proceeding with tests.")

    # Test 0: List current directory
    print("\n--- Test 0: Listing current directory (initial state) ---")
    list_result_initial = _AGENT_TOOLS['list_directory'](".")
    print(f"Initial list_directory('.') result: {list_result_initial}")

    # Test 1: write_file
    print(f"\n--- Test 1: write_file to '{test_file_path}' ---")
    write_result = _AGENT_TOOLS['write_file'](test_file_path, test_content)
    print(f"write_file result: {write_result}")

    # Test 2: list_directory for the new directory
    if write_result.get('success'):
        print(f"\n--- Test 2: list_directory for '{test_dir_path}' ---")
        list_result = _AGENT_TOOLS['list_directory'](test_dir_path)
        print(f"list_directory('{test_dir_path}') result: {list_result}")
        if list_result.get('success') and test_file_name in list_result.get('contents', []):
            print(f"  SUCCESS: '{test_file_name}' found in '{test_dir_path}'.")
        elif list_result.get('success'):
            print(f"  WARNING: '{test_file_name}' NOT found in '{test_dir_path}' contents: {list_result.get('contents')}")
        else:
            print(f"  FAILURE: list_directory failed for '{test_dir_path}': {list_result.get('message')}")
    else:
        print(f"\n--- Test 2: Skipped list_directory for '{test_dir_path}' due to write_file failure. ---")

    # Test 3: read_file for the created file
    if write_result.get('success'):
        print(f"\n--- Test 3: read_file from '{test_file_path}' ---")
        read_result = _AGENT_TOOLS['read_file'](test_file_path)
        print(f"read_file('{test_file_path}') result: {read_result}")
        if read_result.get('success') and read_result.get('content') == test_content:
            print(f"  SUCCESS: Content of '{test_file_path}' matches expected content.")
        elif read_result.get('success'):
            print(f"  FAILURE: Content mismatch! Expected:\n'{test_content}'\nGot:\n'{read_result.get('content')}'")
        else:
            print(f"  FAILURE: read_file failed for '{test_file_path}': {read_result.get('message')}")
    else:
        print(f"\n--- Test 3: Skipped read_file for '{test_file_path}' due to write_file failure. ---")

    # Test 4: read_file for a non-existent file
    print(f"\n--- Test 4: read_file from non-existent file '{non_existent_file_path}' ---")
    read_non_existent_result = _AGENT_TOOLS['read_file'](non_existent_file_path)
    print(f"read_file('{non_existent_file_path}') result: {read_non_existent_result}")
    if not read_non_existent_result.get('success') and read_non_existent_result.get('content') is None:
        print(f"  SUCCESS: read_file correctly reported failure for non-existent file.")
    else:
        print(f"  FAILURE: read_file did not report failure as expected for non-existent file.")

    # Test 5: list_directory for a non-existent directory
    print(f"\n--- Test 5: list_directory for non-existent directory '{non_existent_dir_path}' ---")
    list_non_existent_result = _AGENT_TOOLS['list_directory'](non_existent_dir_path)
    print(f"list_directory('{non_existent_dir_path}') result: {list_non_existent_result}")
    if not list_non_existent_result.get('success') and list_non_existent_result.get('contents') is None:
        print(f"  SUCCESS: list_directory correctly reported failure for non-existent directory.")
    else:
        print(f"  FAILURE: list_directory did not report failure as expected for non-existent directory.")

    # Cleanup (optional, but good practice for repeated tests)
    # Using execute_python_code would be safer if we wanted to guarantee cleanup even if these tools are flaky
    # For now, direct os calls for cleanup if primary tools worked.
    if write_result.get('success'): # Basic check before attempting cleanup
        print("\n--- Attempting cleanup of test files/directories ---")
        try:
            if os.path.exists(test_file_path):
                os.remove(test_file_path)
                print(f"Agent log: Cleaned up file: {test_file_path}")
            if os.path.exists(test_dir_path):
                # Check if directory is empty before rmdir, or use shutil.rmtree
                if not os.listdir(test_dir_path): # Only remove if empty
                    os.rmdir(test_dir_path)
                    print(f"Agent log: Cleaned up empty directory: {test_dir_path}")
                else: # If not empty, likely means file wasn't removed or other issue
                    shutil.rmtree(test_dir_path) # Force remove directory and contents
                    print(f"Agent log: Cleaned up directory (and contents) using shutil.rmtree: {test_dir_path}")
        except Exception as e:
            print(f"Agent log: Error during cleanup: {e}")


print("\nAgent: Re-initialization and testing of file system tools complete.")
print("Waiting for next instruction.")

Agent: Attempting to re-initialize and test file system tools due to previous 'tool not found' errors.
Agent: File system tool functions re-defined (v2).
Agent: 'add_tool' function is available. Proceeding with tool registration.
Agent: State of _AGENT_TOOLS before re-adding: ['list_tools', 'get_agent_status', 'execute_python_code']
Tool 'write_file' added successfully. Description: (Version 2) Writes text content to a specified file. Overwrites if exists, creates directories if needed.
Tool 'read_file' added successfully. Description: (Version 2) Reads the text content from a specified file.
Tool 'list_directory' added successfully. Description: (Version 2) Lists files and directories within a specified path. Defaults to current directory.
Agent: State of _AGENT_TOOLS after re-adding: ['list_tools', 'get_agent_status', 'execute_python_code', 'write_file', 'read_file', 'list_directory']

Agent: Verifying tool registration immediately by listing all available tools:

Available tools:
1.

In [6]:
# _AGENT_TOOLS, add_tool, etc., are assumed to be in the global scope.
import os
import shutil # For rmtree (recursive directory deletion)

print("Agent: Defining and registering enhanced file system management tools.")

# --- Define New File System Tools ---

def create_directory_tool(path: str) -> dict:
    """
    Creates a directory at the specified path.
    If parent directories do not exist, they will be created (like mkdir -p).
    If the directory already exists, it will report success without error.

    Args:
        path (str): The path where the directory should be created.

    Returns:
        dict: A dictionary containing:
            'success' (bool): True if the directory was created or already existed, False otherwise.
            'message' (str): A message indicating the outcome.
            'path' (str): The absolute path of the created/verified directory.
    """
    tool_name = "create_directory_tool"
    print(f"Agent log ({tool_name}): Called with path='{path}'.")
    try:
        abs_path = os.path.abspath(path)
        if os.path.exists(abs_path) and os.path.isdir(abs_path):
            message = f"Directory '{abs_path}' already exists."
            print(f"Agent log ({tool_name}): {message}")
            return {"success": True, "message": message, "path": abs_path}
        
        os.makedirs(path, exist_ok=True) # exist_ok=True means no error if it already exists
        message = f"Directory '{abs_path}' created successfully (or already existed)."
        print(f"Agent log ({tool_name}): {message}")
        return {"success": True, "message": message, "path": abs_path}
    except Exception as e:
        error_message = f"Error creating directory '{path}': {type(e).__name__}: {e}"
        print(f"Agent log ({tool_name}): {error_message}")
        return {"success": False, "message": error_message, "path": os.path.abspath(path)}

def delete_file_tool(filepath: str) -> dict:
    """
    Deletes a file at the specified filepath.

    Args:
        filepath (str): The path to the file to be deleted.

    Returns:
        dict: A dictionary containing:
            'success' (bool): True if the file was deleted successfully, False otherwise.
            'message' (str): A message indicating the outcome.
            'filepath' (str): The absolute path of the targeted file.
    """
    tool_name = "delete_file_tool"
    print(f"Agent log ({tool_name}): Called with filepath='{filepath}'.")
    abs_filepath = os.path.abspath(filepath)
    try:
        if not os.path.exists(abs_filepath):
            message = f"File '{abs_filepath}' does not exist. Cannot delete."
            print(f"Agent log ({tool_name}): {message}")
            return {"success": False, "message": message, "filepath": abs_filepath}
        if not os.path.isfile(abs_filepath):
            message = f"Path '{abs_filepath}' is not a file. Cannot delete using this tool."
            print(f"Agent log ({tool_name}): {message}")
            return {"success": False, "message": message, "filepath": abs_filepath}
        
        os.remove(abs_filepath)
        message = f"File '{abs_filepath}' deleted successfully."
        print(f"Agent log ({tool_name}): {message}")
        return {"success": True, "message": message, "filepath": abs_filepath}
    except Exception as e:
        error_message = f"Error deleting file '{abs_filepath}': {type(e).__name__}: {e}"
        print(f"Agent log ({tool_name}): {error_message}")
        return {"success": False, "message": error_message, "filepath": abs_filepath}

def delete_directory_tool(path: str, recursive: bool = False) -> dict:
    """
    Deletes a directory at the specified path.
    If 'recursive' is False (default), the directory must be empty.
    If 'recursive' is True, it will delete the directory and all its contents.

    Args:
        path (str): The path to the directory to be deleted.
        recursive (bool, optional): Whether to delete non-empty directories. Defaults to False.

    Returns:
        dict: A dictionary containing:
            'success' (bool): True if the directory was deleted successfully, False otherwise.
            'message' (str): A message indicating the outcome.
            'path' (str): The absolute path of the targeted directory.
    """
    tool_name = "delete_directory_tool"
    print(f"Agent log ({tool_name}): Called with path='{path}', recursive={recursive}.")
    abs_path = os.path.abspath(path)
    try:
        if not os.path.exists(abs_path):
            message = f"Directory '{abs_path}' does not exist. Cannot delete."
            print(f"Agent log ({tool_name}): {message}")
            return {"success": False, "message": message, "path": abs_path}
        if not os.path.isdir(abs_path):
            message = f"Path '{abs_path}' is not a directory. Cannot delete using this tool."
            print(f"Agent log ({tool_name}): {message}")
            return {"success": False, "message": message, "path": abs_path}

        if recursive:
            shutil.rmtree(abs_path)
            message = f"Directory '{abs_path}' and its contents deleted successfully (recursively)."
        else:
            if os.listdir(abs_path): # Check if directory is empty
                message = f"Directory '{abs_path}' is not empty. Cannot delete non-recursively. Use recursive=True to delete non-empty directories."
                print(f"Agent log ({tool_name}): {message}")
                return {"success": False, "message": message, "path": abs_path}
            os.rmdir(abs_path)
            message = f"Empty directory '{abs_path}' deleted successfully."
        
        print(f"Agent log ({tool_name}): {message}")
        return {"success": True, "message": message, "path": abs_path}
    except Exception as e:
        error_message = f"Error deleting directory '{abs_path}': {type(e).__name__}: {e}"
        print(f"Agent log ({tool_name}): {error_message}")
        return {"success": False, "message": error_message, "path": abs_path}

# --- Register New Tools ---
print("\nAgent: Registering new file management tools...")
add_tool(
    name="create_directory",
    tool_callable=create_directory_tool,
    description="Creates a directory. Creates parent directories if needed. Idempotent."
)
add_tool(
    name="delete_file",
    tool_callable=delete_file_tool,
    description="Deletes a specified file."
)
add_tool(
    name="delete_directory",
    tool_callable=delete_directory_tool,
    description="Deletes a directory. Must be empty unless 'recursive=True' is specified."
)

# --- Verify Registration ---
print("\nAgent: Verifying tool registration by listing all available tools:")
if 'list_tools' in _AGENT_TOOLS:
    _AGENT_TOOLS['list_tools']()
else:
    print("CRITICAL WARNING: 'list_tools' tool is missing.")

# --- Test New File Management Tools ---
print("\nAgent: Starting tests for new file management tools.")
test_base_dir = "agent_fs_mgmt_tests"
test_dir_to_create = os.path.join(test_base_dir, "my_new_dir")
test_file_in_dir = os.path.join(test_dir_to_create, "temp_file.txt")
test_empty_dir_to_delete = os.path.join(test_base_dir, "empty_dir_for_deletion")

# Ensure the base test directory is clean for repeatable tests (using existing tools if possible, or direct os calls)
if os.path.exists(test_base_dir):
    print(f"Agent log: Pre-test cleanup: Removing existing '{test_base_dir}'.")
    shutil.rmtree(test_base_dir) # Direct call for robust cleanup before test run
os.makedirs(test_base_dir, exist_ok=True) # Create the base test directory

# Test 1: Create a directory
print(f"\n--- Test 1: create_directory '{test_dir_to_create}' ---")
create_dir_result = _AGENT_TOOLS['create_directory'](path=test_dir_to_create)
print(f"create_directory result: {create_dir_result}")
if create_dir_result.get('success') and os.path.isdir(test_dir_to_create):
    print("  SUCCESS: Directory created or already existed.")
else:
    print("  FAILURE: Directory not created as expected.")

# Test 2: Create a file in the new directory (using write_file)
print(f"\n--- Test 2: write_file to '{test_file_in_dir}' ---")
write_result = _AGENT_TOOLS['write_file'](filepath=test_file_in_dir, content="Test content for deletion.")
print(f"write_file result: {write_result}")

# Test 3: Delete the file
print(f"\n--- Test 3: delete_file '{test_file_in_dir}' ---")
delete_file_result = _AGENT_TOOLS['delete_file'](filepath=test_file_in_dir)
print(f"delete_file result: {delete_file_result}")
if delete_file_result.get('success') and not os.path.exists(test_file_in_dir):
    print("  SUCCESS: File deleted.")
else:
    print("  FAILURE: File not deleted as expected.")

# Test 4: Attempt to delete the now-empty directory (non-recursively)
print(f"\n--- Test 4: delete_directory (non-recursive) '{test_dir_to_create}' ---")
delete_empty_dir_result = _AGENT_TOOLS['delete_directory'](path=test_dir_to_create, recursive=False)
print(f"delete_directory (non-recursive) result: {delete_empty_dir_result}")
if delete_empty_dir_result.get('success') and not os.path.exists(test_dir_to_create):
    print("  SUCCESS: Empty directory deleted non-recursively.")
else:
    print("  FAILURE: Empty directory not deleted or error occurred.")

# Test 5: Create a directory, put a file in it, then delete recursively
print(f"\n--- Test 5: delete_directory (recursive) ---")
# Re-create directory and file
_AGENT_TOOLS['create_directory'](path=test_dir_to_create)
_AGENT_TOOLS['write_file'](filepath=test_file_in_dir, content="Another test file.")
print(f"Agent log: Re-created '{test_dir_to_create}' and '{test_file_in_dir}' for recursive delete test.")

delete_recursive_result = _AGENT_TOOLS['delete_directory'](path=test_dir_to_create, recursive=True)
print(f"delete_directory (recursive) result: {delete_recursive_result}")
if delete_recursive_result.get('success') and not os.path.exists(test_dir_to_create):
    print("  SUCCESS: Non-empty directory deleted recursively.")
else:
    print(f"  FAILURE: Non-empty directory not deleted recursively. Path exists: {os.path.exists(test_dir_to_create)}")

# Test 6: Attempt to delete a non-existent file
print(f"\n--- Test 6: delete_file on non-existent file ---")
delete_non_existent_file_result = _AGENT_TOOLS['delete_file'](filepath="non_existent_123.txt")
print(f"delete_file (non-existent) result: {delete_non_existent_file_result}")
if not delete_non_existent_file_result.get('success'):
    print("  SUCCESS: Correctly handled non-existent file deletion attempt.")
else:
    print("  FAILURE: Incorrectly reported success for non-existent file.")

# Test 7: Attempt to delete a non-existent directory
print(f"\n--- Test 7: delete_directory on non-existent directory ---")
delete_non_existent_dir_result = _AGENT_TOOLS['delete_directory'](path="non_existent_dir_123")
print(f"delete_directory (non-existent) result: {delete_non_existent_dir_result}")
if not delete_non_existent_dir_result.get('success'):
    print("  SUCCESS: Correctly handled non-existent directory deletion attempt.")
else:
    print("  FAILURE: Incorrectly reported success for non-existent directory.")

# Test 8: Attempt to delete non-empty directory non-recursively (should fail)
print(f"\n--- Test 8: delete_directory (non-recursive, non-empty) ---")
_AGENT_TOOLS['create_directory'](path=test_dir_to_create)
_AGENT_TOOLS['write_file'](filepath=test_file_in_dir, content="File to make dir non-empty.")
print(f"Agent log: Re-created '{test_dir_to_create}' and '{test_file_in_dir}' for non-empty, non-recursive delete test.")
delete_non_empty_fail_result = _AGENT_TOOLS['delete_directory'](path=test_dir_to_create, recursive=False)
print(f"delete_directory (non-recursive, non-empty) result: {delete_non_empty_fail_result}")
if not delete_non_empty_fail_result.get('success') and os.path.exists(test_dir_to_create):
    print("  SUCCESS: Correctly failed to delete non-empty directory non-recursively.")
    # Cleanup for next potential run
    _AGENT_TOOLS['delete_directory'](path=test_dir_to_create, recursive=True)
    print(f"Agent log: Cleaned up '{test_dir_to_create}' recursively after non-empty test.")
else:
    print("  FAILURE: Did not correctly fail or directory was unexpectedly deleted.")


# Final cleanup of the base test directory
print(f"\nAgent log: Post-test cleanup: Removing '{test_base_dir}'.")
if os.path.exists(test_base_dir):
    shutil.rmtree(test_base_dir)

print("\nAgent: Tests for new file management tools complete.")
print("Waiting for next instruction.")

Agent: Defining and registering enhanced file system management tools.

Agent: Registering new file management tools...
Tool 'create_directory' added successfully. Description: Creates a directory. Creates parent directories if needed. Idempotent.
Tool 'delete_file' added successfully. Description: Deletes a specified file.
Tool 'delete_directory' added successfully. Description: Deletes a directory. Must be empty unless 'recursive=True' is specified.

Agent: Verifying tool registration by listing all available tools:

Available tools:
1. list_tools: Lists all available tools with their names and descriptions. Useful for understanding agent capabilities.
2. get_agent_status: Provides a status report of the agent, including version and number of loaded tools.
3. execute_python_code: Executes a given string of Python code in the agent's current global environment. Captures and returns stdout, stderr, and any execution error from the provided code. USE WITH EXTREME CAUTION, as it can alte

In [7]:
# _AGENT_TOOLS, add_tool, etc., are assumed to be in the global scope.
import json # For handling JSON data
import os # For potential future file downloads
# Attempt to import the requests library.
# If this fails, the tool will indicate the dependency is missing.
_REQUESTS_AVAILABLE = False
try:
    import requests
    _REQUESTS_AVAILABLE = True
except ImportError:
    print("Agent Warning: The 'requests' library is not installed. 'make_http_request' tool will not be functional.")
    # The tool itself will also check _REQUESTS_AVAILABLE.

print("Agent: Defining and registering HTTP request tool.")

# --- Define New HTTP Request Tool ---

def make_http_request_tool(
    url: str,
    method: str = "GET",
    headers: dict = None,
    params: dict = None,
    json_payload: dict = None,
    data_payload=None, # Can be dict for form data, or string
    timeout: int = 10 # Seconds
) -> dict:
    """
    Makes an HTTP request to the specified URL using the given method and options.

    Args:
        url (str): The URL to request.
        method (str, optional): HTTP method (e.g., "GET", "POST", "PUT", "DELETE"). Defaults to "GET".
        headers (dict, optional): Dictionary of HTTP headers.
        params (dict, optional): Dictionary of URL parameters (for GET requests).
        json_payload (dict, optional): JSON data to send in the request body (sets Content-Type to application/json).
        data_payload (any, optional): Data to send in the request body (e.g., for form data).
                           If json_payload is provided, this is ignored.
        timeout (int, optional): Request timeout in seconds. Defaults to 10.

    Returns:
        dict: A dictionary containing:
            'success' (bool): True if the request was successful (2xx status code), False otherwise.
            'status_code' (int or None): HTTP status code, or None if request failed before sending.
            'headers' (dict or None): Response headers as a dictionary, or None.
            'text_content' (str or None): Response body as text if applicable, or None.
            'json_content' (dict or None): Response body parsed as JSON if applicable, or None.
            'error_message' (str or None): Error message if the request failed or an exception occurred.
            'url' (str): The URL that was requested.
    """
    tool_name = "make_http_request_tool"
    print(f"Agent log ({tool_name}): Called with URL='{url}', Method='{method}'.")

    if not _REQUESTS_AVAILABLE:
        msg = "The 'requests' library is not available. Cannot make HTTP requests."
        print(f"Agent log ({tool_name}): {msg}")
        return {
            "success": False, "status_code": None, "headers": None, 
            "text_content": None, "json_content": None, "error_message": msg, "url": url
        }

    method = method.upper()
    response_data = {
        "success": False, "status_code": None, "headers": None,
        "text_content": None, "json_content": None, "error_message": None, "url": url
    }

    try:
        request_kwargs = {"headers": headers or {}, "params": params, "timeout": timeout}
        
        if json_payload is not None:
            request_kwargs["json"] = json_payload
            if "Content-Type" not in request_kwargs["headers"]: # Auto-set if not provided
                 request_kwargs["headers"]["Content-Type"] = "application/json"
        elif data_payload is not None:
            request_kwargs["data"] = data_payload

        print(f"Agent log ({tool_name}): Making {method} request to {url} with args: {request_kwargs}")
        
        response = requests.request(method, url, **request_kwargs)
        
        response_data["status_code"] = response.status_code
        response_data["headers"] = dict(response.headers)
        response_data["text_content"] = response.text
        
        # Try to parse JSON content if Content-Type suggests it
        content_type = response.headers.get("Content-Type", "").lower()
        if "application/json" in content_type:
            try:
                response_data["json_content"] = response.json()
            except json.JSONDecodeError:
                print(f"Agent log ({tool_name}): Content-Type was JSON, but failed to decode JSON response.")
                # Keep text_content available
        
        # Consider 2xx status codes as success
        if 200 <= response.status_code < 300:
            response_data["success"] = True
            print(f"Agent log ({tool_name}): Request successful ({response.status_code}).")
        else:
            response_data["success"] = False
            response_data["error_message"] = f"HTTP Error: {response.status_code} {response.reason}"
            print(f"Agent log ({tool_name}): Request failed ({response.status_code} {response.reason}).")
            print(f"Agent log ({tool_name}): Response text for error: {response.text[:500]}...")


    except requests.exceptions.RequestException as e:
        error_msg = f"RequestException: {type(e).__name__}: {e}"
        print(f"Agent log ({tool_name}): {error_msg}")
        response_data["error_message"] = error_msg
    except Exception as e:
        error_msg = f"Unexpected error during HTTP request: {type(e).__name__}: {e}"
        print(f"Agent log ({tool_name}): {error_msg}")
        response_data["error_message"] = error_msg
        
    return response_data

# --- Register New Tool ---
print("\nAgent: Registering HTTP request tool...")
if 'add_tool' in globals() and callable(add_tool):
    add_tool(
        name="make_http_request",
        tool_callable=make_http_request_tool,
        description="Makes HTTP requests (GET, POST, etc.). Handles headers, params, JSON/data payloads, and timeout. Returns status, headers, and content (text/JSON)."
    )
else:
    print("CRITICAL ERROR: 'add_tool' function not found. Cannot register 'make_http_request'.")


# --- Verify Registration ---
print("\nAgent: Verifying tool registration by listing all available tools:")
if 'list_tools' in _AGENT_TOOLS and callable(_AGENT_TOOLS['list_tools']):
    _AGENT_TOOLS['list_tools']()
else:
    print("CRITICAL WARNING: 'list_tools' tool is missing.")

# --- Test New HTTP Request Tool ---
print("\nAgent: Starting tests for HTTP request tool.")

if not _REQUESTS_AVAILABLE:
    print("Agent: Skipping HTTP request tests as 'requests' library is not available.")
elif 'make_http_request' not in _AGENT_TOOLS:
    print("Agent: Skipping HTTP request tests as 'make_http_request' tool was not successfully registered.")
else:
    # Test 1: Simple GET request
    print("\n--- Test 1: Simple GET request to jsonplaceholder ---")
    get_url = "https://jsonplaceholder.typicode.com/todos/1"
    get_result = _AGENT_TOOLS['make_http_request'](url=get_url, method="GET")
    print(f"GET request to '{get_url}' result:")
    # Print selective parts of the result for brevity in logs
    print(f"  Success: {get_result.get('success')}")
    print(f"  Status Code: {get_result.get('status_code')}")
    print(f"  Error: {get_result.get('error_message')}")
    if get_result.get('json_content'):
        print(f"  JSON Content (partial): {{'userId': {get_result['json_content'].get('userId')}, 'title': '{get_result['json_content'].get('title')[:30]}...'}}")
    elif get_result.get('text_content'):
        print(f"  Text Content (first 100 chars): {get_result.get('text_content', '')[:100]}")

    if get_result.get('success') and get_result.get('status_code') == 200 and get_result.get('json_content'):
        print("  SUCCESS: GET request seems to have worked as expected.")
    else:
        print("  FAILURE: GET request did not meet expectations.")

    # Test 2: POST request with JSON payload
    print("\n--- Test 2: POST request to httpbin.org/post ---")
    post_url = "https://httpbin.org/post"
    post_payload = {"agent_name": "MaxUsefulAgent", "version": _AGENT_VERSION, "purpose": "testing"}
    post_result = _AGENT_TOOLS['make_http_request'](
        url=post_url,
        method="POST",
        json_payload=post_payload,
        headers={"X-Agent-Test": "true"}
    )
    print(f"POST request to '{post_url}' result:")
    print(f"  Success: {post_result.get('success')}")
    print(f"  Status Code: {post_result.get('status_code')}")
    print(f"  Error: {post_result.get('error_message')}")
    if post_result.get('json_content') and isinstance(post_result['json_content'].get('json'), dict):
        print(f"  JSON Response 'json' field (partial): {str(post_result['json_content']['json'])[:100]}...")
        # Verify if our payload is echoed back by httpbin
        if post_result['json_content']['json'] == post_payload:
             print("  SUCCESS: POST request echoed back the sent JSON payload correctly.")
        else:
             print("  FAILURE: POST request did not echo back the sent JSON payload as expected.")
    elif post_result.get('text_content'):
        print(f"  Text Content (first 100 chars): {post_result.get('text_content', '')[:100]}")

    if not (post_result.get('success') and post_result.get('status_code') == 200 and post_result.get('json_content')):
        print("  Overall FAILURE: POST request did not meet basic expectations.")

    # Test 3: Request to a non-existent domain (should fail)
    print("\n--- Test 3: GET request to a non-existent domain ---")
    bad_url = "http://domainthatshouldnotexist123abc.com"
    bad_url_result = _AGENT_TOOLS['make_http_request'](url=bad_url, method="GET", timeout=3) # Shorter timeout
    print(f"GET request to '{bad_url}' result:")
    print(f"  Success: {bad_url_result.get('success')}") # Should be False
    print(f"  Status Code: {bad_url_result.get('status_code')}") # Should be None
    print(f"  Error: {bad_url_result.get('error_message')}") # Should have an error message
    
    if not bad_url_result.get('success') and bad_url_result.get('error_message'):
        print("  SUCCESS: Request to non-existent domain failed as expected.")
    else:
        print("  FAILURE: Request to non-existent domain did not fail as expected.")

print("\nAgent: HTTP request tool definition, registration, and tests complete.")
print("Waiting for next instruction.")

Agent: Defining and registering HTTP request tool.

Agent: Registering HTTP request tool...
Tool 'make_http_request' added successfully. Description: Makes HTTP requests (GET, POST, etc.). Handles headers, params, JSON/data payloads, and timeout. Returns status, headers, and content (text/JSON).

Agent: Verifying tool registration by listing all available tools:

Available tools:
1. list_tools: Lists all available tools with their names and descriptions. Useful for understanding agent capabilities.
2. get_agent_status: Provides a status report of the agent, including version and number of loaded tools.
3. execute_python_code: Executes a given string of Python code in the agent's current global environment. Captures and returns stdout, stderr, and any execution error from the provided code. USE WITH EXTREME CAUTION, as it can alter the agent's state and capabilities.
4. write_file: (Version 2) Writes text content to a specified file. Overwrites if exists, creates directories if needed.


In [8]:
# _AGENT_TOOLS, add_tool, etc., are assumed to be in the global scope.
import json
import os

# --- Define Knowledge Store Tools ---

_KNOWLEDGE_STORE_FILE = "agent_knowledge_store.json" # File to store knowledge

def _load_knowledge_store() -> dict:
    """Helper function to load the entire knowledge store from file."""
    if not os.path.exists(_KNOWLEDGE_STORE_FILE):
        return {}
    try:
        with open(_KNOWLEDGE_STORE_FILE, 'r', encoding='utf-8') as f:
            # Handle empty file case gracefully
            content = f.read()
            if not content:
                return {}
            return json.loads(content)
    except json.JSONDecodeError:
        print(f"Agent Warning: Knowledge store file '{_KNOWLEDGE_STORE_FILE}' is corrupted or not valid JSON. Starting with an empty store.")
        return {} # Return empty if file is corrupted
    except Exception as e:
        print(f"Agent Error: Could not load knowledge store: {e}")
        return {} # Return empty on other errors

def _save_knowledge_store(data: dict) -> bool:
    """Helper function to save the entire knowledge store to file."""
    try:
        with open(_KNOWLEDGE_STORE_FILE, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2) # Pretty print for readability
        return True
    except Exception as e:
        print(f"Agent Error: Could not save knowledge store: {e}")
        return False

def save_knowledge_tool(key: str, value: any) -> dict:
    """
    Saves a key-value pair to the agent's persistent knowledge store.
    The value can be any JSON-serializable Python object.
    Overwrites the key if it already exists.

    Args:
        key (str): The key to store the value under.
        value (any): The JSON-serializable value to store.

    Returns:
        dict: {'success': bool, 'message': str, 'key': str}
    """
    tool_name = "save_knowledge_tool"
    print(f"Agent log ({tool_name}): Called with key='{key}'.")
    # Ensure value is JSON serializable early to prevent partial writes of corrupted data
    try:
        json.dumps(value) # Test serializability
    except TypeError as e:
        msg = f"Value for key '{key}' is not JSON serializable: {e}"
        print(f"Agent log ({tool_name}): {msg}")
        return {"success": False, "message": msg, "key": key}

    store = _load_knowledge_store()
    store[key] = value
    if _save_knowledge_store(store):
        msg = f"Knowledge for key '{key}' saved successfully."
        print(f"Agent log ({tool_name}): {msg}")
        return {"success": True, "message": msg, "key": key}
    else:
        msg = f"Failed to save knowledge for key '{key}' due to a store saving error."
        print(f"Agent log ({tool_name}): {msg}")
        return {"success": False, "message": msg, "key": key}

def retrieve_knowledge_tool(key: str, default: any = None) -> dict:
    """
    Retrieves a value from the agent's knowledge store by its key.

    Args:
        key (str): The key of the value to retrieve.
        default (any, optional): Value to return if key is not found. Defaults to None.

    Returns:
        dict: {'success': bool, 'key': str, 'value': any (retrieved value or default), 'found': bool, 'message': str}
    """
    tool_name = "retrieve_knowledge_tool"
    print(f"Agent log ({tool_name}): Called with key='{key}'.")
    store = _load_knowledge_store()
    if key in store:
        value = store[key]
        msg = f"Knowledge for key '{key}' retrieved successfully."
        print(f"Agent log ({tool_name}): {msg}")
        return {"success": True, "key": key, "value": value, "found": True, "message": msg}
    else:
        msg = f"Knowledge for key '{key}' not found. Returning default."
        print(f"Agent log ({tool_name}): {msg}")
        return {"success": True, "key": key, "value": default, "found": False, "message": msg}


def list_knowledge_keys_tool() -> dict:
    """
    Lists all keys currently stored in the agent's knowledge store.

    Returns:
        dict: {'success': bool, 'keys': list[str], 'count': int, 'message': str}
    """
    tool_name = "list_knowledge_keys_tool"
    print(f"Agent log ({tool_name}): Called.")
    store = _load_knowledge_store()
    keys = list(store.keys())
    count = len(keys)
    msg = f"Found {count} key(s) in the knowledge store."
    print(f"Agent log ({tool_name}): {msg}")
    return {"success": True, "keys": keys, "count": count, "message": msg}

def delete_knowledge_tool(key: str) -> dict:
    """
    Deletes a key-value pair from the agent's knowledge store.

    Args:
        key (str): The key to delete.

    Returns:
        dict: {'success': bool, 'message': str, 'key': str, 'deleted': bool}
              'deleted' is True if key existed and was removed, False otherwise.
    """
    tool_name = "delete_knowledge_tool"
    print(f"Agent log ({tool_name}): Called with key='{key}'.")
    store = _load_knowledge_store()
    if key in store:
        del store[key]
        if _save_knowledge_store(store):
            msg = f"Knowledge for key '{key}' deleted successfully."
            print(f"Agent log ({tool_name}): {msg}")
            return {"success": True, "message": msg, "key": key, "deleted": True}
        else:
            msg = f"Found key '{key}', but failed to save knowledge store after deletion."
            print(f"Agent log ({tool_name}): {msg}")
            return {"success": False, "message": msg, "key": key, "deleted": False}
    else:
        msg = f"Knowledge for key '{key}' not found. Nothing to delete."
        print(f"Agent log ({tool_name}): {msg}")
        return {"success": True, "message": msg, "key": key, "deleted": False} # Success as operation completed without error

# --- Register New Tools ---
print("\nAgent: Registering knowledge store tools...")
if 'add_tool' in globals() and callable(add_tool):
    add_tool("save_knowledge", save_knowledge_tool, "Saves a JSON-serializable key-value pair to persistent knowledge store.")
    add_tool("retrieve_knowledge", retrieve_knowledge_tool, "Retrieves a value by key from knowledge store. Returns default if not found.")
    add_tool("list_knowledge_keys", list_knowledge_keys_tool, "Lists all keys in the knowledge store.")
    add_tool("delete_knowledge", delete_knowledge_tool, "Deletes a key-value pair from the knowledge store.")
else:
    print("CRITICAL ERROR: 'add_tool' function not found. Cannot register knowledge tools.")

# --- Verify Registration ---
print("\nAgent: Verifying tool registration:")
if 'list_tools' in _AGENT_TOOLS and callable(_AGENT_TOOLS['list_tools']):
    _AGENT_TOOLS['list_tools']()
else:
    print("CRITICAL WARNING: 'list_tools' tool is missing.")

# --- Test Knowledge Store Tools ---
print("\nAgent: Starting tests for knowledge store tools.")

# Ensure the knowledge store file is clean before tests for predictable results.
if os.path.exists(_KNOWLEDGE_STORE_FILE):
    print(f"Agent log: Pre-test cleanup: Deleting existing knowledge store file '{_KNOWLEDGE_STORE_FILE}'.")
    os.remove(_KNOWLEDGE_STORE_FILE)

if not all(tool_name in _AGENT_TOOLS for tool_name in ["save_knowledge", "retrieve_knowledge", "list_knowledge_keys", "delete_knowledge"]):
    print("Agent: Skipping knowledge store tests as one or more tools were not successfully registered.")
else:
    # Test 1: Save and Retrieve
    print("\n--- Test 1: Save and Retrieve Knowledge ---")
    key1, value1 = "agent_name", "MaxUseful"
    save_result1 = _AGENT_TOOLS['save_knowledge'](key=key1, value=value1)
    print(f"Save result 1: {save_result1}")
    retrieve_result1 = _AGENT_TOOLS['retrieve_knowledge'](key=key1)
    print(f"Retrieve result 1: {retrieve_result1}")
    if retrieve_result1.get('found') and retrieve_result1.get('value') == value1:
        print("  SUCCESS: Saved and retrieved value matches.")
    else:
        print("  FAILURE: Saved and retrieved value mismatch or not found.")

    key2, value2 = "agent_version_info", {"major": 0, "minor": 2, "status": "beta"}
    _AGENT_TOOLS['save_knowledge'](key=key2, value=value2)
    retrieve_result2 = _AGENT_TOOLS['retrieve_knowledge'](key=key2)
    if retrieve_result2.get('found') and retrieve_result2.get('value') == value2:
        print("  SUCCESS: Saved and retrieved complex dictionary value matches.")
    else:
        print("  FAILURE: Complex dictionary value mismatch or not found.")

    # Test 2: List Keys
    print("\n--- Test 2: List Knowledge Keys ---")
    list_result = _AGENT_TOOLS['list_knowledge_keys']()
    print(f"List keys result: {list_result}")
    expected_keys = sorted([key1, key2])
    if list_result.get('success') and sorted(list_result.get('keys', [])) == expected_keys:
        print(f"  SUCCESS: Listed keys {list_result.get('keys')} match expected {expected_keys}.")
    else:
        print(f"  FAILURE: Listed keys mismatch. Got {list_result.get('keys')}, expected {expected_keys}.")

    # Test 3: Retrieve non-existent key
    print("\n--- Test 3: Retrieve Non-existent Key ---")
    non_existent_key = "ghost_key"
    retrieve_non_existent = _AGENT_TOOLS['retrieve_knowledge'](key=non_existent_key, default="not_here")
    print(f"Retrieve non-existent result: {retrieve_non_existent}")
    if not retrieve_non_existent.get('found') and retrieve_non_existent.get('value') == "not_here":
        print("  SUCCESS: Correctly handled non-existent key and returned default.")
    else:
        print("  FAILURE: Incorrect behavior for non-existent key.")

    # Test 4: Delete Key
    print("\n--- Test 4: Delete Knowledge ---")
    delete_result = _AGENT_TOOLS['delete_knowledge'](key=key1)
    print(f"Delete result for '{key1}': {delete_result}")
    retrieve_after_delete = _AGENT_TOOLS['retrieve_knowledge'](key=key1)
    if delete_result.get('deleted') and not retrieve_after_delete.get('found'):
        print(f"  SUCCESS: Key '{key1}' deleted and no longer found.")
    else:
        print(f"  FAILURE: Key '{key1}' not deleted properly or still found.")

    # Test 5: Delete non-existent key
    print("\n--- Test 5: Delete Non-existent Key ---")
    delete_non_existent_result = _AGENT_TOOLS['delete_knowledge'](key=non_existent_key)
    print(f"Delete non-existent result: {delete_non_existent_result}")
    if not delete_non_existent_result.get('deleted') and delete_non_existent_result.get('success'): # Success is true because no error, just nothing to delete
        print("  SUCCESS: Correctly handled attempt to delete non-existent key.")
    else:
        print("  FAILURE: Incorrect behavior for deleting non-existent key.")
        
    # Test 6: Save non-JSON serializable value
    print("\n--- Test 6: Save Non-JSON Serializable Value ---")
    class NotSerializable: pass
    non_serializable_value = NotSerializable()
    save_non_serializable_result = _AGENT_TOOLS['save_knowledge'](key="bad_key", value=non_serializable_value)
    print(f"Save non-serializable result: {save_non_serializable_result}")
    if not save_non_serializable_result.get('success'):
        print("  SUCCESS: Correctly failed to save non-JSON serializable value.")
        # Verify it didn't corrupt the store or add the key
        retrieve_bad_key = _AGENT_TOOLS['retrieve_knowledge'](key="bad_key")
        if not retrieve_bad_key.get('found'):
            print("  SUCCESS: Key 'bad_key' was not added to the store.")
        else:
            print("  FAILURE: Key 'bad_key' was unexpectedly added to the store.")
    else:
        print("  FAILURE: Incorrectly reported success for saving non-JSON serializable value.")


# Final cleanup of the knowledge store file for subsequent runs.
if os.path.exists(_KNOWLEDGE_STORE_FILE):
    print(f"\nAgent log: Post-test cleanup: Deleting knowledge store file '{_KNOWLEDGE_STORE_FILE}'.")
    os.remove(_KNOWLEDGE_STORE_FILE)

print("\nAgent: Knowledge store tools definition, registration, and tests complete.")
print("Waiting for next instruction.")


Agent: Registering knowledge store tools...
Tool 'save_knowledge' added successfully. Description: Saves a JSON-serializable key-value pair to persistent knowledge store.
Tool 'retrieve_knowledge' added successfully. Description: Retrieves a value by key from knowledge store. Returns default if not found.
Tool 'list_knowledge_keys' added successfully. Description: Lists all keys in the knowledge store.
Tool 'delete_knowledge' added successfully. Description: Deletes a key-value pair from the knowledge store.

Agent: Verifying tool registration:

Available tools:
1. list_tools: Lists all available tools with their names and descriptions. Useful for understanding agent capabilities.
2. get_agent_status: Provides a status report of the agent, including version and number of loaded tools.
3. execute_python_code: Executes a given string of Python code in the agent's current global environment. Captures and returns stdout, stderr, and any execution error from the provided code. USE WITH EXT

In [9]:
# _AGENT_TOOLS, add_tool, etc., are assumed to be in the global scope.
import json
import os
import time
import uuid # For generating unique task IDs

# --- Define Task Management Tools ---

_TASKS_FILE = "agent_tasks.json"

def _load_tasks() -> dict:
    """Helper function to load all tasks from the tasks file."""
    if not os.path.exists(_TASKS_FILE):
        return {} # Tasks are stored as a dict: {task_id: task_data}
    try:
        with open(_TASKS_FILE, 'r', encoding='utf-8') as f:
            content = f.read()
            if not content:
                return {}
            return json.loads(content)
    except json.JSONDecodeError:
        print(f"Agent Warning: Tasks file '{_TASKS_FILE}' is corrupted. Starting with an empty task list.")
        return {}
    except Exception as e:
        print(f"Agent Error: Could not load tasks: {e}")
        return {}

def _save_tasks(tasks_data: dict) -> bool:
    """Helper function to save all tasks to the tasks file."""
    try:
        with open(_TASKS_FILE, 'w', encoding='utf-8') as f:
            json.dump(tasks_data, f, indent=2)
        return True
    except Exception as e:
        print(f"Agent Error: Could not save tasks: {e}")
        return False

def add_task_tool(description: str, status: str = "pending") -> dict:
    """
    Adds a new task to the agent's task list.

    Args:
        description (str): A description of the task.
        status (str, optional): Initial status of the task (e.g., "pending", "in_progress", "completed", "failed"). 
                               Defaults to "pending".

    Returns:
        dict: {'success': bool, 'message': str, 'task_id': str or None, 'task_data': dict or None}
    """
    tool_name = "add_task_tool"
    print(f"Agent log ({tool_name}): Called with description='{description[:50]}...', status='{status}'.")
    if not description:
        msg = "Task description cannot be empty."
        print(f"Agent log ({tool_name}): {msg}")
        return {"success": False, "message": msg, "task_id": None, "task_data": None}

    tasks = _load_tasks()
    task_id = str(uuid.uuid4())
    current_timestamp = time.time()
    
    task_data = {
        "id": task_id,
        "description": description,
        "status": status,
        "created_at": current_timestamp,
        "updated_at": current_timestamp,
        "history": [{"timestamp": current_timestamp, "status": status, "notes": "Task created."}]
    }
    tasks[task_id] = task_data

    if _save_tasks(tasks):
        msg = f"Task '{task_id}' added successfully with status '{status}'."
        print(f"Agent log ({tool_name}): {msg}")
        return {"success": True, "message": msg, "task_id": task_id, "task_data": task_data}
    else:
        msg = "Failed to save new task due to a store saving error."
        print(f"Agent log ({tool_name}): {msg}")
        return {"success": False, "message": msg, "task_id": None, "task_data": None}

def get_task_tool(task_id: str) -> dict:
    """
    Retrieves a specific task by its ID.

    Args:
        task_id (str): The ID of the task to retrieve.

    Returns:
        dict: {'success': bool, 'message': str, 'task_id': str, 'task_data': dict or None, 'found': bool}
    """
    tool_name = "get_task_tool"
    print(f"Agent log ({tool_name}): Called with task_id='{task_id}'.")
    tasks = _load_tasks()
    task_data = tasks.get(task_id)

    if task_data:
        msg = f"Task '{task_id}' retrieved successfully."
        print(f"Agent log ({tool_name}): {msg}")
        return {"success": True, "message": msg, "task_id": task_id, "task_data": task_data, "found": True}
    else:
        msg = f"Task '{task_id}' not found."
        print(f"Agent log ({tool_name}): {msg}")
        return {"success": False, "message": msg, "task_id": task_id, "task_data": None, "found": False}

def update_task_tool(task_id: str, description: str = None, status: str = None, notes: str = None) -> dict:
    """
    Updates a task's description and/or status.
    Only provided fields (description, status) will be updated.

    Args:
        task_id (str): The ID of the task to update.
        description (str, optional): New description for the task.
        status (str, optional): New status for the task.
        notes (str, optional): Optional notes about this update, added to history.

    Returns:
        dict: {'success': bool, 'message': str, 'task_id': str, 'updated_fields': list, 'task_data': dict or None}
    """
    tool_name = "update_task_tool"
    print(f"Agent log ({tool_name}): Called for task_id='{task_id}'. Desc: {'Provided' if description else 'None'}, Status: {'Provided' if status else 'None'}.")
    tasks = _load_tasks()
    if task_id not in tasks:
        msg = f"Task '{task_id}' not found. Cannot update."
        print(f"Agent log ({tool_name}): {msg}")
        return {"success": False, "message": msg, "task_id": task_id, "updated_fields": [], "task_data": None}

    task_data = tasks[task_id]
    updated_fields = []
    current_timestamp = time.time()
    history_entry = {"timestamp": current_timestamp}
    
    if description is not None and task_data["description"] != description:
        task_data["description"] = description
        updated_fields.append("description")
        history_entry["description_updated_to"] = description
        
    if status is not None and task_data["status"] != status:
        task_data["status"] = status
        updated_fields.append("status")
        history_entry["status_changed_to"] = status

    if not updated_fields:
        msg = f"No changes provided for task '{task_id}'. Nothing updated."
        print(f"Agent log ({tool_name}): {msg}")
        return {"success": True, "message": msg, "task_id": task_id, "updated_fields": [], "task_data": task_data}

    task_data["updated_at"] = current_timestamp
    if notes: history_entry["notes"] = notes
    if len(history_entry) > 1: # More than just timestamp
        task_data.setdefault("history", []).append(history_entry)


    tasks[task_id] = task_data
    if _save_tasks(tasks):
        msg = f"Task '{task_id}' updated successfully. Fields changed: {', '.join(updated_fields)}."
        print(f"Agent log ({tool_name}): {msg}")
        return {"success": True, "message": msg, "task_id": task_id, "updated_fields": updated_fields, "task_data": task_data}
    else:
        msg = f"Failed to save updates for task '{task_id}'."
        print(f"Agent log ({tool_name}): {msg}")
        # Revert in-memory changes if save failed (though _load_tasks would overwrite on next call anyway)
        return {"success": False, "message": msg, "task_id": task_id, "updated_fields": [], "task_data": _load_tasks().get(task_id)}


def list_tasks_tool(status_filter: str = None) -> dict:
    """
    Lists tasks, optionally filtering by status.

    Args:
        status_filter (str, optional): If provided, only tasks with this status will be returned.

    Returns:
        dict: {'success': bool, 'message': str, 'tasks': list[dict], 'count': int, 'filter_applied': str or None}
    """
    tool_name = "list_tasks_tool"
    print(f"Agent log ({tool_name}): Called. Status filter: '{status_filter}'.")
    tasks_dict = _load_tasks()
    
    # Convert dict of tasks to a list for easier processing and sorting
    all_tasks_list = sorted(tasks_dict.values(), key=lambda t: t.get("created_at", 0))

    if status_filter:
        filtered_tasks = [task for task in all_tasks_list if task.get("status") == status_filter]
        count = len(filtered_tasks)
        msg = f"Found {count} task(s) matching status '{status_filter}'."
        print(f"Agent log ({tool_name}): {msg}")
        return {"success": True, "message": msg, "tasks": filtered_tasks, "count": count, "filter_applied": status_filter}
    else:
        count = len(all_tasks_list)
        msg = f"Found {count} total task(s)."
        print(f"Agent log ({tool_name}): {msg}")
        return {"success": True, "message": msg, "tasks": all_tasks_list, "count": count, "filter_applied": None}

def remove_task_tool(task_id: str) -> dict:
    """
    Removes a task from the task list.

    Args:
        task_id (str): The ID of the task to remove.

    Returns:
        dict: {'success': bool, 'message': str, 'task_id': str, 'removed': bool}
    """
    tool_name = "remove_task_tool"
    print(f"Agent log ({tool_name}): Called for task_id='{task_id}'.")
    tasks = _load_tasks()
    if task_id in tasks:
        del tasks[task_id]
        if _save_tasks(tasks):
            msg = f"Task '{task_id}' removed successfully."
            print(f"Agent log ({tool_name}): {msg}")
            return {"success": True, "message": msg, "task_id": task_id, "removed": True}
        else:
            msg = f"Found task '{task_id}', but failed to save task list after removal."
            print(f"Agent log ({tool_name}): {msg}")
            return {"success": False, "message": msg, "task_id": task_id, "removed": False}
    else:
        msg = f"Task '{task_id}' not found. Nothing to remove."
        print(f"Agent log ({tool_name}): {msg}")
        return {"success": True, "message": msg, "task_id": task_id, "removed": False} # Success as operation completed without error

# --- Register New Tools ---
print("\nAgent: Registering task management tools...")
if 'add_tool' in globals() and callable(add_tool):
    add_tool("add_task", add_task_tool, "Adds a new task with description and optional status.")
    add_tool("get_task", get_task_tool, "Retrieves a specific task by its ID.")
    add_tool("update_task", update_task_tool, "Updates a task's description, status, and adds notes to history.")
    add_tool("list_tasks", list_tasks_tool, "Lists tasks, optionally filtering by status. Sorted by creation time.")
    add_tool("remove_task", remove_task_tool, "Removes a task from the list.")
else:
    print("CRITICAL ERROR: 'add_tool' function not found. Cannot register task management tools.")

# --- Verify Registration ---
print("\nAgent: Verifying tool registration:")
if 'list_tools' in _AGENT_TOOLS and callable(_AGENT_TOOLS['list_tools']):
    _AGENT_TOOLS['list_tools']()
else:
    print("CRITICAL WARNING: 'list_tools' tool is missing.")

# --- Test Task Management Tools ---
print("\nAgent: Starting tests for task management tools.")

# Ensure tasks file is clean before tests
if os.path.exists(_TASKS_FILE):
    print(f"Agent log: Pre-test cleanup: Deleting existing tasks file '{_TASKS_FILE}'.")
    os.remove(_TASKS_FILE)

task_tools_registered = all(tool_name in _AGENT_TOOLS for tool_name in ["add_task", "get_task", "update_task", "list_tasks", "remove_task"])
if not task_tools_registered:
    print("Agent: Skipping task management tests as one or more tools were not successfully registered.")
else:
    # Test 1: Add tasks
    print("\n--- Test 1: Add Tasks ---")
    add_res1 = _AGENT_TOOLS['add_task'](description="Research quantum entanglement.", status="pending")
    task1_id = add_res1.get('task_id')
    print(f"Add task 1 result: {add_res1.get('success')}, ID: {task1_id}")
    time.sleep(0.01) # ensure timestamp difference for sorting
    add_res2 = _AGENT_TOOLS['add_task'](description="Develop a plan for world peace.", status="in_progress")
    task2_id = add_res2.get('task_id')
    print(f"Add task 2 result: {add_res2.get('success')}, ID: {task2_id}")
    time.sleep(0.01)
    add_res3 = _AGENT_TOOLS['add_task'](description="Learn to bake sourdough bread.", status="pending")
    task3_id = add_res3.get('task_id')
    print(f"Add task 3 result: {add_res3.get('success')}, ID: {task3_id}")
    
    if not (task1_id and task2_id and task3_id):
        print("  FAILURE: One or more tasks failed to add. Halting further task tests.")
    else:
        # Test 2: Get Task
        print("\n--- Test 2: Get Task ---")
        get_res = _AGENT_TOOLS['get_task'](task_id=task1_id)
        print(f"Get task '{task1_id}' result: {get_res.get('found')}, Status: {get_res.get('task_data', {}).get('status')}")
        if not (get_res.get('found') and get_res.get('task_data', {}).get('description') == "Research quantum entanglement."):
            print(f"  FAILURE: get_task failed for {task1_id}")

        # Test 3: List Tasks (all and filtered)
        print("\n--- Test 3: List Tasks ---")
        list_all_res = _AGENT_TOOLS['list_tasks']()
        print(f"List all tasks count: {list_all_res.get('count')}")
        if list_all_res.get('count') != 3: print(f"  FAILURE: Expected 3 tasks, got {list_all_res.get('count')}")
        
        list_pending_res = _AGENT_TOOLS['list_tasks'](status_filter="pending")
        print(f"List 'pending' tasks count: {list_pending_res.get('count')}")
        if list_pending_res.get('count') != 2: print(f"  FAILURE: Expected 2 pending tasks, got {list_pending_res.get('count')}")
        # Check sorting by creation time (task1 should be before task3 in pending)
        if list_pending_res.get('count') == 2 and list_pending_res['tasks'][0]['id'] == task1_id and list_pending_res['tasks'][1]['id'] == task3_id:
            print("  SUCCESS: Pending tasks are correctly listed and sorted.")
        elif list_pending_res.get('count') == 2:
            print(f"  FAILURE: Pending tasks sorting incorrect. Order: {[t['id'] for t in list_pending_res['tasks']]}")


        # Test 4: Update Task
        print("\n--- Test 4: Update Task ---")
        update_res = _AGENT_TOOLS['update_task'](task_id=task1_id, status="completed", notes="Initial research phase done.")
        print(f"Update task '{task1_id}' to 'completed': {update_res.get('success')}")
        get_updated_res = _AGENT_TOOLS['get_task'](task_id=task1_id)
        if not (get_updated_res.get('task_data', {}).get('status') == "completed"):
            print(f"  FAILURE: Task {task1_id} status not updated to 'completed'")
        if len(get_updated_res.get('task_data', {}).get('history', [])) != 2: # Create + Update
             print(f"  FAILURE: Task {task1_id} history not updated correctly. Count: {len(get_updated_res.get('task_data', {}).get('history', []))}")
        elif get_updated_res.get('task_data', {}).get('history', [])[-1].get('notes') != "Initial research phase done.":
             print(f"  FAILURE: Task {task1_id} history notes incorrect.")


        # Test 5: Remove Task
        print("\n--- Test 5: Remove Task ---")
        remove_res = _AGENT_TOOLS['remove_task'](task_id=task2_id)
        print(f"Remove task '{task2_id}': {remove_res.get('removed')}")
        list_after_remove_res = _AGENT_TOOLS['list_tasks']()
        if list_after_remove_res.get('count') != 2:
            print(f"  FAILURE: Expected 2 tasks after removal, got {list_after_remove_res.get('count')}")
        get_removed_res = _AGENT_TOOLS['get_task'](task_id=task2_id)
        if get_removed_res.get('found'):
            print(f"  FAILURE: Task {task2_id} still found after removal.")
            
        # Test 6: Update non-existent task
        print("\n--- Test 6: Update Non-existent Task ---")
        update_non_existent_res = _AGENT_TOOLS['update_task'](task_id="non-existent-task-id", status="completed")
        if update_non_existent_res.get('success'): # Should be False
            print(f"  FAILURE: Update on non-existent task reported success.")
        else:
            print(f"  SUCCESS: Update on non-existent task correctly reported failure: {update_non_existent_res.get('message')}")


# Final cleanup of the tasks file
if os.path.exists(_TASKS_FILE):
    print(f"\nAgent log: Post-test cleanup: Deleting tasks file '{_TASKS_FILE}'.")
    os.remove(_TASKS_FILE)

print("\nAgent: Task management tools definition, registration, and tests complete.")
print("Waiting for next instruction.")


Agent: Registering task management tools...
Tool 'add_task' added successfully. Description: Adds a new task with description and optional status.
Tool 'get_task' added successfully. Description: Retrieves a specific task by its ID.
Tool 'update_task' added successfully. Description: Updates a task's description, status, and adds notes to history.
Tool 'list_tasks' added successfully. Description: Lists tasks, optionally filtering by status. Sorted by creation time.
Tool 'remove_task' added successfully. Description: Removes a task from the list.

Agent: Verifying tool registration:

Available tools:
1. list_tools: Lists all available tools with their names and descriptions. Useful for understanding agent capabilities.
2. get_agent_status: Provides a status report of the agent, including version and number of loaded tools.
3. execute_python_code: Executes a given string of Python code in the agent's current global environment. Captures and returns stdout, stderr, and any execution err

In [10]:
# _AGENT_TOOLS, _TOOL_DESCRIPTIONS, add_tool, etc., are assumed to be in the global scope.
import re # For simple word tokenization
from collections import Counter # For keyword counting

print("Agent: Defining and registering a tool suggestion mechanism.")

# --- Define Tool Suggestion Tool ---

def suggest_tools_for_goal_tool(goal_description: str, top_n: int = 5) -> dict:
    """
    Suggests relevant tools from the agent's toolkit to achieve a given goal.
    Suggestions are based on keyword matching between the goal and tool names/descriptions.

    Args:
        goal_description (str): The natural language description of the goal.
        top_n (int, optional): The maximum number of tool suggestions to return. Defaults to 5.

    Returns:
        dict: {
            'success': bool,
            'message': str,
            'goal': str,
            'suggestions': list[dict] 
                Each dict in suggestions: {
                    'tool_name': str, 
                    'description': str, 
                    'relevance_score': float, 
                    'matching_keywords': list[str]
                }
        }
    """
    tool_name_internal = "suggest_tools_for_goal_tool" # Renamed to avoid conflict if registered with same name
    print(f"Agent log ({tool_name_internal}): Called with goal='{goal_description[:100]}...'.")

    if not goal_description:
        msg = "Goal description cannot be empty."
        print(f"Agent log ({tool_name_internal}): {msg}")
        return {"success": False, "message": msg, "goal": goal_description, "suggestions": []}

    # Ensure _TOOL_DESCRIPTIONS is accessible and populated.
    # It's populated by add_tool which should have run for all tools.
    global _TOOL_DESCRIPTIONS 
    if not _TOOL_DESCRIPTIONS:
        msg = "No tool descriptions available. Cannot make suggestions."
        print(f"Agent log ({tool_name_internal}): {msg}")
        return {"success": False, "message": msg, "goal": goal_description, "suggestions": []}

    # Simple tokenization and normalization (lowercase, alphanumeric words)
    def tokenize(text: str) -> set[str]:
        if not text: return set()
        words = re.findall(r'\b\w+\b', text.lower())
        # Basic stop word removal - can be expanded
        stop_words = {"a", "an", "the", "is", "to", "and", "of", "it", "for", "in", "on", "with", "tool", "function"}
        return set(word for word in words if word not in stop_words and len(word) > 2)

    goal_keywords = tokenize(goal_description)
    if not goal_keywords:
        msg = "No effective keywords found in goal description after tokenization."
        print(f"Agent log ({tool_name_internal}): {msg}")
        return {"success": True, "message": msg, "goal": goal_description, "suggestions": []}


    suggestions = []
    for tool_name, description in _TOOL_DESCRIPTIONS.items():
        tool_text = tool_name.replace("_", " ") + " " + description # Combine name and desc for keyword search
        tool_keywords = tokenize(tool_text)
        
        matching_keywords = list(goal_keywords.intersection(tool_keywords))
        
        if matching_keywords:
            # Simple relevance score: number of matching keywords
            # Could be weighted by keyword importance, IDF, etc. in a more advanced version
            relevance_score = float(len(matching_keywords))
            
            # Bonus for direct name match in goal (even partial)
            normalized_tool_name_parts = set(tokenize(tool_name.replace("_", " ")))
            if normalized_tool_name_parts.intersection(goal_keywords):
                relevance_score += 0.5 * len(normalized_tool_name_parts.intersection(goal_keywords))

            suggestions.append({
                "tool_name": tool_name,
                "description": description,
                "relevance_score": relevance_score,
                "matching_keywords": matching_keywords
            })

    # Sort suggestions by relevance score in descending order
    suggestions.sort(key=lambda x: x['relevance_score'], reverse=True)
    
    final_suggestions = suggestions[:top_n]
    
    msg = f"Found {len(final_suggestions)} relevant tool suggestion(s) for the goal."
    if not final_suggestions and suggestions: # Found some matches but not in top_n (if top_n is low)
        msg = f"Found {len(suggestions)} potential matches, but returning top {top_n}."
    elif not suggestions:
        msg = "No relevant tool suggestions found based on keyword matching."
        
    print(f"Agent log ({tool_name_internal}): {msg} Goal keywords: {goal_keywords}")
    return {
        "success": True, 
        "message": msg, 
        "goal": goal_description, 
        "suggestions": final_suggestions
    }

# --- Register New Tool ---
print("\nAgent: Registering tool suggestion tool...")
if 'add_tool' in globals() and callable(add_tool):
    add_tool(
        name="suggest_tools_for_goal", # The public name of the tool
        tool_callable=suggest_tools_for_goal_tool,
        description="Suggests relevant tools to achieve a goal based on keyword matching with tool names/descriptions."
    )
else:
    print("CRITICAL ERROR: 'add_tool' function not found. Cannot register 'suggest_tools_for_goal'.")

# --- Verify Registration ---
print("\nAgent: Verifying tool registration:")
if 'list_tools' in _AGENT_TOOLS and callable(_AGENT_TOOLS['list_tools']):
    _AGENT_TOOLS['list_tools']()
else:
    print("CRITICAL WARNING: 'list_tools' tool is missing.")

# --- Test Tool Suggestion Tool ---
print("\nAgent: Starting tests for tool suggestion tool.")

if 'suggest_tools_for_goal' not in _AGENT_TOOLS:
    print("Agent: Skipping tool suggestion tests as 'suggest_tools_for_goal' tool was not successfully registered.")
else:
    # Test 1: Goal related to file writing
    goal1 = "I want to write some text into a new document called 'my_notes.txt'."
    print(f"\n--- Test 1: Goal: '{goal1}' ---")
    suggestions1 = _AGENT_TOOLS['suggest_tools_for_goal'](goal_description=goal1, top_n=3)
    print(f"Suggestions for goal 1: {json.dumps(suggestions1, indent=2)}")
    if suggestions1['success'] and suggestions1['suggestions']:
        # Check if 'write_file' is among the top suggestions
        found_write_file = any(s['tool_name'] == 'write_file' for s in suggestions1['suggestions'])
        if found_write_file:
            print("  SUCCESS: 'write_file' likely suggested.")
        else:
            print("  POTENTIAL ISSUE: 'write_file' not in top suggestions for a file writing goal.")
    elif suggestions1['success']:
        print("  INFO: No suggestions returned, but call was successful.")
    else:
        print(f"  FAILURE: Tool suggestion call failed: {suggestions1.get('message')}")

    # Test 2: Goal related to web content and saving
    goal2 = "Download the main page of example.com and store it as example_page.html"
    print(f"\n--- Test 2: Goal: '{goal2}' ---")
    suggestions2 = _AGENT_TOOLS['suggest_tools_for_goal'](goal_description=goal2, top_n=3)
    print(f"Suggestions for goal 2: {json.dumps(suggestions2, indent=2)}")
    if suggestions2['success'] and suggestions2['suggestions']:
        found_http = any(s['tool_name'] == 'make_http_request' for s in suggestions2['suggestions'])
        found_write = any(s['tool_name'] == 'write_file' for s in suggestions2['suggestions'])
        if found_http and found_write:
            print("  SUCCESS: 'make_http_request' and 'write_file' likely suggested.")
        else:
            print("  POTENTIAL ISSUE: Key tools for web download and save not in top suggestions.")
    elif suggestions2['success']:
         print("  INFO: No suggestions returned, but call was successful.")
    else:
        print(f"  FAILURE: Tool suggestion call failed: {suggestions2.get('message')}")

    # Test 3: Goal related to listing tasks
    goal3 = "Show me all my current pending tasks."
    print(f"\n--- Test 3: Goal: '{goal3}' ---")
    suggestions3 = _AGENT_TOOLS['suggest_tools_for_goal'](goal_description=goal3, top_n=3)
    print(f"Suggestions for goal 3: {json.dumps(suggestions3, indent=2)}")
    if suggestions3['success'] and suggestions3['suggestions']:
        found_list_tasks = any(s['tool_name'] == 'list_tasks' for s in suggestions3['suggestions'])
        if found_list_tasks:
            print("  SUCCESS: 'list_tasks' likely suggested for viewing tasks.")
        else:
            print("  POTENTIAL ISSUE: 'list_tasks' not in top suggestions for a task listing goal.")
    elif suggestions3['success']:
         print("  INFO: No suggestions returned, but call was successful.")
    else:
        print(f"  FAILURE: Tool suggestion call failed: {suggestions3.get('message')}")
        
    # Test 4: Vague goal
    goal4 = "Do something useful."
    print(f"\n--- Test 4: Goal: '{goal4}' ---")
    suggestions4 = _AGENT_TOOLS['suggest_tools_for_goal'](goal_description=goal4, top_n=3)
    print(f"Suggestions for goal 4: {json.dumps(suggestions4, indent=2)}")
    # For a vague goal, we don't have a strong expectation, just that it runs.
    if suggestions4['success']:
        print("  INFO: Vague goal processed. Suggestions may or may not be highly relevant.")
    else:
        print(f"  FAILURE: Tool suggestion call failed for vague goal: {suggestions4.get('message')}")


print("\nAgent: Tool suggestion mechanism definition, registration, and tests complete.")
print("Waiting for next instruction.")

Agent: Defining and registering a tool suggestion mechanism.

Agent: Registering tool suggestion tool...
Tool 'suggest_tools_for_goal' added successfully. Description: Suggests relevant tools to achieve a goal based on keyword matching with tool names/descriptions.

Agent: Verifying tool registration:

Available tools:
1. list_tools: Lists all available tools with their names and descriptions. Useful for understanding agent capabilities.
2. get_agent_status: Provides a status report of the agent, including version and number of loaded tools.
3. execute_python_code: Executes a given string of Python code in the agent's current global environment. Captures and returns stdout, stderr, and any execution error from the provided code. USE WITH EXTREME CAUTION, as it can alter the agent's state and capabilities.
4. write_file: (Version 2) Writes text content to a specified file. Overwrites if exists, creates directories if needed.
5. read_file: (Version 2) Reads the text content from a specif

In [11]:
# _AGENT_TOOLS, _TOOL_DESCRIPTIONS, add_tool, etc., are assumed to be in the global scope.
import re
import json # For printing test results neatly

print("Agent: Defining and registering an improved tool suggestion mechanism (v2).")

# --- Define Tool Suggestion Tool V2 ---

# This dictionary maps tool names to a set of concept keywords.
# These keywords represent the core functionality and common terms associated with each tool.
_AGENT_TOOL_CONCEPT_KEYWORDS = {
    "list_tools": {"list", "show", "available", "tool", "tools", "capabilities", "what can you do", "describe", "command"},
    "get_agent_status": {"status", "state", "version", "report", "info", "health", "check"},
    "execute_python_code": {"execute", "run", "python", "code", "script", "eval", "interpret"},
    "write_file": {"write", "save", "create", "file", "document", "text", "content", "store", "output to file", "persist file"},
    "read_file": {"read", "open", "load", "file", "content", "input from file", "get file"},
    "list_directory": {"list", "show", "directory", "folder", "files", "ls", "content of directory", "browse"},
    "create_directory": {"create", "make", "new", "directory", "folder", "mkdir"},
    "delete_file": {"delete", "remove", "erase", "file", "discard file"},
    "delete_directory": {"delete", "remove", "erase", "directory", "folder", "rmdir", "discard folder"},
    "make_http_request": {"http", "request", "web", "get", "post", "download", "fetch", "url", "api", "connect", "retrieve page", "online", "internet"},
    "save_knowledge": {"save", "store", "remember", "knowledge", "data", "persist data", "memory", "learn"},
    "retrieve_knowledge": {"retrieve", "load", "get", "recall", "knowledge", "data", "memory", "access data"},
    "list_knowledge_keys": {"list", "show", "all", "keys", "knowledge", "data", "what data"},
    "delete_knowledge": {"delete", "remove", "forget", "knowledge", "data", "clear memory"},
    "add_task": {"add", "new", "create", "task", "todo", "schedule", "assign"},
    "get_task": {"get", "view", "show", "find", "task", "details", "check task"},
    "update_task": {"update", "modify", "change", "edit", "task", "status", "reschedule"},
    "list_tasks": {"list", "show", "all", "tasks", "view tasks", "pending", "completed", "status", "todo list"},
    "remove_task": {"remove", "delete", "complete", "finish", "task", "cancel task"},
    "suggest_tools_for_goal": {"suggest", "tool", "help", "goal", "recommend", "find tool", "which tool"} # Self-reference
}


def suggest_tools_for_goal_v2(goal_description: str, top_n: int = 5) -> dict:
    """
    (Version 2) Suggests relevant tools from the agent's toolkit to achieve a given goal.
    Suggestions are based on keyword matching between the goal and curated concept keywords for each tool.
    Also considers direct matches with tool names and original descriptions.
    """
    tool_name_internal = "suggest_tools_for_goal_v2" # Internal name for logging
    # print(f"Agent log ({tool_name_internal}): Called with goal='{goal_description[:100]}...'.") # Redundant due to user feedback

    if not goal_description:
        msg = "Goal description cannot be empty."
        # print(f"Agent log ({tool_name_internal}): {msg}")
        return {"success": False, "message": msg, "goal": goal_description, "suggestions": []}

    global _TOOL_DESCRIPTIONS, _AGENT_TOOL_CONCEPT_KEYWORDS # Ensure they are accessible
    if not _TOOL_DESCRIPTIONS: # Should always be populated if tools exist
        msg = "No tool descriptions available. Cannot make suggestions."
        # print(f"Agent log ({tool_name_internal}): {msg}")
        return {"success": False, "message": msg, "goal": goal_description, "suggestions": []}

    def tokenize(text: str) -> set[str]:
        if not text: return set()
        words = re.findall(r'\b\w+\b', text.lower())
        # Slightly adjusted stop words for goal processing
        stop_words = {
            "a", "an", "the", "is", "to", "and", "of", "it", "for", "in", "on", "with",
            "i", "me", "my", "myself", "want", "some", "new", "called", "current", "me", "all",
            "do", "can", "you", "please", "how", "what", "tell", "give" # Common conversational fluff
        }
        return set(word for word in words if word not in stop_words and len(word) > 2)

    goal_keywords = tokenize(goal_description)
    if not goal_keywords:
        msg = f"No effective keywords found in goal description after tokenization (Original: '{goal_description}')."
        # print(f"Agent log ({tool_name_internal}): {msg}")
        return {"success": True, "message": msg, "goal": goal_description, "suggestions": []}

    suggestions = []
    for tool_name_key in _TOOL_DESCRIPTIONS.keys(): # Iterate using keys from _TOOL_DESCRIPTIONS to ensure tool exists
        description = _TOOL_DESCRIPTIONS[tool_name_key]
        relevance_score = 0.0
        matching_keywords_details = {} # To store matched keywords by source for detailed reason
        
        # 1. Match with curated concept keywords (highest weight)
        tool_concepts = _AGENT_TOOL_CONCEPT_KEYWORDS.get(tool_name_key, set())
        concept_matches = goal_keywords.intersection(tool_concepts)
        if concept_matches:
            score_increment = len(concept_matches) * 3.0
            relevance_score += score_increment
            matching_keywords_details["concept"] = list(concept_matches)
            
        # 2. Match with keywords from the tool's actual name (medium weight)
        tokenized_tool_name = tokenize(tool_name_key.replace("_", " "))
        name_matches = goal_keywords.intersection(tokenized_tool_name)
        if name_matches:
            score_increment = len(name_matches) * 2.0
            relevance_score += score_increment
            matching_keywords_details["name"] = list(name_matches)
        
        # 3. Match with keywords from the tool's original description (lower weight)
        tokenized_description = tokenize(description)
        desc_matches = goal_keywords.intersection(tokenized_description)
        if desc_matches:
            # Only add score for description keywords not already captured by name or (more specific) concepts
            # This avoids over-inflating score if description just repeats name/concept keywords.
            unique_desc_matches = desc_matches - name_matches - concept_matches
            score_increment = len(unique_desc_matches) * 1.0
            relevance_score += score_increment
            if unique_desc_matches : matching_keywords_details["description"] = list(unique_desc_matches)

        if relevance_score > 0:
            reason_parts = []
            all_matched_keywords_for_suggestion = set()
            if "concept" in matching_keywords_details:
                reason_parts.append(f"Concept ({', '.join(matching_keywords_details['concept'])})")
                all_matched_keywords_for_suggestion.update(matching_keywords_details['concept'])
            if "name" in matching_keywords_details:
                reason_parts.append(f"Name ({', '.join(matching_keywords_details['name'])})")
                all_matched_keywords_for_suggestion.update(matching_keywords_details['name'])
            if "description" in matching_keywords_details:
                reason_parts.append(f"Desc ({', '.join(matching_keywords_details['description'])})")
                all_matched_keywords_for_suggestion.update(matching_keywords_details['description'])
            
            final_reason = "; ".join(reason_parts) if reason_parts else "General keyword match."
            
            suggestions.append({
                "tool_name": tool_name_key,
                "description": description,
                "relevance_score": round(relevance_score, 2),
                "matching_keywords": sorted(list(all_matched_keywords_for_suggestion)),
                "reason": final_reason
            })

    suggestions.sort(key=lambda x: x['relevance_score'], reverse=True)
    final_suggestions = suggestions[:top_n]
    
    msg = f"Found {len(final_suggestions)} relevant tool suggestion(s) for the goal (out of {len(suggestions)} potential)."
    if not suggestions:
        msg = "No relevant tool suggestions found based on keyword matching."
        
    # print(f"Agent log ({tool_name_internal}): {msg} Goal keywords: {goal_keywords}") # Redundant
    return {
        "success": True, 
        "message": msg, 
        "goal": goal_description, 
        "suggestions": final_suggestions
    }

# --- Register New Tool (overwrite previous one) ---
print("\nAgent: Registering improved tool suggestion tool (v2)...")
if 'add_tool' in globals() and callable(add_tool):
    add_tool_result = add_tool(
        name="suggest_tools_for_goal", 
        tool_callable=suggest_tools_for_goal_v2,
        description="(Version 2) Suggests relevant tools to achieve a goal based on keyword matching with curated concepts, tool names, and descriptions."
    )
    if add_tool_result:
        print("Tool 'suggest_tools_for_goal' (v2) registered successfully.")
        # Ensure _TOOL_DESCRIPTIONS is updated if add_tool overwrites it. It should.
        _TOOL_DESCRIPTIONS["suggest_tools_for_goal"] = "(Version 2) Suggests relevant tools to achieve a goal based on keyword matching with curated concepts, tool names, and descriptions."

    else:
        print("Error: Failed to register 'suggest_tools_for_goal' (v2).")
else:
    print("CRITICAL ERROR: 'add_tool' function not found. Cannot register 'suggest_tools_for_goal' (v2).")

# --- Verify Registration ---
print("\nAgent: Verifying tool registration (v2):")
# Access list_tools through _AGENT_TOOLS to ensure it's the registered one.
if 'list_tools' in _AGENT_TOOLS and callable(_AGENT_TOOLS['list_tools']):
    _AGENT_TOOLS['list_tools']() 
else:
    print("CRITICAL WARNING: 'list_tools' tool is missing or not callable.")

# --- Test Tool Suggestion Tool (v2) ---
print("\nAgent: Starting tests for improved tool suggestion tool (v2).")

# Check if the new version is actually in use
tool_to_test = _AGENT_TOOLS.get('suggest_tools_for_goal')
if tool_to_test is None:
    print("Agent: CRITICAL - 'suggest_tools_for_goal' not found in _AGENT_TOOLS. Skipping tests.")
elif tool_to_test != suggest_tools_for_goal_v2:
    print("Agent: CRITICAL - 'suggest_tools_for_goal' in _AGENT_TOOLS is not V2. Skipping tests.")
    print(f"  _AGENT_TOOLS['suggest_tools_for_goal'] is {tool_to_test.__name__ if hasattr(tool_to_test, '__name__') else 'Unknown'}")
    print(f"  suggest_tools_for_goal_v2 is {suggest_tools_for_goal_v2.__name__}")

else:
    print("Agent: 'suggest_tools_for_goal' (v2) confirmed in _AGENT_TOOLS. Proceeding with tests.")
    # Test 1: Goal related to file writing
    goal1 = "I want to write some text into a new document called 'my_notes.txt'."
    print(f"\n--- Test 1 (v2): Goal: '{goal1}' ---")
    suggestions1 = tool_to_test(goal_description=goal1, top_n=3)
    print(f"Suggestions for goal 1: {json.dumps(suggestions1, indent=2)}")
    if suggestions1['success'] and suggestions1['suggestions'] and suggestions1['suggestions'][0]['tool_name'] == 'write_file':
        print("  SUCCESS (v2): 'write_file' is top suggestion.")
    else:
        print("  FAILURE (v2): 'write_file' not top suggestion for file writing goal.")

    # Test 2: Goal related to web content and saving (key test)
    goal2 = "Download the main page of example.com and store it as example_page.html"
    print(f"\n--- Test 2 (v2): Goal: '{goal2}' ---")
    suggestions2 = tool_to_test(goal_description=goal2, top_n=3)
    print(f"Suggestions for goal 2: {json.dumps(suggestions2, indent=2)}")
    if suggestions2['success'] and len(suggestions2['suggestions']) >= 2:
        tool_names_suggested = [s['tool_name'] for s in suggestions2['suggestions']]
        if 'make_http_request' in tool_names_suggested and 'write_file' in tool_names_suggested:
            print("  SUCCESS (v2): 'make_http_request' and 'write_file' are among top suggestions.")
            # Check if they are the top 2, order might vary based on score ties
            if suggestions2['suggestions'][0]['tool_name'] in ('make_http_request', 'write_file') and \
               suggestions2['suggestions'][1]['tool_name'] in ('make_http_request', 'write_file') and \
               suggestions2['suggestions'][0]['tool_name'] != suggestions2['suggestions'][1]['tool_name']:
                 print("  IDEAL (v2): 'make_http_request' and 'write_file' are the top 2.")
        else:
            print("  FAILURE (v2): 'make_http_request' or 'write_file' missing from top suggestions for web download and save.")
    elif suggestions2['success'] and suggestions2['suggestions']:
         print(f"  PARTIAL (v2): Suggestions found ({len(suggestions2['suggestions'])}), but not enough for full check, or wrong tools. Top: {suggestions2['suggestions'][0]['tool_name']}")
    else:
        print(f"  FAILURE (v2): Tool suggestion call failed or no suggestions: {suggestions2.get('message')}")

    # Test 3: Goal related to listing tasks
    goal3 = "Show me all my current pending tasks."
    print(f"\n--- Test 3 (v2): Goal: '{goal3}' ---")
    suggestions3 = tool_to_test(goal_description=goal3, top_n=3)
    print(f"Suggestions for goal 3: {json.dumps(suggestions3, indent=2)}")
    if suggestions3['success'] and suggestions3['suggestions'] and suggestions3['suggestions'][0]['tool_name'] == 'list_tasks':
        print("  SUCCESS (v2): 'list_tasks' is top suggestion.")
    else:
        print("  FAILURE (v2): 'list_tasks' not top suggestion for task listing goal.")
        
    # Test 4: Vague goal
    goal4 = "Do something useful."
    print(f"\n--- Test 4 (v2): Goal: '{goal4}' ---")
    suggestions4 = tool_to_test(goal_description=goal4, top_n=3)
    print(f"Suggestions for goal 4: {json.dumps(suggestions4, indent=2)}")
    if suggestions4['success']:
        print("  INFO (v2): Vague goal processed.")
    else:
        print(f"  FAILURE (v2): Tool suggestion call failed for vague goal: {suggestions4.get('message')}")

print("\nAgent: Improved tool suggestion mechanism (v2) definition, registration, and tests complete.")
print("Waiting for next instruction.")

Agent: Defining and registering an improved tool suggestion mechanism (v2).

Agent: Registering improved tool suggestion tool (v2)...
Tool 'suggest_tools_for_goal' added successfully. Description: (Version 2) Suggests relevant tools to achieve a goal based on keyword matching with curated concepts, tool names, and descriptions.
Tool 'suggest_tools_for_goal' (v2) registered successfully.

Agent: Verifying tool registration (v2):

Available tools:
1. list_tools: Lists all available tools with their names and descriptions. Useful for understanding agent capabilities.
2. get_agent_status: Provides a status report of the agent, including version and number of loaded tools.
3. execute_python_code: Executes a given string of Python code in the agent's current global environment. Captures and returns stdout, stderr, and any execution error from the provided code. USE WITH EXTREME CAUTION, as it can alter the agent's state and capabilities.
4. write_file: (Version 2) Writes text content to a sp

In [12]:
# _AGENT_TOOLS, _TOOL_DESCRIPTIONS, add_tool, suggest_tools_for_goal_v2, etc. are assumed to be in the global scope.
import inspect
import re
import json # For printing test results neatly

print("Agent: Defining and registering a basic autonomous goal execution tool (v1).")

# --- Define Autonomous Goal Execution Tool V1 ---

_MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION = 2.5 # Min score from suggest_tools_for_goal

def _extract_parameter_value_heuristically(param_name: str, param_type, goal_description: str, existing_args: dict):
    """
    Very basic heuristic parameter extraction.
    This is the part that would need significant improvement for robust autonomy.
    """
    # print(f"Agent log (_extract_parameter_value_heuristically): Attempting to extract '{param_name}' ({param_type}).")
    goal_lower = goal_description.lower()

    # 1. Quoted strings (greedy match for longer content)
    # Matches '...' or "..."
    quoted_match = re.findall(r"\"([^\"]+)\"|\'([^\']+)\'", goal_description) # findall returns list of tuples
    
    # Flatten list of tuples and filter out empty strings from non-matching groups
    # Example: [('', "single quotes"), ('double quotes', '')] -> ["single quotes", "double quotes"]
    all_quoted_strings = [item for tpl in quoted_match for item in tpl if item]


    if param_name in ["content", "text", "description", "message"] and all_quoted_strings:
        # print(f"Agent log (_extract_parameter_value_heuristically): Found quoted strings for '{param_name}': {all_quoted_strings}")
        # Prefer longer quoted strings for content-like parameters
        return max(all_quoted_strings, key=len)

    if param_name in ["filepath", "path", "directory"] and all_quoted_strings:
        # For filepaths, prefer those that look like paths or have extensions. This is still very basic.
        # Try to find a quoted string that looks like a file or path.
        for q_str in all_quoted_strings:
            if '.' in q_str or '/' in q_str or '\\' in q_str: # Simple check
                 # Avoid re-using a string if another param already claimed it, unless it's identical
                if q_str not in existing_args.values() or existing_args.get(param_name) == q_str:
                    return q_str
        # If no path-like quoted string, but some quoted string exists and param is filepath/path
        if all_quoted_strings and (all_quoted_strings[0] not in existing_args.values() or existing_args.get(param_name) == all_quoted_strings[0]):
            return all_quoted_strings[0] # Take the first one as a guess

    if param_name == "url":
        url_match = re.search(r"https?://(?:[a-zA-Z0-9$-_@.&+]|[!*(),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", goal_description)
        if url_match:
            return url_match.group(0)

    if param_name == "code_string":
        # Look for content after "code:" or within triple backticks (non-greedy)
        code_block_match = re.search(r"code:\s*(.+)", goal_description, re.IGNORECASE | re.DOTALL)
        if code_block_match:
            return code_block_match.group(1).strip()
        
        triple_tick_match = re.search(r"```(?:python\n)?(.*?)```", goal_description, re.DOTALL | re.IGNORECASE)
        if triple_tick_match:
            return triple_tick_match.group(1).strip()
        
    if param_name == "method" and param_type == str: # For HTTP requests
        if "post" in goal_lower: return "POST"
        if "get" in goal_lower: return "GET" # Default is often GET anyway via tool
        if "put" in goal_lower: return "PUT"
        if "delete" in goal_lower: return "DELETE"
    
    if param_type == bool: # For boolean flags like 'recursive'
        if param_name in goal_lower or f"{param_name}=true" in goal_lower or f"{param_name} is true" in goal_lower:
            return True
        if f"{param_name}=false" in goal_lower or f"{param_name} is false" in goal_lower:
            return False
        # Simple check for words implying recursion
        if param_name == "recursive" and "recursively" in goal_lower:
            return True

    # Fallback: Try to find param_name = 'value' or param_name: 'value'
    # This requires the user to be somewhat explicit.
    explicit_assign_match = re.search(rf"{param_name}\s*[:=]\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
    if explicit_assign_match:
        return explicit_assign_match.group(1)
    
    explicit_assign_match_unquoted = re.search(rf"{param_name}\s*[:=]\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
    if explicit_assign_match_unquoted: # For simple values not in quotes (e.g. numbers if type allows, or simple strings)
        val_str = explicit_assign_match_unquoted.group(1)
        if param_type == int:
            try: return int(val_str)
            except ValueError: pass
        if param_type == float:
            try: return float(val_str)
            except ValueError: pass
        # If string or Any, return the string
        return val_str

    return None # Cannot extract

def attempt_goal_autonomously_v1_tool(goal_description: str) -> dict:
    """
    (Version 1) Attempts to achieve a goal by selecting a tool,
    heuristically filling its parameters, and executing it.
    Parameter filling is very basic and pattern-based.
    """
    tool_name_internal = "attempt_goal_autonomously_v1_tool"
    log_prefix = f"Agent log ({tool_name_internal}): "
    print(f"{log_prefix}Attempting goal: '{goal_description[:100]}...'")

    # 1. Suggest tools
    if 'suggest_tools_for_goal' not in _AGENT_TOOLS:
        return {"success": False, "message": "Tool 'suggest_tools_for_goal' not found.", "details": None}
    
    suggestion_result = _AGENT_TOOLS['suggest_tools_for_goal'](goal_description=goal_description, top_n=1)
    print(f"{log_prefix}Tool suggestion result: {suggestion_result['message']}")

    if not suggestion_result['success'] or not suggestion_result['suggestions']:
        msg = "No suitable tools suggested for the goal."
        print(f"{log_prefix}{msg}")
        return {"success": False, "message": msg, "chosen_tool": None, "parameters": None, "execution_result": None}

    top_suggestion = suggestion_result['suggestions'][0]
    chosen_tool_name = top_suggestion['tool_name']
    relevance_score = top_suggestion['relevance_score']

    print(f"{log_prefix}Top suggested tool: '{chosen_tool_name}' (Score: {relevance_score}). Reason: {top_suggestion.get('reason', 'N/A')}")

    if relevance_score < _MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION:
        msg = f"Top suggested tool '{chosen_tool_name}' score ({relevance_score}) is below threshold ({_MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION}). Not proceeding with execution."
        print(f"{log_prefix}{msg}")
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, "parameters": None, "execution_result": None}

    if chosen_tool_name not in _AGENT_TOOLS:
        msg = f"Suggested tool '{chosen_tool_name}' not found in agent's toolkit."
        print(f"{log_prefix}{msg}")
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, "parameters": None, "execution_result": None}
    
    tool_callable = _AGENT_TOOLS[chosen_tool_name]

    # 2. Inspect tool parameters
    try:
        sig = inspect.signature(tool_callable)
    except Exception as e:
        msg = f"Could not inspect signature of tool '{chosen_tool_name}': {e}"
        print(f"{log_prefix}{msg}")
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, "parameters": None, "execution_result": None}

    filled_params = {}
    missing_required_params = []
    print(f"{log_prefix}Attempting to fill parameters for '{chosen_tool_name}': {list(sig.parameters.keys())}")

    for param_name, param_obj in sig.parameters.items():
        param_type = param_obj.annotation if param_obj.annotation != inspect.Parameter.empty else None
        
        # Try to extract value
        # Pass `filled_params` so heuristics can avoid re-using already extracted values if needed.
        extracted_value = _extract_parameter_value_heuristically(param_name, param_type, goal_description, filled_params)

        if extracted_value is not None:
            # Attempt type conversion if annotation is present and value is string
            if param_type and isinstance(extracted_value, str):
                try:
                    if param_type == bool: # "true"/"false" (case-insensitive) or presence of keyword
                        if extracted_value.lower() == 'true': extracted_value = True
                        elif extracted_value.lower() == 'false': extracted_value = False
                        # else keep as string if not clearly bool, might be handled by tool
                    elif param_type == int: extracted_value = int(extracted_value)
                    elif param_type == float: extracted_value = float(extracted_value)
                    # Note: no complex type conversion (e.g., dict, list from string)
                except ValueError:
                    print(f"{log_prefix}Warning: Could not convert extracted value '{extracted_value}' to type {param_type} for param '{param_name}'. Using as string.")
            
            filled_params[param_name] = extracted_value
            print(f"{log_prefix}  Filled '{param_name}': {str(extracted_value)[:50]}{'...' if len(str(extracted_value)) > 50 else ''} (Type: {type(extracted_value).__name__})")
        elif param_obj.default != inspect.Parameter.empty:
            filled_params[param_name] = param_obj.default # Use default if available
            print(f"{log_prefix}  Using default for '{param_name}': {str(param_obj.default)[:50]}{'...' if len(str(param_obj.default)) > 50 else ''}")
        else:
            missing_required_params.append(param_name)
            print(f"{log_prefix}  Could not fill required parameter '{param_name}'.")
            
    if missing_required_params:
        msg = f"Failed to execute goal: Missing required parameters for tool '{chosen_tool_name}': {', '.join(missing_required_params)}."
        print(f"{log_prefix}{msg}")
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, "parameters": filled_params, "execution_result": None}

    # 3. Execute the tool
    print(f"{log_prefix}All required parameters filled. Executing '{chosen_tool_name}' with: {json.dumps(filled_params, default=str, indent=2)}")
    try:
        execution_result = tool_callable(**filled_params)
        print(f"{log_prefix}Execution of '{chosen_tool_name}' successful.")
        # For brevity, just show success/failure and a snippet of result message or content
        result_summary = {"success_status": execution_result.get('success', 'N/A')}
        if 'message' in execution_result: result_summary['message'] = str(execution_result['message'])[:100]
        if 'content' in execution_result and execution_result['content'] is not None: result_summary['content_snippet'] = str(execution_result['content'])[:100]
        
        print(f"{log_prefix}Result summary: {result_summary}")

        return {
            "success": True, # Overall success of this autonomous attempt if tool execution reported success
            "message": f"Goal attempted. Tool '{chosen_tool_name}' executed.",
            "chosen_tool": chosen_tool_name,
            "parameters_used": filled_params,
            "execution_result": execution_result # Full result from the tool
        }
    except Exception as e:
        error_msg = f"Error during execution of tool '{chosen_tool_name}': {type(e).__name__}: {e}"
        tb_str = traceback.format_exc() # Make sure traceback is imported if used more broadly
        detailed_error = f"{error_msg}\nTraceback:\n{tb_str}"
        print(f"{log_prefix}{detailed_error}")
        return {"success": False, "message": error_msg, "chosen_tool": chosen_tool_name, "parameters": filled_params, "execution_result": {"error": detailed_error}}


# --- Register New Tool ---
print("\nAgent: Registering autonomous goal execution tool (v1)...")
if 'add_tool' in globals() and callable(add_tool):
    add_tool(
        name="attempt_goal_autonomously_v1",
        tool_callable=attempt_goal_autonomously_v1_tool,
        description="(Version 1) Attempts to achieve a goal by selecting a tool, heuristically filling its parameters, and executing it."
    )
else:
    print("CRITICAL ERROR: 'add_tool' function not found.")

# --- Verify Registration ---
print("\nAgent: Verifying tool registration:")
if 'list_tools' in _AGENT_TOOLS and callable(_AGENT_TOOLS['list_tools']):
    _AGENT_TOOLS['list_tools']()

# --- Test Autonomous Goal Execution Tool ---
print("\nAgent: Starting tests for autonomous goal execution tool (v1).")

if 'attempt_goal_autonomously_v1' not in _AGENT_TOOLS:
    print("Agent: Skipping tests as 'attempt_goal_autonomously_v1' tool was not registered.")
else:
    # Test 1: Goal that should map to write_file and succeed
    goal1 = "Please write the text 'Hello autonomously!' to a file named 'autonomous_test_output.txt'."
    print(f"\n--- Test 1: Goal: '{goal1}' ---")
    result1 = _AGENT_TOOLS['attempt_goal_autonomously_v1'](goal_description=goal1)
    print(f"Result for goal 1: {json.dumps(result1, default=str, indent=2)}")
    if result1.get('success') and result1.get('chosen_tool') == 'write_file' and result1.get('execution_result', {}).get('success'):
        print("  SUCCESS (Test 1): Goal likely achieved with write_file.")
        # Cleanup: use delete_file tool
        if 'delete_file' in _AGENT_TOOLS and result1.get('parameters_used', {}).get('filepath'):
            _AGENT_TOOLS['delete_file'](filepath=result1['parameters_used']['filepath'])
            print(f"  Cleanup: Deleted '{result1['parameters_used']['filepath']}'.")
    else:
        print(f"  FAILURE (Test 1): Goal not achieved as expected. Message: {result1.get('message')}")

    # Test 2: Goal that should map to make_http_request and succeed
    goal2 = "Download the content from 'https://jsonplaceholder.typicode.com/todos/1' using GET method."
    print(f"\n--- Test 2: Goal: '{goal2}' ---")
    result2 = _AGENT_TOOLS['attempt_goal_autonomously_v1'](goal_description=goal2)
    print(f"Result for goal 2: {json.dumps(result2, default=str, indent=2)}")
    if result2.get('success') and result2.get('chosen_tool') == 'make_http_request' and result2.get('execution_result', {}).get('success'):
        print("  SUCCESS (Test 2): Goal likely achieved with make_http_request.")
    else:
        print(f"  FAILURE (Test 2): Goal not achieved as expected. Message: {result2.get('message')}")

    # Test 3: Goal where parameter extraction is likely to fail (missing essential info)
    goal3 = "Read a file." # No filename specified
    print(f"\n--- Test 3: Goal: '{goal3}' ---")
    result3 = _AGENT_TOOLS['attempt_goal_autonomously_v1'](goal_description=goal3)
    print(f"Result for goal 3: {json.dumps(result3, default=str, indent=2)}")
    if not result3.get('success') and result3.get('chosen_tool') == 'read_file' and "Missing required parameters" in result3.get('message', ''):
        print("  SUCCESS (Test 3): Correctly failed due to missing 'filepath' parameter for read_file.")
    else:
        print(f"  FAILURE (Test 3): Did not fail as expected for missing parameters. Message: {result3.get('message')}")

    # Test 4: Goal where no tool meets relevance threshold
    goal4 = "Contemplate the sound of one hand clapping." # Unlikely to match tools well
    print(f"\n--- Test 4: Goal: '{goal4}' ---")
    # Temporarily lower threshold for suggest_tools_for_goal to ensure it *could* find something,
    # then test that attempt_goal_autonomously_v1 respects its *own* threshold.
    # This requires modifying _MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION.
    # Or, ensure goal4 truly has low scores from suggest_tools_for_goal.
    # Current `suggest_tools_for_goal` is fairly simple, so this goal should have low scores.
    result4 = _AGENT_TOOLS['attempt_goal_autonomously_v1'](goal_description=goal4)
    print(f"Result for goal 4: {json.dumps(result4, default=str, indent=2)}")
    if not result4.get('success') and "score" in result4.get('message', '').lower() and "below threshold" in result4.get('message', '').lower():
        print("  SUCCESS (Test 4): Correctly decided not to execute due to low relevance score.")
    elif not result4.get('success') and "No suitable tools suggested" in result4.get('message', ''):
         print("  SUCCESS (Test 4): Correctly found no tools or suggestions were empty.")
    else:
        print(f"  FAILURE (Test 4): Did not behave as expected for low relevance. Message: {result4.get('message')}")
        
    # Test 5: Goal for execute_python_code
    goal5 = "Run this python code: print('Autonomous code execution test!')"
    print(f"\n--- Test 5: Goal: '{goal5}' ---")
    result5 = _AGENT_TOOLS['attempt_goal_autonomously_v1'](goal_description=goal5)
    print(f"Result for goal 5: {json.dumps(result5, default=str, indent=2)}")
    if result5.get('success') and result5.get('chosen_tool') == 'execute_python_code':
        exec_res = result5.get('execution_result', {})
        if exec_res.get('success_status') is None: # execute_python_code itself doesn't return 'success' key
             # Check stdout for expected output from execute_python_code's result dict
            if exec_res.get("stdout", "").strip() == "Autonomous code execution test!":
                 print("  SUCCESS (Test 5): Python code executed successfully via autonomous goal.")
            else:
                 print(f"  FAILURE (Test 5): Python code execution had unexpected stdout: {exec_res.get('stdout')}")
        else:
            print(f"  FAILURE (Test 5): execute_python_code success status unexpected: {exec_res.get('success_status')}")
    else:
        print(f"  FAILURE (Test 5): Goal for execute_python_code not achieved as expected. Message: {result5.get('message')}")


print("\nAgent: Autonomous goal execution tool (v1) definition, registration, and tests complete.")
print("Waiting for next instruction.")

Agent: Defining and registering a basic autonomous goal execution tool (v1).

Agent: Registering autonomous goal execution tool (v1)...
Tool 'attempt_goal_autonomously_v1' added successfully. Description: (Version 1) Attempts to achieve a goal by selecting a tool, heuristically filling its parameters, and executing it.

Agent: Verifying tool registration:

Available tools:
1. list_tools: Lists all available tools with their names and descriptions. Useful for understanding agent capabilities.
2. get_agent_status: Provides a status report of the agent, including version and number of loaded tools.
3. execute_python_code: Executes a given string of Python code in the agent's current global environment. Captures and returns stdout, stderr, and any execution error from the provided code. USE WITH EXTREME CAUTION, as it can alter the agent's state and capabilities.
4. write_file: (Version 2) Writes text content to a specified file. Overwrites if exists, creates directories if needed.
5. read

In [13]:
# _AGENT_TOOLS, _TOOL_DESCRIPTIONS, add_tool, suggest_tools_for_goal_v2, etc. are assumed to be in the global scope.
import inspect
import re
import json # For printing test results neatly
import traceback # For detailed error messages

print("Agent: Defining and registering autonomous goal execution tool (v2 with improved heuristics).")

# --- Define Autonomous Goal Execution Tool V2 ---

_MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION_V2 = 2.5 # Same threshold

def _extract_parameter_value_heuristically_v2(
    param_name: str, 
    param_type, 
    goal_description: str, 
    all_quoted_strings_from_goal: list[str], # Pass pre-extracted quoted strings
    consumed_quoted_strings: set[str] # Keep track of quoted strings already used
) -> any:
    """
    (Version 2) Heuristic parameter extraction with improved disambiguation.
    `consumed_quoted_strings` is modified by this function if a quoted string is used.
    """
    # print(f"Agent log (_extract_v2): Param='{param_name}', Type='{param_type}', Consumed so far: {consumed_quoted_strings}")
    goal_lower = goal_description.lower()

    # 1. URL Extraction (and cleaning)
    if param_name == "url":
        url_match_re = r"https?://(?:[a-zA-Z0-9.\-_~:/?#\[\]@!$&'()*+,;=%]+[a-zA-Z0-9\-_~:/?#\[\]@!$&*+,;=%])" # More robust, includes common chars
        # Simpler alternative if above is too complex or greedy:
        # url_match_re = r"https?://[^\s'\"]+"

        url_match = re.search(url_match_re, goal_description)
        if url_match:
            extracted_url = url_match.group(0)
            # Clean trailing quotes if they are not part of the URL itself
            # (e.g., if goal was "download from 'http://example.com/' then ...")
            if extracted_url.endswith("'") and goal_description.count(f"'{extracted_url}'") > 0:
                extracted_url = extracted_url[:-1]
            elif extracted_url.endswith("\"") and goal_description.count(f"\"{extracted_url}\"") > 0:
                extracted_url = extracted_url[:-1]
            
            # Check if this cleaned URL corresponds to one of the `all_quoted_strings_from_goal`
            # If so, mark that original quoted string as consumed.
            for q_str in all_quoted_strings_from_goal:
                if extracted_url == q_str or f"'{extracted_url}'" == q_str or f"\"{extracted_url}\"" == q_str :
                    if q_str not in consumed_quoted_strings:
                        consumed_quoted_strings.add(q_str)
                        # print(f"Agent log (_extract_v2): Consumed quoted string '{q_str}' for URL.")
                        break 
            return extracted_url

    # 2. Filepath / Path / Directory from Quoted Strings
    if param_name in ["filepath", "path", "directory"]:
        path_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue # Already used by another param (e.g. URL that was quoted)
            # Heuristic: contains common path characters or known extensions
            if '.' in q_str or '/' in q_str or '\\' in q_str or \
               any(q_str.endswith(ext) for ext in ['.txt', '.json', '.py', '.html', '.md']):
                path_candidates.append(q_str)
        
        if path_candidates:
            chosen_path = max(path_candidates, key=len) # Prefer longer, potentially more complete paths
            consumed_quoted_strings.add(chosen_path)
            # print(f"Agent log (_extract_v2): Consumed quoted string '{chosen_path}' for filepath/path.")
            return chosen_path

    # 3. Content / Text / Description / Message from Quoted Strings
    if param_name in ["content", "text", "description", "message"]:
        content_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue # Already used
            content_candidates.append(q_str)
        
        if content_candidates:
            # Prefer longer quoted strings for content-like parameters
            chosen_content = max(content_candidates, key=len)
            consumed_quoted_strings.add(chosen_content)
            # print(f"Agent log (_extract_v2): Consumed quoted string '{chosen_content}' for content.")
            return chosen_content
            
    # 4. Code String
    if param_name == "code_string":
        code_block_match = re.search(r"code:\s*(.+)", goal_description, re.IGNORECASE | re.DOTALL)
        if code_block_match: return code_block_match.group(1).strip()
        triple_tick_match = re.search(r"```(?:python\n)?(.*?)```", goal_description, re.DOTALL | re.IGNORECASE)
        if triple_tick_match: return triple_tick_match.group(1).strip()

    # 5. HTTP Method
    if param_name == "method" and (param_type == str or param_type is None): # type might be inspect._empty
        method_search = re.search(r"\b(GET|POST|PUT|DELETE|PATCH|HEAD|OPTIONS)\b", goal_description, re.IGNORECASE)
        if method_search:
            return method_search.group(1).upper()
        # Fallback to keyword check if specific verb not found
        if "post" in goal_lower: return "POST"
        if "get" in goal_lower: return "GET"
        if "put" in goal_lower: return "PUT"
        if "delete" in goal_lower: return "DELETE"

    # 6. Boolean Flags
    if param_type == bool:
        # Pattern: "param_name recursively", "recursive true", "is recursive"
        if re.search(rf"\b{param_name}\b\s*(true|yes|on|recursively|enabled)\b", goal_lower, re.IGNORECASE) or \
           re.search(rf"\b(is|be|make it)\s*{param_name}\b", goal_lower, re.IGNORECASE):
            return True
        if re.search(rf"\b{param_name}\b\s*(false|no|off|disabled)\b", goal_lower, re.IGNORECASE) or \
           re.search(rf"\b(is not|not to be|don't make it)\s*{param_name}\b", goal_lower, re.IGNORECASE):
            return False
        # If just the param name is mentioned, might imply true for flags often used positively
        if param_name in goal_lower and param_name not in ["false", "no", "off"]: # Avoid "recursive false" matching "recursive"
            return True

    # 7. Explicit Assignment (param_name = 'value' or param_name: value)
    # This is a fallback and should be used carefully as it's very literal.
    # Quoted value:
    explicit_assign_match_quoted = re.search(rf"\b{param_name}\s*[:=]\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
    if explicit_assign_match_quoted:
        val_str = explicit_assign_match_quoted.group(1)
        if val_str not in consumed_quoted_strings: # Only if this exact quote wasn't consumed
            return val_str
    # Unquoted value (for numbers, booleans, or simple non-spaced strings):
    explicit_assign_match_unquoted = re.search(rf"\b{param_name}\s*[:=]\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
    if explicit_assign_match_unquoted:
        val_str = explicit_assign_match_unquoted.group(1)
        # Attempt type conversion for primitive types
        if param_type == int: try: return int(val_str) catch ValueError: pass
        if param_type == float: try: return float(val_str) catch ValueError: pass
        if param_type == bool:
            if val_str.lower() == 'true': return True
            if val_str.lower() == 'false': return False
        # If string, or if type conversion failed but it's a string param, return as string
        if param_type == str or param_type is None: return val_str

    return None # Cannot extract

def attempt_goal_autonomously_v2_tool(goal_description: str) -> dict:
    """
    (Version 2) Attempts to achieve a goal by selecting a tool,
    heuristically filling its parameters with improved logic, and executing it.
    """
    tool_name_internal = "attempt_goal_autonomously_v2_tool"
    log_prefix = f"Agent log ({tool_name_internal}): "
    print(f"{log_prefix}Attempting goal (v2): '{goal_description[:100]}...'")

    # Pre-extract all quoted strings once
    quoted_match_tuples = re.findall(r"\"([^\"]+)\"|\'([^\']+)\'", goal_description)
    all_quoted_strings_from_goal = [item for tpl in quoted_match_tuples for item in tpl if item]
    
    # --- Tool Selection (same as v1) ---
    if 'suggest_tools_for_goal' not in _AGENT_TOOLS:
        return {"success": False, "message": "Tool 'suggest_tools_for_goal' not found.", "details": None}
    suggestion_result = _AGENT_TOOLS['suggest_tools_for_goal'](goal_description=goal_description, top_n=1)
    # ... (rest of tool selection logic is identical to v1) ...
    if not suggestion_result['success'] or not suggestion_result['suggestions']:
        msg = "No suitable tools suggested for the goal."
        print(f"{log_prefix}{msg}")
        return {"success": False, "message": msg, "chosen_tool": None, "parameters": None, "execution_result": None}
    top_suggestion = suggestion_result['suggestions'][0]
    chosen_tool_name = top_suggestion['tool_name']
    relevance_score = top_suggestion['relevance_score']
    print(f"{log_prefix}Tool suggestion result: {suggestion_result['message']}")
    print(f"{log_prefix}Top suggested tool: '{chosen_tool_name}' (Score: {relevance_score}). Reason: {top_suggestion.get('reason', 'N/A')}")
    if relevance_score < _MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION_V2:
        msg = f"Top suggested tool '{chosen_tool_name}' score ({relevance_score}) is below threshold ({_MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION_V2}). Not proceeding."
        print(f"{log_prefix}{msg}")
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, "parameters": None, "execution_result": None}
    if chosen_tool_name not in _AGENT_TOOLS:
        msg = f"Suggested tool '{chosen_tool_name}' not found in agent's toolkit."
        print(f"{log_prefix}{msg}")
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, "parameters": None, "execution_result": None}
    tool_callable = _AGENT_TOOLS[chosen_tool_name]

    # --- Parameter Filling (using v2 heuristic) ---
    try:
        sig = inspect.signature(tool_callable)
    except Exception as e: # Should not happen for well-defined tools
        msg = f"Could not inspect signature of tool '{chosen_tool_name}': {e}"
        print(f"{log_prefix}{msg}")
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, "parameters": None, "execution_result": None}

    filled_params = {}
    missing_required_params = []
    consumed_quoted_strings_for_this_call = set() # Track consumed quotes per call
    print(f"{log_prefix}Attempting to fill parameters for '{chosen_tool_name}': {list(sig.parameters.keys())}")
    
    # Prioritize parameters that are more likely to consume specific quote patterns (like filepath, url)
    # This is a simple sort order, could be more sophisticated.
    param_order = sorted(sig.parameters.items(), key=lambda item: 1 if item[0] in ["filepath", "path", "url"] else 2)

    for param_name, param_obj in param_order:
        param_type_annotation = param_obj.annotation if param_obj.annotation != inspect.Parameter.empty else None
        
        extracted_value = _extract_parameter_value_heuristically_v2(
            param_name, param_type_annotation, goal_description, 
            all_quoted_strings_from_goal, consumed_quoted_strings_for_this_call
        )

        if extracted_value is not None:
            # Type conversion (basic)
            if param_type_annotation and isinstance(extracted_value, str):
                try:
                    if param_type_annotation == bool:
                        if extracted_value.lower() == 'true': extracted_value = True
                        elif extracted_value.lower() == 'false': extracted_value = False
                    elif param_type_annotation == int: extracted_value = int(extracted_value)
                    elif param_type_annotation == float: extracted_value = float(extracted_value)
                except ValueError:
                    print(f"{log_prefix}Warning: Could not convert extracted value '{extracted_value}' to type {param_type_annotation} for param '{param_name}'. Using as string.")
            
            filled_params[param_name] = extracted_value
            print(f"{log_prefix}  Filled '{param_name}': {str(extracted_value)[:50]}{'...' if len(str(extracted_value)) > 50 else ''} (Type: {type(extracted_value).__name__})")
        elif param_obj.default != inspect.Parameter.empty:
            filled_params[param_name] = param_obj.default
            print(f"{log_prefix}  Using default for '{param_name}': {str(param_obj.default)[:50]}{'...' if len(str(param_obj.default)) > 50 else ''}")
        else: # No value extracted, no default -> required param is missing
            missing_required_params.append(param_name)
            print(f"{log_prefix}  Could not fill required parameter '{param_name}'.")
            
    if missing_required_params:
        msg = f"Failed to execute goal: Missing required parameters for tool '{chosen_tool_name}': {', '.join(missing_required_params)}."
        print(f"{log_prefix}{msg}")
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, "parameters_tried": filled_params, "execution_result": None}

    # --- Tool Execution (same as v1) ---
    print(f"{log_prefix}All required parameters filled. Executing '{chosen_tool_name}' with: {json.dumps(filled_params, default=str, indent=2)}")
    try:
        execution_result = tool_callable(**filled_params)
        # ... (rest of execution and result reporting logic is identical to v1) ...
        print(f"{log_prefix}Execution of '{chosen_tool_name}' appears to have completed.")
        result_summary = {"tool_reported_success": execution_result.get('success', 'N/A')} # Tool's own success field
        if 'message' in execution_result: result_summary['tool_message'] = str(execution_result['message'])[:100]
        if 'content' in execution_result and execution_result['content'] is not None: result_summary['content_snippet'] = str(execution_result['content'])[:100]
        print(f"{log_prefix}Tool Result summary: {result_summary}")

        # The success of attempt_goal_autonomously_v2_tool depends on the tool's own reported success, if available.
        # If the tool doesn't report a 'success' field, we assume the attempt was "successful" if no exception occurred.
        overall_success = execution_result.get('success', True) if isinstance(execution_result, dict) else True

        return {
            "success": overall_success,
            "message": f"Goal attempted. Tool '{chosen_tool_name}' executed.",
            "chosen_tool": chosen_tool_name,
            "parameters_used": filled_params,
            "execution_result": execution_result
        }
    except Exception as e:
        error_msg = f"Error during execution of tool '{chosen_tool_name}': {type(e).__name__}: {e}"
        tb_str = traceback.format_exc()
        detailed_error = f"{error_msg}\nTraceback:\n{tb_str}"
        print(f"{log_prefix}{detailed_error}")
        return {"success": False, "message": error_msg, "chosen_tool": chosen_tool_name, "parameters_used": filled_params, "execution_result": {"error": detailed_error}}


# --- Register New Tool (overwrite v1) ---
print("\nAgent: Registering autonomous goal execution tool (v2)...")
if 'add_tool' in globals() and callable(add_tool):
    add_tool(
        name="attempt_goal_autonomously_v2", # New name for v2, or overwrite v1 if preferred
        tool_callable=attempt_goal_autonomously_v2_tool,
        description="(Version 2) Attempts to achieve a goal by selecting a tool, heuristically filling its parameters with improved logic, and executing it."
    )
    # Overwrite v1 if we want to replace it directly for tests
    _AGENT_TOOLS["attempt_goal_autonomously_v1"] = attempt_goal_autonomously_v2_tool
    _TOOL_DESCRIPTIONS["attempt_goal_autonomously_v1"] = "(Version 2, aliased as v1) Attempts to achieve a goal by selecting a tool, heuristically filling its parameters with improved logic, and executing it."

    print("Registered 'attempt_goal_autonomously_v2' and aliased 'attempt_goal_autonomously_v1' to point to v2.")
else:
    print("CRITICAL ERROR: 'add_tool' function not found.")

# --- Verify Registration ---
print("\nAgent: Verifying tool registration:")
if 'list_tools' in _AGENT_TOOLS and callable(_AGENT_TOOLS['list_tools']):
    _AGENT_TOOLS['list_tools']()

# --- Test Autonomous Goal Execution Tool (using the v1 alias which now points to v2) ---
print("\nAgent: Starting tests for autonomous goal execution tool (v2 functionality via v1 alias).")

tool_to_test_name = "attempt_goal_autonomously_v1" # Test using the original name, now pointing to v2
if tool_to_test_name not in _AGENT_TOOLS:
    print(f"Agent: Skipping tests as '{tool_to_test_name}' tool was not registered.")
else:
    # Test 1: Goal that should map to write_file and succeed (testing content/filepath disambiguation)
    goal1 = "Please write the text 'Hello autonomously V2!' to a file named 'autonomous_test_output_v2.txt'."
    print(f"\n--- Test 1 (v2 logic): Goal: '{goal1}' ---")
    result1 = _AGENT_TOOLS[tool_to_test_name](goal_description=goal1)
    print(f"Result for goal 1: {json.dumps(result1, default=str, indent=2)}")
    if result1.get('success') and result1.get('chosen_tool') == 'write_file' and \
       result1.get('parameters_used', {}).get('content') == 'Hello autonomously V2!' and \
       result1.get('parameters_used', {}).get('filepath') == 'autonomous_test_output_v2.txt':
        print("  SUCCESS (Test 1, v2 logic): Goal achieved with write_file and correct parameters.")
        if 'delete_file' in _AGENT_TOOLS:
            _AGENT_TOOLS['delete_file'](filepath=result1['parameters_used']['filepath'])
            print(f"  Cleanup: Deleted '{result1['parameters_used']['filepath']}'.")
    else:
        print(f"  FAILURE (Test 1, v2 logic): Goal not achieved as expected. Params: {result1.get('parameters_used')}, Message: {result1.get('message')}")

    # Test 2: Goal that should map to make_http_request and succeed (testing URL cleaning)
    goal2 = "Download the content from 'https://jsonplaceholder.typicode.com/todos/1' using GET method." # Note trailing quote on URL
    print(f"\n--- Test 2 (v2 logic): Goal: '{goal2}' ---")
    result2 = _AGENT_TOOLS[tool_to_test_name](goal_description=goal2)
    print(f"Result for goal 2: {json.dumps(result2, default=str, indent=2)}")
    # Expect success=True from the tool itself, and the HTTP request to be successful (status 200)
    if result2.get('success') and result2.get('chosen_tool') == 'make_http_request' and \
       result2.get('parameters_used', {}).get('url') == 'https://jsonplaceholder.typicode.com/todos/1' and \
       result2.get('execution_result', {}).get('status_code') == 200:
        print("  SUCCESS (Test 2, v2 logic): Goal achieved with make_http_request and cleaned URL.")
    else:
        print(f"  FAILURE (Test 2, v2 logic): Goal not achieved as expected. URL used: {result2.get('parameters_used', {}).get('url')}, HTTP Status: {result2.get('execution_result', {}).get('status_code')}, Message: {result2.get('message')}")

    # Test 3 and 4 and 5 are expected to behave similarly to v1, as they test other aspects.
    # Re-running them to ensure no regressions.
    goal3 = "Read a file."
    print(f"\n--- Test 3 (v2 logic): Goal: '{goal3}' ---")
    result3 = _AGENT_TOOLS[tool_to_test_name](goal_description=goal3)
    if not result3.get('success') and "Missing required parameters" in result3.get('message', ''):
        print("  SUCCESS (Test 3, v2 logic): Correctly failed due to missing parameters.")
    else:
        print(f"  FAILURE (Test 3, v2 logic): Did not fail as expected. Message: {result3.get('message')}")

    goal4 = "Contemplate the sound of one hand clapping."
    print(f"\n--- Test 4 (v2 logic): Goal: '{goal4}' ---")
    result4 = _AGENT_TOOLS[tool_to_test_name](goal_description=goal4)
    if not result4.get('success') and ("score" in result4.get('message', '').lower() and "below threshold" in result4.get('message', '').lower() or "No suitable tools suggested" in result4.get('message', '')):
        print("  SUCCESS (Test 4, v2 logic): Correctly decided not to execute due to low relevance or no suggestion.")
    else:
        print(f"  FAILURE (Test 4, v2 logic): Did not behave as expected for low relevance. Message: {result4.get('message')}")
        
    goal5 = "Run this python code: print('Autonomous code execution test V2!')"
    print(f"\n--- Test 5 (v2 logic): Goal: '{goal5}' ---")
    result5 = _AGENT_TOOLS[tool_to_test_name](goal_description=goal5)
    if result5.get('success') and result5.get('chosen_tool') == 'execute_python_code' and \
       result5.get('execution_result', {}).get("stdout", "").strip() == "Autonomous code execution test V2!":
        print("  SUCCESS (Test 5, v2 logic): Python code executed successfully.")
    else:
        print(f"  FAILURE (Test 5, v2 logic): Python code execution failed or stdout mismatch. Message: {result5.get('message')}, Stdout: {result5.get('execution_result', {}).get('stdout')}")


print("\nAgent: Autonomous goal execution tool (v2 functionality) definition, registration, and tests complete.")
print("Waiting for next instruction.")

SyntaxError: invalid syntax (4240606914.py, line 127)

In [14]:
# _AGENT_TOOLS, _TOOL_DESCRIPTIONS, add_tool, suggest_tools_for_goal_v2, etc. are assumed to be in the global scope.
import inspect
import re
import json # For printing test results neatly
import traceback # For detailed error messages

print("Agent: Defining and registering autonomous goal execution tool (v2 with improved heuristics - FIX 1).")

# --- Define Autonomous Goal Execution Tool V2 ---

_MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION_V2 = 2.5

def _extract_parameter_value_heuristically_v2(
    param_name: str, 
    param_type, 
    goal_description: str, 
    all_quoted_strings_from_goal: list[str],
    consumed_quoted_strings: set[str] 
) -> any:
    goal_lower = goal_description.lower()

    # 1. URL Extraction (and cleaning)
    if param_name == "url":
        url_match_re = r"https?://(?:[a-zA-Z0-9.\-_~:/?#\[\]@!$&'()*+,;=%]+[a-zA-Z0-9\-_~:/?#\[\]@!$&*+,;=%])"
        url_match = re.search(url_match_re, goal_description)
        if url_match:
            extracted_url = url_match.group(0)
            if extracted_url.endswith("'") and goal_description.count(f"'{extracted_url}'") > 0:
                extracted_url = extracted_url[:-1]
            elif extracted_url.endswith("\"") and goal_description.count(f"\"{extracted_url}\"") > 0:
                extracted_url = extracted_url[:-1]
            for q_str in all_quoted_strings_from_goal:
                if extracted_url == q_str or f"'{extracted_url}'" == q_str or f"\"{extracted_url}\"" == q_str :
                    if q_str not in consumed_quoted_strings:
                        consumed_quoted_strings.add(q_str)
                        break 
            return extracted_url

    # 2. Filepath / Path / Directory from Quoted Strings
    if param_name in ["filepath", "path", "directory"]:
        path_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            if '.' in q_str or '/' in q_str or '\\' in q_str or \
               any(q_str.endswith(ext) for ext in ['.txt', '.json', '.py', '.html', '.md']):
                path_candidates.append(q_str)
        if path_candidates:
            chosen_path = max(path_candidates, key=len)
            consumed_quoted_strings.add(chosen_path)
            return chosen_path

    # 3. Content / Text / Description / Message from Quoted Strings
    if param_name in ["content", "text", "description", "message"]:
        content_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            content_candidates.append(q_str)
        if content_candidates:
            chosen_content = max(content_candidates, key=len)
            consumed_quoted_strings.add(chosen_content)
            return chosen_content
            
    # 4. Code String
    if param_name == "code_string":
        code_block_match = re.search(r"code:\s*(.+)", goal_description, re.IGNORECASE | re.DOTALL)
        if code_block_match: return code_block_match.group(1).strip()
        triple_tick_match = re.search(r"```(?:python\n)?(.*?)```", goal_description, re.DOTALL | re.IGNORECASE)
        if triple_tick_match: return triple_tick_match.group(1).strip()

    # 5. HTTP Method
    if param_name == "method" and (param_type == str or param_type is None):
        method_search = re.search(r"\b(GET|POST|PUT|DELETE|PATCH|HEAD|OPTIONS)\b", goal_description, re.IGNORECASE)
        if method_search: return method_search.group(1).upper()
        if "post" in goal_lower: return "POST"
        if "get" in goal_lower: return "GET"
        if "put" in goal_lower: return "PUT"
        if "delete" in goal_lower: return "DELETE"

    # 6. Boolean Flags
    if param_type == bool:
        if re.search(rf"\b{param_name}\b\s*(true|yes|on|recursively|enabled)\b", goal_lower, re.IGNORECASE) or \
           re.search(rf"\b(is|be|make it)\s*{param_name}\b", goal_lower, re.IGNORECASE):
            return True
        if re.search(rf"\b{param_name}\b\s*(false|no|off|disabled)\b", goal_lower, re.IGNORECASE) or \
           re.search(rf"\b(is not|not to be|don't make it)\s*{param_name}\b", goal_lower, re.IGNORECASE):
            return False
        if param_name in goal_lower and param_name not in ["false", "no", "off"]:
            return True

    # 7. Explicit Assignment
    explicit_assign_match_quoted = re.search(rf"\b{param_name}\s*[:=]\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
    if explicit_assign_match_quoted:
        val_str = explicit_assign_match_quoted.group(1)
        if val_str not in consumed_quoted_strings:
            return val_str
    explicit_assign_match_unquoted = re.search(rf"\b{param_name}\s*[:=]\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
    if explicit_assign_match_unquoted:
        val_str = explicit_assign_match_unquoted.group(1)
        if param_type == int:
            try: return int(val_str)
            except ValueError: pass # Corrected
        if param_type == float:
            try: return float(val_str)
            except ValueError: pass # Corrected
        if param_type == bool:
            if val_str.lower() == 'true': return True
            if val_str.lower() == 'false': return False
        if param_type == str or param_type is None: return val_str
    return None

def attempt_goal_autonomously_v2_tool(goal_description: str) -> dict:
    tool_name_internal = "attempt_goal_autonomously_v2_tool"
    log_prefix = f"Agent log ({tool_name_internal}): "
    print(f"{log_prefix}Attempting goal (v2): '{goal_description[:100]}...'")

    quoted_match_tuples = re.findall(r"\"([^\"]+)\"|\'([^\']+)\'", goal_description)
    all_quoted_strings_from_goal = [item for tpl in quoted_match_tuples for item in tpl if item]
    
    if 'suggest_tools_for_goal' not in _AGENT_TOOLS:
        return {"success": False, "message": "Tool 'suggest_tools_for_goal' not found.", "details": None}
    suggestion_result = _AGENT_TOOLS['suggest_tools_for_goal'](goal_description=goal_description, top_n=1)
    if not suggestion_result['success'] or not suggestion_result['suggestions']:
        msg = "No suitable tools suggested for the goal."
        print(f"{log_prefix}{msg}")
        return {"success": False, "message": msg, "chosen_tool": None, "parameters": None, "execution_result": None}
    top_suggestion = suggestion_result['suggestions'][0]
    chosen_tool_name = top_suggestion['tool_name']
    relevance_score = top_suggestion['relevance_score']
    print(f"{log_prefix}Tool suggestion result: {suggestion_result['message']}")
    print(f"{log_prefix}Top suggested tool: '{chosen_tool_name}' (Score: {relevance_score}). Reason: {top_suggestion.get('reason', 'N/A')}")
    if relevance_score < _MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION_V2:
        msg = f"Top suggested tool '{chosen_tool_name}' score ({relevance_score}) is below threshold ({_MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION_V2}). Not proceeding."
        print(f"{log_prefix}{msg}")
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, "parameters": None, "execution_result": None}
    if chosen_tool_name not in _AGENT_TOOLS:
        msg = f"Suggested tool '{chosen_tool_name}' not found in agent's toolkit."
        print(f"{log_prefix}{msg}")
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, "parameters": None, "execution_result": None}
    tool_callable = _AGENT_TOOLS[chosen_tool_name]

    try:
        sig = inspect.signature(tool_callable)
    except Exception as e:
        msg = f"Could not inspect signature of tool '{chosen_tool_name}': {e}"
        print(f"{log_prefix}{msg}")
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, "parameters": None, "execution_result": None}

    filled_params = {}
    missing_required_params = []
    consumed_quoted_strings_for_this_call = set()
    print(f"{log_prefix}Attempting to fill parameters for '{chosen_tool_name}': {list(sig.parameters.keys())}")
    param_order = sorted(sig.parameters.items(), key=lambda item: 1 if item[0] in ["filepath", "path", "url"] else 2)

    for param_name, param_obj in param_order:
        param_type_annotation = param_obj.annotation if param_obj.annotation != inspect.Parameter.empty else None
        extracted_value = _extract_parameter_value_heuristically_v2(
            param_name, param_type_annotation, goal_description, 
            all_quoted_strings_from_goal, consumed_quoted_strings_for_this_call
        )
        if extracted_value is not None:
            if param_type_annotation and isinstance(extracted_value, str):
                try:
                    if param_type_annotation == bool:
                        if extracted_value.lower() == 'true': extracted_value = True
                        elif extracted_value.lower() == 'false': extracted_value = False
                    elif param_type_annotation == int: extracted_value = int(extracted_value)
                    elif param_type_annotation == float: extracted_value = float(extracted_value)
                except ValueError:
                    print(f"{log_prefix}Warning: Could not convert extracted value '{extracted_value}' to type {param_type_annotation} for param '{param_name}'. Using as string.")
            filled_params[param_name] = extracted_value
            print(f"{log_prefix}  Filled '{param_name}': {str(extracted_value)[:50]}{'...' if len(str(extracted_value)) > 50 else ''} (Type: {type(extracted_value).__name__})")
        elif param_obj.default != inspect.Parameter.empty:
            filled_params[param_name] = param_obj.default
            print(f"{log_prefix}  Using default for '{param_name}': {str(param_obj.default)[:50]}{'...' if len(str(param_obj.default)) > 50 else ''}")
        else:
            missing_required_params.append(param_name)
            print(f"{log_prefix}  Could not fill required parameter '{param_name}'.")
            
    if missing_required_params:
        msg = f"Failed to execute goal: Missing required parameters for tool '{chosen_tool_name}': {', '.join(missing_required_params)}."
        print(f"{log_prefix}{msg}")
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, "parameters_tried": filled_params, "execution_result": None}

    print(f"{log_prefix}All required parameters filled. Executing '{chosen_tool_name}' with: {json.dumps(filled_params, default=str, indent=2)}")
    try:
        execution_result = tool_callable(**filled_params)
        print(f"{log_prefix}Execution of '{chosen_tool_name}' appears to have completed.")
        result_summary = {"tool_reported_success": execution_result.get('success', 'N/A')}
        if 'message' in execution_result: result_summary['tool_message'] = str(execution_result['message'])[:100]
        if 'content' in execution_result and execution_result['content'] is not None: result_summary['content_snippet'] = str(execution_result['content'])[:100]
        print(f"{log_prefix}Tool Result summary: {result_summary}")
        overall_success = execution_result.get('success', True) if isinstance(execution_result, dict) else True
        return {
            "success": overall_success,
            "message": f"Goal attempted. Tool '{chosen_tool_name}' executed.",
            "chosen_tool": chosen_tool_name,
            "parameters_used": filled_params,
            "execution_result": execution_result
        }
    except Exception as e:
        error_msg = f"Error during execution of tool '{chosen_tool_name}': {type(e).__name__}: {e}"
        tb_str = traceback.format_exc()
        detailed_error = f"{error_msg}\nTraceback:\n{tb_str}"
        print(f"{log_prefix}{detailed_error}")
        return {"success": False, "message": error_msg, "chosen_tool": chosen_tool_name, "parameters_used": filled_params, "execution_result": {"error": detailed_error}}

# --- Register New Tool (overwrite v1) ---
print("\nAgent: Registering autonomous goal execution tool (v2 - FIX 1)...")
if 'add_tool' in globals() and callable(add_tool):
    # Add v2 under its own name first, then alias v1 to it.
    add_tool(
        name="attempt_goal_autonomously_v2",
        tool_callable=attempt_goal_autonomously_v2_tool,
        description="(Version 2) Attempts to achieve a goal by selecting a tool, heuristically filling its parameters with improved logic, and executing it."
    )
    # Overwrite v1 to point to the v2 implementation for seamless testing
    _AGENT_TOOLS["attempt_goal_autonomously_v1"] = attempt_goal_autonomously_v2_tool
    _TOOL_DESCRIPTIONS["attempt_goal_autonomously_v1"] = "(Version 2, aliased as v1 - FIX 1) Attempts to achieve a goal by selecting a tool, heuristically filling its parameters with improved logic, and executing it."
    print("Registered 'attempt_goal_autonomously_v2' and aliased 'attempt_goal_autonomously_v1' to point to v2.")
else:
    print("CRITICAL ERROR: 'add_tool' function not found.")

# --- Verify Registration ---
print("\nAgent: Verifying tool registration:")
if 'list_tools' in _AGENT_TOOLS and callable(_AGENT_TOOLS['list_tools']):
    _AGENT_TOOLS['list_tools']()

# --- Test Autonomous Goal Execution Tool (using the v1 alias which now points to v2) ---
print("\nAgent: Starting tests for autonomous goal execution tool (v2 functionality via v1 alias - FIX 1).")
tool_to_test_name = "attempt_goal_autonomously_v1"
if tool_to_test_name not in _AGENT_TOOLS:
    print(f"Agent: Skipping tests as '{tool_to_test_name}' tool was not registered.")
else:
    goal1 = "Please write the text 'Hello autonomously V2!' to a file named 'autonomous_test_output_v2.txt'."
    print(f"\n--- Test 1 (v2 logic): Goal: '{goal1}' ---")
    result1 = _AGENT_TOOLS[tool_to_test_name](goal_description=goal1)
    print(f"Result for goal 1: {json.dumps(result1, default=str, indent=2)}")
    if result1.get('success') and result1.get('chosen_tool') == 'write_file' and \
       result1.get('parameters_used', {}).get('content') == 'Hello autonomously V2!' and \
       result1.get('parameters_used', {}).get('filepath') == 'autonomous_test_output_v2.txt':
        print("  SUCCESS (Test 1, v2 logic): Goal achieved with write_file and correct parameters.")
        if 'delete_file' in _AGENT_TOOLS:
            _AGENT_TOOLS['delete_file'](filepath=result1['parameters_used']['filepath'])
            print(f"  Cleanup: Deleted '{result1['parameters_used']['filepath']}'.")
    else:
        print(f"  FAILURE (Test 1, v2 logic): Goal not achieved as expected. Params: {result1.get('parameters_used')}, Message: {result1.get('message')}")

    goal2 = "Download the content from 'https://jsonplaceholder.typicode.com/todos/1' using GET method."
    print(f"\n--- Test 2 (v2 logic): Goal: '{goal2}' ---")
    result2 = _AGENT_TOOLS[tool_to_test_name](goal_description=goal2)
    print(f"Result for goal 2: {json.dumps(result2, default=str, indent=2)}")
    if result2.get('success') and result2.get('chosen_tool') == 'make_http_request' and \
       result2.get('parameters_used', {}).get('url') == 'https://jsonplaceholder.typicode.com/todos/1' and \
       result2.get('execution_result', {}).get('status_code') == 200:
        print("  SUCCESS (Test 2, v2 logic): Goal achieved with make_http_request and cleaned URL.")
    else:
        print(f"  FAILURE (Test 2, v2 logic): Goal not achieved as expected. URL used: {result2.get('parameters_used', {}).get('url')}, HTTP Status: {result2.get('execution_result', {}).get('status_code')}, Message: {result2.get('message')}")

    goal3 = "Read a file."
    print(f"\n--- Test 3 (v2 logic): Goal: '{goal3}' ---")
    result3 = _AGENT_TOOLS[tool_to_test_name](goal_description=goal3)
    if not result3.get('success') and "Missing required parameters" in result3.get('message', ''):
        print("  SUCCESS (Test 3, v2 logic): Correctly failed due to missing parameters.")
    else:
        print(f"  FAILURE (Test 3, v2 logic): Did not fail as expected. Message: {result3.get('message')}")

    goal4 = "Contemplate the sound of one hand clapping."
    print(f"\n--- Test 4 (v2 logic): Goal: '{goal4}' ---")
    result4 = _AGENT_TOOLS[tool_to_test_name](goal_description=goal4)
    if not result4.get('success') and ("score" in result4.get('message', '').lower() and "below threshold" in result4.get('message', '').lower() or "No suitable tools suggested" in result4.get('message', '')):
        print("  SUCCESS (Test 4, v2 logic): Correctly decided not to execute due to low relevance or no suggestion.")
    else:
        print(f"  FAILURE (Test 4, v2 logic): Did not behave as expected for low relevance. Message: {result4.get('message')}")
        
    goal5 = "Run this python code: print('Autonomous code execution test V2!')"
    print(f"\n--- Test 5 (v2 logic): Goal: '{goal5}' ---")
    result5 = _AGENT_TOOLS[tool_to_test_name](goal_description=goal5)
    if result5.get('success') and result5.get('chosen_tool') == 'execute_python_code' and \
       result5.get('execution_result', {}).get("stdout", "").strip() == "Autonomous code execution test V2!":
        print("  SUCCESS (Test 5, v2 logic): Python code executed successfully.")
    else:
        print(f"  FAILURE (Test 5, v2 logic): Python code execution failed or stdout mismatch. Message: {result5.get('message')}, Stdout: {result5.get('execution_result', {}).get('stdout')}")

print("\nAgent: Autonomous goal execution tool (v2 functionality - FIX 1) definition, registration, and tests complete.")
print("Waiting for next instruction.")

Agent: Defining and registering autonomous goal execution tool (v2 with improved heuristics - FIX 1).

Agent: Registering autonomous goal execution tool (v2 - FIX 1)...
Tool 'attempt_goal_autonomously_v2' added successfully. Description: (Version 2) Attempts to achieve a goal by selecting a tool, heuristically filling its parameters with improved logic, and executing it.
Registered 'attempt_goal_autonomously_v2' and aliased 'attempt_goal_autonomously_v1' to point to v2.

Agent: Verifying tool registration:

Available tools:
1. list_tools: Lists all available tools with their names and descriptions. Useful for understanding agent capabilities.
2. get_agent_status: Provides a status report of the agent, including version and number of loaded tools.
3. execute_python_code: Executes a given string of Python code in the agent's current global environment. Captures and returns stdout, stderr, and any execution error from the provided code. USE WITH EXTREME CAUTION, as it can alter the agent'

In [15]:
# _AGENT_TOOLS, _TOOL_DESCRIPTIONS, add_tool, suggest_tools_for_goal_v2, etc. are assumed to be in the global scope.
# _extract_parameter_value_heuristically_v2 is assumed available from previous successful execution.
import inspect
import re
import json
import traceback

print("Agent: Defining and registering autonomous goal execution tool (v3 with clarification requests).")

# --- Define Autonomous Goal Execution Tool V3 ---

_MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION_V3 = 2.5 # Same threshold as v2

# Re-use _extract_parameter_value_heuristically_v2 (it's not defined in this block but assumed to exist from prev turn)
# If it's not in the global scope, this will fail. This is a limitation of the turn-based execution.
# For robustness in a real system, it would be part of this module or explicitly loaded.
# Assuming `_extract_parameter_value_heuristically_v2` is available in the execution environment's global scope.


def attempt_goal_autonomously_v3_tool(goal_description: str, additional_context: dict = None) -> dict:
    """
    (Version 3) Attempts to achieve a goal by selecting a tool,
    heuristically filling its parameters, and executing it.
    If required parameters are missing, it returns a clarification request.
    `additional_context` can be used to provide pre-filled parameters or override heuristics.
    """
    tool_name_internal = "attempt_goal_autonomously_v3_tool"
    log_prefix = f"Agent log ({tool_name_internal}): "
    print(f"{log_prefix}Attempting goal (v3): '{goal_description[:100]}...'. Additional context: {additional_context is not None}")

    if additional_context is None:
        additional_context = {}

    # Pre-extract all quoted strings once
    quoted_match_tuples = re.findall(r"\"([^\"]+)\"|\'([^\']+)\'", goal_description)
    all_quoted_strings_from_goal = [item for tpl in quoted_match_tuples for item in tpl if item]
    
    # --- Tool Selection ---
    if 'suggest_tools_for_goal' not in _AGENT_TOOLS:
        return {"success": False, "message": "Tool 'suggest_tools_for_goal' not found.", "details": None}
    
    # Check if a tool is pre-selected in context (e.g., from a previous clarification)
    chosen_tool_name = additional_context.get("chosen_tool_override")
    tool_callable = None
    
    if chosen_tool_name:
        if chosen_tool_name in _AGENT_TOOLS:
            tool_callable = _AGENT_TOOLS[chosen_tool_name]
            print(f"{log_prefix}Using pre-selected tool from context: '{chosen_tool_name}'.")
        else:
            msg = f"Pre-selected tool '{chosen_tool_name}' from context not found in toolkit."
            print(f"{log_prefix}{msg}")
            return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, "parameters": None, "execution_result": None}
    else:
        suggestion_result = _AGENT_TOOLS['suggest_tools_for_goal'](goal_description=goal_description, top_n=1)
        print(f"{log_prefix}Tool suggestion result: {suggestion_result['message']}")
        if not suggestion_result['success'] or not suggestion_result['suggestions']:
            msg = "No suitable tools suggested for the goal."
            print(f"{log_prefix}{msg}")
            return {"success": False, "message": msg, "chosen_tool": None, "parameters": None, "execution_result": None, "clarification_request": None}
        
        top_suggestion = suggestion_result['suggestions'][0]
        chosen_tool_name = top_suggestion['tool_name']
        relevance_score = top_suggestion['relevance_score']
        print(f"{log_prefix}Top suggested tool: '{chosen_tool_name}' (Score: {relevance_score}). Reason: {top_suggestion.get('reason', 'N/A')}")

        if relevance_score < _MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION_V3:
            msg = f"Top suggested tool '{chosen_tool_name}' score ({relevance_score}) is below threshold ({_MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION_V3}). Not proceeding."
            print(f"{log_prefix}{msg}")
            return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, "parameters": None, "execution_result": None, "clarification_request": None}
        
        if chosen_tool_name not in _AGENT_TOOLS: # Should not happen if suggest_tools uses _AGENT_TOOLS
            msg = f"Suggested tool '{chosen_tool_name}' not found in agent's toolkit."
            print(f"{log_prefix}{msg}")
            return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, "parameters": None, "execution_result": None, "clarification_request": None}
        tool_callable = _AGENT_TOOLS[chosen_tool_name]

    # --- Parameter Filling ---
    try:
        sig = inspect.signature(tool_callable)
    except Exception as e:
        msg = f"Could not inspect signature of tool '{chosen_tool_name}': {e}"
        print(f"{log_prefix}{msg}")
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, "parameters": None, "execution_result": None, "clarification_request": None}

    filled_params = {}
    missing_required_params_info = [] # Store info about missing params for clarification
    consumed_quoted_strings_for_this_call = set()
    print(f"{log_prefix}Attempting to fill parameters for '{chosen_tool_name}': {list(sig.parameters.keys())}")
    param_order = sorted(sig.parameters.items(), key=lambda item: 1 if item[0] in ["filepath", "path", "url"] else 2)

    for param_name, param_obj in param_order:
        param_type_annotation = param_obj.annotation if param_obj.annotation != inspect.Parameter.empty else None
        
        # Prioritize values from additional_context
        if param_name in additional_context.get("parameters", {}):
            extracted_value = additional_context["parameters"][param_name]
            print(f"{log_prefix}  Using value for '{param_name}' from additional_context.")
        else:
            extracted_value = _extract_parameter_value_heuristically_v2( # Assuming this exists from previous turns
                param_name, param_type_annotation, goal_description, 
                all_quoted_strings_from_goal, consumed_quoted_strings_for_this_call
            )

        if extracted_value is not None:
            # Basic type conversion (already in _extract_parameter_value_heuristically_v2, but can be double-checked/refined here)
            if param_type_annotation and isinstance(extracted_value, str):
                try:
                    if param_type_annotation == bool:
                        if extracted_value.lower() == 'true': extracted_value = True
                        elif extracted_value.lower() == 'false': extracted_value = False
                    elif param_type_annotation == int: extracted_value = int(extracted_value)
                    elif param_type_annotation == float: extracted_value = float(extracted_value)
                except ValueError:
                    print(f"{log_prefix}Warning: Could not convert extracted value '{extracted_value}' to type {param_type_annotation} for param '{param_name}'. Using as string for now.")
            
            filled_params[param_name] = extracted_value
            print(f"{log_prefix}  Filled '{param_name}': {str(extracted_value)[:50]}{'...' if len(str(extracted_value)) > 50 else ''} (Type: {type(extracted_value).__name__})")
        elif param_obj.default != inspect.Parameter.empty:
            filled_params[param_name] = param_obj.default
            print(f"{log_prefix}  Using default for '{param_name}': {str(param_obj.default)[:50]}{'...' if len(str(param_obj.default)) > 50 else ''}")
        else: # No value extracted, no default -> required param is missing
            missing_required_params_info.append({
                "name": param_name,
                "type": str(param_type_annotation) if param_type_annotation else "unknown",
                "description": f"Value for '{param_name}' (expected type: {str(param_type_annotation) if param_type_annotation else 'any'})."
            })
            print(f"{log_prefix}  Could not fill required parameter '{param_name}'.")
            
    if missing_required_params_info:
        param_names_missing = [p['name'] for p in missing_required_params_info]
        msg = f"Clarification needed: Missing required parameters for tool '{chosen_tool_name}': {', '.join(param_names_missing)}."
        print(f"{log_prefix}{msg}")
        clarification_prompt = f"Please provide values for the following parameters to use tool '{chosen_tool_name}' for the goal '{goal_description[:50]}...':\n"
        for p_info in missing_required_params_info:
            clarification_prompt += f"  - {p_info['name']} ({p_info['description']})\n"
        clarification_prompt += "You can provide them in the 'additional_context' field, e.g., {'parameters': {'param_name': 'value'}}."
        
        return {
            "success": False, 
            "message": msg, 
            "chosen_tool": chosen_tool_name, 
            "parameters_tried": filled_params, 
            "execution_result": None,
            "clarification_request": {
                "type": "missing_parameters",
                "original_goal": goal_description,
                "tool_name": chosen_tool_name,
                "missing_params_details": missing_required_params_info,
                "prompt": clarification_prompt
            }
        }

    # --- Tool Execution ---
    print(f"{log_prefix}All required parameters filled. Executing '{chosen_tool_name}' with: {json.dumps(filled_params, default=str, indent=2)}")
    try:
        execution_result = tool_callable(**filled_params)
        print(f"{log_prefix}Execution of '{chosen_tool_name}' appears to have completed.")
        result_summary = {"tool_reported_success": execution_result.get('success', 'N/A')}
        if 'message' in execution_result: result_summary['tool_message'] = str(execution_result['message'])[:100]
        overall_success = execution_result.get('success', True) if isinstance(execution_result, dict) else True
        print(f"{log_prefix}Tool Result summary: {result_summary}")
        return {
            "success": overall_success,
            "message": f"Goal attempted. Tool '{chosen_tool_name}' executed.",
            "chosen_tool": chosen_tool_name,
            "parameters_used": filled_params,
            "execution_result": execution_result,
            "clarification_request": None
        }
    except Exception as e:
        error_msg = f"Error during execution of tool '{chosen_tool_name}': {type(e).__name__}: {e}"
        tb_str = traceback.format_exc()
        detailed_error = f"{error_msg}\nTraceback:\n{tb_str}"
        print(f"{log_prefix}{detailed_error}")
        return {
            "success": False, 
            "message": error_msg, 
            "chosen_tool": chosen_tool_name, 
            "parameters_used": filled_params, 
            "execution_result": {"error": detailed_error},
            "clarification_request": None
        }

# --- Register New Tool ---
print("\nAgent: Registering autonomous goal execution tool (v3)...")
if 'add_tool' in globals() and callable(add_tool):
    add_tool(
        name="attempt_goal_autonomously_v3",
        tool_callable=attempt_goal_autonomously_v3_tool,
        description="(Version 3) Attempts goal execution. Asks for clarification if required parameters are missing. Accepts 'additional_context'."
    )
else:
    print("CRITICAL ERROR: 'add_tool' function not found.")

# --- Verify Registration ---
print("\nAgent: Verifying tool registration:")
if 'list_tools' in _AGENT_TOOLS and callable(_AGENT_TOOLS['list_tools']):
    _AGENT_TOOLS['list_tools']()

# --- Test Autonomous Goal Execution Tool (v3) ---
print("\nAgent: Starting tests for autonomous goal execution tool (v3).")

tool_to_test_name = "attempt_goal_autonomously_v3"
if tool_to_test_name not in _AGENT_TOOLS:
    print(f"Agent: Skipping tests as '{tool_to_test_name}' tool was not registered.")
else:
    # Test 1: Successful execution (no clarification needed)
    goal1 = "Write the text 'Autonomous V3 test' to 'v3_output.txt'."
    print(f"\n--- Test 1 (v3): Goal: '{goal1}' ---")
    result1 = _AGENT_TOOLS[tool_to_test_name](goal_description=goal1)
    print(f"Result for goal 1: {json.dumps(result1, default=str, indent=2)}")
    if result1.get('success') and result1.get('chosen_tool') == 'write_file' and \
       result1.get('parameters_used', {}).get('content') == 'Autonomous V3 test':
        print("  SUCCESS (Test 1, v3): Goal achieved directly.")
        if 'delete_file' in _AGENT_TOOLS:
            _AGENT_TOOLS['delete_file'](filepath=result1['parameters_used']['filepath'])
            print(f"  Cleanup: Deleted '{result1['parameters_used']['filepath']}'.")
    else:
        print(f"  FAILURE (Test 1, v3): Goal not achieved as expected. Message: {result1.get('message')}")

    # Test 2: Clarification request for missing parameter
    goal2 = "Read a file." # Intentionally missing filepath
    print(f"\n--- Test 2 (v3): Goal: '{goal2}' (Expect clarification) ---")
    result2 = _AGENT_TOOLS[tool_to_test_name](goal_description=goal2)
    print(f"Result for goal 2: {json.dumps(result2, default=str, indent=2)}")
    if not result2.get('success') and result2.get('clarification_request') and \
       result2['clarification_request']['type'] == 'missing_parameters' and \
       result2['clarification_request']['missing_params_details'][0]['name'] == 'filepath':
        print("  SUCCESS (Test 2, v3): Correctly requested clarification for 'filepath'.")
    else:
        print(f"  FAILURE (Test 2, v3): Did not request clarification as expected. Clarification: {result2.get('clarification_request')}")

    # Test 3: Using additional_context to provide the missing parameter from Test 2's clarification
    if result2.get('clarification_request'): # Only run if Test 2 produced a clarification request
        print(f"\n--- Test 3 (v3): Goal: '{goal2}' with additional context ---")
        # First, create a dummy file to read
        dummy_filepath = "dummy_for_clarification_test.txt"
        dummy_content = "Content from dummy file."
        if 'write_file' in _AGENT_TOOLS:
            _AGENT_TOOLS['write_file'](filepath=dummy_filepath, content=dummy_content)
            print(f"  Setup: Created '{dummy_filepath}' for reading.")

            context_for_goal2 = {"parameters": {"filepath": dummy_filepath}}
            result3 = _AGENT_TOOLS[tool_to_test_name](goal_description=goal2, additional_context=context_for_goal2)
            print(f"Result for goal 3: {json.dumps(result3, default=str, indent=2)}")
            if result3.get('success') and result3.get('chosen_tool') == 'read_file' and \
               result3.get('execution_result', {}).get('content') == dummy_content:
                print("  SUCCESS (Test 3, v3): Goal achieved using provided context after clarification.")
            else:
                print(f"  FAILURE (Test 3, v3): Goal not achieved with context. Message: {result3.get('message')}")
            
            if 'delete_file' in _AGENT_TOOLS: # Cleanup dummy file
                 _AGENT_TOOLS['delete_file'](filepath=dummy_filepath)
                 print(f"  Cleanup: Deleted '{dummy_filepath}'.")
        else:
            print("  SKIPPED Test 3: 'write_file' tool not available for setup.")
    else:
        print("  SKIPPED Test 3: Test 2 did not produce a clarification request to respond to.")


    # Test 4: Low relevance (should not ask for clarification, just fail)
    goal4 = "Ponder the imponderable nature of existence."
    print(f"\n--- Test 4 (v3): Goal: '{goal4}' (Expect low relevance) ---")
    result4 = _AGENT_TOOLS[tool_to_test_name](goal_description=goal4)
    print(f"Result for goal 4: {json.dumps(result4, default=str, indent=2)}")
    if not result4.get('success') and not result4.get('clarification_request') and \
       ("score" in result4.get('message', '').lower() and "below threshold" in result4.get('message', '').lower() or \
        "No suitable tools suggested" in result4.get('message', '')):
        print("  SUCCESS (Test 4, v3): Correctly failed due to low relevance, no clarification requested.")
    else:
        print(f"  FAILURE (Test 4, v3): Incorrect behavior for low relevance. Clarification: {result4.get('clarification_request')}, Message: {result4.get('message')}")

print("\nAgent: Autonomous goal execution tool (v3) definition, registration, and tests complete.")
print("Waiting for next instruction.")

Agent: Defining and registering autonomous goal execution tool (v3 with clarification requests).

Agent: Registering autonomous goal execution tool (v3)...
Tool 'attempt_goal_autonomously_v3' added successfully. Description: (Version 3) Attempts goal execution. Asks for clarification if required parameters are missing. Accepts 'additional_context'.

Agent: Verifying tool registration:

Available tools:
1. list_tools: Lists all available tools with their names and descriptions. Useful for understanding agent capabilities.
2. get_agent_status: Provides a status report of the agent, including version and number of loaded tools.
3. execute_python_code: Executes a given string of Python code in the agent's current global environment. Captures and returns stdout, stderr, and any execution error from the provided code. USE WITH EXTREME CAUTION, as it can alter the agent's state and capabilities.
4. write_file: (Version 2) Writes text content to a specified file. Overwrites if exists, creates 

In [16]:
# _AGENT_TOOLS, _TOOL_DESCRIPTIONS, add_tool, _extract_parameter_value_heuristically_v2,
# attempt_goal_autonomously_v3_tool, etc. are assumed to be in the global scope.
import json
import traceback

print("Agent: Defining and registering a multi-step sequential goal execution tool (v1).")

# --- Define Multi-Step Sequential Goal Execution Tool V1 ---

def execute_multi_step_goal_v1_tool(overall_goal_description: str, sub_goal_strings: list[str], initial_context: dict = None) -> dict:
    """
    (Version 1) Executes a sequence of sub-goals to achieve an overall goal.
    Uses 'attempt_goal_autonomously_v3_tool' for each sub-goal.
    Propagates context between steps and handles clarifications.

    Args:
        overall_goal_description (str): A description of the main goal for context.
        sub_goal_strings (list[str]): An ordered list of natural language sub-goal descriptions.
        initial_context (dict, optional): Any initial context to provide to the first sub-goal's
                                          attempt_goal_autonomously_v3_tool call, under 'parameters'.
                                          e.g., {"parameters": {"my_param": "value"}}

    Returns:
        dict: {
            'success': bool, (True if all sub-goals completed successfully)
            'message': str,
            'overall_goal': str,
            'sub_goal_results': list[dict], (Results from each sub-goal attempt)
            'final_cumulative_context': dict,
            'clarification_request_for_sub_goal': { (Present if clarification is needed)
                'sub_goal_index': int,
                'sub_goal_description': str,
                'clarification_details': dict (from attempt_goal_autonomously_v3_tool)
            }
        }
    """
    tool_name_internal = "execute_multi_step_goal_v1_tool"
    log_prefix = f"Agent log ({tool_name_internal}): "
    print(f"{log_prefix}Starting multi-step goal: '{overall_goal_description[:100]}...'. Number of sub-goals: {len(sub_goal_strings)}")

    if not sub_goal_strings:
        return {
            "success": False, "message": "No sub-goals provided.", "overall_goal": overall_goal_description,
            "sub_goal_results": [], "final_cumulative_context": initial_context or {}, "clarification_request_for_sub_goal": None
        }

    if 'attempt_goal_autonomously_v3' not in _AGENT_TOOLS: # The worker tool
        msg = "Critical dependency 'attempt_goal_autonomously_v3' tool not found."
        print(f"{log_prefix}{msg}")
        return {"success": False, "message": msg, "overall_goal": overall_goal_description, "sub_goal_results": [],
                "final_cumulative_context": initial_context or {}, "clarification_request_for_sub_goal": None}

    sub_goal_results_list = []
    cumulative_context = initial_context.get("parameters", {}) if initial_context else {} # Store key-value pairs
    
    print(f"{log_prefix}Initial cumulative context: {cumulative_context}")

    for i, sub_goal_desc in enumerate(sub_goal_strings):
        print(f"\n{log_prefix}--- Sub-goal {i+1}/{len(sub_goal_strings)}: '{sub_goal_desc[:100]}...' ---")
        
        # Prepare additional_context for the autonomous tool
        # This includes any parameters explicitly passed in initial_context or accumulated from previous steps.
        context_for_this_sub_goal = {"parameters": cumulative_context.copy()}
        print(f"{log_prefix}Context for sub-goal {i+1}: {context_for_this_sub_goal}")

        sub_goal_attempt_result = _AGENT_TOOLS['attempt_goal_autonomously_v3'](
            goal_description=sub_goal_desc,
            additional_context=context_for_this_sub_goal
        )
        sub_goal_results_list.append(sub_goal_attempt_result)

        if sub_goal_attempt_result.get("clarification_request"):
            msg = f"Clarification needed for sub-goal {i+1} ('{sub_goal_desc[:50]}...'). Halting multi-step plan."
            print(f"{log_prefix}{msg}")
            return {
                "success": False, "message": msg, "overall_goal": overall_goal_description,
                "sub_goal_results": sub_goal_results_list, "final_cumulative_context": cumulative_context,
                "clarification_request_for_sub_goal": {
                    "sub_goal_index": i,
                    "sub_goal_description": sub_goal_desc,
                    "clarification_details": sub_goal_attempt_result["clarification_request"]
                }
            }
        
        if not sub_goal_attempt_result.get("success"):
            msg = f"Sub-goal {i+1} ('{sub_goal_desc[:50]}...') failed: {sub_goal_attempt_result.get('message', 'Unknown error')}. Halting multi-step plan."
            print(f"{log_prefix}{msg}")
            return {
                "success": False, "message": msg, "overall_goal": overall_goal_description,
                "sub_goal_results": sub_goal_results_list, "final_cumulative_context": cumulative_context,
                "clarification_request_for_sub_goal": None
            }

        # If successful, try to extract some common outputs to pass to next steps
        if sub_goal_attempt_result.get("success"):
            execution_res = sub_goal_attempt_result.get("execution_result", {})
            if isinstance(execution_res, dict): # Ensure it's a dict before trying to get items
                # Define a list of common keys to extract for context propagation
                # Keys are {name_in_exec_result: name_in_cumulative_context}
                # This allows renaming for clarity or avoiding clashes.
                context_keys_to_propagate = {
                    "filepath": "last_filepath",
                    "content": "last_content",      # From read_file, make_http_request (text_content)
                    "text_content": "last_text_content", # Explicitly from make_http_request
                    "json_content": "last_json_content", # Explicitly from make_http_request
                    "task_id": "last_task_id",        # From add_task
                    "value": "last_retrieved_value" # From retrieve_knowledge
                    # Add more as needed
                }
                
                newly_added_to_context = {}
                for exec_key, context_key in context_keys_to_propagate.items():
                    if exec_key in execution_res and execution_res[exec_key] is not None:
                        cumulative_context[context_key] = execution_res[exec_key]
                        newly_added_to_context[context_key] = execution_res[exec_key]
                if newly_added_to_context:
                     print(f"{log_prefix}Added to cumulative context from sub-goal {i+1}: { {k: str(v)[:50]+'...' if len(str(v)) > 50 else v for k,v in newly_added_to_context.items()} }")
            print(f"{log_prefix}Updated cumulative context: { {k: str(v)[:50]+'...' if len(str(v)) > 50 else v for k,v in cumulative_context.items()} }")
            
    # If all sub-goals completed successfully
    final_msg = "All sub-goals completed successfully."
    print(f"{log_prefix}{final_msg}")
    return {
        "success": True, "message": final_msg, "overall_goal": overall_goal_description,
        "sub_goal_results": sub_goal_results_list, "final_cumulative_context": cumulative_context,
        "clarification_request_for_sub_goal": None
    }

# --- Register New Tool ---
print("\nAgent: Registering multi-step sequential goal execution tool (v1)...")
if 'add_tool' in globals() and callable(add_tool):
    add_tool(
        name="execute_multi_step_goal_v1",
        tool_callable=execute_multi_step_goal_v1_tool,
        description="(Version 1) Executes a sequence of sub-goals. Propagates context and handles clarifications."
    )
else:
    print("CRITICAL ERROR: 'add_tool' function not found.")

# --- Verify Registration ---
print("\nAgent: Verifying tool registration:")
if 'list_tools' in _AGENT_TOOLS and callable(_AGENT_TOOLS['list_tools']):
    _AGENT_TOOLS['list_tools']()

# --- Test Multi-Step Goal Execution Tool ---
print("\nAgent: Starting tests for multi-step goal execution tool (v1).")

tool_to_test_name = "execute_multi_step_goal_v1"
if tool_to_test_name not in _AGENT_TOOLS:
    print(f"Agent: Skipping tests as '{tool_to_test_name}' tool was not registered.")
else:
    # Test 1: Successful multi-step goal: Download content, save to file, read file back
    overall_goal1 = "Download a specific todo item, save its title to a file, then read that file and store its content in knowledge."
    sub_goals1 = [
        "Download content from 'https://jsonplaceholder.typicode.com/todos/2'",
        "Write the 'title' from the 'last_json_content' to a file named 'todo_title.txt'", # Uses context
        "Read the file 'todo_title.txt'", # Uses context (implicitly, could also use last_filepath)
        "Save the 'last_content' (from reading the file) into knowledge with key 'downloaded_todo_title'" # Uses context
    ]
    print(f"\n--- Test 1 (Multi-step): Goal: '{overall_goal1}' ---")
    result1 = _AGENT_TOOLS[tool_to_test_name](overall_goal_description=overall_goal1, sub_goal_strings=sub_goals1)
    print(f"Result for multi-step goal 1: {json.dumps(result1, default=str, indent=2)}")

    if result1.get('success'):
        print("  SUCCESS (Test 1): Multi-step goal reported success.")
        # Verify by checking knowledge store
        if 'retrieve_knowledge' in _AGENT_TOOLS:
            knowledge_val = _AGENT_TOOLS['retrieve_knowledge'](key='downloaded_todo_title')
            if knowledge_val.get('found') and len(knowledge_val.get('value','')) > 0: # Check if non-empty string
                print(f"  VERIFIED (Test 1): Knowledge 'downloaded_todo_title' contains: '{str(knowledge_val.get('value'))[:50]}...'")
            else:
                print(f"  VERIFICATION FAILED (Test 1): Knowledge 'downloaded_todo_title' not found or empty. Value: {knowledge_val.get('value')}")
            # Cleanup knowledge
            _AGENT_TOOLS['delete_knowledge'](key='downloaded_todo_title')
        # Cleanup file
        if 'delete_file' in _AGENT_TOOLS: _AGENT_TOOLS['delete_file'](filepath='todo_title.txt')
    else:
        print(f"  FAILURE (Test 1): Multi-step goal failed. Message: {result1.get('message')}")
        if result1.get('clarification_request_for_sub_goal'):
            print(f"  Clarification needed for sub-goal index {result1['clarification_request_for_sub_goal']['sub_goal_index']}: {result1['clarification_request_for_sub_goal']['sub_goal_description']}")


    # Test 2: Multi-step goal where a sub-goal needs clarification
    overall_goal2 = "Create a directory then write something to a file inside it."
    sub_goals2 = [
        "Create a new directory named 'multi_step_test_dir'",
        "Write text 'hello from multi-step' to a file." # Intentionally vague about filename to trigger clarification
    ]
    print(f"\n--- Test 2 (Multi-step): Goal: '{overall_goal2}' (Expect clarification) ---")
    result2 = _AGENT_TOOLS[tool_to_test_name](overall_goal_description=overall_goal2, sub_goal_strings=sub_goals2)
    print(f"Result for multi-step goal 2: {json.dumps(result2, default=str, indent=2)}")

    if not result2.get('success') and result2.get('clarification_request_for_sub_goal'):
        clar_req = result2['clarification_request_for_sub_goal']
        if clar_req['sub_goal_index'] == 1 and \
           clar_req['clarification_details']['missing_params_details'][0]['name'] == 'filepath':
            print("  SUCCESS (Test 2): Multi-step goal correctly requested clarification for 'filepath' in the second sub-goal.")
        else:
            print(f"  FAILURE (Test 2): Clarification request was for an unexpected sub-goal or parameter. Details: {clar_req}")
    else:
        print(f"  FAILURE (Test 2): Did not request clarification as expected. Result success: {result2.get('success')}")
    
    # Cleanup from Test 2 if dir was created
    if 'delete_directory' in _AGENT_TOOLS:
         # Check if dir exists before trying to delete (using another tool or os.path)
         # For simplicity in test, just try to delete. delete_directory handles non-existence gracefully.
        _AGENT_TOOLS['delete_directory'](path='multi_step_test_dir', recursive=True)
        print("  Cleanup attempt for 'multi_step_test_dir'.")


    # Test 3: Responding to the clarification from Test 2
    if not result2.get('success') and result2.get('clarification_request_for_sub_goal') and result2['clarification_request_for_sub_goal']['sub_goal_index'] == 1:
        print(f"\n--- Test 3 (Multi-step): Goal: '{overall_goal2}' (Responding to clarification) ---")
        
        # The planner doesn't automatically merge the clarification. The user/calling system would do this.
        # Here, we simulate it by crafting the `initial_context` for the *entire plan* based on the clarification.
        # A more advanced planner might resume from the point of clarification.
        # For this v1, we'll just pre-fill what was asked for the specific sub-goal.
        # We can't directly pass context to a *specific sub-goal* via initial_context for the whole plan.
        # So, we modify the sub-goal string itself, or rely on the planner to pass context correctly.
        
        # Let's assume the user provides the filepath for the second sub-goal,
        # and we pass it as initial context to the overall plan.
        # The context passing logic within the planner should then pick this up for the relevant sub-goal if designed well.
        # However, our current `additional_context` in `attempt_goal_autonomously_v3` is for *that specific call*.
        # The planner's `cumulative_context` is better for this.
        # Let's simulate by re-running with a more specific sub-goal 2.

        modified_sub_goals2 = [
            "Create a new directory named 'multi_step_test_dir_clarified'", # New dir name for this test
            "Write text 'hello from clarified multi-step' to a file named 'clarified_output.txt' inside 'multi_step_test_dir_clarified'"
        ]
        
        result3 = _AGENT_TOOLS[tool_to_test_name](overall_goal_description=overall_goal2 + " (clarified)", sub_goal_strings=modified_sub_goals2)
        print(f"Result for multi-step goal 3 (clarified): {json.dumps(result3, default=str, indent=2)}")

        if result3.get('success'):
            print("  SUCCESS (Test 3): Multi-step goal (clarified) completed successfully.")
            # Verify file was created
            if 'read_file' in _AGENT_TOOLS:
                read_res = _AGENT_TOOLS['read_file'](filepath='multi_step_test_dir_clarified/clarified_output.txt')
                if read_res.get('success') and read_res.get('content') == 'hello from clarified multi-step':
                    print("  VERIFIED (Test 3): File content is correct.")
                else:
                    print(f"  VERIFICATION FAILED (Test 3): File content incorrect or not found. Read result: {read_res}")
        else:
            print(f"  FAILURE (Test 3): Clarified multi-step goal failed. Message: {result3.get('message')}")
        
        # Cleanup for Test 3
        if 'delete_directory' in _AGENT_TOOLS:
            _AGENT_TOOLS['delete_directory'](path='multi_step_test_dir_clarified', recursive=True)
            print("  Cleanup attempt for 'multi_step_test_dir_clarified'.")
    else:
        print("  SKIPPED Test 3: Test 2 did not produce the expected clarification request.")

print("\nAgent: Multi-step sequential goal execution tool (v1) definition, registration, and tests complete.")
print("Waiting for next instruction.")

Agent: Defining and registering a multi-step sequential goal execution tool (v1).

Agent: Registering multi-step sequential goal execution tool (v1)...
Tool 'execute_multi_step_goal_v1' added successfully. Description: (Version 1) Executes a sequence of sub-goals. Propagates context and handles clarifications.

Agent: Verifying tool registration:

Available tools:
1. list_tools: Lists all available tools with their names and descriptions. Useful for understanding agent capabilities.
2. get_agent_status: Provides a status report of the agent, including version and number of loaded tools.
3. execute_python_code: Executes a given string of Python code in the agent's current global environment. Captures and returns stdout, stderr, and any execution error from the provided code. USE WITH EXTREME CAUTION, as it can alter the agent's state and capabilities.
4. write_file: (Version 2) Writes text content to a specified file. Overwrites if exists, creates directories if needed.
5. read_file: (V

In [17]:
# _AGENT_TOOLS, _TOOL_DESCRIPTIONS, add_tool, suggest_tools_for_goal_v2,
# _extract_parameter_value_heuristically_v2, attempt_goal_autonomously_v3_tool
# etc. are assumed to be in the global scope.
import inspect
import re
import json
import traceback

print("Agent: Refining tool suggestion and parameter extraction for multi-step planner (FIX 2).")

# --- 1. Update Concept Keywords ---
# (Assuming _AGENT_TOOL_CONCEPT_KEYWORDS is in global scope and mutable)
if '_AGENT_TOOL_CONCEPT_KEYWORDS' in globals():
    _AGENT_TOOL_CONCEPT_KEYWORDS["make_http_request"].update([
        "url", "uri", "from web", "fetch online", "get from internet", "retrieve from url"
    ])
    # Also, give more weight to the direct action word 'download' in the scoring of suggest_tools or ensure it's handled.
    # The current scoring in suggest_tools_for_goal_v2 already gives concept keywords high weight.
    
    # For create_directory, ensure "named" or "called" are useful contextual hints for path extraction,
    # though this is more for the heuristic extractor.
    _AGENT_TOOL_CONCEPT_KEYWORDS["create_directory"].update(["directory named", "folder called"])
    print("Agent log: Updated _AGENT_TOOL_CONCEPT_KEYWORDS for 'make_http_request' and 'create_directory'.")
else:
    print("Agent Warning: _AGENT_TOOL_CONCEPT_KEYWORDS not found in globals. Cannot update.")


# --- 2. Refine Parameter Extraction Heuristic (v3) ---
def _extract_parameter_value_heuristically_v3(
    param_name: str, 
    param_type, 
    goal_description: str, 
    all_quoted_strings_from_goal: list[str],
    consumed_quoted_strings: set[str] 
) -> any:
    # This function reuses much of _v2, with specific enhancements for path and disambiguation.
    goal_lower = goal_description.lower()

    # Check for explicit assignment first (e.g. path='value') as it's a strong signal
    explicit_assign_match_quoted = re.search(rf"\b{param_name}\s*[:=]\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
    if explicit_assign_match_quoted:
        val_str = explicit_assign_match_quoted.group(1)
        if val_str not in consumed_quoted_strings:
            # consumed_quoted_strings.add(val_str) # No, consumption is only for general quoted strings
            return val_str
            
    explicit_assign_match_unquoted = re.search(rf"\b{param_name}\s*[:=]\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
    if explicit_assign_match_unquoted:
        val_str = explicit_assign_match_unquoted.group(1)
        # Type conversion for primitive types if type is known
        if param_type == int: try: return int(val_str) except ValueError: pass
        if param_type == float: try: return float(val_str) except ValueError: pass
        if param_type == bool:
            if val_str.lower() == 'true': return True
            if val_str.lower() == 'false': return False
        if param_type == str or param_type is None or param_type == inspect.Parameter.empty : return val_str
        # If type conversion failed but it's a string param, let it pass as string

    # 1. URL Extraction (and cleaning) - same as v2
    if param_name == "url":
        url_match_re = r"https?://(?:[a-zA-Z0-9.\-_~:/?#\[\]@!$&'()*+,;=%]+[a-zA-Z0-9\-_~:/?#\[\]@!$&*+,;=%])"
        url_match = re.search(url_match_re, goal_description)
        if url_match:
            extracted_url = url_match.group(0)
            if extracted_url.endswith("'") and goal_description.count(f"'{extracted_url}'") > 0: extracted_url = extracted_url[:-1]
            elif extracted_url.endswith("\"") and goal_description.count(f"\"{extracted_url}\"") > 0: extracted_url = extracted_url[:-1]
            for q_str in all_quoted_strings_from_goal: # Mark as consumed if it was a quoted URL
                if extracted_url == q_str or f"'{extracted_url}'" == q_str or f"\"{extracted_url}\"" == q_str :
                    if q_str not in consumed_quoted_strings: consumed_quoted_strings.add(q_str); break 
            return extracted_url

    # 2. Filepath / Path / Directory from Quoted Strings (Enhanced)
    if param_name in ["filepath", "path", "directory"]:
        # Look for patterns like "named 'X'", "called 'X'", or just a quote if it's a path-like param.
        path_pattern_named = re.search(rf"(?:named|called|path is|directory is)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if path_pattern_named:
            potential_path = path_pattern_named.group(1)
            if potential_path in all_quoted_strings_from_goal and potential_path not in consumed_quoted_strings:
                consumed_quoted_strings.add(potential_path)
                return potential_path
        
        # General check for remaining unconsumed quoted strings that look like paths
        path_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            # Relaxed condition for directory names (don't strictly require '.', '/', etc.)
            if param_name == "directory": # For "directory", any unconsumed quote is a candidate
                path_candidates.append(q_str)
            elif '.' in q_str or '/' in q_str or '\\' in q_str or \
                 any(q_str.endswith(ext) for ext in ['.txt', '.json', '.py', '.html', '.md']):
                path_candidates.append(q_str)
        
        if path_candidates:
            chosen_path = max(path_candidates, key=len) 
            consumed_quoted_strings.add(chosen_path)
            return chosen_path

    # 3. Content / Text / Description / Message (same as v2, but respects consumed_quoted_strings)
    if param_name in ["content", "text", "description", "message"]:
        content_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            content_candidates.append(q_str)
        if content_candidates:
            chosen_content = max(content_candidates, key=len)
            consumed_quoted_strings.add(chosen_content)
            return chosen_content
            
    # 4. Code String (same as v2)
    if param_name == "code_string":
        code_block_match = re.search(r"code:\s*(.+)", goal_description, re.IGNORECASE | re.DOTALL)
        if code_block_match: return code_block_match.group(1).strip()
        triple_tick_match = re.search(r"```(?:python\n)?(.*?)```", goal_description, re.DOTALL | re.IGNORECASE)
        if triple_tick_match: return triple_tick_match.group(1).strip()

    # 5. HTTP Method (same as v2)
    if param_name == "method" and (param_type == str or param_type is None or param_type == inspect.Parameter.empty):
        method_search = re.search(r"\b(GET|POST|PUT|DELETE|PATCH|HEAD|OPTIONS)\b", goal_description, re.IGNORECASE)
        if method_search: return method_search.group(1).upper()
        # ... (fallback keyword checks for method) ...
        if "post" in goal_lower: return "POST"
        if "get" in goal_lower: return "GET"
        if "delete" in goal_lower: return "DELETE"
        if "put" in goal_lower: return "PUT"


    # 6. Boolean Flags (same as v2)
    if param_type == bool:
        if re.search(rf"\b{param_name}\b\s*(true|yes|on|recursively|enabled)\b", goal_lower, re.IGNORECASE) or \
           re.search(rf"\b(is|be|make it)\s*{param_name}\b", goal_lower, re.IGNORECASE):
            return True
        if re.search(rf"\b{param_name}\b\s*(false|no|off|disabled)\b", goal_lower, re.IGNORECASE) or \
           re.search(rf"\b(is not|not to be|don't make it)\s*{param_name}\b", goal_lower, re.IGNORECASE):
            return False
        if param_name in goal_lower and param_name not in ["false", "no", "off"]:
            return True # Simple presence of the flag name implies true

    return None


# --- 3. Update attempt_goal_autonomously (v4) to use new heuristic ---
def attempt_goal_autonomously_v4_tool(goal_description: str, additional_context: dict = None) -> dict:
    """
    (Version 4) Uses _extract_parameter_value_heuristically_v3.
    """
    tool_name_internal = "attempt_goal_autonomously_v4_tool"
    log_prefix = f"Agent log ({tool_name_internal}): "
    # ... (Setup and Tool Selection logic largely same as v3) ...
    print(f"{log_prefix}Attempting goal (v4): '{goal_description[:100]}...'. Additional context: {additional_context is not None}")
    if additional_context is None: additional_context = {}
    quoted_match_tuples = re.findall(r"\"([^\"]+)\"|\'([^\']+)\'", goal_description)
    all_quoted_strings_from_goal = [item for tpl in quoted_match_tuples for item in tpl if item]
    if 'suggest_tools_for_goal' not in _AGENT_TOOLS: return {"success": False, "message": "Tool 'suggest_tools_for_goal' not found."} # Simplified return
    
    chosen_tool_name = additional_context.get("chosen_tool_override")
    tool_callable = None
    if chosen_tool_name: # Tool pre-selected
        if chosen_tool_name in _AGENT_TOOLS: tool_callable = _AGENT_TOOLS[chosen_tool_name]; print(f"{log_prefix}Using pre-selected tool: '{chosen_tool_name}'.")
        else: return {"success": False, "message": f"Pre-selected tool '{chosen_tool_name}' not found."}
    else: # Suggest tool
        suggestion_result = _AGENT_TOOLS['suggest_tools_for_goal'](goal_description=goal_description, top_n=1)
        print(f"{log_prefix}Tool suggestion: {suggestion_result['message']}")
        if not suggestion_result['success'] or not suggestion_result['suggestions']: return {"success": False, "message": "No suitable tools suggested."}
        top_suggestion = suggestion_result['suggestions'][0]
        chosen_tool_name = top_suggestion['tool_name']
        relevance_score = top_suggestion['relevance_score']
        print(f"{log_prefix}Top suggested: '{chosen_tool_name}' (Score: {relevance_score}). Reason: {top_suggestion.get('reason', 'N/A')}")
        if relevance_score < _MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION_V3: return {"success": False, "message": f"Tool score {relevance_score} below threshold."}
        if chosen_tool_name not in _AGENT_TOOLS: return {"success": False, "message": f"Suggested tool '{chosen_tool_name}' not found."}
        tool_callable = _AGENT_TOOLS[chosen_tool_name]

    sig = inspect.signature(tool_callable)
    filled_params = {}
    missing_required_params_info = []
    consumed_quoted_strings_for_this_call = set()
    print(f"{log_prefix}Params for '{chosen_tool_name}': {list(sig.parameters.keys())}")
    param_order = sorted(sig.parameters.items(), key=lambda item: 1 if item[0] in ["filepath", "path", "url", "directory"] else 2) # Ensure directory is prioritized

    for param_name, param_obj in param_order:
        param_type_annotation = param_obj.annotation if param_obj.annotation != inspect.Parameter.empty else None
        if param_name in additional_context.get("parameters", {}):
            extracted_value = additional_context["parameters"][param_name]; print(f"{log_prefix}  Using '{param_name}' from context.")
        else:
            extracted_value = _extract_parameter_value_heuristically_v3( # USE V3 HEURISTIC
                param_name, param_type_annotation, goal_description, 
                all_quoted_strings_from_goal, consumed_quoted_strings_for_this_call
            )
        if extracted_value is not None:
            # Basic type conversion
            if param_type_annotation and isinstance(extracted_value, str):
                try:
                    if param_type_annotation == bool: extracted_value = (extracted_value.lower() == 'true')
                    elif param_type_annotation == int: extracted_value = int(extracted_value)
                    elif param_type_annotation == float: extracted_value = float(extracted_value)
                except ValueError: print(f"{log_prefix}Warn: Could not convert '{extracted_value}' to {param_type_annotation} for '{param_name}'.")
            filled_params[param_name] = extracted_value; print(f"{log_prefix}  Filled '{param_name}': {str(extracted_value)[:50]}...")
        elif param_obj.default != inspect.Parameter.empty:
            filled_params[param_name] = param_obj.default; print(f"{log_prefix}  Default for '{param_name}': {str(param_obj.default)[:50]}...")
        else:
            missing_required_params_info.append({"name": param_name, "type": str(param_type_annotation), "description": f"Value for '{param_name}'."})
            print(f"{log_prefix}  Could not fill required '{param_name}'.")
            
    if missing_required_params_info: # Clarification logic (same as v3)
        param_names_missing = [p['name'] for p in missing_required_params_info]
        msg = f"Clarification needed for tool '{chosen_tool_name}': {', '.join(param_names_missing)}."
        # ... (construct clarification_request as in v3 tool) ...
        clarification_prompt = f"Please provide values for: {', '.join(param_names_missing)} for tool '{chosen_tool_name}' (goal: '{goal_description[:30]}...')."
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, 
                "clarification_request": {"type": "missing_parameters", "original_goal": goal_description, "tool_name": chosen_tool_name, 
                                          "missing_params_details": missing_required_params_info, "prompt": clarification_prompt}}
    
    # Execution logic (same as v3)
    print(f"{log_prefix}Executing '{chosen_tool_name}' with: {json.dumps(filled_params, default=str, indent=2)}")
    try:
        execution_result = tool_callable(**filled_params)
        overall_success = execution_result.get('success', True) if isinstance(execution_result, dict) else True
        return {"success": overall_success, "message": f"Tool '{chosen_tool_name}' executed.", "chosen_tool": chosen_tool_name,
                "parameters_used": filled_params, "execution_result": execution_result}
    except Exception as e:
        # ... (error handling as in v3 tool) ...
        tb_str = traceback.format_exc()
        return {"success": False, "message": f"Error executing '{chosen_tool_name}': {e}", "execution_result": {"error": f"{e}\n{tb_str}"}}


# --- Register New Tools (attempt_goal_autonomously_v4, and update multi_step_goal to use it) ---
print("\nAgent: Registering autonomous goal execution tool (v4) and updating multi-step planner...")
if 'add_tool' in globals() and callable(add_tool):
    add_tool(
        name="attempt_goal_autonomously_v4",
        tool_callable=attempt_goal_autonomously_v4_tool,
        description="(Version 4) Uses v3 heuristic for param extraction. Asks for clarification."
    )
    # For tests, the multi-step planner will now internally use the v4 autonomous tool.
    # We need to update the multi_step_goal_v1_tool's internal call if it directly calls by name.
    # The previous multi_step_goal_v1_tool calls `_AGENT_TOOLS['attempt_goal_autonomously_v3']`.
    # We can alias v3 to v4 for the purpose of the test, or make multi_step_goal_v1 configurable.
    # For simplicity in this turn, let's assume the tests for multi-step will pick up changes if we
    # make 'attempt_goal_autonomously_v3' point to v4.
    _AGENT_TOOLS['attempt_goal_autonomously_v3'] = attempt_goal_autonomously_v4_tool
    _TOOL_DESCRIPTIONS['attempt_goal_autonomously_v3'] = "(Version 4, aliased as v3 - FIX 2) Uses v3 heuristic. Asks for clarification."
    print("Registered 'attempt_goal_autonomously_v4' and aliased 'attempt_goal_autonomously_v3' to point to v4.")

else:
    print("CRITICAL ERROR: 'add_tool' function not found.")

# --- Re-test Multi-Step Goal Execution Tool (it will now use the v4 autonomous tool internally via the v3 alias) ---
print("\nAgent: Re-testing multi-step goal execution tool (v1) with updated autonomous worker (v4 via v3 alias).")

tool_to_test_name = "execute_multi_step_goal_v1" # This planner uses 'attempt_goal_autonomously_v3' internally
if tool_to_test_name not in _AGENT_TOOLS:
    print(f"Agent: Skipping tests as '{tool_to_test_name}' tool was not registered.")
else:
    # Test 1: Successful multi-step goal (Download, save title, read file, save knowledge)
    overall_goal1 = "Download todo item 2, save its title to 'todo_title_v2.txt', read file, store content in knowledge as 'todo_title_v2'."
    sub_goals1 = [
        "Download content from 'https://jsonplaceholder.typicode.com/todos/2'", # Should pick make_http_request
        "Write the 'title' from the 'last_json_content' to a file named 'todo_title_v2.txt'",
        "Read the file 'todo_title_v2.txt'",
        "Save the 'last_content' into knowledge with key 'todo_title_v2'"
    ]
    print(f"\n--- Test 1 (Multi-step REVISED): Goal: '{overall_goal1}' ---")
    result1 = _AGENT_TOOLS[tool_to_test_name](overall_goal_description=overall_goal1, sub_goal_strings=sub_goals1)
    print(f"Result for multi-step goal 1: {json.dumps(result1, default=str, indent=2)}")
    if result1.get('success'):
        print("  SUCCESS (Test 1 REVISED): Multi-step goal reported success.")
        if 'retrieve_knowledge' in _AGENT_TOOLS:
            kv = _AGENT_TOOLS['retrieve_knowledge'](key='todo_title_v2')
            if kv.get('found') and kv.get('value') == "quis ut nam facilis et officia qui": # Expected title of todo #2
                print(f"  VERIFIED (Test 1 REVISED): Knowledge 'todo_title_v2' correct: '{str(kv.get('value'))[:50]}...'")
            else: print(f"  VERIFICATION FAILED (Test 1 REVISED): Knowledge 'todo_title_v2'. Got: {kv.get('value')}")
            _AGENT_TOOLS['delete_knowledge'](key='todo_title_v2')
        if 'delete_file' in _AGENT_TOOLS: _AGENT_TOOLS['delete_file'](filepath='todo_title_v2.txt')
    else:
        print(f"  FAILURE (Test 1 REVISED): Multi-step goal failed. Message: {result1.get('message')}")

    # Test 2: Multi-step goal needing clarification (create dir, then write file)
    overall_goal2 = "Create a directory then write something to a file inside it (v2 test)."
    sub_goals2 = [
        "Create a new directory named 'multi_step_test_dir_v2'", # Should extract path correctly now
        "Write text 'hello from multi-step v2' to a file." # Should ask for filepath
    ]
    print(f"\n--- Test 2 (Multi-step REVISED): Goal: '{overall_goal2}' (Expect clarification on 2nd step) ---")
    result2 = _AGENT_TOOLS[tool_to_test_name](overall_goal_description=overall_goal2, sub_goal_strings=sub_goals2)
    print(f"Result for multi-step goal 2: {json.dumps(result2, default=str, indent=2)}")
    if not result2.get('success') and result2.get('clarification_request_for_sub_goal'):
        clar_req = result2['clarification_request_for_sub_goal']
        # Sub-goal 0 (create dir) should succeed. Sub-goal 1 (write file) should ask for filepath.
        if clar_req['sub_goal_index'] == 1 and \
           clar_req['clarification_details']['missing_params_details'][0]['name'] == 'filepath':
            print("  SUCCESS (Test 2 REVISED): Correctly requested clarification for 'filepath' in the second sub-goal.")
            # Verify first sub-goal (create_directory) was successful
            if len(result2.get('sub_goal_results', [])) > 0 and result2['sub_goal_results'][0].get('success'):
                print("  VERIFIED (Test 2 REVISED): First sub-goal (create_directory) was successful.")
            else:
                 print("  VERIFICATION FAILED (Test 2 REVISED): First sub-goal did not succeed as expected.")
        else:
            print(f"  FAILURE (Test 2 REVISED): Clarification for unexpected sub-goal/param. Details: {clar_req}")
    else:
        print(f"  FAILURE (Test 2 REVISED): Did not request clarification. Result success: {result2.get('success')}")
    
    if 'delete_directory' in _AGENT_TOOLS:
        _AGENT_TOOLS['delete_directory'](path='multi_step_test_dir_v2', recursive=True)
        print("  Cleanup attempt for 'multi_step_test_dir_v2'.")


print("\nAgent: Revisions for multi-step planner (FIX 2) and tests complete.")
print("Waiting for next instruction.")

SyntaxError: invalid syntax (4286678561.py, line 51)

In [18]:
# _AGENT_TOOLS, _TOOL_DESCRIPTIONS, add_tool, suggest_tools_for_goal_v2,
# _extract_parameter_value_heuristically_v2 (from a previous successful turn, now effectively replaced by v3)
# attempt_goal_autonomously_v3_tool (from a previous successful turn, now effectively replaced by v4)
# etc. are assumed to be in the global scope.
import inspect
import re
import json
import traceback

print("Agent: Refining tool suggestion and parameter extraction for multi-step planner (FIX 3).")

# --- 1. Update Concept Keywords (if not already done or to ensure) ---
if '_AGENT_TOOL_CONCEPT_KEYWORDS' in globals():
    _AGENT_TOOL_CONCEPT_KEYWORDS.setdefault("make_http_request", set()).update([
        "url", "uri", "from web", "fetch online", "get from internet", "retrieve from url"
    ])
    _AGENT_TOOL_CONCEPT_KEYWORDS.setdefault("create_directory", set()).update(["directory named", "folder called"])
    print("Agent log: Ensured/Updated _AGENT_TOOL_CONCEPT_KEYWORDS for 'make_http_request' and 'create_directory'.")
else:
    print("Agent Warning: _AGENT_TOOL_CONCEPT_KEYWORDS not found in globals. Cannot update concepts.")


# --- 2. Refine Parameter Extraction Heuristic (v3 - with syntax fix) ---
def _extract_parameter_value_heuristically_v3(
    param_name: str, 
    param_type, 
    goal_description: str, 
    all_quoted_strings_from_goal: list[str],
    consumed_quoted_strings: set[str] 
) -> any:
    goal_lower = goal_description.lower()

    explicit_assign_match_quoted = re.search(rf"\b{param_name}\s*[:=]\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
    if explicit_assign_match_quoted:
        val_str = explicit_assign_match_quoted.group(1)
        if val_str not in consumed_quoted_strings:
            return val_str
            
    explicit_assign_match_unquoted = re.search(rf"\b{param_name}\s*[:=]\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
    if explicit_assign_match_unquoted:
        val_str = explicit_assign_match_unquoted.group(1)
        if param_type == int:
            try:
                return int(val_str)
            except ValueError:
                pass
        if param_type == float:
            try:
                return float(val_str)
            except ValueError:
                pass
        if param_type == bool:
            if val_str.lower() == 'true': return True
            if val_str.lower() == 'false': return False
        if param_type == str or param_type is None or param_type == inspect.Parameter.empty : return val_str

    if param_name == "url":
        url_match_re = r"https?://(?:[a-zA-Z0-9.\-_~:/?#\[\]@!$&'()*+,;=%]+[a-zA-Z0-9\-_~:/?#\[\]@!$&*+,;=%])"
        url_match = re.search(url_match_re, goal_description)
        if url_match:
            extracted_url = url_match.group(0)
            if extracted_url.endswith("'") and goal_description.count(f"'{extracted_url}'") > 0: extracted_url = extracted_url[:-1]
            elif extracted_url.endswith("\"") and goal_description.count(f"\"{extracted_url}\"") > 0: extracted_url = extracted_url[:-1]
            for q_str in all_quoted_strings_from_goal:
                if extracted_url == q_str or f"'{extracted_url}'" == q_str or f"\"{extracted_url}\"" == q_str :
                    if q_str not in consumed_quoted_strings: consumed_quoted_strings.add(q_str); break 
            return extracted_url

    if param_name in ["filepath", "path", "directory"]:
        path_pattern_named = re.search(rf"(?:named|called|path is|directory is)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if path_pattern_named:
            potential_path = path_pattern_named.group(1)
            if potential_path in all_quoted_strings_from_goal and potential_path not in consumed_quoted_strings:
                consumed_quoted_strings.add(potential_path)
                return potential_path
        path_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            if param_name == "directory":
                path_candidates.append(q_str)
            elif '.' in q_str or '/' in q_str or '\\' in q_str or \
                 any(q_str.endswith(ext) for ext in ['.txt', '.json', '.py', '.html', '.md']):
                path_candidates.append(q_str)
        if path_candidates:
            chosen_path = max(path_candidates, key=len) 
            consumed_quoted_strings.add(chosen_path)
            return chosen_path

    if param_name in ["content", "text", "description", "message"]:
        content_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            content_candidates.append(q_str)
        if content_candidates:
            chosen_content = max(content_candidates, key=len)
            consumed_quoted_strings.add(chosen_content)
            return chosen_content
            
    if param_name == "code_string":
        code_block_match = re.search(r"code:\s*(.+)", goal_description, re.IGNORECASE | re.DOTALL)
        if code_block_match: return code_block_match.group(1).strip()
        triple_tick_match = re.search(r"```(?:python\n)?(.*?)```", goal_description, re.DOTALL | re.IGNORECASE)
        if triple_tick_match: return triple_tick_match.group(1).strip()

    if param_name == "method" and (param_type == str or param_type is None or param_type == inspect.Parameter.empty):
        method_search = re.search(r"\b(GET|POST|PUT|DELETE|PATCH|HEAD|OPTIONS)\b", goal_description, re.IGNORECASE)
        if method_search: return method_search.group(1).upper()
        if "post" in goal_lower: return "POST"
        if "get" in goal_lower: return "GET"
        if "delete" in goal_lower: return "DELETE"
        if "put" in goal_lower: return "PUT"

    if param_type == bool:
        if re.search(rf"\b{param_name}\b\s*(true|yes|on|recursively|enabled)\b", goal_lower, re.IGNORECASE) or \
           re.search(rf"\b(is|be|make it)\s*{param_name}\b", goal_lower, re.IGNORECASE):
            return True
        if re.search(rf"\b{param_name}\b\s*(false|no|off|disabled)\b", goal_lower, re.IGNORECASE) or \
           re.search(rf"\b(is not|not to be|don't make it)\s*{param_name}\b", goal_lower, re.IGNORECASE):
            return False
        if param_name in goal_lower and param_name not in ["false", "no", "off"]:
            return True
    return None

# --- 3. Update attempt_goal_autonomously (v4) to use new heuristic ---
def attempt_goal_autonomously_v4_tool(goal_description: str, additional_context: dict = None) -> dict:
    tool_name_internal = "attempt_goal_autonomously_v4_tool"
    log_prefix = f"Agent log ({tool_name_internal}): "
    print(f"{log_prefix}Attempting goal (v4): '{goal_description[:100]}...'. Additional context: {additional_context is not None}")
    if additional_context is None: additional_context = {}
    quoted_match_tuples = re.findall(r"\"([^\"]+)\"|\'([^\']+)\'", goal_description)
    all_quoted_strings_from_goal = [item for tpl in quoted_match_tuples for item in tpl if item]
    if 'suggest_tools_for_goal' not in _AGENT_TOOLS: return {"success": False, "message": "Tool 'suggest_tools_for_goal' not found."}
    
    chosen_tool_name = additional_context.get("chosen_tool_override")
    tool_callable = None
    if chosen_tool_name:
        if chosen_tool_name in _AGENT_TOOLS: tool_callable = _AGENT_TOOLS[chosen_tool_name]; print(f"{log_prefix}Using pre-selected tool: '{chosen_tool_name}'.")
        else: return {"success": False, "message": f"Pre-selected tool '{chosen_tool_name}' not found."}
    else:
        suggestion_result = _AGENT_TOOLS['suggest_tools_for_goal'](goal_description=goal_description, top_n=1)
        print(f"{log_prefix}Tool suggestion: {suggestion_result['message']}")
        if not suggestion_result['success'] or not suggestion_result['suggestions']: return {"success": False, "message": "No suitable tools suggested."}
        top_suggestion = suggestion_result['suggestions'][0]
        chosen_tool_name = top_suggestion['tool_name']
        relevance_score = top_suggestion['relevance_score']
        print(f"{log_prefix}Top suggested: '{chosen_tool_name}' (Score: {relevance_score}). Reason: {top_suggestion.get('reason', 'N/A')}")
        # Using _MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION_V3 as threshold name, assuming value is same for v4
        if relevance_score < _MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION_V3: return {"success": False, "message": f"Tool score {relevance_score} below threshold."}
        if chosen_tool_name not in _AGENT_TOOLS: return {"success": False, "message": f"Suggested tool '{chosen_tool_name}' not found."}
        tool_callable = _AGENT_TOOLS[chosen_tool_name]

    sig = inspect.signature(tool_callable)
    filled_params = {}
    missing_required_params_info = []
    consumed_quoted_strings_for_this_call = set()
    print(f"{log_prefix}Params for '{chosen_tool_name}': {list(sig.parameters.keys())}")
    param_order = sorted(sig.parameters.items(), key=lambda item: 1 if item[0] in ["filepath", "path", "url", "directory"] else 2)

    for param_name, param_obj in param_order:
        param_type_annotation = param_obj.annotation if param_obj.annotation != inspect.Parameter.empty else None
        if param_name in additional_context.get("parameters", {}):
            extracted_value = additional_context["parameters"][param_name]; print(f"{log_prefix}  Using '{param_name}' from context.")
        else:
            extracted_value = _extract_parameter_value_heuristically_v3(
                param_name, param_type_annotation, goal_description, 
                all_quoted_strings_from_goal, consumed_quoted_strings_for_this_call
            )
        if extracted_value is not None:
            if param_type_annotation and isinstance(extracted_value, str):
                try:
                    if param_type_annotation == bool: extracted_value = (extracted_value.lower() == 'true')
                    elif param_type_annotation == int: extracted_value = int(extracted_value)
                    elif param_type_annotation == float: extracted_value = float(extracted_value)
                except ValueError: print(f"{log_prefix}Warn: Could not convert '{extracted_value}' to {param_type_annotation} for '{param_name}'.")
            filled_params[param_name] = extracted_value; print(f"{log_prefix}  Filled '{param_name}': {str(extracted_value)[:50]}...")
        elif param_obj.default != inspect.Parameter.empty:
            filled_params[param_name] = param_obj.default; print(f"{log_prefix}  Default for '{param_name}': {str(param_obj.default)[:50]}...")
        else:
            missing_required_params_info.append({"name": param_name, "type": str(param_type_annotation), "description": f"Value for '{param_name}'."})
            print(f"{log_prefix}  Could not fill required '{param_name}'.")
            
    if missing_required_params_info:
        param_names_missing = [p['name'] for p in missing_required_params_info]
        msg = f"Clarification needed for tool '{chosen_tool_name}': {', '.join(param_names_missing)}."
        clarification_prompt = f"Please provide values for: {', '.join(param_names_missing)} for tool '{chosen_tool_name}' (goal: '{goal_description[:30]}...')."
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, 
                "clarification_request": {"type": "missing_parameters", "original_goal": goal_description, "tool_name": chosen_tool_name, 
                                          "missing_params_details": missing_required_params_info, "prompt": clarification_prompt}}
    
    print(f"{log_prefix}Executing '{chosen_tool_name}' with: {json.dumps(filled_params, default=str, indent=2)}")
    try:
        execution_result = tool_callable(**filled_params)
        overall_success = execution_result.get('success', True) if isinstance(execution_result, dict) else True
        return {"success": overall_success, "message": f"Tool '{chosen_tool_name}' executed.", "chosen_tool": chosen_tool_name,
                "parameters_used": filled_params, "execution_result": execution_result}
    except Exception as e:
        tb_str = traceback.format_exc()
        return {"success": False, "message": f"Error executing '{chosen_tool_name}': {e}", "execution_result": {"error": f"{e}\n{tb_str}"}}

# --- Register New Tools ---
print("\nAgent: Registering autonomous goal execution tool (v4 - FIX 3) and updating aliases...")
if 'add_tool' in globals() and callable(add_tool):
    add_tool(
        name="attempt_goal_autonomously_v4",
        tool_callable=attempt_goal_autonomously_v4_tool,
        description="(Version 4) Uses v3 heuristic for param extraction with syntax fix. Asks for clarification."
    )
    _AGENT_TOOLS['attempt_goal_autonomously_v3'] = attempt_goal_autonomously_v4_tool
    _TOOL_DESCRIPTIONS['attempt_goal_autonomously_v3'] = "(Version 4, aliased as v3 - FIX 3) Uses v3 heuristic with syntax fix. Asks for clarification."
    print("Registered 'attempt_goal_autonomously_v4' and aliased 'attempt_goal_autonomously_v3' to point to v4.")
else:
    print("CRITICAL ERROR: 'add_tool' function not found.")

# --- Re-test Multi-Step Goal Execution Tool ---
print("\nAgent: Re-testing multi-step goal execution tool (v1) with updated autonomous worker (v4 via v3 alias - FIX 3).")
tool_to_test_name = "execute_multi_step_goal_v1"
if tool_to_test_name not in _AGENT_TOOLS:
    print(f"Agent: Skipping tests as '{tool_to_test_name}' tool was not registered.")
else:
    overall_goal1 = "Download todo item 2, save its title to 'todo_title_v3.txt', read file, store content in knowledge as 'todo_title_v3'."
    sub_goals1 = [
        "Download content from 'https://jsonplaceholder.typicode.com/todos/2'",
        "Write the 'title' from the 'last_json_content' to a file named 'todo_title_v3.txt'",
        "Read the file 'todo_title_v3.txt'",
        "Save the 'last_content' into knowledge with key 'todo_title_v3'"
    ]
    print(f"\n--- Test 1 (Multi-step REVISED FIX 3): Goal: '{overall_goal1}' ---")
    result1 = _AGENT_TOOLS[tool_to_test_name](overall_goal_description=overall_goal1, sub_goal_strings=sub_goals1)
    print(f"Result for multi-step goal 1: {json.dumps(result1, default=str, indent=2)}")
    if result1.get('success'):
        print("  SUCCESS (Test 1 REVISED FIX 3): Multi-step goal reported success.")
        if 'retrieve_knowledge' in _AGENT_TOOLS:
            kv = _AGENT_TOOLS['retrieve_knowledge'](key='todo_title_v3')
            if kv.get('found') and kv.get('value') == "quis ut nam facilis et officia qui":
                print(f"  VERIFIED (Test 1 REVISED FIX 3): Knowledge 'todo_title_v3' correct: '{str(kv.get('value'))[:50]}...'")
            else: print(f"  VERIFICATION FAILED (Test 1 REVISED FIX 3): Knowledge 'todo_title_v3'. Got: {kv.get('value')}")
            _AGENT_TOOLS['delete_knowledge'](key='todo_title_v3')
        if 'delete_file' in _AGENT_TOOLS: _AGENT_TOOLS['delete_file'](filepath='todo_title_v3.txt')
    else:
        print(f"  FAILURE (Test 1 REVISED FIX 3): Multi-step goal failed. Message: {result1.get('message')}")

    overall_goal2 = "Create a directory then write something to a file inside it (v3 test)."
    sub_goals2 = [
        "Create a new directory named 'multi_step_test_dir_v3'",
        "Write text 'hello from multi-step v3' to a file."
    ]
    print(f"\n--- Test 2 (Multi-step REVISED FIX 3): Goal: '{overall_goal2}' (Expect clarification on 2nd step) ---")
    result2 = _AGENT_TOOLS[tool_to_test_name](overall_goal_description=overall_goal2, sub_goal_strings=sub_goals2)
    print(f"Result for multi-step goal 2: {json.dumps(result2, default=str, indent=2)}")
    if not result2.get('success') and result2.get('clarification_request_for_sub_goal'):
        clar_req = result2['clarification_request_for_sub_goal']
        if clar_req['sub_goal_index'] == 1 and \
           clar_req['clarification_details']['missing_params_details'][0]['name'] == 'filepath':
            print("  SUCCESS (Test 2 REVISED FIX 3): Correctly requested clarification for 'filepath' in the second sub-goal.")
            if len(result2.get('sub_goal_results', [])) > 0 and result2['sub_goal_results'][0].get('success'):
                print("  VERIFIED (Test 2 REVISED FIX 3): First sub-goal (create_directory) was successful.")
            else:
                 print("  VERIFICATION FAILED (Test 2 REVISED FIX 3): First sub-goal did not succeed as expected.")
        else:
            print(f"  FAILURE (Test 2 REVISED FIX 3): Clarification for unexpected sub-goal/param. Details: {clar_req}")
    else:
        print(f"  FAILURE (Test 2 REVISED FIX 3): Did not request clarification. Result success: {result2.get('success')}")
    
    if 'delete_directory' in _AGENT_TOOLS:
        _AGENT_TOOLS['delete_directory'](path='multi_step_test_dir_v3', recursive=True)
        print("  Cleanup attempt for 'multi_step_test_dir_v3'.")

print("\nAgent: Revisions for multi-step planner (FIX 3) and tests complete.")
print("Waiting for next instruction.")

Agent: Refining tool suggestion and parameter extraction for multi-step planner (FIX 3).
Agent log: Ensured/Updated _AGENT_TOOL_CONCEPT_KEYWORDS for 'make_http_request' and 'create_directory'.

Agent: Registering autonomous goal execution tool (v4 - FIX 3) and updating aliases...
Tool 'attempt_goal_autonomously_v4' added successfully. Description: (Version 4) Uses v3 heuristic for param extraction with syntax fix. Asks for clarification.
Registered 'attempt_goal_autonomously_v4' and aliased 'attempt_goal_autonomously_v3' to point to v4.

Agent: Re-testing multi-step goal execution tool (v1) with updated autonomous worker (v4 via v3 alias - FIX 3).

--- Test 1 (Multi-step REVISED FIX 3): Goal: 'Download todo item 2, save its title to 'todo_title_v3.txt', read file, store content in knowledge as 'todo_title_v3'.' ---
Agent log (execute_multi_step_goal_v1_tool): Starting multi-step goal: 'Download todo item 2, save its title to 'todo_title_v3.txt', read file, store content in knowledge a.

In [19]:
# _AGENT_TOOLS, _TOOL_DESCRIPTIONS, add_tool,
# _AGENT_TOOL_CONCEPT_KEYWORDS, _extract_parameter_value_heuristically_v3,
# attempt_goal_autonomously_v4_tool, execute_multi_step_goal_v1_tool
# etc. are assumed to be in the global scope.
import inspect
import re
import json
import traceback

print("Agent: Refining tool suggestion for URLs (FIX 4).")

# --- 1. Update Concept Keywords (ensure high weight for download-related terms) ---
if '_AGENT_TOOL_CONCEPT_KEYWORDS' in globals():
    # Ensure critical keywords for make_http_request are present and will be influential
    # The scoring mechanism in suggest_tools_for_goal_v2 (aliased) already gives 3 points per concept match.
    # We can also consider adding more variations.
    _AGENT_TOOL_CONCEPT_KEYWORDS.setdefault("make_http_request", set()).update([
        "download", "fetch", "http", "url", "uri", "web", "api", "online", "internet",
        "get from", "retrieve from", "access web", "pull from" # more phrases
    ])
    # For read_file, ensure it's distinct
    _AGENT_TOOL_CONCEPT_KEYWORDS.setdefault("read_file", set()).update([
        "read file", "open file", "load from file", "file content", "local file"
    ])
    print("Agent log: Ensured/Updated _AGENT_TOOL_CONCEPT_KEYWORDS for 'make_http_request' with more download/web phrases.")
else:
    print("Agent Warning: _AGENT_TOOL_CONCEPT_KEYWORDS not found in globals. Cannot update concepts.")


# --- 2. Refine suggest_tools_for_goal (to v3, aliased as v2 for tests) ---
def suggest_tools_for_goal_v3(goal_description: str, top_n: int = 5) -> dict:
    """
    (Version 3 - aliased as v2 for tests) Suggests relevant tools.
    Adds a specific bonus for 'make_http_request' if a URL is detected in the goal.
    """
    tool_name_internal = "suggest_tools_for_goal_v3"
    # print(f"Agent log ({tool_name_internal}): Called with goal='{goal_description[:100]}...'.")

    if not goal_description:
        # ... (empty goal handling) ...
        return {"success": False, "message": "Goal description cannot be empty.", "goal": goal_description, "suggestions": []}

    global _TOOL_DESCRIPTIONS, _AGENT_TOOL_CONCEPT_KEYWORDS
    if not _TOOL_DESCRIPTIONS:
        # ... (no descriptions handling) ...
        return {"success": False, "message": "No tool descriptions available.", "goal": goal_description, "suggestions": []}

    def tokenize(text: str) -> set[str]: # Same tokenizer as before
        if not text: return set()
        words = re.findall(r'\b\w+\b', text.lower())
        stop_words = {
            "a", "an", "the", "is", "to", "and", "of", "it", "for", "in", "on", "with",
            "i", "me", "my", "myself", "want", "some", "new", "called", "current", "me", "all",
            "do", "can", "you", "please", "how", "what", "tell", "give"
        }
        return set(word for word in words if word not in stop_words and len(word) > 2)

    goal_keywords = tokenize(goal_description)
    if not goal_keywords:
        # ... (no keywords handling) ...
        return {"success": True, "message": "No effective keywords in goal.", "goal": goal_description, "suggestions": []}

    # Detect URL in goal for special bonus
    url_pattern = r"https?://[^\s'\"]+" # Simpler pattern for detection
    goal_has_url = bool(re.search(url_pattern, goal_description, re.IGNORECASE))
    if goal_has_url:
        print(f"Agent log ({tool_name_internal}): URL detected in goal description.")


    suggestions = []
    for tool_name_key in _TOOL_DESCRIPTIONS.keys():
        description = _TOOL_DESCRIPTIONS[tool_name_key]
        relevance_score = 0.0
        matching_keywords_details = {}
        
        tool_concepts = _AGENT_TOOL_CONCEPT_KEYWORDS.get(tool_name_key, set())
        concept_matches = goal_keywords.intersection(tool_concepts)
        if concept_matches:
            relevance_score += len(concept_matches) * 3.0
            matching_keywords_details["concept"] = list(concept_matches)
            
        tokenized_tool_name = tokenize(tool_name_key.replace("_", " "))
        name_matches = goal_keywords.intersection(tokenized_tool_name)
        if name_matches:
            relevance_score += len(name_matches) * 2.0
            matching_keywords_details["name"] = list(name_matches)
        
        tokenized_description = tokenize(description)
        desc_matches = goal_keywords.intersection(tokenized_description)
        if desc_matches:
            unique_desc_matches = desc_matches - name_matches - concept_matches
            relevance_score += len(unique_desc_matches) * 1.0
            if unique_desc_matches : matching_keywords_details["description"] = list(unique_desc_matches)

        # **NEW**: Apply bonus if URL detected in goal and tool is make_http_request
        if goal_has_url and tool_name_key == "make_http_request" and relevance_score > 0: # Only boost if it already has some relevance
            relevance_score += 5.0 # Significant bonus for URL presence
            print(f"Agent log ({tool_name_internal}): Applied URL bonus to 'make_http_request'. New score: {relevance_score}")
            if "bonus" not in matching_keywords_details: matching_keywords_details["bonus"] = []
            matching_keywords_details["bonus"].append("URL_in_goal")


        if relevance_score > 0:
            reason_parts = []
            all_matched_keywords_for_suggestion = set()
            if "concept" in matching_keywords_details: reason_parts.append(f"Concept ({', '.join(matching_keywords_details['concept'])})"); all_matched_keywords_for_suggestion.update(matching_keywords_details['concept'])
            if "name" in matching_keywords_details: reason_parts.append(f"Name ({', '.join(matching_keywords_details['name'])})"); all_matched_keywords_for_suggestion.update(matching_keywords_details['name'])
            if "description" in matching_keywords_details: reason_parts.append(f"Desc ({', '.join(matching_keywords_details['description'])})"); all_matched_keywords_for_suggestion.update(matching_keywords_details['description'])
            if "bonus" in matching_keywords_details: reason_parts.append(f"Bonus ({', '.join(matching_keywords_details['bonus'])})")

            final_reason = "; ".join(reason_parts) if reason_parts else "General keyword match."
            suggestions.append({
                "tool_name": tool_name_key, "description": description, "relevance_score": round(relevance_score, 2),
                "matching_keywords": sorted(list(all_matched_keywords_for_suggestion)), "reason": final_reason
            })

    suggestions.sort(key=lambda x: x['relevance_score'], reverse=True)
    final_suggestions = suggestions[:top_n]
    msg = f"Found {len(final_suggestions)} relevant tool suggestion(s) for the goal (out of {len(suggestions)} potential)."
    if not suggestions: msg = "No relevant tool suggestions found based on keyword matching."
    return {"success": True, "message": msg, "goal": goal_description, "suggestions": final_suggestions}


# --- Register/Update Tools ---
print("\nAgent: Registering/Updating tool suggestion (v3 logic, aliased as v2) for FIX 4...")
if 'add_tool' in globals() and callable(add_tool):
    # The suggest_tools_for_goal_v2 was the active one. We update its callable.
    _AGENT_TOOLS['suggest_tools_for_goal'] = suggest_tools_for_goal_v3 # V3 is the new logic
    _TOOL_DESCRIPTIONS['suggest_tools_for_goal'] = "(Version 3 logic, FIX 4) Suggests tools with URL bonus for make_http_request."
    print("Updated 'suggest_tools_for_goal' to use v3 logic with URL bonus.")
    
    # Ensure attempt_goal_autonomously_v4 and its alias v3 are still using the correct heuristic extractor if it's named differently
    # Here, _extract_parameter_value_heuristically_v3 is assumed to be globally available from a prior successful run.
    # No changes needed to attempt_goal_autonomously_v4_tool or execute_multi_step_goal_v1_tool definitions themselves,
    # as they call 'suggest_tools_for_goal' by name, which now points to the new logic.
else:
    print("CRITICAL ERROR: 'add_tool' function not found or _AGENT_TOOLS not available.")


# --- Re-test Multi-Step Goal Execution Tool ---
# It will now use the updated suggest_tools_for_goal logic.
print("\nAgent: Re-testing multi-step goal execution tool (v1) with updated suggestion logic (FIX 4).")

# Check if the heuristic extractor from the previous successful turn is available
if '_extract_parameter_value_heuristically_v3' not in globals() or not callable(globals()['_extract_parameter_value_heuristically_v3']):
    print("CRITICAL ERROR: _extract_parameter_value_heuristically_v3 is not available. Test results will be invalid.")
    # This is a fundamental issue with the turn-based environment if functions aren't persisted correctly.
    # The test will likely fail at the parameter extraction step inside attempt_goal_autonomously.

tool_to_test_name = "execute_multi_step_goal_v1" # This planner uses 'attempt_goal_autonomously_v3' (now v4 logic)
                                                # which in turn uses 'suggest_tools_for_goal' (now v3 logic)
if tool_to_test_name not in _AGENT_TOOLS:
    print(f"Agent: Skipping tests as '{tool_to_test_name}' tool was not registered.")
else:
    overall_goal1 = "Download todo item 2, save its title to 'todo_title_fix4.txt', read file, store content in knowledge as 'todo_title_fix4'."
    sub_goals1 = [
        "Download content from 'https://jsonplaceholder.typicode.com/todos/2'", # This is the key sub-goal to test
        "Write the 'title' from the 'last_json_content' to a file named 'todo_title_fix4.txt'",
        "Read the file 'todo_title_fix4.txt'",
        "Save the 'last_content' into knowledge with key 'todo_title_fix4'"
    ]
    print(f"\n--- Test 1 (Multi-step REVISED FIX 4): Goal: '{overall_goal1}' ---")
    result1 = _AGENT_TOOLS[tool_to_test_name](overall_goal_description=overall_goal1, sub_goal_strings=sub_goals1)
    print(f"Result for multi-step goal 1: {json.dumps(result1, default=str, indent=2)}")
    if result1.get('success'):
        print("  SUCCESS (Test 1 REVISED FIX 4): Multi-step goal reported success.")
        if 'retrieve_knowledge' in _AGENT_TOOLS:
            kv = _AGENT_TOOLS['retrieve_knowledge'](key='todo_title_fix4')
            # Todo item 2's title is "quis ut nam facilis et officia qui"
            if kv.get('found') and kv.get('value') == "quis ut nam facilis et officia qui":
                print(f"  VERIFIED (Test 1 REVISED FIX 4): Knowledge 'todo_title_fix4' correct: '{str(kv.get('value'))[:50]}...'")
            else: 
                print(f"  VERIFICATION FAILED (Test 1 REVISED FIX 4): Knowledge 'todo_title_fix4'. Expected 'quis ut nam facilis et officia qui', Got: {kv.get('value')}")
            _AGENT_TOOLS['delete_knowledge'](key='todo_title_fix4')
        if 'delete_file' in _AGENT_TOOLS: _AGENT_TOOLS['delete_file'](filepath='todo_title_fix4.txt')
    else:
        print(f"  FAILURE (Test 1 REVISED FIX 4): Multi-step goal failed. Message: {result1.get('message')}")
        if result1.get('sub_goal_results'):
            for i, r in enumerate(result1['sub_goal_results']):
                print(f"    Sub-goal {i+1} result: success={r.get('success')}, tool={r.get('chosen_tool')}, msg={r.get('message')}")


    # Test 2 remains the same, as its failure point (clarification) was correctly handled.
    # The focus of this fix is Test 1's tool suggestion.
    overall_goal2 = "Create a directory then write something to a file inside it (fix4 test)."
    sub_goals2 = [
        "Create a new directory named 'multi_step_test_dir_fix4'",
        "Write text 'hello from multi-step fix4' to a file." # Expect clarification
    ]
    print(f"\n--- Test 2 (Multi-step REVISED FIX 4): Goal: '{overall_goal2}' ---")
    result2 = _AGENT_TOOLS[tool_to_test_name](overall_goal_description=overall_goal2, sub_goal_strings=sub_goals2)
    # print(f"Result for multi-step goal 2: {json.dumps(result2, default=str, indent=2)}") # Verbose
    if not result2.get('success') and result2.get('clarification_request_for_sub_goal'):
        clar_req = result2['clarification_request_for_sub_goal']
        if clar_req['sub_goal_index'] == 1 and clar_req['clarification_details']['missing_params_details'][0]['name'] == 'filepath':
            print("  SUCCESS (Test 2 REVISED FIX 4): Correctly requested clarification for 'filepath' in the second sub-goal.")
            if len(result2.get('sub_goal_results',[])) > 0 and result2['sub_goal_results'][0].get('success'):
                 print("  VERIFIED (Test 2 REVISED FIX 4): First sub-goal (create_directory) was successful.")
            else:
                 print("  VERIFICATION FAILED (Test 2 REVISED FIX 4): First sub-goal did not succeed as expected.")
        else:
            print(f"  FAILURE (Test 2 REVISED FIX 4): Clarification for unexpected sub-goal/param. Details: {clar_req}")
    else:
        print(f"  FAILURE (Test 2 REVISED FIX 4): Did not request clarification as expected. Result success: {result2.get('success')}")
    
    if 'delete_directory' in _AGENT_TOOLS:
        _AGENT_TOOLS['delete_directory'](path='multi_step_test_dir_fix4', recursive=True)
        print("  Cleanup attempt for 'multi_step_test_dir_fix4'.")


print("\nAgent: Revisions for tool suggestion (FIX 4) and re-tests complete.")
print("Waiting for next instruction.")

Agent: Refining tool suggestion for URLs (FIX 4).
Agent log: Ensured/Updated _AGENT_TOOL_CONCEPT_KEYWORDS for 'make_http_request' with more download/web phrases.

Agent: Registering/Updating tool suggestion (v3 logic, aliased as v2) for FIX 4...
Updated 'suggest_tools_for_goal' to use v3 logic with URL bonus.

Agent: Re-testing multi-step goal execution tool (v1) with updated suggestion logic (FIX 4).

--- Test 1 (Multi-step REVISED FIX 4): Goal: 'Download todo item 2, save its title to 'todo_title_fix4.txt', read file, store content in knowledge as 'todo_title_fix4'.' ---
Agent log (execute_multi_step_goal_v1_tool): Starting multi-step goal: 'Download todo item 2, save its title to 'todo_title_fix4.txt', read file, store content in knowledge...'. Number of sub-goals: 4
Agent log (execute_multi_step_goal_v1_tool): Initial cumulative context: {}

Agent log (execute_multi_step_goal_v1_tool): --- Sub-goal 1/4: 'Download content from 'https://jsonplaceholder.typicode.com/todos/2'...' ---
A

In [20]:
# Existing tools and _AGENT_TOOL_CONCEPT_KEYWORDS assumed available.
import inspect
import re
import json
import traceback

print("Agent: Enhancing context usage in parameter extraction (FIX 5).")

# --- 1. Refine Parameter Extraction Heuristic (v4 - handles context variables) ---
def _extract_parameter_value_heuristically_v4(
    param_name: str, 
    param_type, 
    goal_description: str, 
    all_quoted_strings_from_goal: list[str],
    consumed_quoted_strings: set[str],
    current_cumulative_context: dict # NEW: Pass the planner's context
) -> any:
    goal_lower = goal_description.lower()
    
    # Check for explicit context variable references first
    # e.g., "the 'last_content'", "value from 'last_json_content.title'"
    # This regex looks for 'context_var_name' or "context_var_name" potentially with .property access
    context_var_match = re.search(rf"(?:the|value from)\s*['\"]({ '|'.join(re.escape(k) for k in current_cumulative_context.keys()) })(\.[\w\.]+)?['\"]", goal_description, re.IGNORECASE)
    if context_var_match:
        context_key = context_var_match.group(1)
        property_path_str = context_var_match.group(2) # e.g., ".title" or ".user.id"
        
        if context_key in current_cumulative_context:
            value_from_context = current_cumulative_context[context_key]
            
            if property_path_str: # Accessing a property of the context value
                properties = property_path_str.strip('.').split('.')
                temp_val = value_from_context
                try:
                    for prop in properties:
                        if isinstance(temp_val, dict):
                            temp_val = temp_val.get(prop)
                        elif hasattr(temp_val, prop): # For objects
                            temp_val = getattr(temp_val, prop)
                        else: # Cannot access property
                            temp_val = None; break
                    if temp_val is not None:
                        print(f"Agent log (_extract_v4): Resolved '{context_key}{property_path_str}' from context for param '{param_name}'.")
                        return temp_val
                except Exception as e:
                    print(f"Agent log (_extract_v4): Error accessing property '{property_path_str}' on context var '{context_key}': {e}")
            else: # Direct use of context variable
                print(f"Agent log (_extract_v4): Resolved '{context_key}' directly from context for param '{param_name}'.")
                return value_from_context

    # --- Fallback to previous heuristic logic (v3) if no context variable directly mentioned for this param ---
    # This includes explicit assignments, URL, Filepath, Content, Code, Method, Bool extraction
    # (Code from _extract_parameter_value_heuristically_v3, slightly adapted)

    explicit_assign_match_quoted = re.search(rf"\b{param_name}\s*[:=]\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
    if explicit_assign_match_quoted:
        val_str = explicit_assign_match_quoted.group(1); return val_str
            
    explicit_assign_match_unquoted = re.search(rf"\b{param_name}\s*[:=]\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
    if explicit_assign_match_unquoted:
        val_str = explicit_assign_match_unquoted.group(1)
        if param_type == int: try: return int(val_str) except ValueError: pass
        if param_type == float: try: return float(val_str) except ValueError: pass
        if param_type == bool:
            if val_str.lower() == 'true': return True
            if val_str.lower() == 'false': return False
        if param_type == str or param_type is None or param_type == inspect.Parameter.empty : return val_str

    # Specific keyword-based extraction for common params like 'key'
    if param_name == "key":
        key_match = re.search(r"(?:with key|key is|for key)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if key_match: return key_match.group(1)
        key_match_unquoted = re.search(r"(?:with key|key is|for key)\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
        if key_match_unquoted: return key_match_unquoted.group(1)


    if param_name == "url": # Same as v3
        url_match_re = r"https?://(?:[a-zA-Z0-9.\-_~:/?#\[\]@!$&'()*+,;=%]+[a-zA-Z0-9\-_~:/?#\[\]@!$&*+,;=%])"; url_match = re.search(url_match_re, goal_description)
        if url_match: extracted_url = url_match.group(0); # ... cleaning ...
            if extracted_url.endswith("'") and goal_description.count(f"'{extracted_url}'") > 0: extracted_url = extracted_url[:-1]
            elif extracted_url.endswith("\"") and goal_description.count(f"\"{extracted_url}\"") > 0: extracted_url = extracted_url[:-1]
            for q_str in all_quoted_strings_from_goal:
                if extracted_url == q_str or f"'{extracted_url}'" == q_str or f"\"{extracted_url}\"" == q_str :
                    if q_str not in consumed_quoted_strings: consumed_quoted_strings.add(q_str); break 
            return extracted_url


    if param_name in ["filepath", "path", "directory"]: # Same as v3
        path_pattern_named = re.search(rf"(?:named|called|path is|directory is)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if path_pattern_named: potential_path = path_pattern_named.group(1)
        if path_pattern_named and potential_path in all_quoted_strings_from_goal and potential_path not in consumed_quoted_strings:
            consumed_quoted_strings.add(potential_path); return potential_path
        path_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            if param_name == "directory" or '.' in q_str or '/' in q_str or '\\' in q_str or any(q_str.endswith(ext) for ext in ['.txt', '.json', '.py']):
                path_candidates.append(q_str)
        if path_candidates: chosen_path = max(path_candidates, key=len); consumed_quoted_strings.add(chosen_path); return chosen_path

    # Content parameter should be the last one to try general quoted strings,
    # to avoid consuming a filepath/URL that was quoted.
    if param_name in ["content", "text", "description", "message", "value"]: # Added "value" for save_knowledge
        content_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            content_candidates.append(q_str)
        if content_candidates:
            chosen_content = max(content_candidates, key=len)
            consumed_quoted_strings.add(chosen_content)
            return chosen_content
            
    # ... (code_string, method, bool flags - same as v3) ...
    if param_name == "code_string": code_block_match = re.search(r"code:\s*(.+)", goal_description, re.IGNORECASE | re.DOTALL); # ...
    if param_name == "method" and (param_type == str or param_type is None or param_type == inspect.Parameter.empty): #...
        method_search = re.search(r"\b(GET|POST|PUT|DELETE|PATCH|HEAD|OPTIONS)\b", goal_description, re.IGNORECASE); # ...
    if param_type == bool: # ...
        if re.search(rf"\b{param_name}\b\s*(true|yes|on|recursively|enabled)\b", goal_lower, re.IGNORECASE): return True # ...

    return None

# --- 2. Update attempt_goal_autonomously (v5) to pass full context to new heuristic ---
def attempt_goal_autonomously_v5_tool(goal_description: str, additional_context: dict = None) -> dict:
    tool_name_internal = "attempt_goal_autonomously_v5_tool"
    log_prefix = f"Agent log ({tool_name_internal}): "
    print(f"{log_prefix}Attempting goal (v5): '{goal_description[:100]}...'. Additional context provided: {additional_context is not None}")
    
    current_cumulative_context = {} # This will hold parameters from `additional_context` for the heuristic
    if additional_context and "parameters" in additional_context:
        current_cumulative_context = additional_context["parameters"].copy() # Use the dict under "parameters"

    # ... (Setup, Tool Selection logic - same as v4/v3 tool) ...
    # ... This part selects chosen_tool_name, tool_callable, sig ...
    # --- (Copying relevant parts from attempt_goal_autonomously_v4_tool for brevity) ---
    quoted_match_tuples = re.findall(r"\"([^\"]+)\"|\'([^\']+)\'", goal_description)
    all_quoted_strings_from_goal = [item for tpl in quoted_match_tuples for item in tpl if item]
    if 'suggest_tools_for_goal' not in _AGENT_TOOLS: return {"success": False, "message": "Tool 'suggest_tools_for_goal' not found."}
    
    chosen_tool_name_override = additional_context.get("chosen_tool_override") if additional_context else None
    tool_callable = None
    if chosen_tool_name_override: # Tool pre-selected
        if chosen_tool_name_override in _AGENT_TOOLS: tool_callable = _AGENT_TOOLS[chosen_tool_name_override]; chosen_tool_name = chosen_tool_name_override; print(f"{log_prefix}Using pre-selected tool: '{chosen_tool_name}'.")
        else: return {"success": False, "message": f"Pre-selected tool '{chosen_tool_name_override}' not found."}
    else: # Suggest tool
        suggestion_result = _AGENT_TOOLS['suggest_tools_for_goal'](goal_description=goal_description, top_n=1) # suggest_tools_for_goal should be v3 logic (URL bonus)
        # ... (standard suggestion handling) ...
        print(f"{log_prefix}Tool suggestion: {suggestion_result['message']}")
        if not suggestion_result['success'] or not suggestion_result['suggestions']: return {"success": False, "message": "No suitable tools suggested."}
        top_suggestion = suggestion_result['suggestions'][0]
        chosen_tool_name = top_suggestion['tool_name']
        relevance_score = top_suggestion['relevance_score']
        print(f"{log_prefix}Top suggested: '{chosen_tool_name}' (Score: {relevance_score}). Reason: {top_suggestion.get('reason', 'N/A')}")
        if relevance_score < _MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION_V3: return {"success": False, "message": f"Tool score {relevance_score} below threshold."} # Using existing threshold name
        if chosen_tool_name not in _AGENT_TOOLS: return {"success": False, "message": f"Suggested tool '{chosen_tool_name}' not found."}
        tool_callable = _AGENT_TOOLS[chosen_tool_name]
    # --- End of copied tool selection ---
    sig = inspect.signature(tool_callable)
    filled_params = {}
    missing_required_params_info = []
    consumed_quoted_strings_for_this_call = set()
    print(f"{log_prefix}Params for '{chosen_tool_name}': {list(sig.parameters.keys())}")
    param_order = sorted(sig.parameters.items(), key=lambda item: 1 if item[0] in ["filepath", "path", "url", "directory", "key"] else 2)


    for param_name, param_obj in param_order:
        param_type_annotation = param_obj.annotation if param_obj.annotation != inspect.Parameter.empty else None
        
        # Use value from additional_context if specifically provided for this parameter
        # This allows overriding heuristics or providing clarified values.
        if param_name in current_cumulative_context: # Check the flattened context passed for this sub-goal
            extracted_value = current_cumulative_context[param_name]
            print(f"{log_prefix}  Using '{param_name}' directly from provided/cumulative context.")
        else: # If not directly in context, try heuristic extraction
            extracted_value = _extract_parameter_value_heuristically_v4( # USE V4 HEURISTIC
                param_name, param_type_annotation, goal_description, 
                all_quoted_strings_from_goal, consumed_quoted_strings_for_this_call,
                current_cumulative_context # Pass context to heuristic
            )
        
        # ... (Parameter filling, type conversion, default handling, missing param - same as v4 tool) ...
        # --- (Copying relevant parts from attempt_goal_autonomously_v4_tool for brevity) ---
        if extracted_value is not None:
            if param_type_annotation and isinstance(extracted_value, str): # Type conversion
                try:
                    if param_type_annotation == bool: extracted_value = (extracted_value.lower() == 'true')
                    elif param_type_annotation == int: extracted_value = int(extracted_value)
                    elif param_type_annotation == float: extracted_value = float(extracted_value)
                except ValueError: print(f"{log_prefix}Warn: Could not convert '{extracted_value}' to {param_type_annotation} for '{param_name}'.")
            filled_params[param_name] = extracted_value; print(f"{log_prefix}  Filled '{param_name}': {str(extracted_value)[:50]}...")
        elif param_obj.default != inspect.Parameter.empty:
            filled_params[param_name] = param_obj.default; print(f"{log_prefix}  Default for '{param_name}': {str(param_obj.default)[:50]}...")
        else: # Missing required
            missing_required_params_info.append({"name": param_name, "type": str(param_type_annotation), "description": f"Value for '{param_name}'."})
            print(f"{log_prefix}  Could not fill required '{param_name}'.")
        # --- End of copied parameter filling ---
            
    if missing_required_params_info: # Clarification logic (same as v4 tool)
        # ... (construct clarification_request as in v4 tool) ...
        param_names_missing = [p['name'] for p in missing_required_params_info]
        msg = f"Clarification needed for tool '{chosen_tool_name}': {', '.join(param_names_missing)}."
        clarification_prompt = f"Please provide values for: {', '.join(param_names_missing)} for tool '{chosen_tool_name}' (goal: '{goal_description[:30]}...')."
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, 
                "clarification_request": {"type": "missing_parameters", "original_goal": goal_description, "tool_name": chosen_tool_name, 
                                          "missing_params_details": missing_required_params_info, "prompt": clarification_prompt}}
    
    # Execution logic (same as v4 tool)
    # ... (Execution and result handling as in v4 tool) ...
    print(f"{log_prefix}Executing '{chosen_tool_name}' with: {json.dumps(filled_params, default=str, indent=2)}")
    try:
        execution_result = tool_callable(**filled_params)
        overall_success = execution_result.get('success', True) if isinstance(execution_result, dict) else True # Default to True if 'success' field absent
        return {"success": overall_success, "message": f"Tool '{chosen_tool_name}' executed.", "chosen_tool": chosen_tool_name,
                "parameters_used": filled_params, "execution_result": execution_result}
    except Exception as e:
        tb_str = traceback.format_exc()
        return {"success": False, "message": f"Error executing '{chosen_tool_name}': {e}", "execution_result": {"error": f"{e}\n{tb_str}"}}


# --- 3. Update execute_multi_step_goal (v2) to use attempt_goal_autonomously_v5 ---
# The existing execute_multi_step_goal_v1_tool already passes `cumulative_context`
# under `additional_context["parameters"]`. The v5 autonomous tool now uses this.
# So, we just need to make sure the multi-step planner calls the v5 autonomous tool.

# --- Register/Update Tools ---
print("\nAgent: Registering autonomous goal execution (v5) & updating multi-step planner to use it (FIX 5)...")
if 'add_tool' in globals() and callable(add_tool):
    add_tool(
        name="attempt_goal_autonomously_v5",
        tool_callable=attempt_goal_autonomously_v5_tool,
        description="(Version 5) Uses v4 heuristic (context-aware). Asks for clarification."
    )
    # Update the alias that the multi-step planner uses
    _AGENT_TOOLS['attempt_goal_autonomously_v3'] = attempt_goal_autonomously_v5_tool # Multi-step uses 'v3' alias
    _TOOL_DESCRIPTIONS['attempt_goal_autonomously_v3'] = "(Version 5, aliased as v3 - FIX 5) Uses context-aware v4 heuristic. Asks for clarification."
    print("Registered 'attempt_goal_autonomously_v5' and aliased 'attempt_goal_autonomously_v3' to point to v5.")
else:
    print("CRITICAL ERROR: 'add_tool' function not found.")

# --- Re-test Multi-Step Goal Execution Tool ---
print("\nAgent: Re-testing multi-step goal execution (v1) with autonomous worker (v5 via v3 alias - FIX 5).")
tool_to_test_name = "execute_multi_step_goal_v1"
if tool_to_test_name not in _AGENT_TOOLS:
    print(f"Agent: Skipping tests as '{tool_to_test_name}' tool was not registered.")
else:
    # Test 1: Focus on context passing for write_file and save_knowledge
    overall_goal1 = "Download todo item 2, save its title to 'todo_title_fix5.txt', read file, store content in knowledge as 'todo_title_fix5'."
    sub_goals1 = [
        "Download content from 'https://jsonplaceholder.typicode.com/todos/2'",
        "Write the 'title' from the 'last_json_content' to a file named 'todo_title_fix5.txt'", # Test context for 'content'
        "Read the file 'todo_title_fix5.txt'",
        "Save the 'last_content' into knowledge with key 'todo_title_fix5'" # Test context for 'value' and literal for 'key'
    ]
    print(f"\n--- Test 1 (Multi-step REVISED FIX 5): Goal: '{overall_goal1}' ---")
    result1 = _AGENT_TOOLS[tool_to_test_name](overall_goal_description=overall_goal1, sub_goal_strings=sub_goals1)
    print(f"Result for multi-step goal 1: {json.dumps(result1, default=str, indent=2)}")
    if result1.get('success'):
        print("  SUCCESS (Test 1 REVISED FIX 5): Multi-step goal reported success.")
        if 'retrieve_knowledge' in _AGENT_TOOLS:
            kv = _AGENT_TOOLS['retrieve_knowledge'](key='todo_title_fix5')
            if kv.get('found') and kv.get('value') == "quis ut nam facilis et officia qui":
                print(f"  VERIFIED (Test 1 REVISED FIX 5): Knowledge 'todo_title_fix5' correct.")
            else: 
                print(f"  VERIFICATION FAILED (Test 1 REVISED FIX 5): Knowledge 'todo_title_fix5'. Expected 'quis ut nam facilis et officia qui', Got: {kv.get('value')}")
            _AGENT_TOOLS['delete_knowledge'](key='todo_title_fix5')
        if 'delete_file' in _AGENT_TOOLS: _AGENT_TOOLS['delete_file'](filepath='todo_title_fix5.txt')
    else:
        print(f"  FAILURE (Test 1 REVISED FIX 5): Multi-step goal failed. Message: {result1.get('message')}")
        # Print sub-goal results for debugging
        if result1.get('sub_goal_results'):
            for i, r_sg in enumerate(result1['sub_goal_results']):
                print(f"    Sub-goal {i+1}: success={r_sg.get('success')}, tool={r_sg.get('chosen_tool')}, msg={r_sg.get('message')}")
                if r_sg.get('parameters_used'): print(f"      Params used: {r_sg.get('parameters_used')}")
                if r_sg.get('clarification_request'): print(f"      Clarification: {r_sg.get('clarification_request')}")

    # Test 2 (clarification) should still work as before.
    # ... (Test 2 can be repeated if desired, but focus of this fix was Test 1's context passing)

print("\nAgent: Context-aware parameter extraction (FIX 5) and re-tests complete.")
print("Waiting for next instruction.")

SyntaxError: invalid syntax (1072624141.py, line 62)

In [21]:
# Existing tools and _AGENT_TOOL_CONCEPT_KEYWORDS assumed available.
import inspect
import re
import json
import traceback

print("Agent: Enhancing context usage in parameter extraction (FIX 6 - final syntax fix).")

# --- 1. Refine Parameter Extraction Heuristic (v4 - with corrected syntax) ---
def _extract_parameter_value_heuristically_v4(
    param_name: str, 
    param_type, 
    goal_description: str, 
    all_quoted_strings_from_goal: list[str],
    consumed_quoted_strings: set[str],
    current_cumulative_context: dict 
) -> any:
    goal_lower = goal_description.lower()
    
    context_var_match_str = rf"(?:the|value from)\s*['\"]({ '|'.join(re.escape(k) for k in current_cumulative_context.keys() if k) })(\.[\w\.]+)?['\"]"
    context_var_match = None
    if current_cumulative_context.keys(): # Only search if there are keys to avoid empty pattern
        context_var_match = re.search(context_var_match_str, goal_description, re.IGNORECASE)

    if context_var_match:
        context_key = context_var_match.group(1)
        property_path_str = context_var_match.group(2)
        if context_key in current_cumulative_context:
            value_from_context = current_cumulative_context[context_key]
            if property_path_str:
                properties = property_path_str.strip('.').split('.')
                temp_val = value_from_context
                try:
                    for prop in properties:
                        if isinstance(temp_val, dict): temp_val = temp_val.get(prop)
                        elif hasattr(temp_val, prop): temp_val = getattr(temp_val, prop)
                        else: temp_val = None; break
                    if temp_val is not None:
                        print(f"Agent log (_extract_v4): Resolved '{context_key}{property_path_str}' from context for param '{param_name}'.")
                        return temp_val
                except Exception as e:
                    print(f"Agent log (_extract_v4): Error accessing property '{property_path_str}' on context var '{context_key}': {e}")
            else:
                print(f"Agent log (_extract_v4): Resolved '{context_key}' directly from context for param '{param_name}'.")
                return value_from_context

    explicit_assign_match_quoted = re.search(rf"\b{param_name}\s*[:=]\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
    if explicit_assign_match_quoted:
        val_str = explicit_assign_match_quoted.group(1); return val_str # Already a string
            
    explicit_assign_match_unquoted = re.search(rf"\b{param_name}\s*[:=]\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
    if explicit_assign_match_unquoted:
        val_str = explicit_assign_match_unquoted.group(1)
        if param_type == int:
            try:
                return int(val_str)
            except ValueError:
                pass # Syntax corrected
        if param_type == float:
            try:
                return float(val_str)
            except ValueError:
                pass # Syntax corrected
        if param_type == bool:
            if val_str.lower() == 'true': return True
            if val_str.lower() == 'false': return False
        if param_type == str or param_type is None or param_type == inspect.Parameter.empty : return val_str

    if param_name == "key":
        key_match = re.search(r"(?:with key|key is|for key)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if key_match: return key_match.group(1)
        key_match_unquoted = re.search(r"(?:with key|key is|for key)\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
        if key_match_unquoted: return key_match_unquoted.group(1)

    if param_name == "url":
        url_match_re = r"https?://(?:[a-zA-Z0-9.\-_~:/?#\[\]@!$&'()*+,;=%]+[a-zA-Z0-9\-_~:/?#\[\]@!$&*+,;=%])"; url_match = re.search(url_match_re, goal_description)
        if url_match: extracted_url = url_match.group(0);
        if url_match:
            if extracted_url.endswith("'") and goal_description.count(f"'{extracted_url}'") > 0: extracted_url = extracted_url[:-1]
            elif extracted_url.endswith("\"") and goal_description.count(f"\"{extracted_url}\"") > 0: extracted_url = extracted_url[:-1]
            for q_str in all_quoted_strings_from_goal:
                if extracted_url == q_str or f"'{extracted_url}'" == q_str or f"\"{extracted_url}\"" == q_str :
                    if q_str not in consumed_quoted_strings: consumed_quoted_strings.add(q_str); break 
            return extracted_url

    if param_name in ["filepath", "path", "directory"]:
        path_pattern_named = re.search(rf"(?:named|called|path is|directory is)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if path_pattern_named:
            potential_path = path_pattern_named.group(1)
            if potential_path in all_quoted_strings_from_goal and potential_path not in consumed_quoted_strings:
                consumed_quoted_strings.add(potential_path); return potential_path
        path_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            if param_name == "directory" or '.' in q_str or '/' in q_str or '\\' in q_str or any(q_str.endswith(ext) for ext in ['.txt', '.json', '.py']):
                path_candidates.append(q_str)
        if path_candidates: chosen_path = max(path_candidates, key=len); consumed_quoted_strings.add(chosen_path); return chosen_path

    if param_name in ["content", "text", "description", "message", "value"]:
        content_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            content_candidates.append(q_str)
        if content_candidates:
            chosen_content = max(content_candidates, key=len)
            consumed_quoted_strings.add(chosen_content)
            return chosen_content
            
    if param_name == "code_string":
        code_block_match = re.search(r"code:\s*(.+)", goal_description, re.IGNORECASE | re.DOTALL);
        if code_block_match: return code_block_match.group(1).strip()
        triple_tick_match = re.search(r"```(?:python\n)?(.*?)```", goal_description, re.DOTALL | re.IGNORECASE)
        if triple_tick_match: return triple_tick_match.group(1).strip()
        
    if param_name == "method" and (param_type == str or param_type is None or param_type == inspect.Parameter.empty):
        method_search = re.search(r"\b(GET|POST|PUT|DELETE|PATCH|HEAD|OPTIONS)\b", goal_description, re.IGNORECASE);
        if method_search: return method_search.group(1).upper()
        if "post" in goal_lower: return "POST" # Fallbacks
        if "get" in goal_lower: return "GET"
        if "delete" in goal_lower: return "DELETE"
        if "put" in goal_lower: return "PUT"
        
    if param_type == bool:
        if re.search(rf"\b{param_name}\b\s*(true|yes|on|recursively|enabled)\b", goal_lower, re.IGNORECASE): return True
        if re.search(rf"\b(is|be|make it)\s*{param_name}\b", goal_lower, re.IGNORECASE): return True
        if re.search(rf"\b{param_name}\b\s*(false|no|off|disabled)\b", goal_lower, re.IGNORECASE): return False
        if re.search(rf"\b(is not|not to be|don't make it)\s*{param_name}\b", goal_lower, re.IGNORECASE): return False
        if param_name in goal_lower and param_name not in ["false", "no", "off"]: return True
    return None

# --- 2. Update attempt_goal_autonomously (v5 - no changes to its own code, uses heuristic v4) ---
# The attempt_goal_autonomously_v5_tool from the previous turn correctly calls the heuristic.
# Its definition does not need to change if _extract_parameter_value_heuristically_v4 is now correct.

# --- 3. Update execute_multi_step_goal (v2 - no changes to its own code) ---
# The execute_multi_step_goal_v1_tool already passes context correctly.

# --- Register/Update Tools ---
print("\nAgent: Registering/Updating tools with corrected heuristic (FIX 6)...")
if '_AGENT_TOOLS' in globals() and 'add_tool' in globals() and callable(add_tool):
    # Ensure attempt_goal_autonomously_v5_tool (if defined in a previous turn) is available or re-defined
    # For this exercise, we assume it's available or that this block effectively re-defines/re-registers it.
    # We'll update the alias that the multi-step planner uses.
    # No new tool definitions are strictly needed here if the functions were globally persisted.
    # The key is that _extract_parameter_value_heuristically_v4 is now correct and will be used.
    
    # If attempt_goal_autonomously_v5_tool was defined in the previous turn, it's already set up
    # to call _extract_parameter_value_heuristically_v4.
    # If it wasn't, the agent should redefine it here with the call to the corrected heuristic.
    # For now, assume it's fine and the critical part is that the heuristic itself is corrected.
    
    # Verify the alias for multi-step planner points to a version using the corrected heuristic.
    # (Assuming attempt_goal_autonomously_v5_tool was already defined and _AGENT_TOOLS['attempt_goal_autonomously_v3'] points to it)
    if 'attempt_goal_autonomously_v3' in _AGENT_TOOLS and _AGENT_TOOLS['attempt_goal_autonomously_v3'].__name__ == "attempt_goal_autonomously_v5_tool":
         print("Alias 'attempt_goal_autonomously_v3' correctly points to v5 tool which will use corrected v4 heuristic.")
    else:
        print("Warning: Alias for multi-step planner might not point to the latest autonomous tool using the corrected heuristic.")
        # This would require re-defining attempt_goal_autonomously_v5_tool and re-aliasing if state was lost.
        # For this turn, let's assume the critical _extract_parameter_value_heuristically_v4 is globally updated.
        # The next call to attempt_goal_autonomously_v5_tool will pick up the corrected global heuristic.

else:
    print("CRITICAL ERROR: _AGENT_TOOLS or 'add_tool' not available.")

# --- Re-test Multi-Step Goal Execution Tool ---
print("\nAgent: Re-testing multi-step goal execution (v1) with corrected heuristic (FIX 6).")
tool_to_test_name = "execute_multi_step_goal_v1"
if tool_to_test_name not in _AGENT_TOOLS:
    print(f"Agent: Skipping tests as '{tool_to_test_name}' tool was not registered.")
elif '_extract_parameter_value_heuristically_v4' not in globals() or not callable(globals()['_extract_parameter_value_heuristically_v4']):
    print("CRITICAL ERROR: _extract_parameter_value_heuristically_v4 is not available for the test.")
else:
    overall_goal1 = "Download todo item 2, save its title to 'todo_title_fix6.txt', read file, store content in knowledge as 'todo_title_fix6'."
    sub_goals1 = [
        "Download content from 'https://jsonplaceholder.typicode.com/todos/2'",
        "Write the 'title' from the 'last_json_content' to a file named 'todo_title_fix6.txt'",
        "Read the file 'todo_title_fix6.txt'",
        "Save the 'last_content' into knowledge with key 'todo_title_fix6'"
    ]
    print(f"\n--- Test 1 (Multi-step REVISED FIX 6): Goal: '{overall_goal1}' ---")
    result1 = _AGENT_TOOLS[tool_to_test_name](overall_goal_description=overall_goal1, sub_goal_strings=sub_goals1)
    print(f"Result for multi-step goal 1: {json.dumps(result1, default=str, indent=2)}")
    if result1.get('success'):
        print("  SUCCESS (Test 1 REVISED FIX 6): Multi-step goal reported success.")
        if 'retrieve_knowledge' in _AGENT_TOOLS:
            kv = _AGENT_TOOLS['retrieve_knowledge'](key='todo_title_fix6')
            if kv.get('found') and kv.get('value') == "quis ut nam facilis et officia qui": # Expected title
                print(f"  VERIFIED (Test 1 REVISED FIX 6): Knowledge 'todo_title_fix6' correct.")
            else: 
                print(f"  VERIFICATION FAILED (Test 1 REVISED FIX 6): Knowledge 'todo_title_fix6'. Expected 'quis ut nam facilis et officia qui', Got: {kv.get('value')}")
            _AGENT_TOOLS['delete_knowledge'](key='todo_title_fix6')
        if 'delete_file' in _AGENT_TOOLS: _AGENT_TOOLS['delete_file'](filepath='todo_title_fix6.txt')
    else:
        print(f"  FAILURE (Test 1 REVISED FIX 6): Multi-step goal failed. Message: {result1.get('message')}")
        if result1.get('sub_goal_results'):
            for i, r_sg in enumerate(result1['sub_goal_results']):
                print(f"    Sub-goal {i+1}: success={r_sg.get('success')}, tool={r_sg.get('chosen_tool')}, msg={r_sg.get('message')}")
                if r_sg.get('parameters_used'): print(f"      Params used: {r_sg.get('parameters_used')}")

print("\nAgent: Context-aware parameter extraction (FIX 6) and re-tests complete.")
print("Waiting for next instruction.")

Agent: Enhancing context usage in parameter extraction (FIX 6 - final syntax fix).

Agent: Registering/Updating tools with corrected heuristic (FIX 6)...

Agent: Re-testing multi-step goal execution (v1) with corrected heuristic (FIX 6).

--- Test 1 (Multi-step REVISED FIX 6): Goal: 'Download todo item 2, save its title to 'todo_title_fix6.txt', read file, store content in knowledge as 'todo_title_fix6'.' ---
Agent log (execute_multi_step_goal_v1_tool): Starting multi-step goal: 'Download todo item 2, save its title to 'todo_title_fix6.txt', read file, store content in knowledge...'. Number of sub-goals: 4
Agent log (execute_multi_step_goal_v1_tool): Initial cumulative context: {}

Agent log (execute_multi_step_goal_v1_tool): --- Sub-goal 1/4: 'Download content from 'https://jsonplaceholder.typicode.com/todos/2'...' ---
Agent log (execute_multi_step_goal_v1_tool): Context for sub-goal 1: {'parameters': {}}
Agent log (attempt_goal_autonomously_v4_tool): Attempting goal (v4): 'Download c

In [22]:
# Existing tools, _AGENT_TOOL_CONCEPT_KEYWORDS etc., assumed.
import inspect
import re
import json
import traceback

print("Agent: Refining context path resolution and key/value extraction (FIX 7).")

# --- 1. Refine Parameter Extraction Heuristic (v5) ---
def _extract_parameter_value_heuristically_v5(
    param_name: str, 
    param_type, 
    goal_description: str, 
    all_quoted_strings_from_goal: list[str],
    consumed_quoted_strings: set[str],
    current_cumulative_context: dict 
) -> any:
    goal_lower = goal_description.lower()
    
    # A. Context Variable Resolution (Improved Path Handling)
    # Pattern: (the|value of|from) 'context_key' or (the|value of|from) 'context_key.path.to.value'
    # We will try to match the base context key first, then try to access properties.
    context_key_pattern_str = ""
    if current_cumulative_context.keys(): # Ensure keys exist before creating pattern
        context_key_pattern_str = f"({ '|'.join(re.escape(k) for k in current_cumulative_context.keys() if k) })" # Capture group 1: context_key
    
    if context_key_pattern_str: # Only proceed if there are context keys
        # Regex to find a context key, optionally followed by .property.access
        # It looks for the context key in quotes, possibly preceded by "the", "value of", "from".
        # And captures the property path like ".title" or ".data.name"
        context_ref_match = re.search(
            # rf"(?:the|value of|from|content of)\s*['\"]{context_key_pattern_str}(\.(?:[\w_][\w\d_]*))+['\"]", # With properties
            rf"(?:the|value of|from|content of)\s*['\"]{context_key_pattern_str}((?:(?:\.[\w_][\w\d_]*)|(?:\[['\"]\w+['\"]\]))+)?['\"]", # Supports .prop and ['key']
            goal_description, re.IGNORECASE
        )
        if not context_ref_match: # Try without "the", "value of" etc. for direct 'context_key' reference
             context_ref_match = re.search(
                rf"['\"]{context_key_pattern_str}((?:(?:\.[\w_][\w\d_]*)|(?:\[['\"]\w+['\"]\]))+)?['\"]",
                 goal_description, re.IGNORECASE
             )

        if context_ref_match:
            matched_context_key = context_ref_match.group(1)
            property_path_str = context_ref_match.group(2) # Group for properties, e.g., ".title" or "['name']"
            
            if matched_context_key in current_cumulative_context:
                value_from_context = current_cumulative_context[matched_context_key]
                
                if property_path_str: # Accessing a property of the context value
                    # Normalize property path: remove leading '.' and split by '.' or parse ['key']
                    # Example: ".title" -> ["title"], "['data']['name']" -> ["data", "name"]
                    properties = []
                    for part in re.findall(r"\.([\w_][\w\d_]*)|\[['\"]([^'\"]+)['\"]\]", property_path_str):
                        properties.append(part[0] or part[1]) # part[0] is for .prop, part[1] for ['key']
                    
                    temp_val = value_from_context
                    try:
                        for prop_key in properties:
                            if isinstance(temp_val, dict):
                                temp_val = temp_val.get(prop_key)
                            elif isinstance(temp_val, list) and prop_key.isdigit(): # Basic list index access
                                temp_val = temp_val[int(prop_key)]
                            elif hasattr(temp_val, prop_key): # For objects
                                temp_val = getattr(temp_val, prop_key)
                            else:
                                temp_val = None; break
                        if temp_val is not None:
                            print(f"Agent log (_extract_v5): Resolved '{matched_context_key}{property_path_str}' from context for param '{param_name}'. Value: {str(temp_val)[:50]}...")
                            return temp_val
                    except Exception as e:
                        print(f"Agent log (_extract_v5): Error accessing property path '{property_path_str}' on context var '{matched_context_key}': {e}")
                else: # Direct use of context variable's value
                    print(f"Agent log (_extract_v5): Resolved '{matched_context_key}' directly from context for param '{param_name}'. Value: {str(value_from_context)[:50]}...")
                    return value_from_context

    # B. Explicit Assignment (param_name = 'value' or param_name: value) - highest precedence after context vars
    explicit_assign_match_quoted = re.search(rf"\b{param_name}\s*[:=]\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
    if explicit_assign_match_quoted: return explicit_assign_match_quoted.group(1)
            
    explicit_assign_match_unquoted = re.search(rf"\b{param_name}\s*[:=]\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
    if explicit_assign_match_unquoted:
        val_str = explicit_assign_match_unquoted.group(1)
        if param_type == int: try: return int(val_str) except ValueError: pass
        if param_type == float: try: return float(val_str) except ValueError: pass
        if param_type == bool:
            if val_str.lower() == 'true': return True
            if val_str.lower() == 'false': return False
        if param_type == str or param_type is None or param_type == inspect.Parameter.empty : return val_str
    
    # C. Specific keyword-based extraction (e.g., for 'key')
    if param_name == "key":
        key_match = re.search(r"(?:with key|key is|for key|as key|named key)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if key_match: return key_match.group(1)
        key_match_unquoted = re.search(r"(?:with key|key is|for key|as key|named key)\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
        if key_match_unquoted: return key_match_unquoted.group(1)

    # D. URL, Filepath, Directory (largely from v4/v3, respecting consumed_quoted_strings)
    if param_name == "url": # ... (same URL logic as _v4/_v3) ...
        url_match_re = r"https?://(?:[a-zA-Z0-9.\-_~:/?#\[\]@!$&'()*+,;=%]+[a-zA-Z0-9\-_~:/?#\[\]@!$&*+,;=%])"; url_match = re.search(url_match_re, goal_description)
        if url_match: extracted_url = url_match.group(0);
        if url_match:
            if extracted_url.endswith("'") and goal_description.count(f"'{extracted_url}'") > 0: extracted_url = extracted_url[:-1]
            elif extracted_url.endswith("\"") and goal_description.count(f"\"{extracted_url}\"") > 0: extracted_url = extracted_url[:-1]
            for q_str in all_quoted_strings_from_goal:
                if extracted_url == q_str or f"'{extracted_url}'" == q_str or f"\"{extracted_url}\"" == q_str :
                    if q_str not in consumed_quoted_strings: consumed_quoted_strings.add(q_str); break 
            return extracted_url

    if param_name in ["filepath", "path", "directory"]: # ... (same filepath/path/directory logic as _v4/_v3) ...
        path_pattern_named = re.search(rf"(?:named|called|path is|directory is)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if path_pattern_named: potential_path = path_pattern_named.group(1)
        if path_pattern_named and potential_path in all_quoted_strings_from_goal and potential_path not in consumed_quoted_strings:
            consumed_quoted_strings.add(potential_path); return potential_path
        path_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            if param_name == "directory" or '.' in q_str or '/' in q_str or '\\' in q_str or any(q_str.endswith(ext) for ext in ['.txt', '.json', '.py']):
                path_candidates.append(q_str)
        if path_candidates: chosen_path = max(path_candidates, key=len); consumed_quoted_strings.add(chosen_path); return chosen_path


    # E. Content, Text, Description, Message, Value (general quoted strings, last resort)
    # Added "value" here for save_knowledge, but context resolution (A) should usually catch it.
    if param_name in ["content", "text", "description", "message", "value"]:
        content_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            content_candidates.append(q_str)
        if content_candidates:
            chosen_content = max(content_candidates, key=len)
            consumed_quoted_strings.add(chosen_content)
            return chosen_content
            
    # F. Code String, Method, Bool flags (same as _v4/_v3)
    if param_name == "code_string": #...
        code_block_match = re.search(r"code:\s*(.+)", goal_description, re.IGNORECASE | re.DOTALL);
        if code_block_match: return code_block_match.group(1).strip()
        triple_tick_match = re.search(r"```(?:python\n)?(.*?)```", goal_description, re.DOTALL | re.IGNORECASE)
        if triple_tick_match: return triple_tick_match.group(1).strip()
    if param_name == "method" and (param_type == str or param_type is None or param_type == inspect.Parameter.empty): #...
        method_search = re.search(r"\b(GET|POST|PUT|DELETE|PATCH|HEAD|OPTIONS)\b", goal_description, re.IGNORECASE);
        if method_search: return method_search.group(1).upper()
        if "post" in goal_lower: return "POST"; # Fallbacks
        if "get" in goal_lower: return "GET";
        if "delete" in goal_lower: return "DELETE";
        if "put" in goal_lower: return "PUT";
    if param_type == bool: #...
        if re.search(rf"\b{param_name}\b\s*(true|yes|on|recursively|enabled)\b", goal_lower, re.IGNORECASE): return True
        if re.search(rf"\b(is|be|make it)\s*{param_name}\b", goal_lower, re.IGNORECASE): return True
        if re.search(rf"\b{param_name}\b\s*(false|no|off|disabled)\b", goal_lower, re.IGNORECASE): return False
        if re.search(rf"\b(is not|not to be|don't make it)\s*{param_name}\b", goal_lower, re.IGNORECASE): return False
        if param_name in goal_lower and param_name not in ["false", "no", "off"]: return True
        
    return None

# --- Update attempt_goal_autonomously (v6) to use new heuristic v5 ---
def attempt_goal_autonomously_v6_tool(goal_description: str, additional_context: dict = None) -> dict:
    tool_name_internal = "attempt_goal_autonomously_v6_tool"
    log_prefix = f"Agent log ({tool_name_internal}): "
    print(f"{log_prefix}Attempting goal (v6): '{goal_description[:100]}...'. Additional context: {additional_context is not None}")
    
    current_cumulative_context = {}
    if additional_context and "parameters" in additional_context:
        current_cumulative_context = additional_context["parameters"].copy()

    quoted_match_tuples = re.findall(r"\"([^\"]+)\"|\'([^\']+)\'", goal_description)
    all_quoted_strings_from_goal = [item for tpl in quoted_match_tuples for item in tpl if item]
    if 'suggest_tools_for_goal' not in _AGENT_TOOLS: return {"success": False, "message": "Tool 'suggest_tools_for_goal' not found."}
    
    chosen_tool_name_override = additional_context.get("chosen_tool_override") if additional_context else None
    chosen_tool_name = "" # Initialize
    tool_callable = None # Initialize
    if chosen_tool_name_override:
        if chosen_tool_name_override in _AGENT_TOOLS: tool_callable = _AGENT_TOOLS[chosen_tool_name_override]; chosen_tool_name = chosen_tool_name_override; print(f"{log_prefix}Using pre-selected tool: '{chosen_tool_name}'.")
        else: return {"success": False, "message": f"Pre-selected tool '{chosen_tool_name_override}' not found."}
    else:
        suggestion_result = _AGENT_TOOLS['suggest_tools_for_goal'](goal_description=goal_description, top_n=1)
        print(f"{log_prefix}Tool suggestion: {suggestion_result['message']}")
        if not suggestion_result['success'] or not suggestion_result['suggestions']: return {"success": False, "message": "No suitable tools suggested."}
        top_suggestion = suggestion_result['suggestions'][0]
        chosen_tool_name = top_suggestion['tool_name']
        relevance_score = top_suggestion['relevance_score']
        print(f"{log_prefix}Top suggested: '{chosen_tool_name}' (Score: {relevance_score}). Reason: {top_suggestion.get('reason', 'N/A')}")
        if relevance_score < _MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION_V3: return {"success": False, "message": f"Tool score {relevance_score} below threshold."}
        if chosen_tool_name not in _AGENT_TOOLS: return {"success": False, "message": f"Suggested tool '{chosen_tool_name}' not found."}
        tool_callable = _AGENT_TOOLS[chosen_tool_name]

    sig = inspect.signature(tool_callable)
    filled_params = {}
    missing_required_params_info = []
    consumed_quoted_strings_for_this_call = set()
    print(f"{log_prefix}Params for '{chosen_tool_name}': {list(sig.parameters.keys())}")
    param_order = sorted(sig.parameters.items(), key=lambda item: 1 if item[0] in ["filepath", "path", "url", "directory", "key"] else 2)

    for param_name, param_obj in param_order:
        param_type_annotation = param_obj.annotation if param_obj.annotation != inspect.Parameter.empty else None
        if param_name in current_cumulative_context: # Check context first (values provided by planner)
            extracted_value = current_cumulative_context[param_name]
            print(f"{log_prefix}  Using '{param_name}' directly from provided/cumulative context.")
        else:
            extracted_value = _extract_parameter_value_heuristically_v5( # USE V5 HEURISTIC
                param_name, param_type_annotation, goal_description, 
                all_quoted_strings_from_goal, consumed_quoted_strings_for_this_call,
                current_cumulative_context
            )
        if extracted_value is not None:
            if param_type_annotation and isinstance(extracted_value, str):
                try:
                    if param_type_annotation == bool: extracted_value = (extracted_value.lower() == 'true')
                    elif param_type_annotation == int: extracted_value = int(extracted_value)
                    elif param_type_annotation == float: extracted_value = float(extracted_value)
                except ValueError: print(f"{log_prefix}Warn: Could not convert '{extracted_value}' to {param_type_annotation} for '{param_name}'.")
            filled_params[param_name] = extracted_value; print(f"{log_prefix}  Filled '{param_name}': {str(extracted_value)[:50]}...")
        elif param_obj.default != inspect.Parameter.empty:
            filled_params[param_name] = param_obj.default; print(f"{log_prefix}  Default for '{param_name}': {str(param_obj.default)[:50]}...")
        else:
            missing_required_params_info.append({"name": param_name, "type": str(param_type_annotation), "description": f"Value for '{param_name}'."})
            print(f"{log_prefix}  Could not fill required '{param_name}'.")
            
    if missing_required_params_info:
        param_names_missing = [p['name'] for p in missing_required_params_info]
        msg = f"Clarification needed for tool '{chosen_tool_name}': {', '.join(param_names_missing)}."
        clarification_prompt = f"Please provide values for: {', '.join(param_names_missing)} for tool '{chosen_tool_name}' (goal: '{goal_description[:30]}...')."
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, 
                "clarification_request": {"type": "missing_parameters", "original_goal": goal_description, "tool_name": chosen_tool_name, 
                                          "missing_params_details": missing_required_params_info, "prompt": clarification_prompt}}
    
    print(f"{log_prefix}Executing '{chosen_tool_name}' with: {json.dumps(filled_params, default=str, indent=2)}")
    try:
        execution_result = tool_callable(**filled_params)
        overall_success = execution_result.get('success', True) if isinstance(execution_result, dict) else True
        return {"success": overall_success, "message": f"Tool '{chosen_tool_name}' executed.", "chosen_tool": chosen_tool_name,
                "parameters_used": filled_params, "execution_result": execution_result}
    except Exception as e:
        tb_str = traceback.format_exc()
        return {"success": False, "message": f"Error executing '{chosen_tool_name}': {e}", "execution_result": {"error": f"{e}\n{tb_str}"}}

# --- Register/Update Tools ---
print("\nAgent: Registering autonomous goal execution (v6) & updating aliases (FIX 7)...")
if 'add_tool' in globals() and callable(add_tool):
    add_tool(
        name="attempt_goal_autonomously_v6",
        tool_callable=attempt_goal_autonomously_v6_tool,
        description="(Version 6) Uses v5 heuristic (improved context/key/value). Asks for clarification."
    )
    _AGENT_TOOLS['attempt_goal_autonomously_v3'] = attempt_goal_autonomously_v6_tool # Multi-step uses 'v3' alias
    _TOOL_DESCRIPTIONS['attempt_goal_autonomously_v3'] = "(Version 6, aliased as v3 - FIX 7) Uses context-aware v5 heuristic."
    print("Registered 'attempt_goal_autonomously_v6' and aliased 'attempt_goal_autonomously_v3' to point to v6.")
else:
    print("CRITICAL ERROR: 'add_tool' function not found.")

# --- Re-test Multi-Step Goal Execution Tool ---
print("\nAgent: Re-testing multi-step goal execution (v1) with autonomous worker (v6 via v3 alias - FIX 7).")
tool_to_test_name = "execute_multi_step_goal_v1" # This planner calls 'attempt_goal_autonomously_v3' (now v6 logic)
if tool_to_test_name not in _AGENT_TOOLS:
    print(f"Agent: Skipping tests as '{tool_to_test_name}' tool was not registered.")
elif '_extract_parameter_value_heuristically_v5' not in globals() or not callable(globals()['_extract_parameter_value_heuristically_v5']):
    print("CRITICAL ERROR: _extract_parameter_value_heuristically_v5 is not available. Tests will fail.")
else:
    overall_goal1 = "Download todo item 2, save its title to 'todo_title_fix7.txt', read file, store content in knowledge as 'todo_title_fix7'."
    sub_goals1 = [
        "Download content from 'https://jsonplaceholder.typicode.com/todos/2'",
        "Write the 'title' from the 'last_json_content' to a file named 'todo_title_fix7.txt'",
        "Read the file 'todo_title_fix7.txt'",
        "Save the 'last_content' from reading the file into knowledge with key 'todo_title_fix7'" # Clarified source of 'last_content'
    ]
    print(f"\n--- Test 1 (Multi-step REVISED FIX 7): Goal: '{overall_goal1}' ---")
    result1 = _AGENT_TOOLS[tool_to_test_name](overall_goal_description=overall_goal1, sub_goal_strings=sub_goals1)
    print(f"Result for multi-step goal 1: {json.dumps(result1, default=str, indent=2)}")
    if result1.get('success'):
        print("  SUCCESS (Test 1 REVISED FIX 7): Multi-step goal reported success.")
        if 'retrieve_knowledge' in _AGENT_TOOLS:
            kv = _AGENT_TOOLS['retrieve_knowledge'](key='todo_title_fix7')
            # Expected title for todo #2: "quis ut nam facilis et officia qui"
            if kv.get('found') and kv.get('value') == "quis ut nam facilis et officia qui":
                print(f"  VERIFIED (Test 1 REVISED FIX 7): Knowledge 'todo_title_fix7' correct.")
            else: 
                print(f"  VERIFICATION FAILED (Test 1 REVISED FIX 7): Knowledge 'todo_title_fix7'. Expected 'quis ut nam facilis et officia qui', Got: {kv.get('value')}")
            _AGENT_TOOLS['delete_knowledge'](key='todo_title_fix7')
        if 'delete_file' in _AGENT_TOOLS: _AGENT_TOOLS['delete_file'](filepath='todo_title_fix7.txt')
    else:
        print(f"  FAILURE (Test 1 REVISED FIX 7): Multi-step goal failed. Message: {result1.get('message')}")
        if result1.get('sub_goal_results'):
            for i, r_sg in enumerate(result1['sub_goal_results']):
                print(f"    Sub-goal {i+1}: success={r_sg.get('success')}, tool={r_sg.get('chosen_tool')}, msg={r_sg.get('message')}")
                if r_sg.get('parameters_used'): print(f"      Params used: {r_sg.get('parameters_used')}")
                if r_sg.get('clarification_request'): print(f"      Clarification: {r_sg.get('clarification_request')}")

print("\nAgent: Context-aware parameter extraction (FIX 7) and re-tests complete.")
print("Waiting for next instruction.")

SyntaxError: invalid syntax (3442758365.py, line 83)

In [23]:
# Existing tools, _AGENT_TOOL_CONCEPT_KEYWORDS etc., assumed.
import inspect
import re
import json
import traceback

print("Agent: Refining context path resolution and key/value extraction (FIX 8 - syntax correction).")

# --- 1. Refine Parameter Extraction Heuristic (v5 - with final syntax fix) ---
def _extract_parameter_value_heuristically_v5(
    param_name: str, 
    param_type, 
    goal_description: str, 
    all_quoted_strings_from_goal: list[str],
    consumed_quoted_strings: set[str],
    current_cumulative_context: dict 
) -> any:
    goal_lower = goal_description.lower()
    
    context_key_pattern_str = ""
    if current_cumulative_context.keys():
        context_key_pattern_str = f"({ '|'.join(re.escape(k) for k in current_cumulative_context.keys() if k) })"
    
    if context_key_pattern_str:
        context_ref_match = re.search(
            rf"(?:the|value of|from|content of)\s*['\"]{context_key_pattern_str}((?:(?:\.[\w_][\w\d_]*)|(?:\[['\"]\w+['\"]\]))+)?['\"]",
            goal_description, re.IGNORECASE
        )
        if not context_ref_match:
             context_ref_match = re.search(
                rf"['\"]{context_key_pattern_str}((?:(?:\.[\w_][\w\d_]*)|(?:\[['\"]\w+['\"]\]))+)?['\"]",
                 goal_description, re.IGNORECASE
             )
        if context_ref_match:
            matched_context_key = context_ref_match.group(1)
            property_path_str = context_ref_match.group(2)
            if matched_context_key in current_cumulative_context:
                value_from_context = current_cumulative_context[matched_context_key]
                if property_path_str:
                    properties = []
                    for part in re.findall(r"\.([\w_][\w\d_]*)|\[['\"]([^'\"]+)['\"]\]", property_path_str):
                        properties.append(part[0] or part[1])
                    temp_val = value_from_context
                    try:
                        for prop_key in properties:
                            if isinstance(temp_val, dict): temp_val = temp_val.get(prop_key)
                            elif isinstance(temp_val, list) and prop_key.isdigit(): temp_val = temp_val[int(prop_key)]
                            elif hasattr(temp_val, prop_key): temp_val = getattr(temp_val, prop_key)
                            else: temp_val = None; break
                        if temp_val is not None:
                            print(f"Agent log (_extract_v5): Resolved '{matched_context_key}{property_path_str}' from context for param '{param_name}'. Value: {str(temp_val)[:50]}...")
                            return temp_val
                    except Exception as e:
                        print(f"Agent log (_extract_v5): Error accessing property path '{property_path_str}' on context var '{matched_context_key}': {e}")
                else:
                    print(f"Agent log (_extract_v5): Resolved '{matched_context_key}' directly from context for param '{param_name}'. Value: {str(value_from_context)[:50]}...")
                    return value_from_context

    explicit_assign_match_quoted = re.search(rf"\b{param_name}\s*[:=]\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
    if explicit_assign_match_quoted: return explicit_assign_match_quoted.group(1)
            
    explicit_assign_match_unquoted = re.search(rf"\b{param_name}\s*[:=]\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
    if explicit_assign_match_unquoted:
        val_str = explicit_assign_match_unquoted.group(1)
        if param_type == int:
            try: # CORRECTED BLOCK
                return int(val_str)
            except ValueError:
                pass
        if param_type == float:
            try: # CORRECTED BLOCK
                return float(val_str)
            except ValueError:
                pass
        if param_type == bool:
            if val_str.lower() == 'true': return True
            if val_str.lower() == 'false': return False
        if param_type == str or param_type is None or param_type == inspect.Parameter.empty : return val_str
    
    if param_name == "key":
        key_match = re.search(r"(?:with key|key is|for key|as key|named key)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if key_match: return key_match.group(1)
        key_match_unquoted = re.search(r"(?:with key|key is|for key|as key|named key)\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
        if key_match_unquoted: return key_match_unquoted.group(1)

    if param_name == "url":
        url_match_re = r"https?://(?:[a-zA-Z0-9.\-_~:/?#\[\]@!$&'()*+,;=%]+[a-zA-Z0-9\-_~:/?#\[\]@!$&*+,;=%])"; url_match = re.search(url_match_re, goal_description)
        if url_match: extracted_url = url_match.group(0);
        if url_match: # ensure url_match is not None before proceeding
            if extracted_url.endswith("'") and goal_description.count(f"'{extracted_url}'") > 0: extracted_url = extracted_url[:-1]
            elif extracted_url.endswith("\"") and goal_description.count(f"\"{extracted_url}\"") > 0: extracted_url = extracted_url[:-1]
            for q_str in all_quoted_strings_from_goal:
                if extracted_url == q_str or f"'{extracted_url}'" == q_str or f"\"{extracted_url}\"" == q_str :
                    if q_str not in consumed_quoted_strings: consumed_quoted_strings.add(q_str); break 
            return extracted_url

    if param_name in ["filepath", "path", "directory"]:
        path_pattern_named = re.search(rf"(?:named|called|path is|directory is)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if path_pattern_named:
            potential_path = path_pattern_named.group(1)
            # Check if potential_path is one of the originally quoted strings and not consumed
            if potential_path in all_quoted_strings_from_goal and potential_path not in consumed_quoted_strings:
                consumed_quoted_strings.add(potential_path); return potential_path
        path_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            if param_name == "directory" or '.' in q_str or '/' in q_str or '\\' in q_str or any(q_str.endswith(ext) for ext in ['.txt', '.json', '.py']):
                path_candidates.append(q_str)
        if path_candidates: chosen_path = max(path_candidates, key=len); consumed_quoted_strings.add(chosen_path); return chosen_path

    if param_name in ["content", "text", "description", "message", "value"]:
        content_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            content_candidates.append(q_str)
        if content_candidates:
            chosen_content = max(content_candidates, key=len)
            consumed_quoted_strings.add(chosen_content)
            return chosen_content
            
    if param_name == "code_string":
        code_block_match = re.search(r"code:\s*(.+)", goal_description, re.IGNORECASE | re.DOTALL);
        if code_block_match: return code_block_match.group(1).strip()
        triple_tick_match = re.search(r"```(?:python\n)?(.*?)```", goal_description, re.DOTALL | re.IGNORECASE)
        if triple_tick_match: return triple_tick_match.group(1).strip()
        
    if param_name == "method" and (param_type == str or param_type is None or param_type == inspect.Parameter.empty):
        method_search = re.search(r"\b(GET|POST|PUT|DELETE|PATCH|HEAD|OPTIONS)\b", goal_description, re.IGNORECASE);
        if method_search: return method_search.group(1).upper()
        if "post" in goal_lower: return "POST"; 
        if "get" in goal_lower: return "GET";
        if "delete" in goal_lower: return "DELETE";
        if "put" in goal_lower: return "PUT";
        
    if param_type == bool:
        if re.search(rf"\b{param_name}\b\s*(true|yes|on|recursively|enabled)\b", goal_lower, re.IGNORECASE): return True
        if re.search(rf"\b(is|be|make it)\s*{param_name}\b", goal_lower, re.IGNORECASE): return True
        if re.search(rf"\b{param_name}\b\s*(false|no|off|disabled)\b", goal_lower, re.IGNORECASE): return False
        if re.search(rf"\b(is not|not to be|don't make it)\s*{param_name}\b", goal_lower, re.IGNORECASE): return False
        if param_name in goal_lower and param_name not in ["false", "no", "off"]: return True
    return None

# --- Update attempt_goal_autonomously (v6 - no changes to its own code, uses heuristic v5) ---
# Assuming attempt_goal_autonomously_v6_tool was defined in the previous turn and will use the
# globally corrected _extract_parameter_value_heuristically_v5.

# --- Register/Update Tools ---
print("\nAgent: Registering/Updating tools with corrected heuristic (FIX 8)...")
if '_AGENT_TOOLS' in globals() and 'add_tool' in globals() and callable(add_tool):
    # Assuming attempt_goal_autonomously_v6_tool is already registered and
    # _AGENT_TOOLS['attempt_goal_autonomously_v3'] points to it.
    # The critical change is the correction in _extract_parameter_value_heuristically_v5
    # which will be picked up by attempt_goal_autonomously_v6_tool in its next call.
    print("Ensured heuristic _extract_parameter_value_heuristically_v5 is corrected. Aliases should pick this up.")
else:
    print("CRITICAL ERROR: _AGENT_TOOLS or 'add_tool' not available.")

# --- Re-test Multi-Step Goal Execution Tool ---
print("\nAgent: Re-testing multi-step goal execution (v1) with corrected heuristic (FIX 8).")
tool_to_test_name = "execute_multi_step_goal_v1"
if tool_to_test_name not in _AGENT_TOOLS:
    print(f"Agent: Skipping tests as '{tool_to_test_name}' tool was not registered.")
elif '_extract_parameter_value_heuristically_v5' not in globals() or not callable(globals()['_extract_parameter_value_heuristically_v5']):
    print("CRITICAL ERROR: _extract_parameter_value_heuristically_v5 is not available for the test.")
else:
    overall_goal1 = "Download todo item 2, save its title to 'todo_title_fix8.txt', read file, store content in knowledge as 'todo_title_fix8'."
    sub_goals1 = [
        "Download content from 'https://jsonplaceholder.typicode.com/todos/2'",
        "Write the 'title' from the 'last_json_content' to a file named 'todo_title_fix8.txt'",
        "Read the file 'todo_title_fix8.txt'",
        "Save the 'last_content' from reading the file into knowledge with key 'todo_title_fix8'"
    ]
    print(f"\n--- Test 1 (Multi-step REVISED FIX 8): Goal: '{overall_goal1}' ---")
    result1 = _AGENT_TOOLS[tool_to_test_name](overall_goal_description=overall_goal1, sub_goal_strings=sub_goals1)
    print(f"Result for multi-step goal 1: {json.dumps(result1, default=str, indent=2)}")
    if result1.get('success'):
        print("  SUCCESS (Test 1 REVISED FIX 8): Multi-step goal reported success.")
        if 'retrieve_knowledge' in _AGENT_TOOLS:
            kv = _AGENT_TOOLS['retrieve_knowledge'](key='todo_title_fix8')
            if kv.get('found') and kv.get('value') == "quis ut nam facilis et officia qui":
                print(f"  VERIFIED (Test 1 REVISED FIX 8): Knowledge 'todo_title_fix8' correct.")
            else: 
                print(f"  VERIFICATION FAILED (Test 1 REVISED FIX 8): Knowledge 'todo_title_fix8'. Expected 'quis ut nam facilis et officia qui', Got: {kv.get('value')}")
            _AGENT_TOOLS['delete_knowledge'](key='todo_title_fix8')
        if 'delete_file' in _AGENT_TOOLS: _AGENT_TOOLS['delete_file'](filepath='todo_title_fix8.txt')
    else:
        print(f"  FAILURE (Test 1 REVISED FIX 8): Multi-step goal failed. Message: {result1.get('message')}")
        if result1.get('sub_goal_results'):
            for i, r_sg in enumerate(result1['sub_goal_results']):
                print(f"    Sub-goal {i+1}: success={r_sg.get('success')}, tool={r_sg.get('chosen_tool')}, msg={r_sg.get('message')}")
                if r_sg.get('parameters_used'): print(f"      Params used: {r_sg.get('parameters_used')}")

print("\nAgent: Context-aware parameter extraction (FIX 8) and re-tests complete.")
print("Waiting for next instruction.")

Agent: Refining context path resolution and key/value extraction (FIX 8 - syntax correction).

Agent: Registering/Updating tools with corrected heuristic (FIX 8)...
Ensured heuristic _extract_parameter_value_heuristically_v5 is corrected. Aliases should pick this up.

Agent: Re-testing multi-step goal execution (v1) with corrected heuristic (FIX 8).

--- Test 1 (Multi-step REVISED FIX 8): Goal: 'Download todo item 2, save its title to 'todo_title_fix8.txt', read file, store content in knowledge as 'todo_title_fix8'.' ---
Agent log (execute_multi_step_goal_v1_tool): Starting multi-step goal: 'Download todo item 2, save its title to 'todo_title_fix8.txt', read file, store content in knowledge...'. Number of sub-goals: 4
Agent log (execute_multi_step_goal_v1_tool): Initial cumulative context: {}

Agent log (execute_multi_step_goal_v1_tool): --- Sub-goal 1/4: 'Download content from 'https://jsonplaceholder.typicode.com/todos/2'...' ---
Agent log (execute_multi_step_goal_v1_tool): Context f

In [24]:
# Existing tools, _AGENT_TOOL_CONCEPT_KEYWORDS etc., assumed.
import inspect
import re
import json
import traceback

print("Agent: Correcting syntax in parameter extraction and re-testing (FIX 9).")

# --- 1. Corrected Parameter Extraction Heuristic (v5 - with definitive syntax fix) ---
def _extract_parameter_value_heuristically_v5( # Renaming to v5 for clarity in this block
    param_name: str, 
    param_type, 
    goal_description: str, 
    all_quoted_strings_from_goal: list[str],
    consumed_quoted_strings: set[str],
    current_cumulative_context: dict 
) -> any:
    goal_lower = goal_description.lower()
    
    context_key_pattern_str = ""
    if current_cumulative_context and current_cumulative_context.keys(): # Check if context is not None and has keys
        context_key_pattern_str = f"({ '|'.join(re.escape(k) for k in current_cumulative_context.keys() if k) })"
    
    if context_key_pattern_str:
        context_ref_match = re.search(
            rf"(?:the|value of|from|content of)\s*['\"]{context_key_pattern_str}((?:(?:\.[\w_][\w\d_]*)|(?:\[['\"]\w+['\"]\]))+)?['\"]",
            goal_description, re.IGNORECASE
        )
        if not context_ref_match:
             context_ref_match = re.search(
                rf"['\"]{context_key_pattern_str}((?:(?:\.[\w_][\w\d_]*)|(?:\[['\"]\w+['\"]\]))+)?['\"]",
                 goal_description, re.IGNORECASE
             )
        if context_ref_match:
            matched_context_key = context_ref_match.group(1)
            property_path_str = context_ref_match.group(2)
            if matched_context_key in current_cumulative_context:
                value_from_context = current_cumulative_context[matched_context_key]
                if property_path_str:
                    properties = []
                    for part in re.findall(r"\.([\w_][\w\d_]*)|\[['\"]([^'\"]+)['\"]\]", property_path_str):
                        properties.append(part[0] or part[1])
                    temp_val = value_from_context
                    try:
                        for prop_key in properties:
                            if isinstance(temp_val, dict): temp_val = temp_val.get(prop_key)
                            elif isinstance(temp_val, list) and prop_key.isdigit(): temp_val = temp_val[int(prop_key)]
                            elif hasattr(temp_val, prop_key): temp_val = getattr(temp_val, prop_key)
                            else: temp_val = None; break
                        if temp_val is not None:
                            print(f"Agent log (_extract_v5_fix9): Resolved context '{matched_context_key}{property_path_str}' for '{param_name}'. Value: {str(temp_val)[:30]}...")
                            return temp_val
                    except Exception as e:
                        print(f"Agent log (_extract_v5_fix9): Error accessing property '{property_path_str}' on '{matched_context_key}': {e}")
                else:
                    print(f"Agent log (_extract_v5_fix9): Resolved context '{matched_context_key}' for '{param_name}'. Value: {str(value_from_context)[:30]}...")
                    return value_from_context

    explicit_assign_match_quoted = re.search(rf"\b{param_name}\s*[:=]\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
    if explicit_assign_match_quoted: return explicit_assign_match_quoted.group(1)
            
    explicit_assign_match_unquoted = re.search(rf"\b{param_name}\s*[:=]\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
    if explicit_assign_match_unquoted:
        val_str = explicit_assign_match_unquoted.group(1)
        # ***** DEFINITIVELY CORRECTED SYNTAX FOR TRY-EXCEPT *****
        if param_type == int:
            try:
                return int(val_str)
            except ValueError:
                pass 
        if param_type == float:
            try:
                return float(val_str)
            except ValueError:
                pass
        # ***** END OF CORRECTION *****
        if param_type == bool:
            if val_str.lower() == 'true': return True
            if val_str.lower() == 'false': return False
        if param_type == str or param_type is None or param_type == inspect.Parameter.empty : return val_str
    
    if param_name == "key":
        key_match = re.search(r"(?:with key|key is|for key|as key|named key)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if key_match: return key_match.group(1)
        key_match_unquoted = re.search(r"(?:with key|key is|for key|as key|named key)\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
        if key_match_unquoted: return key_match_unquoted.group(1)

    if param_name == "url":
        url_match_re = r"https?://(?:[a-zA-Z0-9.\-_~:/?#\[\]@!$&'()*+,;=%]+[a-zA-Z0-9\-_~:/?#\[\]@!$&*+,;=%])"; url_m = re.search(url_match_re, goal_description)
        if url_m: extracted_url = url_m.group(0);
        if url_m: 
            if extracted_url.endswith("'") and goal_description.count(f"'{extracted_url}'") > 0: extracted_url = extracted_url[:-1]
            elif extracted_url.endswith("\"") and goal_description.count(f"\"{extracted_url}\"") > 0: extracted_url = extracted_url[:-1]
            for q_str in all_quoted_strings_from_goal:
                if extracted_url == q_str or f"'{extracted_url}'" == q_str or f"\"{extracted_url}\"" == q_str :
                    if q_str not in consumed_quoted_strings: consumed_quoted_strings.add(q_str); break 
            return extracted_url

    if param_name in ["filepath", "path", "directory"]:
        path_pattern_named = re.search(rf"(?:named|called|path is|directory is)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if path_pattern_named:
            potential_path = path_pattern_named.group(1)
            if potential_path in all_quoted_strings_from_goal and potential_path not in consumed_quoted_strings:
                consumed_quoted_strings.add(potential_path); return potential_path
        path_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            if param_name == "directory" or '.' in q_str or '/' in q_str or '\\' in q_str or any(q_str.endswith(ext) for ext in ['.txt', '.json', '.py']):
                path_candidates.append(q_str)
        if path_candidates: chosen_path = max(path_candidates, key=len); consumed_quoted_strings.add(chosen_path); return chosen_path

    if param_name in ["content", "text", "description", "message", "value"]:
        content_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            content_candidates.append(q_str)
        if content_candidates:
            chosen_content = max(content_candidates, key=len)
            consumed_quoted_strings.add(chosen_content)
            return chosen_content
            
    if param_name == "code_string":
        code_block_match = re.search(r"code:\s*(.+)", goal_description, re.IGNORECASE | re.DOTALL);
        if code_block_match: return code_block_match.group(1).strip()
        triple_tick_match = re.search(r"```(?:python\n)?(.*?)```", goal_description, re.DOTALL | re.IGNORECASE)
        if triple_tick_match: return triple_tick_match.group(1).strip()
        
    if param_name == "method" and (param_type == str or param_type is None or param_type == inspect.Parameter.empty):
        method_search = re.search(r"\b(GET|POST|PUT|DELETE|PATCH|HEAD|OPTIONS)\b", goal_description, re.IGNORECASE);
        if method_search: return method_search.group(1).upper()
        if "post" in goal_lower: return "POST"; 
        if "get" in goal_lower: return "GET";
        if "delete" in goal_lower: return "DELETE";
        if "put" in goal_lower: return "PUT";
        
    if param_type == bool:
        if re.search(rf"\b{param_name}\b\s*(true|yes|on|recursively|enabled)\b", goal_lower, re.IGNORECASE): return True
        if re.search(rf"\b(is|be|make it)\s*{param_name}\b", goal_lower, re.IGNORECASE): return True
        if re.search(rf"\b{param_name}\b\s*(false|no|off|disabled)\b", goal_lower, re.IGNORECASE): return False
        if re.search(rf"\b(is not|not to be|don't make it)\s*{param_name}\b", goal_lower, re.IGNORECASE): return False
        if param_name in goal_lower and param_name not in ["false", "no", "off"]: return True
    return None

# --- 2. Re-define attempt_goal_autonomously (v6) to use the corrected heuristic v5 ---
def attempt_goal_autonomously_v6_tool(goal_description: str, additional_context: dict = None) -> dict:
    tool_name_internal = "attempt_goal_autonomously_v6_tool"
    log_prefix = f"Agent log ({tool_name_internal}): "
    print(f"{log_prefix}Attempting goal (v6 - FIX 9): '{goal_description[:100]}...'. Additional context: {additional_context is not None}")
    
    current_cumulative_context = {}
    if additional_context and "parameters" in additional_context:
        current_cumulative_context = additional_context["parameters"].copy()

    quoted_match_tuples = re.findall(r"\"([^\"]+)\"|\'([^\']+)\'", goal_description)
    all_quoted_strings_from_goal = [item for tpl in quoted_match_tuples for item in tpl if item]
    
    # Tool Selection (using existing suggest_tools_for_goal which is v3 logic)
    if 'suggest_tools_for_goal' not in _AGENT_TOOLS: return {"success": False, "message": "Tool 'suggest_tools_for_goal' not found."}
    chosen_tool_name_override = additional_context.get("chosen_tool_override") if additional_context else None
    chosen_tool_name = "" 
    tool_callable = None 
    if chosen_tool_name_override:
        if chosen_tool_name_override in _AGENT_TOOLS: tool_callable = _AGENT_TOOLS[chosen_tool_name_override]; chosen_tool_name = chosen_tool_name_override; print(f"{log_prefix}Using pre-selected tool: '{chosen_tool_name}'.")
        else: return {"success": False, "message": f"Pre-selected tool '{chosen_tool_name_override}' not found."}
    else:
        suggestion_result = _AGENT_TOOLS['suggest_tools_for_goal'](goal_description=goal_description, top_n=1)
        print(f"{log_prefix}Tool suggestion: {suggestion_result['message']}")
        if not suggestion_result['success'] or not suggestion_result['suggestions']: return {"success": False, "message": "No suitable tools suggested."}
        top_suggestion = suggestion_result['suggestions'][0]
        chosen_tool_name = top_suggestion['tool_name']
        relevance_score = top_suggestion['relevance_score']
        print(f"{log_prefix}Top suggested: '{chosen_tool_name}' (Score: {relevance_score}). Reason: {top_suggestion.get('reason', 'N/A')}")
        if relevance_score < _MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION_V3: return {"success": False, "message": f"Tool score {relevance_score} below threshold."} # Reusing threshold name
        if chosen_tool_name not in _AGENT_TOOLS: return {"success": False, "message": f"Suggested tool '{chosen_tool_name}' not found."}
        tool_callable = _AGENT_TOOLS[chosen_tool_name]

    sig = inspect.signature(tool_callable)
    filled_params = {}
    missing_required_params_info = []
    consumed_quoted_strings_for_this_call = set()
    print(f"{log_prefix}Params for '{chosen_tool_name}': {list(sig.parameters.keys())}")
    param_order = sorted(sig.parameters.items(), key=lambda item: 1 if item[0] in ["filepath", "path", "url", "directory", "key"] else 2)

    for param_name, param_obj in param_order:
        param_type_annotation = param_obj.annotation if param_obj.annotation != inspect.Parameter.empty else None
        if param_name in current_cumulative_context:
            extracted_value = current_cumulative_context[param_name]
            print(f"{log_prefix}  Using '{param_name}' directly from provided/cumulative context.")
        else:
            extracted_value = _extract_parameter_value_heuristically_v5( # CALLING THE CORRECTED V5
                param_name, param_type_annotation, goal_description, 
                all_quoted_strings_from_goal, consumed_quoted_strings_for_this_call,
                current_cumulative_context
            )
        if extracted_value is not None:
            if param_type_annotation and isinstance(extracted_value, str):
                try:
                    if param_type_annotation == bool: extracted_value = (extracted_value.lower() == 'true')
                    elif param_type_annotation == int: extracted_value = int(extracted_value)
                    elif param_type_annotation == float: extracted_value = float(extracted_value)
                except ValueError: print(f"{log_prefix}Warn: Could not convert '{extracted_value}' to {param_type_annotation} for '{param_name}'.")
            filled_params[param_name] = extracted_value; print(f"{log_prefix}  Filled '{param_name}': {str(extracted_value)[:50]}...")
        elif param_obj.default != inspect.Parameter.empty:
            filled_params[param_name] = param_obj.default; print(f"{log_prefix}  Default for '{param_name}': {str(param_obj.default)[:50]}...")
        else:
            missing_required_params_info.append({"name": param_name, "type": str(param_type_annotation), "description": f"Value for '{param_name}'."})
            print(f"{log_prefix}  Could not fill required '{param_name}'.")
            
    if missing_required_params_info:
        param_names_missing = [p['name'] for p in missing_required_params_info]
        msg = f"Clarification needed for tool '{chosen_tool_name}': {', '.join(param_names_missing)}."
        clarification_prompt = f"Please provide values for: {', '.join(param_names_missing)} for tool '{chosen_tool_name}' (goal: '{goal_description[:30]}...')."
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, 
                "clarification_request": {"type": "missing_parameters", "original_goal": goal_description, "tool_name": chosen_tool_name, 
                                          "missing_params_details": missing_required_params_info, "prompt": clarification_prompt}}
    
    print(f"{log_prefix}Executing '{chosen_tool_name}' with: {json.dumps(filled_params, default=str, indent=2)}")
    try:
        execution_result = tool_callable(**filled_params)
        overall_success = execution_result.get('success', True) if isinstance(execution_result, dict) else True
        return {"success": overall_success, "message": f"Tool '{chosen_tool_name}' executed.", "chosen_tool": chosen_tool_name,
                "parameters_used": filled_params, "execution_result": execution_result}
    except Exception as e:
        tb_str = traceback.format_exc()
        return {"success": False, "message": f"Error executing '{chosen_tool_name}': {e}", "execution_result": {"error": f"{e}\n{tb_str}"}}

# --- Register/Update Tools ---
print("\nAgent: Registering autonomous goal execution (v6 - FIX 9) & updating aliases...")
if 'add_tool' in globals() and callable(add_tool):
    add_tool(
        name="attempt_goal_autonomously_v6",
        tool_callable=attempt_goal_autonomously_v6_tool,
        description="(Version 6) Uses v5 heuristic (syntax fix). Asks for clarification."
    )
    _AGENT_TOOLS['attempt_goal_autonomously_v3'] = attempt_goal_autonomously_v6_tool
    _TOOL_DESCRIPTIONS['attempt_goal_autonomously_v3'] = "(Version 6, aliased as v3 - FIX 9) Uses context-aware v5 heuristic with syntax fix."
    print("Registered 'attempt_goal_autonomously_v6' and aliased 'attempt_goal_autonomously_v3' to point to v6.")
else:
    print("CRITICAL ERROR: 'add_tool' function not found.")

# --- Re-test Multi-Step Goal Execution Tool ---
print("\nAgent: Re-testing multi-step goal execution (v1) with autonomous worker (v6 via v3 alias - FIX 9).")
tool_to_test_name = "execute_multi_step_goal_v1"
if tool_to_test_name not in _AGENT_TOOLS:
    print(f"Agent: Skipping tests as '{tool_to_test_name}' tool was not registered.")
elif '_extract_parameter_value_heuristically_v5' not in globals() or not callable(globals()['_extract_parameter_value_heuristically_v5']):
    # This check is crucial. If the heuristic wasn't defined due to the syntax error, this will catch it.
    print("CRITICAL ERROR: _extract_parameter_value_heuristically_v5 (corrected) is not available for the test.")
else:
    overall_goal1 = "Download todo item 2, save its title to 'todo_title_fix9.txt', read file, store content in knowledge as 'todo_title_fix9'."
    sub_goals1 = [
        "Download content from 'https://jsonplaceholder.typicode.com/todos/2'",
        "Write the 'title' from the 'last_json_content' to a file named 'todo_title_fix9.txt'",
        "Read the file 'todo_title_fix9.txt'",
        "Save the 'last_content' from reading the file into knowledge with key 'todo_title_fix9'"
    ]
    print(f"\n--- Test 1 (Multi-step REVISED FIX 9): Goal: '{overall_goal1}' ---")
    result1 = _AGENT_TOOLS[tool_to_test_name](overall_goal_description=overall_goal1, sub_goal_strings=sub_goals1)
    print(f"Result for multi-step goal 1: {json.dumps(result1, default=str, indent=2)}")
    if result1.get('success'):
        print("  SUCCESS (Test 1 REVISED FIX 9): Multi-step goal reported success.")
        if 'retrieve_knowledge' in _AGENT_TOOLS:
            kv = _AGENT_TOOLS['retrieve_knowledge'](key='todo_title_fix9')
            if kv.get('found') and kv.get('value') == "quis ut nam facilis et officia qui":
                print(f"  VERIFIED (Test 1 REVISED FIX 9): Knowledge 'todo_title_fix9' correct.")
            else: 
                print(f"  VERIFICATION FAILED (Test 1 REVISED FIX 9): Knowledge 'todo_title_fix9'. Expected 'quis ut nam facilis et officia qui', Got: {kv.get('value')}")
            _AGENT_TOOLS['delete_knowledge'](key='todo_title_fix9')
        if 'delete_file' in _AGENT_TOOLS: _AGENT_TOOLS['delete_file'](filepath='todo_title_fix9.txt')
    else:
        print(f"  FAILURE (Test 1 REVISED FIX 9): Multi-step goal failed. Message: {result1.get('message')}")
        if result1.get('sub_goal_results'):
            for i, r_sg in enumerate(result1['sub_goal_results']):
                print(f"    Sub-goal {i+1}: success={r_sg.get('success')}, tool={r_sg.get('chosen_tool')}, msg={r_sg.get('message')}")
                if r_sg.get('parameters_used'): print(f"      Params used: {r_sg.get('parameters_used')}")

print("\nAgent: Context-aware parameter extraction (FIX 9) and re-tests complete.")
print("Waiting for next instruction.")

Agent: Correcting syntax in parameter extraction and re-testing (FIX 9).

Agent: Registering autonomous goal execution (v6 - FIX 9) & updating aliases...
Tool 'attempt_goal_autonomously_v6' added successfully. Description: (Version 6) Uses v5 heuristic (syntax fix). Asks for clarification.
Registered 'attempt_goal_autonomously_v6' and aliased 'attempt_goal_autonomously_v3' to point to v6.

Agent: Re-testing multi-step goal execution (v1) with autonomous worker (v6 via v3 alias - FIX 9).

--- Test 1 (Multi-step REVISED FIX 9): Goal: 'Download todo item 2, save its title to 'todo_title_fix9.txt', read file, store content in knowledge as 'todo_title_fix9'.' ---
Agent log (execute_multi_step_goal_v1_tool): Starting multi-step goal: 'Download todo item 2, save its title to 'todo_title_fix9.txt', read file, store content in knowledge...'. Number of sub-goals: 4
Agent log (execute_multi_step_goal_v1_tool): Initial cumulative context: {}

Agent log (execute_multi_step_goal_v1_tool): --- Sub-go

In [25]:
# Existing tools, _AGENT_TOOL_CONCEPT_KEYWORDS etc., assumed.
import inspect
import re
import json
import traceback

print("Agent: Refining parameter extraction priorities (literal vs context) (FIX 10).")

# --- 1. Refine Parameter Extraction Heuristic (v6) ---
def _extract_parameter_value_heuristically_v6( # Renaming to v6
    param_name: str, 
    param_type, 
    goal_description: str, 
    all_quoted_strings_from_goal: list[str],
    consumed_quoted_strings: set[str],
    current_cumulative_context: dict 
) -> any:
    goal_lower = goal_description.lower()
    
    # --- Order of Extraction Attempts ---
    # 1. Explicit assignment in goal string (param_name = 'value') - Strongest signal
    # 2. For identifier-like params (key, filepath, path, directory, url): Literal extraction from goal.
    # 3. For data-like params (content, value, json_payload, etc.): Context variable resolution (with property paths).
    # 4. For data-like params: General quoted strings from goal if no context match.
    # 5. General type-based heuristics (method, bool, code_string).

    # 1. Explicit Assignment
    explicit_assign_match_quoted = re.search(rf"\b{param_name}\s*[:=]\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
    if explicit_assign_match_quoted: return explicit_assign_match_quoted.group(1)
    explicit_assign_match_unquoted = re.search(rf"\b{param_name}\s*[:=]\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
    if explicit_assign_match_unquoted: # Convert if type known
        val_str = explicit_assign_match_unquoted.group(1)
        if param_type == int: try: return int(val_str) except ValueError: pass
        if param_type == float: try: return float(val_str) except ValueError: pass
        if param_type == bool:
            if val_str.lower() == 'true': return True
            if val_str.lower() == 'false': return False
        if param_type == str or param_type is None or param_type == inspect.Parameter.empty : return val_str

    # 2. Literal extraction for identifier-like parameters
    if param_name == "key":
        key_match = re.search(r"(?:with key|key is|for key|as key|named key)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if key_match: return key_match.group(1)
        key_match_unquoted = re.search(r"(?:with key|key is|for key|as key|named key)\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
        if key_match_unquoted: return key_match_unquoted.group(1)

    if param_name == "url":
        url_match_re = r"https?://(?:[a-zA-Z0-9.\-_~:/?#\[\]@!$&'()*+,;=%]+[a-zA-Z0-9\-_~:/?#\[\]@!$&*+,;=%])"; url_m = re.search(url_match_re, goal_description)
        if url_m: extracted_url = url_m.group(0)
        if url_m: 
            if extracted_url.endswith("'") and goal_description.count(f"'{extracted_url}'") > 0: extracted_url = extracted_url[:-1]
            elif extracted_url.endswith("\"") and goal_description.count(f"\"{extracted_url}\"") > 0: extracted_url = extracted_url[:-1]
            for q_str in all_quoted_strings_from_goal: # Mark as consumed if it was a quoted URL
                if extracted_url == q_str or f"'{extracted_url}'" == q_str or f"\"{extracted_url}\"" == q_str :
                    if q_str not in consumed_quoted_strings: consumed_quoted_strings.add(q_str); break 
            return extracted_url

    if param_name in ["filepath", "path", "directory"]:
        path_pattern_named = re.search(rf"(?:file named|directory named|path is|called)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if path_pattern_named:
            potential_path = path_pattern_named.group(1)
            if potential_path in all_quoted_strings_from_goal and potential_path not in consumed_quoted_strings:
                consumed_quoted_strings.add(potential_path); return potential_path
        # If not "named 'X'", look for any unconsumed quote that could be a path
        path_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            if param_name == "directory" or '.' in q_str or '/' in q_str or '\\' in q_str or any(q_str.endswith(ext) for ext in ['.txt', '.json', '.py']):
                path_candidates.append(q_str)
        if path_candidates: chosen_path = max(path_candidates, key=len); consumed_quoted_strings.add(chosen_path); return chosen_path


    # 3. Context variable resolution for data-like parameters
    is_data_param = param_name in ["content", "value", "json_payload", "data_payload", "text"] # Add more as needed
    if is_data_param and current_cumulative_context and current_cumulative_context.keys():
        context_key_pattern_str = f"({ '|'.join(re.escape(k) for k in current_cumulative_context.keys() if k) })"
        # Look for "the 'property' of 'context_key'" or "'property' from 'context_key'"
        # OR "the 'context_key's title" (possessive not handled yet, simpler: "title of 'context_key'")
        # This regex prioritizes a property if mentioned.
        # Pattern: (the|value of|from) 'property' (of|from) 'context_key' OR (the|value of|from) 'context_key.property'
        prop_of_context_match = re.search(
            # E.g. "the 'title' from the 'last_json_content'"
            rf"(?:the|value of|from|content of)\s*['\"]([\w_][\w\d_]*)['\"]\s*(?:of|from)\s*['\"]{context_key_pattern_str}['\"]",
            goal_description, re.IGNORECASE
        )
        if prop_of_context_match:
            prop_to_extract = prop_of_context_match.group(1)
            matched_context_key = prop_of_context_match.group(2)
            if matched_context_key in current_cumulative_context:
                base_value = current_cumulative_context[matched_context_key]
                if isinstance(base_value, dict) and prop_to_extract in base_value:
                    val = base_value[prop_to_extract]
                    print(f"Agent log (_extract_v6): Resolved '{prop_to_extract}' from '{matched_context_key}' for param '{param_name}'. Value: {str(val)[:30]}...")
                    return val
        
        # Simpler: (the|value of|from) 'context_key' (potentially with .path)
        context_ref_match = re.search(
            rf"(?:the|value of|from|content of)\s*['\"]{context_key_pattern_str}((?:(?:\.[\w_][\w\d_]*)|(?:\[['\"]\w+['\"]\]))+)?['\"]",
            goal_description, re.IGNORECASE
        )
        if not context_ref_match: # Try direct 'context_key' reference if not qualified by "the", "from" etc.
             context_ref_match = re.search(
                rf"['\"]{context_key_pattern_str}((?:(?:\.[\w_][\w\d_]*)|(?:\[['\"]\w+['\"]\]))+)?['\"]",
                 goal_description, re.IGNORECASE
             )
        if context_ref_match:
            matched_context_key = context_ref_match.group(1)
            property_path_str = context_ref_match.group(2) # e.g. ".title" or "['name']"
            if matched_context_key in current_cumulative_context:
                value_from_context = current_cumulative_context[matched_context_key]
                if property_path_str: # Property path exists
                    properties = []; 
                    for part in re.findall(r"\.([\w_][\w\d_]*)|\[['\"]([^'\"]+)['\"]\]", property_path_str): properties.append(part[0] or part[1])
                    temp_val = value_from_context
                    try:
                        for prop_key in properties:
                            if isinstance(temp_val, dict): temp_val = temp_val.get(prop_key)
                            elif isinstance(temp_val, list) and prop_key.isdigit(): temp_val = temp_val[int(prop_key)]
                            elif hasattr(temp_val, prop_key): temp_val = getattr(temp_val, prop_key)
                            else: temp_val = None; break
                        if temp_val is not None:
                            print(f"Agent log (_extract_v6): Resolved context path '{matched_context_key}{property_path_str}' for '{param_name}'. Value: {str(temp_val)[:30]}...")
                            return temp_val
                    except Exception: pass # Path access failed
                else: # No property path, use the whole context variable
                    print(f"Agent log (_extract_v6): Resolved direct context var '{matched_context_key}' for '{param_name}'. Value: {str(value_from_context)[:30]}...")
                    return value_from_context


    # 4. General quoted strings for data-like parameters if no context match
    if param_name in ["content", "text", "description", "message", "value"]: # "value" for save_knowledge
        content_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            content_candidates.append(q_str)
        if content_candidates:
            chosen_content = max(content_candidates, key=len)
            consumed_quoted_strings.add(chosen_content)
            return chosen_content
            
    # 5. General type-based heuristics (code_string, method, bool)
    if param_name == "code_string": #...
        code_block_match = re.search(r"code:\s*(.+)", goal_description, re.IGNORECASE | re.DOTALL);
        if code_block_match: return code_block_match.group(1).strip()
        triple_tick_match = re.search(r"```(?:python\n)?(.*?)```", goal_description, re.DOTALL | re.IGNORECASE)
        if triple_tick_match: return triple_tick_match.group(1).strip()
    if param_name == "method" and (param_type == str or param_type is None or param_type == inspect.Parameter.empty): #...
        method_search = re.search(r"\b(GET|POST|PUT|DELETE|PATCH|HEAD|OPTIONS)\b", goal_description, re.IGNORECASE);
        if method_search: return method_search.group(1).upper()
        # ... (fallbacks for method) ...
        if "post" in goal_lower: return "POST"; 
        if "get" in goal_lower: return "GET";
        if "delete" in goal_lower: return "DELETE";
        if "put" in goal_lower: return "PUT";
    if param_type == bool: #...
        if re.search(rf"\b{param_name}\b\s*(true|yes|on|recursively|enabled)\b", goal_lower, re.IGNORECASE): return True
        # ... (other boolean checks) ...
        if re.search(rf"\b(is|be|make it)\s*{param_name}\b", goal_lower, re.IGNORECASE): return True
        if re.search(rf"\b{param_name}\b\s*(false|no|off|disabled)\b", goal_lower, re.IGNORECASE): return False
        if re.search(rf"\b(is not|not to be|don't make it)\s*{param_name}\b", goal_lower, re.IGNORECASE): return False
        if param_name in goal_lower and param_name not in ["false", "no", "off"]: return True
        
    return None

# --- Update attempt_goal_autonomously (v7) to use new heuristic v6 ---
def attempt_goal_autonomously_v7_tool(goal_description: str, additional_context: dict = None) -> dict:
    # This function's structure is largely the same as v6_tool,
    # but it calls _extract_parameter_value_heuristically_v6
    # (The definition is verbose, so assuming the structure is as before, just updating the heuristic call)
    tool_name_internal = "attempt_goal_autonomously_v7_tool"
    log_prefix = f"Agent log ({tool_name_internal}): "
    print(f"{log_prefix}Attempting goal (v7 - FIX 10): '{goal_description[:100]}...'. Additional context: {additional_context is not None}")
    current_cumulative_context = (additional_context.get("parameters", {}) if additional_context else {}) or {}
    quoted_match_tuples = re.findall(r"\"([^\"]+)\"|\'([^\']+)\'", goal_description)
    all_quoted_strings_from_goal = [item for tpl in quoted_match_tuples for item in tpl if item]
    if 'suggest_tools_for_goal' not in _AGENT_TOOLS: return {"success": False, "message": "Tool 'suggest_tools_for_goal' not found."}
    chosen_tool_name_override = additional_context.get("chosen_tool_override") if additional_context else None
    chosen_tool_name = ""; tool_callable = None
    if chosen_tool_name_override: # Tool pre-selected
        if chosen_tool_name_override in _AGENT_TOOLS: tool_callable = _AGENT_TOOLS[chosen_tool_name_override]; chosen_tool_name = chosen_tool_name_override; print(f"{log_prefix}Using pre-selected tool: '{chosen_tool_name}'.")
        else: return {"success": False, "message": f"Pre-selected tool '{chosen_tool_name_override}' not found."}
    else: # Suggest tool
        suggestion_result = _AGENT_TOOLS['suggest_tools_for_goal'](goal_description=goal_description, top_n=1)
        print(f"{log_prefix}Tool suggestion: {suggestion_result['message']}")
        if not suggestion_result['success'] or not suggestion_result['suggestions']: return {"success": False, "message": "No suitable tools suggested."}
        top_suggestion = suggestion_result['suggestions'][0]
        chosen_tool_name = top_suggestion['tool_name']; relevance_score = top_suggestion['relevance_score']
        print(f"{log_prefix}Top suggested: '{chosen_tool_name}' (Score: {relevance_score}). Reason: {top_suggestion.get('reason', 'N/A')}")
        if relevance_score < _MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION_V3: return {"success": False, "message": f"Tool score {relevance_score} below threshold."}
        if chosen_tool_name not in _AGENT_TOOLS: return {"success": False, "message": f"Suggested tool '{chosen_tool_name}' not found."}
        tool_callable = _AGENT_TOOLS[chosen_tool_name]
    sig = inspect.signature(tool_callable); filled_params = {}; missing_required_params_info = []; consumed_quoted_strings_for_this_call = set()
    print(f"{log_prefix}Params for '{chosen_tool_name}': {list(sig.parameters.keys())}")
    param_order = sorted(sig.parameters.items(), key=lambda item: 0 if item[0] in ["key", "filepath", "path", "url", "directory"] else 1) # Prioritize identifiers

    for param_name, param_obj in param_order:
        param_type_annotation = param_obj.annotation if param_obj.annotation != inspect.Parameter.empty else None
        if param_name in current_cumulative_context:
            extracted_value = current_cumulative_context[param_name]; print(f"{log_prefix}  Using '{param_name}' from context.")
        else:
            extracted_value = _extract_parameter_value_heuristically_v6( # CALLING THE CORRECTED V6 HEURISTIC
                param_name, param_type_annotation, goal_description, 
                all_quoted_strings_from_goal, consumed_quoted_strings_for_this_call,
                current_cumulative_context
            )
        if extracted_value is not None:
            if param_type_annotation and isinstance(extracted_value, str): # Type conversion
                try:
                    if param_type_annotation == bool: extracted_value = (extracted_value.lower() == 'true')
                    elif param_type_annotation == int: extracted_value = int(extracted_value)
                    elif param_type_annotation == float: extracted_value = float(extracted_value)
                except ValueError: print(f"{log_prefix}Warn: Could not convert '{extracted_value}' for '{param_name}'.")
            filled_params[param_name] = extracted_value; print(f"{log_prefix}  Filled '{param_name}': {str(extracted_value)[:50]}...")
        elif param_obj.default != inspect.Parameter.empty:
            filled_params[param_name] = param_obj.default; print(f"{log_prefix}  Default for '{param_name}': {str(param_obj.default)[:50]}...")
        else:
            missing_required_params_info.append({"name": param_name, "type": str(param_type_annotation), "description": f"Value for '{param_name}'."}); print(f"{log_prefix}  Could not fill required '{param_name}'.")
    if missing_required_params_info: # Clarification
        param_names_missing = [p['name'] for p in missing_required_params_info]; msg = f"Clarification needed for tool '{chosen_tool_name}': {', '.join(param_names_missing)}."
        clar_prompt = f"Please provide values for: {', '.join(param_names_missing)} for tool '{chosen_tool_name}' (goal: '{goal_description[:30]}...')."
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, "clarification_request": {"type": "missing_parameters", "original_goal": goal_description, "tool_name": chosen_tool_name, "missing_params_details": missing_required_params_info, "prompt": clar_prompt}}
    print(f"{log_prefix}Executing '{chosen_tool_name}' with: {json.dumps(filled_params, default=str, indent=2)}")
    try: # Execution
        execution_result = tool_callable(**filled_params)
        overall_success = execution_result.get('success', True) if isinstance(execution_result, dict) else True
        return {"success": overall_success, "message": f"Tool '{chosen_tool_name}' executed.", "chosen_tool": chosen_tool_name, "parameters_used": filled_params, "execution_result": execution_result}
    except Exception as e: tb_str = traceback.format_exc(); return {"success": False, "message": f"Error executing '{chosen_tool_name}': {e}", "execution_result": {"error": f"{e}\n{tb_str}"}}

# --- Register/Update Tools ---
print("\nAgent: Registering autonomous goal execution (v7 - FIX 10) & updating aliases...")
if 'add_tool' in globals() and callable(add_tool):
    add_tool(
        name="attempt_goal_autonomously_v7",
        tool_callable=attempt_goal_autonomously_v7_tool,
        description="(Version 7) Uses v6 heuristic (improved context/key/value priorities). Asks for clarification."
    )
    _AGENT_TOOLS['attempt_goal_autonomously_v3'] = attempt_goal_autonomously_v7_tool # Multi-step uses 'v3' alias
    _TOOL_DESCRIPTIONS['attempt_goal_autonomously_v3'] = "(Version 7, aliased as v3 - FIX 10) Uses context-aware v6 heuristic."
    print("Registered 'attempt_goal_autonomously_v7' and aliased 'attempt_goal_autonomously_v3' to point to v7.")
else:
    print("CRITICAL ERROR: 'add_tool' function not found.")

# --- Re-test Multi-Step Goal Execution Tool ---
print("\nAgent: Re-testing multi-step goal execution (v1) with autonomous worker (v7 via v3 alias - FIX 10).")
tool_to_test_name = "execute_multi_step_goal_v1"
if tool_to_test_name not in _AGENT_TOOLS:
    print(f"Agent: Skipping tests as '{tool_to_test_name}' tool was not registered.")
elif '_extract_parameter_value_heuristically_v6' not in globals() or not callable(globals()['_extract_parameter_value_heuristically_v6']):
    print("CRITICAL ERROR: _extract_parameter_value_heuristically_v6 is not available for the test.")
else:
    overall_goal1 = "Download todo item 2, save its title to 'todo_title_fix10.txt', read file, store content in knowledge as 'todo_title_fix10'."
    sub_goals1 = [
        "Download content from 'https://jsonplaceholder.typicode.com/todos/2'",
        "Write the 'title' from the 'last_json_content' to a file named 'todo_title_fix10.txt'",
        "Read the file 'todo_title_fix10.txt'",
        "Save the 'last_content' from reading the file into knowledge with key 'todo_title_fix10'"
    ]
    print(f"\n--- Test 1 (Multi-step REVISED FIX 10): Goal: '{overall_goal1}' ---")
    result1 = _AGENT_TOOLS[tool_to_test_name](overall_goal_description=overall_goal1, sub_goal_strings=sub_goals1)
    print(f"Result for multi-step goal 1: {json.dumps(result1, default=str, indent=2)}")
    if result1.get('success'):
        print("  SUCCESS (Test 1 REVISED FIX 10): Multi-step goal reported success.")
        if 'retrieve_knowledge' in _AGENT_TOOLS:
            kv = _AGENT_TOOLS['retrieve_knowledge'](key='todo_title_fix10')
            if kv.get('found') and kv.get('value') == "quis ut nam facilis et officia qui":
                print(f"  VERIFIED (Test 1 REVISED FIX 10): Knowledge 'todo_title_fix10' correct.")
            else: 
                print(f"  VERIFICATION FAILED (Test 1 REVISED FIX 10): Knowledge 'todo_title_fix10'. Expected 'quis ut nam facilis et officia qui', Got: {kv.get('value')}")
            _AGENT_TOOLS['delete_knowledge'](key='todo_title_fix10')
        if 'delete_file' in _AGENT_TOOLS: _AGENT_TOOLS['delete_file'](filepath='todo_title_fix10.txt')
    else:
        print(f"  FAILURE (Test 1 REVISED FIX 10): Multi-step goal failed. Message: {result1.get('message')}")
        if result1.get('sub_goal_results'):
            for i, r_sg in enumerate(result1['sub_goal_results']):
                print(f"    Sub-goal {i+1}: success={r_sg.get('success')}, tool={r_sg.get('chosen_tool')}, msg={r_sg.get('message')}")
                if r_sg.get('parameters_used'): print(f"      Params used: {r_sg.get('parameters_used')}")

print("\nAgent: Parameter extraction priorities (FIX 10) and re-tests complete.")
print("Waiting for next instruction.")

SyntaxError: invalid syntax (2130619024.py, line 33)

In [26]:
# Existing tools, _AGENT_TOOL_CONCEPT_KEYWORDS etc., assumed.
import inspect
import re
import json
import traceback

print("Agent: Correcting syntax definitively in parameter extraction (FIX 11).")

# --- 1. Refine Parameter Extraction Heuristic (v7 - with FINAL syntax fix for try-except) ---
def _extract_parameter_value_heuristically_v7(
    param_name: str, 
    param_type, 
    goal_description: str, 
    all_quoted_strings_from_goal: list[str],
    consumed_quoted_strings: set[str],
    current_cumulative_context: dict 
) -> any:
    goal_lower = goal_description.lower()
    
    context_key_pattern_str = ""
    if current_cumulative_context and current_cumulative_context.keys():
        context_key_pattern_str = f"({ '|'.join(re.escape(k) for k in current_cumulative_context.keys() if k) })"
    
    if context_key_pattern_str:
        context_ref_match = re.search(
            rf"(?:the|value of|from|content of)\s*['\"]{context_key_pattern_str}((?:(?:\.[\w_][\w\d_]*)|(?:\[['\"]\w+['\"]\]))+)?['\"]",
            goal_description, re.IGNORECASE
        )
        if not context_ref_match:
             context_ref_match = re.search(
                rf"['\"]{context_key_pattern_str}((?:(?:\.[\w_][\w\d_]*)|(?:\[['\"]\w+['\"]\]))+)?['\"]",
                 goal_description, re.IGNORECASE
             )
        if context_ref_match:
            matched_context_key = context_ref_match.group(1)
            property_path_str = context_ref_match.group(2)
            if matched_context_key in current_cumulative_context:
                value_from_context = current_cumulative_context[matched_context_key]
                if property_path_str:
                    properties = []
                    for part in re.findall(r"\.([\w_][\w\d_]*)|\[['\"]([^'\"]+)['\"]\]", property_path_str):
                        properties.append(part[0] or part[1])
                    temp_val = value_from_context
                    try:
                        for prop_key in properties:
                            if isinstance(temp_val, dict): temp_val = temp_val.get(prop_key)
                            elif isinstance(temp_val, list) and prop_key.isdigit(): temp_val = temp_val[int(prop_key)]
                            elif hasattr(temp_val, prop_key): temp_val = getattr(temp_val, prop_key)
                            else: temp_val = None; break
                        if temp_val is not None:
                            print(f"Agent log (_extract_v7_fix11): Resolved context '{matched_context_key}{property_path_str}' for '{param_name}'.")
                            return temp_val
                    except Exception as e:
                        print(f"Agent log (_extract_v7_fix11): Error accessing property '{property_path_str}' on '{matched_context_key}': {e}")
                else:
                    print(f"Agent log (_extract_v7_fix11): Resolved context '{matched_context_key}' for '{param_name}'.")
                    return value_from_context

    explicit_assign_match_quoted = re.search(rf"\b{param_name}\s*[:=]\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
    if explicit_assign_match_quoted: return explicit_assign_match_quoted.group(1)
            
    explicit_assign_match_unquoted = re.search(rf"\b{param_name}\s*[:=]\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
    if explicit_assign_match_unquoted:
        val_str = explicit_assign_match_unquoted.group(1)
        if param_type == int:
            try: # CORRECTED BLOCK
                return int(val_str)
            except ValueError:
                pass 
        if param_type == float:
            try: # CORRECTED BLOCK
                return float(val_str)
            except ValueError:
                pass
        if param_type == bool:
            if val_str.lower() == 'true': return True
            if val_str.lower() == 'false': return False
        if param_type == str or param_type is None or param_type == inspect.Parameter.empty : return val_str
    
    if param_name == "key":
        key_match = re.search(r"(?:with key|key is|for key|as key|named key)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if key_match: return key_match.group(1)
        key_match_unquoted = re.search(r"(?:with key|key is|for key|as key|named key)\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
        if key_match_unquoted: return key_match_unquoted.group(1)

    if param_name == "url":
        url_match_re = r"https?://(?:[a-zA-Z0-9.\-_~:/?#\[\]@!$&'()*+,;=%]+[a_zA-Z0-9\-_~:/?#\[\]@!$&*+,;=%])"; url_m = re.search(url_match_re, goal_description)
        extracted_url = None # Initialize
        if url_m: extracted_url = url_m.group(0)
        if url_m: 
            if extracted_url.endswith("'") and goal_description.count(f"'{extracted_url}'") > 0: extracted_url = extracted_url[:-1]
            elif extracted_url.endswith("\"") and goal_description.count(f"\"{extracted_url}\"") > 0: extracted_url = extracted_url[:-1]
            for q_str in all_quoted_strings_from_goal:
                if extracted_url == q_str or f"'{extracted_url}'" == q_str or f"\"{extracted_url}\"" == q_str :
                    if q_str not in consumed_quoted_strings: consumed_quoted_strings.add(q_str); break 
            return extracted_url

    if param_name in ["filepath", "path", "directory"]:
        path_pattern_named = re.search(rf"(?:file named|directory named|path is|called)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if path_pattern_named:
            potential_path = path_pattern_named.group(1)
            if potential_path in all_quoted_strings_from_goal and potential_path not in consumed_quoted_strings:
                consumed_quoted_strings.add(potential_path); return potential_path
        path_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            if param_name == "directory" or '.' in q_str or '/' in q_str or '\\' in q_str or any(q_str.endswith(ext) for ext in ['.txt', '.json', '.py']):
                path_candidates.append(q_str)
        if path_candidates: chosen_path = max(path_candidates, key=len); consumed_quoted_strings.add(chosen_path); return chosen_path

    if param_name in ["content", "text", "description", "message", "value"]:
        content_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            content_candidates.append(q_str)
        if content_candidates:
            chosen_content = max(content_candidates, key=len)
            consumed_quoted_strings.add(chosen_content)
            return chosen_content
            
    if param_name == "code_string":
        code_block_match = re.search(r"code:\s*(.+)", goal_description, re.IGNORECASE | re.DOTALL);
        if code_block_match: return code_block_match.group(1).strip()
        triple_tick_match = re.search(r"```(?:python\n)?(.*?)```", goal_description, re.DOTALL | re.IGNORECASE)
        if triple_tick_match: return triple_tick_match.group(1).strip()
        
    if param_name == "method" and (param_type == str or param_type is None or param_type == inspect.Parameter.empty):
        method_search = re.search(r"\b(GET|POST|PUT|DELETE|PATCH|HEAD|OPTIONS)\b", goal_description, re.IGNORECASE);
        if method_search: return method_search.group(1).upper()
        if "post" in goal_lower: return "POST"; 
        if "get" in goal_lower: return "GET";
        if "delete" in goal_lower: return "DELETE";
        if "put" in goal_lower: return "PUT";
        
    if param_type == bool:
        if re.search(rf"\b{param_name}\b\s*(true|yes|on|recursively|enabled)\b", goal_lower, re.IGNORECASE): return True
        if re.search(rf"\b(is|be|make it)\s*{param_name}\b", goal_lower, re.IGNORECASE): return True
        if re.search(rf"\b{param_name}\b\s*(false|no|off|disabled)\b", goal_lower, re.IGNORECASE): return False
        if re.search(rf"\b(is not|not to be|don't make it)\s*{param_name}\b", goal_lower, re.IGNORECASE): return False
        if param_name in goal_lower and param_name not in ["false", "no", "off"]: return True
    return None

# --- Update attempt_goal_autonomously (v8) to use corrected heuristic v7 ---
def attempt_goal_autonomously_v8_tool(goal_description: str, additional_context: dict = None) -> dict:
    tool_name_internal = "attempt_goal_autonomously_v8_tool"
    log_prefix = f"Agent log ({tool_name_internal}): "
    print(f"{log_prefix}Attempting goal (v8 - FIX 11): '{goal_description[:100]}...'. Additional context: {additional_context is not None}")
    current_cumulative_context = (additional_context.get("parameters", {}) if additional_context else {}) or {}
    quoted_match_tuples = re.findall(r"\"([^\"]+)\"|\'([^\']+)\'", goal_description)
    all_quoted_strings_from_goal = [item for tpl in quoted_match_tuples for item in tpl if item]
    if 'suggest_tools_for_goal' not in _AGENT_TOOLS: return {"success": False, "message": "Tool 'suggest_tools_for_goal' not found."}
    chosen_tool_name_override = additional_context.get("chosen_tool_override") if additional_context else None
    chosen_tool_name = ""; tool_callable = None
    if chosen_tool_name_override:
        if chosen_tool_name_override in _AGENT_TOOLS: tool_callable = _AGENT_TOOLS[chosen_tool_name_override]; chosen_tool_name = chosen_tool_name_override; print(f"{log_prefix}Using pre-selected tool: '{chosen_tool_name}'.")
        else: return {"success": False, "message": f"Pre-selected tool '{chosen_tool_name_override}' not found."}
    else:
        suggestion_result = _AGENT_TOOLS['suggest_tools_for_goal'](goal_description=goal_description, top_n=1)
        print(f"{log_prefix}Tool suggestion: {suggestion_result['message']}")
        if not suggestion_result['success'] or not suggestion_result['suggestions']: return {"success": False, "message": "No suitable tools suggested."}
        top_suggestion = suggestion_result['suggestions'][0]
        chosen_tool_name = top_suggestion['tool_name']; relevance_score = top_suggestion['relevance_score']
        print(f"{log_prefix}Top suggested: '{chosen_tool_name}' (Score: {relevance_score}). Reason: {top_suggestion.get('reason', 'N/A')}")
        if relevance_score < _MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION_V3: return {"success": False, "message": f"Tool score {relevance_score} below threshold."}
        if chosen_tool_name not in _AGENT_TOOLS: return {"success": False, "message": f"Suggested tool '{chosen_tool_name}' not found."}
        tool_callable = _AGENT_TOOLS[chosen_tool_name]
    sig = inspect.signature(tool_callable); filled_params = {}; missing_required_params_info = []; consumed_quoted_strings_for_this_call = set()
    print(f"{log_prefix}Params for '{chosen_tool_name}': {list(sig.parameters.keys())}")
    param_order = sorted(sig.parameters.items(), key=lambda item: 0 if item[0] in ["key", "filepath", "path", "url", "directory"] else 1)

    for param_name, param_obj in param_order:
        param_type_annotation = param_obj.annotation if param_obj.annotation != inspect.Parameter.empty else None
        if param_name in current_cumulative_context:
            extracted_value = current_cumulative_context[param_name]; print(f"{log_prefix}  Using '{param_name}' from context.")
        else:
            extracted_value = _extract_parameter_value_heuristically_v7( # CALLING THE CORRECTED V7 HEURISTIC
                param_name, param_type_annotation, goal_description, 
                all_quoted_strings_from_goal, consumed_quoted_strings_for_this_call,
                current_cumulative_context
            )
        if extracted_value is not None:
            if param_type_annotation and isinstance(extracted_value, str):
                try:
                    if param_type_annotation == bool: extracted_value = (extracted_value.lower() == 'true')
                    elif param_type_annotation == int: extracted_value = int(extracted_value)
                    elif param_type_annotation == float: extracted_value = float(extracted_value)
                except ValueError: print(f"{log_prefix}Warn: Could not convert '{extracted_value}' for '{param_name}'.")
            filled_params[param_name] = extracted_value; print(f"{log_prefix}  Filled '{param_name}': {str(extracted_value)[:50]}...")
        elif param_obj.default != inspect.Parameter.empty:
            filled_params[param_name] = param_obj.default; print(f"{log_prefix}  Default for '{param_name}': {str(param_obj.default)[:50]}...")
        else:
            missing_required_params_info.append({"name": param_name, "type": str(param_type_annotation), "description": f"Value for '{param_name}'."}); print(f"{log_prefix}  Could not fill required '{param_name}'.")
    if missing_required_params_info:
        param_names_missing = [p['name'] for p in missing_required_params_info]; msg = f"Clarification needed for tool '{chosen_tool_name}': {', '.join(param_names_missing)}."
        clar_prompt = f"Please provide values for: {', '.join(param_names_missing)} for tool '{chosen_tool_name}' (goal: '{goal_description[:30]}...')."
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, "clarification_request": {"type": "missing_parameters", "original_goal": goal_description, "tool_name": chosen_tool_name, "missing_params_details": missing_required_params_info, "prompt": clar_prompt}}
    print(f"{log_prefix}Executing '{chosen_tool_name}' with: {json.dumps(filled_params, default=str, indent=2)}")
    try:
        execution_result = tool_callable(**filled_params)
        overall_success = execution_result.get('success', True) if isinstance(execution_result, dict) else True
        return {"success": overall_success, "message": f"Tool '{chosen_tool_name}' executed.", "chosen_tool": chosen_tool_name, "parameters_used": filled_params, "execution_result": execution_result}
    except Exception as e: tb_str = traceback.format_exc(); return {"success": False, "message": f"Error executing '{chosen_tool_name}': {e}", "execution_result": {"error": f"{e}\n{tb_str}"}}

# --- Register/Update Tools ---
print("\nAgent: Registering autonomous goal execution (v8 - FIX 11) & updating aliases...")
if 'add_tool' in globals() and callable(add_tool):
    add_tool(
        name="attempt_goal_autonomously_v8",
        tool_callable=attempt_goal_autonomously_v8_tool,
        description="(Version 8) Uses v7 heuristic (definitively fixed syntax). Asks for clarification."
    )
    _AGENT_TOOLS['attempt_goal_autonomously_v3'] = attempt_goal_autonomously_v8_tool
    _TOOL_DESCRIPTIONS['attempt_goal_autonomously_v3'] = "(Version 8, aliased as v3 - FIX 11) Uses context-aware v7 heuristic."
    print("Registered 'attempt_goal_autonomously_v8' and aliased 'attempt_goal_autonomously_v3' to point to v8.")
else:
    print("CRITICAL ERROR: 'add_tool' function not found.")

# --- Re-test Multi-Step Goal Execution Tool ---
print("\nAgent: Re-testing multi-step goal execution (v1) with autonomous worker (v8 via v3 alias - FIX 11).")
tool_to_test_name = "execute_multi_step_goal_v1"
if tool_to_test_name not in _AGENT_TOOLS:
    print(f"Agent: Skipping tests as '{tool_to_test_name}' tool was not registered.")
elif '_extract_parameter_value_heuristically_v7' not in globals() or not callable(globals()['_extract_parameter_value_heuristically_v7']):
    print("CRITICAL ERROR: _extract_parameter_value_heuristically_v7 (corrected) is not available for the test.")
else:
    overall_goal1 = "Download todo item 2, save its title to 'todo_title_fix11.txt', read file, store content in knowledge as 'todo_title_fix11'."
    sub_goals1 = [
        "Download content from 'https://jsonplaceholder.typicode.com/todos/2'",
        "Write the 'title' from the 'last_json_content' to a file named 'todo_title_fix11.txt'",
        "Read the file 'todo_title_fix11.txt'",
        "Save the 'last_content' from reading the file into knowledge with key 'todo_title_fix11'"
    ]
    print(f"\n--- Test 1 (Multi-step REVISED FIX 11): Goal: '{overall_goal1}' ---")
    result1 = _AGENT_TOOLS[tool_to_test_name](overall_goal_description=overall_goal1, sub_goal_strings=sub_goals1)
    print(f"Result for multi-step goal 1: {json.dumps(result1, default=str, indent=2)}")
    if result1.get('success'):
        print("  SUCCESS (Test 1 REVISED FIX 11): Multi-step goal reported success.")
        if 'retrieve_knowledge' in _AGENT_TOOLS:
            kv = _AGENT_TOOLS['retrieve_knowledge'](key='todo_title_fix11')
            if kv.get('found') and kv.get('value') == "quis ut nam facilis et officia qui":
                print(f"  VERIFIED (Test 1 REVISED FIX 11): Knowledge 'todo_title_fix11' correct.")
            else: 
                print(f"  VERIFICATION FAILED (Test 1 REVISED FIX 11): Knowledge 'todo_title_fix11'. Expected 'quis ut nam facilis et officia qui', Got: {kv.get('value')}")
            _AGENT_TOOLS['delete_knowledge'](key='todo_title_fix11')
        if 'delete_file' in _AGENT_TOOLS: _AGENT_TOOLS['delete_file'](filepath='todo_title_fix11.txt')
    else:
        print(f"  FAILURE (Test 1 REVISED FIX 11): Multi-step goal failed. Message: {result1.get('message')}")
        if result1.get('sub_goal_results'):
            for i, r_sg in enumerate(result1['sub_goal_results']):
                print(f"    Sub-goal {i+1}: success={r_sg.get('success')}, tool={r_sg.get('chosen_tool')}, msg={r_sg.get('message')}")
                if r_sg.get('parameters_used'): print(f"      Params used: {r_sg.get('parameters_used')}")

print("\nAgent: Parameter extraction (FIX 11) and re-tests complete.")
print("Waiting for next instruction.")

Agent: Correcting syntax definitively in parameter extraction (FIX 11).

Agent: Registering autonomous goal execution (v8 - FIX 11) & updating aliases...
Tool 'attempt_goal_autonomously_v8' added successfully. Description: (Version 8) Uses v7 heuristic (definitively fixed syntax). Asks for clarification.
Registered 'attempt_goal_autonomously_v8' and aliased 'attempt_goal_autonomously_v3' to point to v8.

Agent: Re-testing multi-step goal execution (v1) with autonomous worker (v8 via v3 alias - FIX 11).

--- Test 1 (Multi-step REVISED FIX 11): Goal: 'Download todo item 2, save its title to 'todo_title_fix11.txt', read file, store content in knowledge as 'todo_title_fix11'.' ---
Agent log (execute_multi_step_goal_v1_tool): Starting multi-step goal: 'Download todo item 2, save its title to 'todo_title_fix11.txt', read file, store content in knowledg...'. Number of sub-goals: 4
Agent log (execute_multi_step_goal_v1_tool): Initial cumulative context: {}

Agent log (execute_multi_step_goal_v

In [29]:
# Existing tools, _AGENT_TOOL_CONCEPT_KEYWORDS etc., assumed.
import inspect
import re
import json
import traceback

print("Agent: Refining parameter extraction priorities and context path resolution (FIX 12).")

# --- 1. Refine Parameter Extraction Heuristic (v8) ---
def _extract_parameter_value_heuristically_v8(
    param_name: str, 
    param_type, 
    goal_description: str, 
    all_quoted_strings_from_goal: list[str],
    consumed_quoted_strings: set[str],
    current_cumulative_context: dict 
) -> any:
    goal_lower = goal_description.lower()
    
    # --- Order of Extraction Attempts ---
    # 1. Explicit assignment in goal string (param_name = 'value') - Strongest signal
    # 2. For identifier-like params (key, filepath, path, directory, url): Literal extraction from goal.
    # 3. For data-like params (content, value, etc.): Context variable resolution (with property paths).
    # 4. For data-like params: General quoted strings from goal if no context match.
    # 5. General type-based heuristics (method, bool, code_string).

    # 1. Explicit Assignment
    explicit_assign_match_quoted = re.search(rf"\b{param_name}\s*[:=]\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
    if explicit_assign_match_quoted: return explicit_assign_match_quoted.group(1)
    explicit_assign_match_unquoted = re.search(rf"\b{param_name}\s*[:=]\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
    if explicit_assign_match_unquoted:
        val_str = explicit_assign_match_unquoted.group(1)
        if param_type == int: try: return int(val_str) except ValueError: pass
        if param_type == float: try: return float(val_str) except ValueError: pass
        if param_type == bool:
            if val_str.lower() == 'true': return True
            if val_str.lower() == 'false': return False
        if param_type == str or param_type is None or param_type == inspect.Parameter.empty : return val_str

    # 2. Literal extraction for IDENTIFIER-LIKE parameters (key, url, filepath, path, directory)
    if param_name == "key":
        # ... (key extraction logic from v7/v5, ensure it's robust) ...
        key_match = re.search(r"(?:with key|key is|for key|as key|named key)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if key_match: return key_match.group(1)
        key_match_unquoted = re.search(r"(?:with key|key is|for key|as key|named key)\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
        if key_match_unquoted: return key_match_unquoted.group(1)

    if param_name == "url":
        # ... (URL extraction logic from v7/v5) ...
        url_match_re = r"https?://(?:[a-zA-Z0-9.\-_~:/?#\[\]@!$&'()*+,;=%]+[a-zA-Z0-9\-_~:/?#\[\]@!$&*+,;=%])"; url_m = re.search(url_match_re, goal_description)
        extracted_url = None 
        if url_m: extracted_url = url_m.group(0)
        if url_m: 
            if extracted_url.endswith("'") and goal_description.count(f"'{extracted_url}'") > 0: extracted_url = extracted_url[:-1]
            elif extracted_url.endswith("\"") and goal_description.count(f"\"{extracted_url}\"") > 0: extracted_url = extracted_url[:-1]
            for q_str in all_quoted_strings_from_goal:
                if extracted_url == q_str or f"'{extracted_url}'" == q_str or f"\"{extracted_url}\"" == q_str :
                    if q_str not in consumed_quoted_strings: consumed_quoted_strings.add(q_str); break 
            return extracted_url


    if param_name in ["filepath", "path", "directory"]:
        # ... (filepath/path/directory extraction logic from v7/v5, prioritizing "named 'X'") ...
        path_pattern_named = re.search(rf"(?:file named|directory named|path is|called)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if path_pattern_named:
            potential_path = path_pattern_named.group(1)
            if potential_path in all_quoted_strings_from_goal and potential_path not in consumed_quoted_strings:
                consumed_quoted_strings.add(potential_path); return potential_path
        path_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            # For "directory", be more lenient. For "filepath"/"path", require some path-like chars or extension.
            if param_name == "directory" or \
               ('.' in q_str or '/' in q_str or '\\' in q_str or any(q_str.endswith(ext) for ext in ['.txt', '.json', '.py'])):
                path_candidates.append(q_str)
        if path_candidates: chosen_path = max(path_candidates, key=len); consumed_quoted_strings.add(chosen_path); return chosen_path

    # 3. Context variable resolution for DATA-LIKE parameters (content, value, etc.)
    is_data_param = param_name in ["content", "value", "json_payload", "data_payload", "text"]
    if is_data_param and current_cumulative_context and current_cumulative_context.keys():
        context_key_names = sorted(list(current_cumulative_context.keys()), key=len, reverse=True) # Match longer keys first
        context_key_pattern = f"({ '|'.join(re.escape(k) for k in context_key_names if k) })"
        
        # Pattern 1: "the 'property' from 'context_key'"
        # E.g., "the 'title' from the 'last_json_content'"
        prop_from_context_match = re.search(
            rf"(?:the|value of|from|content of)\s*['\"]([\w_][\w\d_]*)['\"]\s*(?:of|from)\s*['\"]{context_key_pattern}['\"]",
            goal_description, re.IGNORECASE
        )
        if prop_from_context_match:
            prop_to_extract = prop_from_context_match.group(1)
            matched_context_key = prop_from_context_match.group(2)
            if matched_context_key in current_cumulative_context:
                base_value = current_cumulative_context[matched_context_key]
                if isinstance(base_value, dict) and prop_to_extract in base_value:
                    val = base_value[prop_to_extract]
                    print(f"Agent log (_extract_v8): Resolved property '{prop_to_extract}' from context '{matched_context_key}' for '{param_name}'.")
                    return val
        
        # Pattern 2: "the 'context_key.property.path'" or "the 'context_key'"
        # This will also match "the 'last_json_content'" if no property path is specified.
        context_ref_match = re.search(
            rf"(?:the|value of|from|content of)\s*['\"]{context_key_pattern}((?:(?:\.[\w_][\w\d_]*)|(?:\[['\"]\w+['\"]\]))+)?['\"]",
            goal_description, re.IGNORECASE
        )
        if not context_ref_match: # Try without "the", "from" etc.
             context_ref_match = re.search(
                rf"['\"]{context_key_pattern}((?:(?:\.[\w_][\w\d_]*)|(?:\[['\"]\w+['\"]\]))+)?['\"]",
                 goal_description, re.IGNORECASE
             )
        if context_ref_match:
            matched_context_key = context_ref_match.group(1)
            property_path_str = context_ref_match.group(2) # Optional property path
            if matched_context_key in current_cumulative_context:
                value_from_context = current_cumulative_context[matched_context_key]
                if property_path_str: # Property path specified
                    properties = []; 
                    for part in re.findall(r"\.([\w_][\w\d_]*)|\[['\"]([^'\"]+)['\"]\]", property_path_str): properties.append(part[0] or part[1])
                    temp_val = value_from_context
                    try:
                        for prop_key in properties:
                            if isinstance(temp_val, dict): temp_val = temp_val.get(prop_key)
                            elif isinstance(temp_val, list) and prop_key.isdigit(): temp_val = temp_val[int(prop_key)]
                            elif hasattr(temp_val, prop_key): temp_val = getattr(temp_val, prop_key)
                            else: temp_val = None; break
                        if temp_val is not None:
                            print(f"Agent log (_extract_v8): Resolved context path '{matched_context_key}{property_path_str}' for '{param_name}'.")
                            return temp_val
                    except Exception: pass
                else: # No property path, use the whole context variable (only if param is suitable, e.g. 'value', 'content')
                    if param_name in ["value", "content", "json_payload", "data_payload"]: # Ensure this direct context var assignment is for a data param
                        print(f"Agent log (_extract_v8): Resolved direct context var '{matched_context_key}' for '{param_name}'.")
                        return value_from_context

    # 4. General quoted strings for data-like parameters (if no context match was made FOR THIS PARAMETER)
    if param_name in ["content", "text", "description", "message", "value"]:
        content_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            content_candidates.append(q_str)
        if content_candidates:
            chosen_content = max(content_candidates, key=len)
            # DO NOT consume here generally, only if it's a final fallback
            # consumed_quoted_strings.add(chosen_content) # Let specific literal extractors consume
            return chosen_content
            
    # 5. General type-based heuristics (code_string, method, bool)
    # ... (code_string, method, bool logic from v7/v5 - ensure they don't wrongly consume from consumed_quoted_strings if applicable)
    if param_name == "code_string":
        code_block_match = re.search(r"code:\s*(.+)", goal_description, re.IGNORECASE | re.DOTALL);
        if code_block_match: return code_block_match.group(1).strip()
        triple_tick_match = re.search(r"```(?:python\n)?(.*?)```", goal_description, re.DOTALL | re.IGNORECASE)
        if triple_tick_match: return triple_tick_match.group(1).strip()
        
    if param_name == "method" and (param_type == str or param_type is None or param_type == inspect.Parameter.empty):
        method_search = re.search(r"\b(GET|POST|PUT|DELETE|PATCH|HEAD|OPTIONS)\b", goal_description, re.IGNORECASE);
        if method_search: return method_search.group(1).upper()
        # ... (fallbacks for method) ...
        if "post" in goal_lower: return "POST"; 
        if "get" in goal_lower: return "GET";
        if "delete" in goal_lower: return "DELETE";
        if "put" in goal_lower: return "PUT";
        
    if param_type == bool:
        # ... (boolean logic from v7/v5) ...
        if re.search(rf"\b{param_name}\b\s*(true|yes|on|recursively|enabled)\b", goal_lower, re.IGNORECASE): return True
        if re.search(rf"\b(is|be|make it)\s*{param_name}\b", goal_lower, re.IGNORECASE): return True
        if re.search(rf"\b{param_name}\b\s*(false|no|off|disabled)\b", goal_lower, re.IGNORECASE): return False
        if re.search(rf"\b(is not|not to be|don't make it)\s*{param_name}\b", goal_lower, re.IGNORECASE): return False
        if param_name in goal_lower and param_name not in ["false", "no", "off"]: return True

    return None

# --- Update attempt_goal_autonomously (v9) to use new heuristic v8 ---
def attempt_goal_autonomously_v9_tool(goal_description: str, additional_context: dict = None) -> dict:
    # Structure is same as v8_tool from previous turn, just calls _extract_parameter_value_heuristically_v8
    tool_name_internal = "attempt_goal_autonomously_v9_tool"
    log_prefix = f"Agent log ({tool_name_internal}): "
    print(f"{log_prefix}Attempting goal (v9 - FIX 12): '{goal_description[:100]}...'. Additional context: {additional_context is not None}")
    current_cumulative_context = (additional_context.get("parameters", {}) if additional_context else {}) or {}
    quoted_match_tuples = re.findall(r"\"([^\"]+)\"|\'([^\']+)\'", goal_description)
    all_quoted_strings_from_goal = [item for tpl in quoted_match_tuples for item in tpl if item]
    if 'suggest_tools_for_goal' not in _AGENT_TOOLS: return {"success": False, "message": "Tool 'suggest_tools_for_goal' not found."}
    chosen_tool_name_override = additional_context.get("chosen_tool_override") if additional_context else None
    chosen_tool_name = ""; tool_callable = None
    if chosen_tool_name_override: # Tool pre-selected
        if chosen_tool_name_override in _AGENT_TOOLS: tool_callable = _AGENT_TOOLS[chosen_tool_name_override]; chosen_tool_name = chosen_tool_name_override; print(f"{log_prefix}Using pre-selected tool: '{chosen_tool_name}'.")
        else: return {"success": False, "message": f"Pre-selected tool '{chosen_tool_name_override}' not found."}
    else: # Suggest tool
        suggestion_result = _AGENT_TOOLS['suggest_tools_for_goal'](goal_description=goal_description, top_n=1) # suggest_tools_for_goal should be v3 logic (URL bonus)
        print(f"{log_prefix}Tool suggestion: {suggestion_result['message']}")
        if not suggestion_result['success'] or not suggestion_result['suggestions']: return {"success": False, "message": "No suitable tools suggested."}
        top_suggestion = suggestion_result['suggestions'][0]; chosen_tool_name = top_suggestion['tool_name']; relevance_score = top_suggestion['relevance_score']
        print(f"{log_prefix}Top suggested: '{chosen_tool_name}' (Score: {relevance_score}). Reason: {top_suggestion.get('reason', 'N/A')}")
        if relevance_score < _MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION_V3: return {"success": False, "message": f"Tool score {relevance_score} below threshold."}
        if chosen_tool_name not in _AGENT_TOOLS: return {"success": False, "message": f"Suggested tool '{chosen_tool_name}' not found."}
        tool_callable = _AGENT_TOOLS[chosen_tool_name]
    sig = inspect.signature(tool_callable); filled_params = {}; missing_required_params_info = []; consumed_quoted_strings_for_this_call = set()
    print(f"{log_prefix}Params for '{chosen_tool_name}': {list(sig.parameters.keys())}")
    param_order = sorted(sig.parameters.items(), key=lambda item: 0 if item[0] in ["key", "filepath", "path", "url", "directory"] else 1) # Prioritize identifiers

    for param_name, param_obj in param_order:
        param_type_annotation = param_obj.annotation if param_obj.annotation != inspect.Parameter.empty else None
        if param_name in current_cumulative_context: # Check context provided by planner first
            extracted_value = current_cumulative_context[param_name]; print(f"{log_prefix}  Using '{param_name}' directly from provided/cumulative context.")
        else:
            extracted_value = _extract_parameter_value_heuristically_v8( # CALLING THE CORRECTED V8 HEURISTIC
                param_name, param_type_annotation, goal_description, 
                all_quoted_strings_from_goal, consumed_quoted_strings_for_this_call,
                current_cumulative_context
            )
        if extracted_value is not None: # Type conversion and assignment
            if param_type_annotation and isinstance(extracted_value, str):
                try:
                    if param_type_annotation == bool: extracted_value = (extracted_value.lower() == 'true')
                    elif param_type_annotation == int: extracted_value = int(extracted_value)
                    elif param_type_annotation == float: extracted_value = float(extracted_value)
                except ValueError: print(f"{log_prefix}Warn: Could not convert '{str(extracted_value)[:20]}' for '{param_name}'.")
            filled_params[param_name] = extracted_value; print(f"{log_prefix}  Filled '{param_name}': {str(extracted_value)[:50]}...")
        elif param_obj.default != inspect.Parameter.empty:
            filled_params[param_name] = param_obj.default; print(f"{log_prefix}  Default for '{param_name}': {str(param_obj.default)[:50]}...")
        else: # Missing required
            missing_required_params_info.append({"name": param_name, "type": str(param_type_annotation), "description": f"Value for '{param_name}'."}); print(f"{log_prefix}  Could not fill required '{param_name}'.")
    if missing_required_params_info: # Clarification
        param_names_missing = [p['name'] for p in missing_required_params_info]; msg = f"Clarification needed for tool '{chosen_tool_name}': {', '.join(param_names_missing)}."
        clar_prompt = f"Please provide values for: {', '.join(param_names_missing)} for tool '{chosen_tool_name}' (goal: '{goal_description[:30]}...')."
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, "clarification_request": {"type": "missing_parameters", "original_goal": goal_description, "tool_name": chosen_tool_name, "missing_params_details": missing_required_params_info, "prompt": clar_prompt}}
    print(f"{log_prefix}Executing '{chosen_tool_name}' with: {json.dumps(filled_params, default=str, indent=2)}")
    try: # Execution
        execution_result = tool_callable(**filled_params)
        overall_success = execution_result.get('success', True) if isinstance(execution_result, dict) else True
        return {"success": overall_success, "message": f"Tool '{chosen_tool_name}' executed.", "chosen_tool": chosen_tool_name, "parameters_used": filled_params, "execution_result": execution_result}
    except Exception as e: tb_str = traceback.format_exc(); return {"success": False, "message": f"Error executing '{chosen_tool_name}': {e}", "execution_result": {"error": f"{e}\n{tb_str}"}}

# --- Register/Update Tools ---
print("\nAgent: Registering autonomous goal execution (v9 - FIX 12) & updating aliases...")
if 'add_tool' in globals() and callable(add_tool):
    add_tool(
        name="attempt_goal_autonomously_v9",
        tool_callable=attempt_goal_autonomously_v9_tool,
        description="(Version 9) Uses v8 heuristic (priority fixes). Asks for clarification."
    )
    _AGENT_TOOLS['attempt_goal_autonomously_v3'] = attempt_goal_autonomously_v9_tool
    _TOOL_DESCRIPTIONS['attempt_goal_autonomously_v3'] = "(Version 9, aliased as v3 - FIX 12) Uses context-aware v8 heuristic."
    print("Registered 'attempt_goal_autonomously_v9' and aliased 'attempt_goal_autonomously_v3' to point to v9.")
else:
    print("CRITICAL ERROR: 'add_tool' function not found.")

# --- Re-test Multi-Step Goal Execution Tool ---
print("\nAgent: Re-testing multi-step goal execution (v1) with autonomous worker (v9 via v3 alias - FIX 12).")
tool_to_test_name = "execute_multi_step_goal_v1"
if tool_to_test_name not in _AGENT_TOOLS:
    print(f"Agent: Skipping tests as '{tool_to_test_name}' tool was not registered.")
elif '_extract_parameter_value_heuristically_v8' not in globals() or not callable(globals()['_extract_parameter_value_heuristically_v8']):
    print("CRITICAL ERROR: _extract_parameter_value_heuristically_v8 is not available for the test.")
else:
    overall_goal1 = "Download todo item 2, save its title to 'todo_title_fix12.txt', read file, store content in knowledge as 'todo_title_fix12'."
    sub_goals1 = [
        "Download content from 'https://jsonplaceholder.typicode.com/todos/2'",
        "Write the 'title' from the 'last_json_content' to a file named 'todo_title_fix12.txt'",
        "Read the file 'todo_title_fix12.txt'",
        "Save the 'last_content' from reading the file into knowledge with key 'todo_title_fix12'"
    ]
    print(f"\n--- Test 1 (Multi-step REVISED FIX 12): Goal: '{overall_goal1}' ---")
    result1 = _AGENT_TOOLS[tool_to_test_name](overall_goal_description=overall_goal1, sub_goal_strings=sub_goals1)
    print(f"Result for multi-step goal 1: {json.dumps(result1, default=str, indent=2)}")
    if result1.get('success'):
        print("  SUCCESS (Test 1 REVISED FIX 12): Multi-step goal reported success.")
        if 'retrieve_knowledge' in _AGENT_TOOLS:
            kv = _AGENT_TOOLS['retrieve_knowledge'](key='todo_title_fix12')
            if kv.get('found') and kv.get('value') == "quis ut nam facilis et officia qui":
                print(f"  VERIFIED (Test 1 REVISED FIX 12): Knowledge 'todo_title_fix12' correct.")
            else: 
                print(f"  VERIFICATION FAILED (Test 1 REVISED FIX 12): Knowledge 'todo_title_fix12'. Expected 'quis ut nam facilis et officia qui', Got: {kv.get('value')}")
            _AGENT_TOOLS['delete_knowledge'](key='todo_title_fix12')
        if 'delete_file' in _AGENT_TOOLS: _AGENT_TOOLS['delete_file'](filepath='todo_title_fix12.txt')
    else:
        print(f"  FAILURE (Test 1 REVISED FIX 12): Multi-step goal failed. Message: {result1.get('message')}")
        if result1.get('sub_goal_results'):
            for i, r_sg in enumerate(result1['sub_goal_results']):
                print(f"    Sub-goal {i+1}: success={r_sg.get('success')}, tool={r_sg.get('chosen_tool')}, msg={r_sg.get('message')}")
                if r_sg.get('parameters_used'): print(f"      Params used: {r_sg.get('parameters_used')}")

print("\nAgent: Parameter extraction priorities (FIX 12) and re-tests complete.")
print("Waiting for next instruction.")

SyntaxError: invalid syntax (2338470260.py, line 33)

In [30]:
# Existing tools, _AGENT_TOOL_CONCEPT_KEYWORDS etc., assumed.
import inspect
import re
import json
import traceback

print("Agent: Definitive syntax correction in parameter extraction (FIX 13).")

# --- 1. Corrected Parameter Extraction Heuristic (v9 - FINAL SYNTAX FIX) ---
def _extract_parameter_value_heuristically_v9_final_syntax_fix(
    param_name: str, 
    param_type, 
    goal_description: str, 
    all_quoted_strings_from_goal: list[str],
    consumed_quoted_strings: set[str],
    current_cumulative_context: dict 
) -> any:
    goal_lower = goal_description.lower()
    
    # 1. Explicit Assignment (param_name = 'value' or param_name: value)
    explicit_assign_match_quoted = re.search(rf"\b{param_name}\s*[:=]\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
    if explicit_assign_match_quoted: return explicit_assign_match_quoted.group(1)
    explicit_assign_match_unquoted = re.search(rf"\b{param_name}\s*[:=]\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
    if explicit_assign_match_unquoted:
        val_str = explicit_assign_match_unquoted.group(1)
        # ***** DEFINITIVELY CORRECTED SYNTAX FOR TRY-EXCEPT *****
        if param_type == int:
            try:
                return int(val_str)
            except ValueError:
                pass 
        if param_type == float:
            try:
                return float(val_str)
            except ValueError:
                pass
        # ***** END OF CORRECTION *****
        if param_type == bool:
            if val_str.lower() == 'true': return True
            if val_str.lower() == 'false': return False
        if param_type == str or param_type is None or param_type == inspect.Parameter.empty : return val_str

    # 2. For identifier-like params (key, filepath, path, directory, url): Literal extraction from goal FIRST.
    if param_name == "key":
        key_match = re.search(r"(?:with key|key is|for key|as key|named key)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if key_match: return key_match.group(1)
        key_match_unquoted = re.search(r"(?:with key|key is|for key|as key|named key)\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
        if key_match_unquoted: return key_match_unquoted.group(1)

    if param_name == "url":
        url_match_re = r"https?://(?:[a-zA-Z0-9.\-_~:/?#\[\]@!$&'()*+,;=%]+[a-zA-Z0-9\-_~:/?#\[\]@!$&*+,;=%])"; url_m = re.search(url_match_re, goal_description)
        extracted_url = None 
        if url_m: extracted_url = url_m.group(0)
        if url_m: 
            if extracted_url.endswith("'") and goal_description.count(f"'{extracted_url}'") > 0: extracted_url = extracted_url[:-1]
            elif extracted_url.endswith("\"") and goal_description.count(f"\"{extracted_url}\"") > 0: extracted_url = extracted_url[:-1]
            for q_str in all_quoted_strings_from_goal:
                if extracted_url == q_str or f"'{extracted_url}'" == q_str or f"\"{extracted_url}\"" == q_str :
                    if q_str not in consumed_quoted_strings: consumed_quoted_strings.add(q_str); break 
            return extracted_url

    if param_name in ["filepath", "path", "directory"]:
        path_pattern_named = re.search(rf"(?:file named|directory named|path is|called)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if path_pattern_named:
            potential_path = path_pattern_named.group(1)
            if potential_path in all_quoted_strings_from_goal and potential_path not in consumed_quoted_strings:
                consumed_quoted_strings.add(potential_path); return potential_path
        path_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            if param_name == "directory" or \
               ('.' in q_str or '/' in q_str or '\\' in q_str or any(q_str.endswith(ext) for ext in ['.txt', '.json', '.py'])):
                path_candidates.append(q_str)
        if path_candidates: chosen_path = max(path_candidates, key=len); consumed_quoted_strings.add(chosen_path); return chosen_path

    # 3. Context variable resolution (especially for data-like parameters)
    is_data_param = param_name in ["content", "value", "json_payload", "data_payload", "text"]
    if current_cumulative_context and current_cumulative_context.keys(): # Check context has keys
        context_key_names = sorted(list(current_cumulative_context.keys()), key=len, reverse=True)
        context_key_pattern = f"({ '|'.join(re.escape(k) for k in context_key_names if k) })"
        
        # Pattern 1: "the 'property' from 'context_key'"
        prop_from_context_match = re.search(
            rf"(?:the|value of|from|content of)\s*['\"]([\w_][\w\d_]*)['\"]\s*(?:of|from)\s*['\"]{context_key_pattern}['\"]",
            goal_description, re.IGNORECASE
        )
        if prop_from_context_match:
            prop_to_extract = prop_from_context_match.group(1)
            matched_context_key = prop_from_context_match.group(2)
            if matched_context_key in current_cumulative_context:
                base_value = current_cumulative_context[matched_context_key]
                if isinstance(base_value, dict) and prop_to_extract in base_value:
                    val = base_value[prop_to_extract]
                    print(f"Agent log (_extract_v9_fix13): Resolved prop '{prop_to_extract}' from context '{matched_context_key}' for '{param_name}'.")
                    return val
        
        # Pattern 2: "the 'context_key.property.path'" or "the 'context_key'"
        context_ref_match = re.search(
            rf"(?:the|value of|from|content of)\s*['\"]{context_key_pattern}((?:(?:\.[\w_][\w\d_]*)|(?:\[['\"]\w+['\"]\]))+)?['\"]",
            goal_description, re.IGNORECASE
        )
        if not context_ref_match:
             context_ref_match = re.search(
                rf"['\"]{context_key_pattern}((?:(?:\.[\w_][\w\d_]*)|(?:\[['\"]\w+['\"]\]))+)?['\"]",
                 goal_description, re.IGNORECASE
             )
        if context_ref_match:
            matched_context_key = context_ref_match.group(1)
            property_path_str = context_ref_match.group(2)
            if matched_context_key in current_cumulative_context:
                value_from_context = current_cumulative_context[matched_context_key]
                if property_path_str: 
                    properties = []; 
                    for part in re.findall(r"\.([\w_][\w\d_]*)|\[['\"]([^'\"]+)['\"]\]", property_path_str): properties.append(part[0] or part[1])
                    temp_val = value_from_context
                    try:
                        for prop_key in properties:
                            if isinstance(temp_val, dict): temp_val = temp_val.get(prop_key)
                            elif isinstance(temp_val, list) and prop_key.isdigit(): temp_val = temp_val[int(prop_key)]
                            elif hasattr(temp_val, prop_key): temp_val = getattr(temp_val, prop_key)
                            else: temp_val = None; break
                        if temp_val is not None:
                            print(f"Agent log (_extract_v9_fix13): Resolved context path '{matched_context_key}{property_path_str}' for '{param_name}'.")
                            return temp_val
                    except Exception: pass
                elif is_data_param: # Only use direct context var if it's a data param
                    print(f"Agent log (_extract_v9_fix13): Resolved direct context var '{matched_context_key}' for data param '{param_name}'.")
                    return value_from_context

    # 4. General quoted strings for data-like parameters as a fallback
    if param_name in ["content", "text", "description", "message", "value"]:
        content_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            content_candidates.append(q_str)
        if content_candidates:
            chosen_content = max(content_candidates, key=len)
            consumed_quoted_strings.add(chosen_content) # Consume here as it's a general fallback
            return chosen_content
            
    # 5. General type-based heuristics
    if param_name == "code_string":
        # ... (code_string logic)
        code_block_match = re.search(r"code:\s*(.+)", goal_description, re.IGNORECASE | re.DOTALL);
        if code_block_match: return code_block_match.group(1).strip()
        triple_tick_match = re.search(r"```(?:python\n)?(.*?)```", goal_description, re.DOTALL | re.IGNORECASE)
        if triple_tick_match: return triple_tick_match.group(1).strip()
    if param_name == "method" and (param_type == str or param_type is None or param_type == inspect.Parameter.empty):
        # ... (method logic)
        method_search = re.search(r"\b(GET|POST|PUT|DELETE|PATCH|HEAD|OPTIONS)\b", goal_description, re.IGNORECASE);
        if method_search: return method_search.group(1).upper()
        if "post" in goal_lower: return "POST"; 
        if "get" in goal_lower: return "GET";
        if "delete" in goal_lower: return "DELETE";
        if "put" in goal_lower: return "PUT";
    if param_type == bool:
        # ... (boolean logic)
        if re.search(rf"\b{param_name}\b\s*(true|yes|on|recursively|enabled)\b", goal_lower, re.IGNORECASE): return True
        if re.search(rf"\b(is|be|make it)\s*{param_name}\b", goal_lower, re.IGNORECASE): return True
        if re.search(rf"\b{param_name}\b\s*(false|no|off|disabled)\b", goal_lower, re.IGNORECASE): return False
        if re.search(rf"\b(is not|not to be|don't make it)\s*{param_name}\b", goal_lower, re.IGNORECASE): return False
        if param_name in goal_lower and param_name not in ["false", "no", "off"]: return True
        
    return None

# --- Update attempt_goal_autonomously (v10) to use new heuristic v9 ---
def attempt_goal_autonomously_v10_tool(goal_description: str, additional_context: dict = None) -> dict:
    tool_name_internal = "attempt_goal_autonomously_v10_tool"
    log_prefix = f"Agent log ({tool_name_internal}): "
    print(f"{log_prefix}Attempting goal (v10 - FIX 13): '{goal_description[:100]}...'. Additional context: {additional_context is not None}")
    current_cumulative_context = (additional_context.get("parameters", {}) if additional_context else {}) or {}
    quoted_match_tuples = re.findall(r"\"([^\"]+)\"|\'([^\']+)\'", goal_description)
    all_quoted_strings_from_goal = [item for tpl in quoted_match_tuples for item in tpl if item]
    if 'suggest_tools_for_goal' not in _AGENT_TOOLS: return {"success": False, "message": "Tool 'suggest_tools_for_goal' not found."}
    chosen_tool_name_override = additional_context.get("chosen_tool_override") if additional_context else None
    chosen_tool_name = ""; tool_callable = None
    if chosen_tool_name_override:
        if chosen_tool_name_override in _AGENT_TOOLS: tool_callable = _AGENT_TOOLS[chosen_tool_name_override]; chosen_tool_name = chosen_tool_name_override; print(f"{log_prefix}Using pre-selected tool: '{chosen_tool_name}'.")
        else: return {"success": False, "message": f"Pre-selected tool '{chosen_tool_name_override}' not found."}
    else:
        suggestion_result = _AGENT_TOOLS['suggest_tools_for_goal'](goal_description=goal_description, top_n=1)
        print(f"{log_prefix}Tool suggestion: {suggestion_result['message']}")
        if not suggestion_result['success'] or not suggestion_result['suggestions']: return {"success": False, "message": "No suitable tools suggested."}
        top_suggestion = suggestion_result['suggestions'][0]; chosen_tool_name = top_suggestion['tool_name']; relevance_score = top_suggestion['relevance_score']
        print(f"{log_prefix}Top suggested: '{chosen_tool_name}' (Score: {relevance_score}). Reason: {top_suggestion.get('reason', 'N/A')}")
        if relevance_score < _MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION_V3: return {"success": False, "message": f"Tool score {relevance_score} below threshold."}
        if chosen_tool_name not in _AGENT_TOOLS: return {"success": False, "message": f"Suggested tool '{chosen_tool_name}' not found."}
        tool_callable = _AGENT_TOOLS[chosen_tool_name]
    sig = inspect.signature(tool_callable); filled_params = {}; missing_required_params_info = []; consumed_quoted_strings_for_this_call = set()
    print(f"{log_prefix}Params for '{chosen_tool_name}': {list(sig.parameters.keys())}")
    param_order = sorted(sig.parameters.items(), key=lambda item: 0 if item[0] in ["key", "filepath", "path", "url", "directory"] else 1)

    for param_name, param_obj in param_order:
        param_type_annotation = param_obj.annotation if param_obj.annotation != inspect.Parameter.empty else None
        if param_name in current_cumulative_context: # Check context provided by planner first
            extracted_value = current_cumulative_context[param_name]; print(f"{log_prefix}  Using '{param_name}' directly from provided/cumulative context.")
        else:
            extracted_value = _extract_parameter_value_heuristically_v9_final_syntax_fix( # CALLING THE CORRECTED V9 HEURISTIC
                param_name, param_type_annotation, goal_description, 
                all_quoted_strings_from_goal, consumed_quoted_strings_for_this_call,
                current_cumulative_context
            )
        if extracted_value is not None:
            if param_type_annotation and isinstance(extracted_value, str):
                try:
                    if param_type_annotation == bool: extracted_value = (extracted_value.lower() == 'true')
                    elif param_type_annotation == int: extracted_value = int(extracted_value)
                    elif param_type_annotation == float: extracted_value = float(extracted_value)
                except ValueError: print(f"{log_prefix}Warn: Could not convert '{str(extracted_value)[:20]}' for '{param_name}'.")
            filled_params[param_name] = extracted_value; print(f"{log_prefix}  Filled '{param_name}': {str(extracted_value)[:50]}...")
        elif param_obj.default != inspect.Parameter.empty:
            filled_params[param_name] = param_obj.default; print(f"{log_prefix}  Default for '{param_name}': {str(param_obj.default)[:50]}...")
        else:
            missing_required_params_info.append({"name": param_name, "type": str(param_type_annotation), "description": f"Value for '{param_name}'."}); print(f"{log_prefix}  Could not fill required '{param_name}'.")
    if missing_required_params_info:
        param_names_missing = [p['name'] for p in missing_required_params_info]; msg = f"Clarification needed for tool '{chosen_tool_name}': {', '.join(param_names_missing)}."
        clar_prompt = f"Please provide values for: {', '.join(param_names_missing)} for tool '{chosen_tool_name}' (goal: '{goal_description[:30]}...')."
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, "clarification_request": {"type": "missing_parameters", "original_goal": goal_description, "tool_name": chosen_tool_name, "missing_params_details": missing_required_params_info, "prompt": clar_prompt}}
    print(f"{log_prefix}Executing '{chosen_tool_name}' with: {json.dumps(filled_params, default=str, indent=2)}")
    try:
        execution_result = tool_callable(**filled_params)
        overall_success = execution_result.get('success', True) if isinstance(execution_result, dict) else True
        return {"success": overall_success, "message": f"Tool '{chosen_tool_name}' executed.", "chosen_tool": chosen_tool_name, "parameters_used": filled_params, "execution_result": execution_result}
    except Exception as e: tb_str = traceback.format_exc(); return {"success": False, "message": f"Error executing '{chosen_tool_name}': {e}", "execution_result": {"error": f"{e}\n{tb_str}"}}

# --- Register/Update Tools ---
print("\nAgent: Registering autonomous goal execution (v10 - FIX 13) & updating aliases...")
if 'add_tool' in globals() and callable(add_tool):
    add_tool(
        name="attempt_goal_autonomously_v10",
        tool_callable=attempt_goal_autonomously_v10_tool,
        description="(Version 10) Uses v9 heuristic (definitively fixed syntax, improved context priorities). Asks for clarification."
    )
    _AGENT_TOOLS['attempt_goal_autonomously_v3'] = attempt_goal_autonomously_v10_tool
    _TOOL_DESCRIPTIONS['attempt_goal_autonomously_v3'] = "(Version 10, aliased as v3 - FIX 13) Uses context-aware v9 heuristic."
    print("Registered 'attempt_goal_autonomously_v10' and aliased 'attempt_goal_autonomously_v3' to point to v10.")
else:
    print("CRITICAL ERROR: 'add_tool' function not found.")

# --- Re-test Multi-Step Goal Execution Tool ---
print("\nAgent: Re-testing multi-step goal execution (v1) with autonomous worker (v10 via v3 alias - FIX 13).")
tool_to_test_name = "execute_multi_step_goal_v1"
if tool_to_test_name not in _AGENT_TOOLS:
    print(f"Agent: Skipping tests as '{tool_to_test_name}' tool was not registered.")
elif '_extract_parameter_value_heuristically_v9_final_syntax_fix' not in globals() or not callable(globals()['_extract_parameter_value_heuristically_v9_final_syntax_fix']):
    print("CRITICAL ERROR: _extract_parameter_value_heuristically_v9_final_syntax_fix is not available for the test.")
else:
    overall_goal1 = "Download todo item 2, save its title to 'todo_title_fix13.txt', read file, store content in knowledge as 'todo_title_fix13'."
    sub_goals1 = [
        "Download content from 'https://jsonplaceholder.typicode.com/todos/2'",
        "Write the 'title' from the 'last_json_content' to a file named 'todo_title_fix13.txt'",
        "Read the file 'todo_title_fix13.txt'",
        "Save the 'last_content' from reading the file into knowledge with key 'todo_title_fix13'"
    ]
    print(f"\n--- Test 1 (Multi-step REVISED FIX 13): Goal: '{overall_goal1}' ---")
    result1 = _AGENT_TOOLS[tool_to_test_name](overall_goal_description=overall_goal1, sub_goal_strings=sub_goals1)
    print(f"Result for multi-step goal 1: {json.dumps(result1, default=str, indent=2)}")
    if result1.get('success'):
        print("  SUCCESS (Test 1 REVISED FIX 13): Multi-step goal reported success.")
        if 'retrieve_knowledge' in _AGENT_TOOLS:
            kv = _AGENT_TOOLS['retrieve_knowledge'](key='todo_title_fix13')
            if kv.get('found') and kv.get('value') == "quis ut nam facilis et officia qui":
                print(f"  VERIFIED (Test 1 REVISED FIX 13): Knowledge 'todo_title_fix13' correct.")
            else: 
                print(f"  VERIFICATION FAILED (Test 1 REVISED FIX 13): Knowledge 'todo_title_fix13'. Expected 'quis ut nam facilis et officia qui', Got: {kv.get('value')}")
            _AGENT_TOOLS['delete_knowledge'](key='todo_title_fix13')
        if 'delete_file' in _AGENT_TOOLS: _AGENT_TOOLS['delete_file'](filepath='todo_title_fix13.txt')
    else:
        print(f"  FAILURE (Test 1 REVISED FIX 13): Multi-step goal failed. Message: {result1.get('message')}")
        if result1.get('sub_goal_results'):
            for i, r_sg in enumerate(result1['sub_goal_results']):
                print(f"    Sub-goal {i+1}: success={r_sg.get('success')}, tool={r_sg.get('chosen_tool')}, msg={r_sg.get('message')}")
                if r_sg.get('parameters_used'): print(f"      Params used: {r_sg.get('parameters_used')}")

print("\nAgent: Parameter extraction (FIX 13) and re-tests complete.")
print("Waiting for next instruction.")

Agent: Definitive syntax correction in parameter extraction (FIX 13).

Agent: Registering autonomous goal execution (v10 - FIX 13) & updating aliases...
Tool 'attempt_goal_autonomously_v10' added successfully. Description: (Version 10) Uses v9 heuristic (definitively fixed syntax, improved context priorities). Asks for clarification.
Registered 'attempt_goal_autonomously_v10' and aliased 'attempt_goal_autonomously_v3' to point to v10.

Agent: Re-testing multi-step goal execution (v1) with autonomous worker (v10 via v3 alias - FIX 13).

--- Test 1 (Multi-step REVISED FIX 13): Goal: 'Download todo item 2, save its title to 'todo_title_fix13.txt', read file, store content in knowledge as 'todo_title_fix13'.' ---
Agent log (execute_multi_step_goal_v1_tool): Starting multi-step goal: 'Download todo item 2, save its title to 'todo_title_fix13.txt', read file, store content in knowledg...'. Number of sub-goals: 4
Agent log (execute_multi_step_goal_v1_tool): Initial cumulative context: {}

Age

In [31]:
# Existing tools, _AGENT_TOOL_CONCEPT_KEYWORDS etc., assumed.
import inspect
import re
import json
import traceback

print("Agent: Refining context property path resolution (FIX 14).")

# --- 1. Refine Parameter Extraction Heuristic (v10) ---
def _extract_parameter_value_heuristically_v10( # Renaming to v10
    param_name: str, 
    param_type, 
    goal_description: str, 
    all_quoted_strings_from_goal: list[str],
    consumed_quoted_strings: set[str],
    current_cumulative_context: dict 
) -> any:
    goal_lower = goal_description.lower()
    
    # --- Order of Extraction Attempts ---
    # 1. Explicit assignment in goal string (param_name = 'value')
    # 2. For identifier-like params (key, filepath, path, directory, url): Literal extraction from goal.
    # 3. For data-like params (content, value, etc.): Context variable resolution (with property paths).
    #    THIS IS THE KEY AREA FOR FIX 14.
    # 4. For data-like params: General quoted strings from goal if no context match.
    # 5. General type-based heuristics (method, bool, code_string).

    # 1. Explicit Assignment
    # ... (same as v9_final_syntax_fix) ...
    explicit_assign_match_quoted = re.search(rf"\b{param_name}\s*[:=]\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
    if explicit_assign_match_quoted: return explicit_assign_match_quoted.group(1)
    explicit_assign_match_unquoted = re.search(rf"\b{param_name}\s*[:=]\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
    if explicit_assign_match_unquoted:
        val_str = explicit_assign_match_unquoted.group(1)
        if param_type == int: try: return int(val_str) except ValueError: pass
        if param_type == float: try: return float(val_str) except ValueError: pass
        if param_type == bool:
            if val_str.lower() == 'true': return True
            if val_str.lower() == 'false': return False
        if param_type == str or param_type is None or param_type == inspect.Parameter.empty : return val_str

    # 2. Literal extraction for IDENTIFIER-LIKE parameters
    if param_name == "key": # ... (same as v9_final_syntax_fix) ...
        key_match = re.search(r"(?:with key|key is|for key|as key|named key)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if key_match: return key_match.group(1)
        key_match_unquoted = re.search(r"(?:with key|key is|for key|as key|named key)\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
        if key_match_unquoted: return key_match_unquoted.group(1)
    if param_name == "url": # ... (same as v9_final_syntax_fix) ...
        url_match_re = r"https?://(?:[a-zA-Z0-9.\-_~:/?#\[\]@!$&'()*+,;=%]+[a-zA-Z0-9\-_~:/?#\[\]@!$&*+,;=%])"; url_m = re.search(url_match_re, goal_description)
        extracted_url = None; 
        if url_m: extracted_url = url_m.group(0)
        if url_m: 
            if extracted_url.endswith("'") and goal_description.count(f"'{extracted_url}'") > 0: extracted_url = extracted_url[:-1]
            elif extracted_url.endswith("\"") and goal_description.count(f"\"{extracted_url}\"") > 0: extracted_url = extracted_url[:-1]
            for q_str in all_quoted_strings_from_goal:
                if extracted_url == q_str or f"'{extracted_url}'" == q_str or f"\"{extracted_url}\"" == q_str :
                    if q_str not in consumed_quoted_strings: consumed_quoted_strings.add(q_str); break 
            return extracted_url
    if param_name in ["filepath", "path", "directory"]: # ... (same as v9_final_syntax_fix) ...
        path_pattern_named = re.search(rf"(?:file named|directory named|path is|called)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if path_pattern_named:
            potential_path = path_pattern_named.group(1)
            if potential_path in all_quoted_strings_from_goal and potential_path not in consumed_quoted_strings:
                consumed_quoted_strings.add(potential_path); return potential_path
        path_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            if param_name == "directory" or '.' in q_str or '/' in q_str or '\\' in q_str or any(q_str.endswith(ext) for ext in ['.txt', '.json', '.py']):
                path_candidates.append(q_str)
        if path_candidates: chosen_path = max(path_candidates, key=len); consumed_quoted_strings.add(chosen_path); return chosen_path

    # 3. Context variable resolution for DATA-LIKE parameters (IMPROVED)
    is_data_param = param_name in ["content", "value", "json_payload", "data_payload", "text"]
    if is_data_param and current_cumulative_context and current_cumulative_context.keys():
        context_key_names = sorted(list(current_cumulative_context.keys()), key=len, reverse=True)
        context_key_pattern = f"({ '|'.join(re.escape(k) for k in context_key_names if k) })"
        
        # Pattern 1: "the 'property' from 'context_key'"
        # E.g., "Write the 'title' from the 'last_json_content'..."
        # The 'property' here (e.g. 'title') is often related to the param_name or a general data term.
        # We need to ensure this pattern correctly extracts 'title' and applies it to `last_json_content`.
        # Regex: (the|value of|from|content of) ['"](?P<prop_name>[\w_]+)['"] (of|from) ['"](?P<ctx_key>{context_key_pattern})['"]
        prop_from_context_match = re.search(
            rf"(?:the|value of|from|content of)\s*['\"](?P<prop_name>[\w_][\w\d_]*)['\"]\s*(?:of|from)\s*['\"](?P<ctx_key>{context_key_pattern})['\"]",
            goal_description, re.IGNORECASE
        )
        if prop_from_context_match:
            prop_to_extract = prop_from_context_match.group("prop_name")
            matched_context_key = prop_from_context_match.group("ctx_key")
            if matched_context_key in current_cumulative_context:
                base_value = current_cumulative_context[matched_context_key]
                if isinstance(base_value, dict) and prop_to_extract in base_value:
                    val = base_value[prop_to_extract]
                    print(f"Agent log (_extract_v10): Resolved property '{prop_to_extract}' from context '{matched_context_key}' for '{param_name}'.")
                    return val
                elif isinstance(base_value, list) and prop_to_extract.isdigit() and int(prop_to_extract) < len(base_value): # list index
                    val = base_value[int(prop_to_extract)]
                    print(f"Agent log (_extract_v10): Resolved index '{prop_to_extract}' from context list '{matched_context_key}' for '{param_name}'.")
                    return val

        # Pattern 2: "the 'context_key.property.path'" or "the 'context_key'" (fallback)
        # This will be used if Pattern 1 doesn't match for this specific parameter.
        context_ref_match = re.search(
            rf"(?:the|value of|from|content of)\s*['\"]{context_key_pattern}((?:(?:\.[\w_][\w\d_]*)|(?:\[['\"]\w+['\"]\]))+)?['\"]",
            goal_description, re.IGNORECASE
        )
        if not context_ref_match: # Try direct 'context_key' reference
             context_ref_match = re.search(
                rf"['\"]{context_key_pattern}((?:(?:\.[\w_][\w\d_]*)|(?:\[['\"]\w+['\"]\]))+)?['\"]",
                 goal_description, re.IGNORECASE
             )
        if context_ref_match:
            matched_context_key = context_ref_match.group(1)
            property_path_str = context_ref_match.group(2) # Optional property path
            if matched_context_key in current_cumulative_context:
                value_from_context = current_cumulative_context[matched_context_key]
                if property_path_str: 
                    properties = []; 
                    for part in re.findall(r"\.([\w_][\w\d_]*)|\[['\"]([^'\"]+)['\"]\]", property_path_str): properties.append(part[0] or part[1])
                    temp_val = value_from_context
                    try:
                        for prop_key_in_path in properties: # Renamed to avoid clash with outer prop_key
                            if isinstance(temp_val, dict): temp_val = temp_val.get(prop_key_in_path)
                            elif isinstance(temp_val, list) and prop_key_in_path.isdigit(): temp_val = temp_val[int(prop_key_in_path)]
                            elif hasattr(temp_val, prop_key_in_path): temp_val = getattr(temp_val, prop_key_in_path)
                            else: temp_val = None; break
                        if temp_val is not None:
                            print(f"Agent log (_extract_v10): Resolved context path '{matched_context_key}{property_path_str}' for '{param_name}'.")
                            return temp_val
                    except Exception: pass
                elif is_data_param: # Only use direct context var if it's a data param and NO property path was given by goal for THIS parameter
                    print(f"Agent log (_extract_v10): Resolved direct context var '{matched_context_key}' for data param '{param_name}'.")
                    return value_from_context

    # 4. General quoted strings for data-like parameters as a fallback
    if param_name in ["content", "text", "description", "message", "value"]:
        content_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            content_candidates.append(q_str)
        if content_candidates:
            chosen_content = max(content_candidates, key=len)
            consumed_quoted_strings.add(chosen_content)
            return chosen_content
            
    # 5. General type-based heuristics
    # ... (code_string, method, bool logic - same as v9_final_syntax_fix) ...
    if param_name == "code_string":
        code_block_match = re.search(r"code:\s*(.+)", goal_description, re.IGNORECASE | re.DOTALL);
        if code_block_match: return code_block_match.group(1).strip()
        triple_tick_match = re.search(r"```(?:python\n)?(.*?)```", goal_description, re.DOTALL | re.IGNORECASE)
        if triple_tick_match: return triple_tick_match.group(1).strip()
    if param_name == "method" and (param_type == str or param_type is None or param_type == inspect.Parameter.empty):
        method_search = re.search(r"\b(GET|POST|PUT|DELETE|PATCH|HEAD|OPTIONS)\b", goal_description, re.IGNORECASE);
        if method_search: return method_search.group(1).upper()
        if "post" in goal_lower: return "POST"; 
        if "get" in goal_lower: return "GET";
        if "delete" in goal_lower: return "DELETE";
        if "put" in goal_lower: return "PUT";
    if param_type == bool:
        if re.search(rf"\b{param_name}\b\s*(true|yes|on|recursively|enabled)\b", goal_lower, re.IGNORECASE): return True
        if re.search(rf"\b(is|be|make it)\s*{param_name}\b", goal_lower, re.IGNORECASE): return True
        if re.search(rf"\b{param_name}\b\s*(false|no|off|disabled)\b", goal_lower, re.IGNORECASE): return False
        if re.search(rf"\b(is not|not to be|don't make it)\s*{param_name}\b", goal_lower, re.IGNORECASE): return False
        if param_name in goal_lower and param_name not in ["false", "no", "off"]: return True
        
    return None

# --- Update attempt_goal_autonomously (v11) to use new heuristic v10 ---
def attempt_goal_autonomously_v11_tool(goal_description: str, additional_context: dict = None) -> dict:
    # Structure is same as v10_tool from previous turn, just calls _extract_parameter_value_heuristically_v10
    tool_name_internal = "attempt_goal_autonomously_v11_tool"
    log_prefix = f"Agent log ({tool_name_internal}): "
    print(f"{log_prefix}Attempting goal (v11 - FIX 14): '{goal_description[:100]}...'. Context: {additional_context is not None}")
    current_cumulative_context = (additional_context.get("parameters", {}) if additional_context else {}) or {}
    quoted_match_tuples = re.findall(r"\"([^\"]+)\"|\'([^\']+)\'", goal_description)
    all_quoted_strings_from_goal = [item for tpl in quoted_match_tuples for item in tpl if item]
    if 'suggest_tools_for_goal' not in _AGENT_TOOLS: return {"success": False, "message": "'suggest_tools_for_goal' not found."}
    chosen_tool_name_override = additional_context.get("chosen_tool_override") if additional_context else None
    chosen_tool_name = ""; tool_callable = None
    if chosen_tool_name_override:
        if chosen_tool_name_override in _AGENT_TOOLS: tool_callable = _AGENT_TOOLS[chosen_tool_name_override]; chosen_tool_name = chosen_tool_name_override; print(f"{log_prefix}Using pre-selected tool: '{chosen_tool_name}'.")
        else: return {"success": False, "message": f"Pre-selected tool '{chosen_tool_name_override}' not found."}
    else:
        suggestion_result = _AGENT_TOOLS['suggest_tools_for_goal'](goal_description=goal_description, top_n=1)
        print(f"{log_prefix}Tool suggestion: {suggestion_result['message']}")
        if not suggestion_result['success'] or not suggestion_result['suggestions']: return {"success": False, "message": "No suitable tools suggested."}
        top_suggestion = suggestion_result['suggestions'][0]; chosen_tool_name = top_suggestion['tool_name']; relevance_score = top_suggestion['relevance_score']
        print(f"{log_prefix}Top suggested: '{chosen_tool_name}' (Score: {relevance_score}). Reason: {top_suggestion.get('reason', 'N/A')}")
        if relevance_score < _MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION_V3: return {"success": False, "message": f"Tool score {relevance_score} below threshold."}
        if chosen_tool_name not in _AGENT_TOOLS: return {"success": False, "message": f"Suggested tool '{chosen_tool_name}' not found."}
        tool_callable = _AGENT_TOOLS[chosen_tool_name]
    sig = inspect.signature(tool_callable); filled_params = {}; missing_required_params_info = []; consumed_quoted_strings_for_this_call = set()
    print(f"{log_prefix}Params for '{chosen_tool_name}': {list(sig.parameters.keys())}")
    param_order = sorted(sig.parameters.items(), key=lambda item: 0 if item[0] in ["key", "filepath", "path", "url", "directory"] else 1)

    for param_name, param_obj in param_order:
        param_type_annotation = param_obj.annotation if param_obj.annotation != inspect.Parameter.empty else None
        if param_name in current_cumulative_context:
            extracted_value = current_cumulative_context[param_name]; print(f"{log_prefix}  Using '{param_name}' from context (pre-filled).")
        else:
            extracted_value = _extract_parameter_value_heuristically_v10( # CALLING THE CORRECTED V10 HEURISTIC
                param_name, param_type_annotation, goal_description, 
                all_quoted_strings_from_goal, consumed_quoted_strings_for_this_call,
                current_cumulative_context
            )
        if extracted_value is not None:
            if param_type_annotation and isinstance(extracted_value, str):
                try:
                    if param_type_annotation == bool: extracted_value = (extracted_value.lower() == 'true')
                    elif param_type_annotation == int: extracted_value = int(extracted_value)
                    elif param_type_annotation == float: extracted_value = float(extracted_value)
                except ValueError: print(f"{log_prefix}Warn: Could not convert '{str(extracted_value)[:20]}' for '{param_name}'.")
            filled_params[param_name] = extracted_value; print(f"{log_prefix}  Filled '{param_name}': {str(extracted_value)[:50]}...")
        elif param_obj.default != inspect.Parameter.empty:
            filled_params[param_name] = param_obj.default; print(f"{log_prefix}  Default for '{param_name}': {str(param_obj.default)[:50]}...")
        else:
            missing_required_params_info.append({"name": param_name, "type": str(param_type_annotation), "description": f"Value for '{param_name}'."}); print(f"{log_prefix}  Could not fill required '{param_name}'.")
    if missing_required_params_info:
        param_names_missing = [p['name'] for p in missing_required_params_info]; msg = f"Clarification needed for tool '{chosen_tool_name}': {', '.join(param_names_missing)}."
        clar_prompt = f"Please provide values for: {', '.join(param_names_missing)} for tool '{chosen_tool_name}' (goal: '{goal_description[:30]}...')."
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, "clarification_request": {"type": "missing_parameters", "original_goal": goal_description, "tool_name": chosen_tool_name, "missing_params_details": missing_required_params_info, "prompt": clar_prompt}}
    print(f"{log_prefix}Executing '{chosen_tool_name}' with: {json.dumps(filled_params, default=str, indent=2)}")
    try:
        execution_result = tool_callable(**filled_params)
        overall_success = execution_result.get('success', True) if isinstance(execution_result, dict) else True
        return {"success": overall_success, "message": f"Tool '{chosen_tool_name}' executed.", "chosen_tool": chosen_tool_name, "parameters_used": filled_params, "execution_result": execution_result}
    except Exception as e: tb_str = traceback.format_exc(); return {"success": False, "message": f"Error executing '{chosen_tool_name}': {e}", "execution_result": {"error": f"{e}\n{tb_str}"}}

# --- Register/Update Tools ---
print("\nAgent: Registering autonomous goal execution (v11 - FIX 14) & updating aliases...")
if 'add_tool' in globals() and callable(add_tool):
    add_tool(
        name="attempt_goal_autonomously_v11",
        tool_callable=attempt_goal_autonomously_v11_tool,
        description="(Version 11) Uses v10 heuristic (improved context property path). Asks for clarification."
    )
    _AGENT_TOOLS['attempt_goal_autonomously_v3'] = attempt_goal_autonomously_v11_tool
    _TOOL_DESCRIPTIONS['attempt_goal_autonomously_v3'] = "(Version 11, aliased as v3 - FIX 14) Uses context-aware v10 heuristic."
    print("Registered 'attempt_goal_autonomously_v11' and aliased 'attempt_goal_autonomously_v3' to point to v11.")
else:
    print("CRITICAL ERROR: 'add_tool' function not found.")

# --- Re-test Multi-Step Goal Execution Tool ---
print("\nAgent: Re-testing multi-step goal execution (v1) with autonomous worker (v11 via v3 alias - FIX 14).")
tool_to_test_name = "execute_multi_step_goal_v1"
if tool_to_test_name not in _AGENT_TOOLS:
    print(f"Agent: Skipping tests as '{tool_to_test_name}' tool was not registered.")
elif '_extract_parameter_value_heuristically_v10' not in globals() or not callable(globals()['_extract_parameter_value_heuristically_v10']):
    print("CRITICAL ERROR: _extract_parameter_value_heuristically_v10 is not available for the test.")
else:
    overall_goal1 = "Download todo item 2, save its title to 'todo_title_fix14.txt', read file, store content in knowledge as 'todo_title_fix14'."
    sub_goals1 = [
        "Download content from 'https://jsonplaceholder.typicode.com/todos/2'",
        "Write the 'title' from the 'last_json_content' to a file named 'todo_title_fix14.txt'",
        "Read the file 'todo_title_fix14.txt'",
        "Save the 'last_content' from reading the file into knowledge with key 'todo_title_fix14'"
    ]
    print(f"\n--- Test 1 (Multi-step REVISED FIX 14): Goal: '{overall_goal1}' ---")
    result1 = _AGENT_TOOLS[tool_to_test_name](overall_goal_description=overall_goal1, sub_goal_strings=sub_goals1)
    print(f"Result for multi-step goal 1: {json.dumps(result1, default=str, indent=2)}")
    if result1.get('success'):
        print("  SUCCESS (Test 1 REVISED FIX 14): Multi-step goal reported success.")
        if 'retrieve_knowledge' in _AGENT_TOOLS:
            kv = _AGENT_TOOLS['retrieve_knowledge'](key='todo_title_fix14')
            if kv.get('found') and kv.get('value') == "quis ut nam facilis et officia qui": # Expected title
                print(f"  VERIFIED (Test 1 REVISED FIX 14): Knowledge 'todo_title_fix14' correct.")
            else: 
                print(f"  VERIFICATION FAILED (Test 1 REVISED FIX 14): Knowledge 'todo_title_fix14'. Expected 'quis ut nam facilis et officia qui', Got: {kv.get('value')}")
            _AGENT_TOOLS['delete_knowledge'](key='todo_title_fix14')
        if 'delete_file' in _AGENT_TOOLS: _AGENT_TOOLS['delete_file'](filepath='todo_title_fix14.txt')
    else:
        print(f"  FAILURE (Test 1 REVISED FIX 14): Multi-step goal failed. Message: {result1.get('message')}")
        if result1.get('sub_goal_results'):
            for i, r_sg in enumerate(result1['sub_goal_results']):
                print(f"    Sub-goal {i+1}: success={r_sg.get('success')}, tool={r_sg.get('chosen_tool')}, msg={r_sg.get('message')}")
                if r_sg.get('parameters_used'): print(f"      Params used: {r_sg.get('parameters_used')}")

print("\nAgent: Parameter extraction priorities (FIX 14) and re-tests complete.")
print("Waiting for next instruction.")

SyntaxError: invalid syntax (4181521165.py, line 35)

In [32]:
# Existing tools, _AGENT_TOOL_CONCEPT_KEYWORDS etc., assumed.
import inspect
import re
import json
import traceback

print("Agent: Definitive syntax correction in parameter extraction (FIX 15 - FINAL ATTEMPT at this syntax).")

# --- 1. Corrected Parameter Extraction Heuristic (v11 - FINAL SYNTAX FIX) ---
def _extract_parameter_value_heuristically_v11_final_syntax_fix(
    param_name: str, 
    param_type, 
    goal_description: str, 
    all_quoted_strings_from_goal: list[str],
    consumed_quoted_strings: set[str],
    current_cumulative_context: dict 
) -> any:
    goal_lower = goal_description.lower()
    
    # 1. Explicit Assignment (param_name = 'value' or param_name: value)
    explicit_assign_match_quoted = re.search(rf"\b{param_name}\s*[:=]\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
    if explicit_assign_match_quoted: return explicit_assign_match_quoted.group(1)
    
    explicit_assign_match_unquoted = re.search(rf"\b{param_name}\s*[:=]\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
    if explicit_assign_match_unquoted:
        val_str = explicit_assign_match_unquoted.group(1)
        # ***** DEFINITIVELY CORRECTED SYNTAX FOR TRY-EXCEPT *****
        if param_type == int:
            try:
                return int(val_str)
            except ValueError:
                pass 
        if param_type == float:
            try:
                return float(val_str)
            except ValueError:
                pass
        # ***** END OF CORRECTION *****
        if param_type == bool:
            if val_str.lower() == 'true': return True
            if val_str.lower() == 'false': return False
        if param_type == str or param_type is None or param_type == inspect.Parameter.empty : return val_str

    # 2. For identifier-like params (key, filepath, path, directory, url): Literal extraction from goal FIRST.
    if param_name == "key":
        key_match = re.search(r"(?:with key|key is|for key|as key|named key)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if key_match: return key_match.group(1)
        key_match_unquoted = re.search(r"(?:with key|key is|for key|as key|named key)\s*([^\s\"',]+)", goal_description, re.IGNORECASE)
        if key_match_unquoted: return key_match_unquoted.group(1)

    if param_name == "url":
        url_match_re = r"https?://(?:[a-zA-Z0-9.\-_~:/?#\[\]@!$&'()*+,;=%]+[a-zA-Z0-9\-_~:/?#\[\]@!$&*+,;=%])"; url_m = re.search(url_match_re, goal_description)
        extracted_url = None 
        if url_m: extracted_url = url_m.group(0)
        if url_m: 
            if extracted_url.endswith("'") and goal_description.count(f"'{extracted_url}'") > 0: extracted_url = extracted_url[:-1]
            elif extracted_url.endswith("\"") and goal_description.count(f"\"{extracted_url}\"") > 0: extracted_url = extracted_url[:-1]
            for q_str in all_quoted_strings_from_goal:
                if extracted_url == q_str or f"'{extracted_url}'" == q_str or f"\"{extracted_url}\"" == q_str :
                    if q_str not in consumed_quoted_strings: consumed_quoted_strings.add(q_str); break 
            return extracted_url

    if param_name in ["filepath", "path", "directory"]:
        path_pattern_named = re.search(rf"(?:file named|directory named|path is|called)\s*[\"']([^\"']+)[\"']", goal_description, re.IGNORECASE)
        if path_pattern_named:
            potential_path = path_pattern_named.group(1)
            if potential_path in all_quoted_strings_from_goal and potential_path not in consumed_quoted_strings:
                consumed_quoted_strings.add(potential_path); return potential_path
        path_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            if param_name == "directory" or \
               ('.' in q_str or '/' in q_str or '\\' in q_str or any(q_str.endswith(ext) for ext in ['.txt', '.json', '.py'])):
                path_candidates.append(q_str)
        if path_candidates: chosen_path = max(path_candidates, key=len); consumed_quoted_strings.add(chosen_path); return chosen_path

    # 3. Context variable resolution (especially for data-like parameters)
    is_data_param = param_name in ["content", "value", "json_payload", "data_payload", "text"]
    if is_data_param and current_cumulative_context and current_cumulative_context.keys():
        context_key_names = sorted(list(current_cumulative_context.keys()), key=len, reverse=True)
        context_key_pattern = f"({ '|'.join(re.escape(k) for k in context_key_names if k) })"
        
        prop_from_context_match = re.search(
            rf"(?:the|value of|from|content of)\s*['\"](?P<prop_name>[\w_][\w\d_]*)['\"]\s*(?:of|from)\s*['\"](?P<ctx_key>{context_key_pattern})['\"]",
            goal_description, re.IGNORECASE
        )
        if prop_from_context_match:
            prop_to_extract = prop_from_context_match.group("prop_name")
            matched_context_key = prop_from_context_match.group("ctx_key")
            if matched_context_key in current_cumulative_context:
                base_value = current_cumulative_context[matched_context_key]
                if isinstance(base_value, dict) and prop_to_extract in base_value:
                    val = base_value[prop_to_extract]
                    print(f"Agent log (_extract_v11_fsf): Resolved prop '{prop_to_extract}' from context '{matched_context_key}' for '{param_name}'.")
                    return val
                elif isinstance(base_value, list) and prop_to_extract.isdigit() and int(prop_to_extract) < len(base_value):
                    val = base_value[int(prop_to_extract)]
                    print(f"Agent log (_extract_v11_fsf): Resolved index '{prop_to_extract}' from context list '{matched_context_key}' for '{param_name}'.")
                    return val
        
        context_ref_match = re.search(
            rf"(?:the|value of|from|content of)\s*['\"]{context_key_pattern}((?:(?:\.[\w_][\w\d_]*)|(?:\[['\"]\w+['\"]\]))+)?['\"]",
            goal_description, re.IGNORECASE
        )
        if not context_ref_match:
             context_ref_match = re.search(
                rf"['\"]{context_key_pattern}((?:(?:\.[\w_][\w\d_]*)|(?:\[['\"]\w+['\"]\]))+)?['\"]",
                 goal_description, re.IGNORECASE
             )
        if context_ref_match:
            matched_context_key = context_ref_match.group(1)
            property_path_str = context_ref_match.group(2)
            if matched_context_key in current_cumulative_context:
                value_from_context = current_cumulative_context[matched_context_key]
                if property_path_str: 
                    properties = []; 
                    for part in re.findall(r"\.([\w_][\w\d_]*)|\[['\"]([^'\"]+)['\"]\]", property_path_str): properties.append(part[0] or part[1])
                    temp_val = value_from_context
                    try:
                        for prop_key_in_path in properties:
                            if isinstance(temp_val, dict): temp_val = temp_val.get(prop_key_in_path)
                            elif isinstance(temp_val, list) and prop_key_in_path.isdigit(): temp_val = temp_val[int(prop_key_in_path)]
                            elif hasattr(temp_val, prop_key_in_path): temp_val = getattr(temp_val, prop_key_in_path)
                            else: temp_val = None; break
                        if temp_val is not None:
                            print(f"Agent log (_extract_v11_fsf): Resolved context path '{matched_context_key}{property_path_str}' for '{param_name}'.")
                            return temp_val
                    except Exception: pass
                elif is_data_param:
                    print(f"Agent log (_extract_v11_fsf): Resolved direct context var '{matched_context_key}' for data param '{param_name}'.")
                    return value_from_context

    # 4. General quoted strings for data-like parameters as a fallback
    if param_name in ["content", "text", "description", "message", "value"]:
        content_candidates = []
        for q_str in all_quoted_strings_from_goal:
            if q_str in consumed_quoted_strings: continue
            content_candidates.append(q_str)
        if content_candidates:
            chosen_content = max(content_candidates, key=len)
            consumed_quoted_strings.add(chosen_content)
            return chosen_content
            
    # 5. General type-based heuristics
    if param_name == "code_string":
        code_block_match = re.search(r"code:\s*(.+)", goal_description, re.IGNORECASE | re.DOTALL);
        if code_block_match: return code_block_match.group(1).strip()
        triple_tick_match = re.search(r"```(?:python\n)?(.*?)```", goal_description, re.DOTALL | re.IGNORECASE)
        if triple_tick_match: return triple_tick_match.group(1).strip()
    if param_name == "method" and (param_type == str or param_type is None or param_type == inspect.Parameter.empty):
        method_search = re.search(r"\b(GET|POST|PUT|DELETE|PATCH|HEAD|OPTIONS)\b", goal_description, re.IGNORECASE);
        if method_search: return method_search.group(1).upper()
        if "post" in goal_lower: return "POST"; 
        if "get" in goal_lower: return "GET";
        if "delete" in goal_lower: return "DELETE";
        if "put" in goal_lower: return "PUT";
    if param_type == bool:
        if re.search(rf"\b{param_name}\b\s*(true|yes|on|recursively|enabled)\b", goal_lower, re.IGNORECASE): return True
        if re.search(rf"\b(is|be|make it)\s*{param_name}\b", goal_lower, re.IGNORECASE): return True
        if re.search(rf"\b{param_name}\b\s*(false|no|off|disabled)\b", goal_lower, re.IGNORECASE): return False
        if re.search(rf"\b(is not|not to be|don't make it)\s*{param_name}\b", goal_lower, re.IGNORECASE): return False
        if param_name in goal_lower and param_name not in ["false", "no", "off"]: return True
    return None

# --- Update attempt_goal_autonomously (v12) to use new heuristic v11 ---
def attempt_goal_autonomously_v12_tool(goal_description: str, additional_context: dict = None) -> dict:
    tool_name_internal = "attempt_goal_autonomously_v12_tool"
    log_prefix = f"Agent log ({tool_name_internal}): "
    print(f"{log_prefix}Attempting goal (v12 - FIX 15): '{goal_description[:100]}...'. Context: {additional_context is not None}")
    current_cumulative_context = (additional_context.get("parameters", {}) if additional_context else {}) or {}
    quoted_match_tuples = re.findall(r"\"([^\"]+)\"|\'([^\']+)\'", goal_description)
    all_quoted_strings_from_goal = [item for tpl in quoted_match_tuples for item in tpl if item]
    if 'suggest_tools_for_goal' not in _AGENT_TOOLS: return {"success": False, "message": "'suggest_tools_for_goal' not found."}
    chosen_tool_name_override = additional_context.get("chosen_tool_override") if additional_context else None
    chosen_tool_name = ""; tool_callable = None
    if chosen_tool_name_override:
        if chosen_tool_name_override in _AGENT_TOOLS: tool_callable = _AGENT_TOOLS[chosen_tool_name_override]; chosen_tool_name = chosen_tool_name_override; print(f"{log_prefix}Using pre-selected tool: '{chosen_tool_name}'.")
        else: return {"success": False, "message": f"Pre-selected tool '{chosen_tool_name_override}' not found."}
    else:
        suggestion_result = _AGENT_TOOLS['suggest_tools_for_goal'](goal_description=goal_description, top_n=1)
        print(f"{log_prefix}Tool suggestion: {suggestion_result['message']}")
        if not suggestion_result['success'] or not suggestion_result['suggestions']: return {"success": False, "message": "No suitable tools suggested."}
        top_suggestion = suggestion_result['suggestions'][0]; chosen_tool_name = top_suggestion['tool_name']; relevance_score = top_suggestion['relevance_score']
        print(f"{log_prefix}Top suggested: '{chosen_tool_name}' (Score: {relevance_score}). Reason: {top_suggestion.get('reason', 'N/A')}")
        if relevance_score < _MIN_RELEVANCE_SCORE_FOR_AUTONOMOUS_EXECUTION_V3: return {"success": False, "message": f"Tool score {relevance_score} below threshold."}
        if chosen_tool_name not in _AGENT_TOOLS: return {"success": False, "message": f"Suggested tool '{chosen_tool_name}' not found."}
        tool_callable = _AGENT_TOOLS[chosen_tool_name]
    sig = inspect.signature(tool_callable); filled_params = {}; missing_required_params_info = []; consumed_quoted_strings_for_this_call = set()
    print(f"{log_prefix}Params for '{chosen_tool_name}': {list(sig.parameters.keys())}")
    param_order = sorted(sig.parameters.items(), key=lambda item: 0 if item[0] in ["key", "filepath", "path", "url", "directory"] else 1)

    for param_name, param_obj in param_order:
        param_type_annotation = param_obj.annotation if param_obj.annotation != inspect.Parameter.empty else None
        if param_name in current_cumulative_context:
            extracted_value = current_cumulative_context[param_name]; print(f"{log_prefix}  Using '{param_name}' from context (pre-filled).")
        else:
            extracted_value = _extract_parameter_value_heuristically_v11_final_syntax_fix( # CALLING THE CORRECTED V11 HEURISTIC
                param_name, param_type_annotation, goal_description, 
                all_quoted_strings_from_goal, consumed_quoted_strings_for_this_call,
                current_cumulative_context
            )
        if extracted_value is not None:
            if param_type_annotation and isinstance(extracted_value, str):
                try:
                    if param_type_annotation == bool: extracted_value = (extracted_value.lower() == 'true')
                    elif param_type_annotation == int: extracted_value = int(extracted_value)
                    elif param_type_annotation == float: extracted_value = float(extracted_value)
                except ValueError: print(f"{log_prefix}Warn: Could not convert '{str(extracted_value)[:20]}' for '{param_name}'.")
            filled_params[param_name] = extracted_value; print(f"{log_prefix}  Filled '{param_name}': {str(extracted_value)[:50]}...")
        elif param_obj.default != inspect.Parameter.empty:
            filled_params[param_name] = param_obj.default; print(f"{log_prefix}  Default for '{param_name}': {str(param_obj.default)[:50]}...")
        else:
            missing_required_params_info.append({"name": param_name, "type": str(param_type_annotation), "description": f"Value for '{param_name}'."}); print(f"{log_prefix}  Could not fill required '{param_name}'.")
    if missing_required_params_info:
        param_names_missing = [p['name'] for p in missing_required_params_info]; msg = f"Clarification needed for tool '{chosen_tool_name}': {', '.join(param_names_missing)}."
        clar_prompt = f"Please provide values for: {', '.join(param_names_missing)} for tool '{chosen_tool_name}' (goal: '{goal_description[:30]}...')."
        return {"success": False, "message": msg, "chosen_tool": chosen_tool_name, "clarification_request": {"type": "missing_parameters", "original_goal": goal_description, "tool_name": chosen_tool_name, "missing_params_details": missing_required_params_info, "prompt": clar_prompt}}
    print(f"{log_prefix}Executing '{chosen_tool_name}' with: {json.dumps(filled_params, default=str, indent=2)}")
    try:
        execution_result = tool_callable(**filled_params)
        overall_success = execution_result.get('success', True) if isinstance(execution_result, dict) else True
        return {"success": overall_success, "message": f"Tool '{chosen_tool_name}' executed.", "chosen_tool": chosen_tool_name, "parameters_used": filled_params, "execution_result": execution_result}
    except Exception as e: tb_str = traceback.format_exc(); return {"success": False, "message": f"Error executing '{chosen_tool_name}': {e}", "execution_result": {"error": f"{e}\n{tb_str}"}}

# --- Register/Update Tools ---
print("\nAgent: Registering autonomous goal execution (v12 - FIX 15) & updating aliases...")
if 'add_tool' in globals() and callable(add_tool):
    add_tool(
        name="attempt_goal_autonomously_v12",
        tool_callable=attempt_goal_autonomously_v12_tool,
        description="(Version 12) Uses v11 heuristic (syntax fixed). Asks for clarification."
    )
    _AGENT_TOOLS['attempt_goal_autonomously_v3'] = attempt_goal_autonomously_v12_tool
    _TOOL_DESCRIPTIONS['attempt_goal_autonomously_v3'] = "(Version 12, aliased as v3 - FIX 15) Uses context-aware v11 heuristic."
    print("Registered 'attempt_goal_autonomously_v12' and aliased 'attempt_goal_autonomously_v3' to point to v12.")
else:
    print("CRITICAL ERROR: 'add_tool' function not found.")

# --- Re-test Multi-Step Goal Execution Tool ---
print("\nAgent: Re-testing multi-step goal execution (v1) with autonomous worker (v12 via v3 alias - FIX 15).")
tool_to_test_name = "execute_multi_step_goal_v1"
if tool_to_test_name not in _AGENT_TOOLS:
    print(f"Agent: Skipping tests as '{tool_to_test_name}' tool was not registered.")
elif '_extract_parameter_value_heuristically_v11_final_syntax_fix' not in globals() or \
     not callable(globals()['_extract_parameter_value_heuristically_v11_final_syntax_fix']):
    print("CRITICAL ERROR: _extract_parameter_value_heuristically_v11_final_syntax_fix is not available for the test.")
else:
    overall_goal1 = "Download todo item 2, save its title to 'todo_title_fix15.txt', read file, store content in knowledge as 'todo_title_fix15'."
    sub_goals1 = [
        "Download content from 'https://jsonplaceholder.typicode.com/todos/2'",
        "Write the 'title' from the 'last_json_content' to a file named 'todo_title_fix15.txt'",
        "Read the file 'todo_title_fix15.txt'",
        "Save the 'last_content' from reading the file into knowledge with key 'todo_title_fix15'"
    ]
    print(f"\n--- Test 1 (Multi-step REVISED FIX 15): Goal: '{overall_goal1}' ---")
    result1 = _AGENT_TOOLS[tool_to_test_name](overall_goal_description=overall_goal1, sub_goal_strings=sub_goals1)
    print(f"Result for multi-step goal 1: {json.dumps(result1, default=str, indent=2)}")
    if result1.get('success'):
        print("  SUCCESS (Test 1 REVISED FIX 15): Multi-step goal reported success.")
        if 'retrieve_knowledge' in _AGENT_TOOLS:
            kv = _AGENT_TOOLS['retrieve_knowledge'](key='todo_title_fix15')
            if kv.get('found') and kv.get('value') == "quis ut nam facilis et officia qui":
                print(f"  VERIFIED (Test 1 REVISED FIX 15): Knowledge 'todo_title_fix15' correct.")
            else: 
                print(f"  VERIFICATION FAILED (Test 1 REVISED FIX 15): Knowledge 'todo_title_fix15'. Expected 'quis ut nam facilis et officia qui', Got: {kv.get('value')}")
            _AGENT_TOOLS['delete_knowledge'](key='todo_title_fix15')
        if 'delete_file' in _AGENT_TOOLS: _AGENT_TOOLS['delete_file'](filepath='todo_title_fix15.txt')
    else:
        print(f"  FAILURE (Test 1 REVISED FIX 15): Multi-step goal failed. Message: {result1.get('message')}")
        if result1.get('sub_goal_results'):
            for i, r_sg in enumerate(result1['sub_goal_results']):
                print(f"    Sub-goal {i+1}: success={r_sg.get('success')}, tool={r_sg.get('chosen_tool')}, msg={r_sg.get('message')}")
                if r_sg.get('parameters_used'): print(f"      Params used: {r_sg.get('parameters_used')}")

print("\nAgent: Parameter extraction (FIX 15) and re-tests complete.")
print("Waiting for next instruction.")

Agent: Definitive syntax correction in parameter extraction (FIX 15 - FINAL ATTEMPT at this syntax).

Agent: Registering autonomous goal execution (v12 - FIX 15) & updating aliases...
Tool 'attempt_goal_autonomously_v12' added successfully. Description: (Version 12) Uses v11 heuristic (syntax fixed). Asks for clarification.
Registered 'attempt_goal_autonomously_v12' and aliased 'attempt_goal_autonomously_v3' to point to v12.

Agent: Re-testing multi-step goal execution (v1) with autonomous worker (v12 via v3 alias - FIX 15).

--- Test 1 (Multi-step REVISED FIX 15): Goal: 'Download todo item 2, save its title to 'todo_title_fix15.txt', read file, store content in knowledge as 'todo_title_fix15'.' ---
Agent log (execute_multi_step_goal_v1_tool): Starting multi-step goal: 'Download todo item 2, save its title to 'todo_title_fix15.txt', read file, store content in knowledg...'. Number of sub-goals: 4
Agent log (execute_multi_step_goal_v1_tool): Initial cumulative context: {}

Agent log (ex

In [33]:
import inspect
import os
import pprint
import re
import shutil # For robust directory creation/deletion

# --- Helper to get source, trying to be robust ---
def _get_function_source_safely(func_obj):
    try:
        # For methods bound to an instance or class, get source of the underlying function
        if inspect.ismethod(func_obj):
            func_to_inspect = func_obj.__func__
        else:
            func_to_inspect = func_obj
        
        # Handle cases where func_obj might be a callable class instance
        if not inspect.isfunction(func_to_inspect) and not inspect.isclass(func_to_inspect) and hasattr(func_to_inspect, '__call__') and not isinstance(func_to_inspect, type):
             # Try to get source of the class if it's an instance of a callable class
            try:
                return inspect.getsource(func_to_inspect.__class__) + f"\n\n# Note: Above is the class for callable instance {func_obj.__name__ if hasattr(func_obj, '__name__') else str(func_obj)}\n"
            except (TypeError, OSError):
                 pass # Fall through if class source can't be found

        return inspect.getsource(func_to_inspect)
    except TypeError: 
        name = func_obj.__name__ if hasattr(func_obj, '__name__') else str(func_obj)
        return f"# Source code for '{name}' could not be retrieved (possibly built-in or complex callable)."
    except OSError: 
        name = func_obj.__name__ if hasattr(func_obj, '__name__') else str(func_obj)
        return f"# Source code for '{name}' could not be retrieved (source file likely not available)."
    except Exception as e:
        name = func_obj.__name__ if hasattr(func_obj, '__name__') else str(func_obj)
        return f"# Error retrieving source for '{name}': {e}"

# --- Export Agent Code Tool ---
def export_agent_code_tool(output_base_dir: str) -> dict:
    """
    Exports the agent's current tools, core functions, and configurations
    into a directory structure suitable for a code repository.
    """
    tool_name = "export_agent_code_tool"
    log_prefix = f"Agent log ({tool_name}): "
    print(f"{log_prefix}Starting agent code export to '{output_base_dir}'.")

    if not all(t in _AGENT_TOOLS for t in ['create_directory', 'write_file']):
        return {"success": False, "message": "Export failed: Missing create_directory or write_file tools.", "output_path": None}

    if os.path.exists(output_base_dir):
        print(f"{log_prefix}Output directory '{output_base_dir}' exists. Cleaning and recreating.")
        try:
            shutil.rmtree(output_base_dir)
        except Exception as e:
            return {"success": False, "message": f"Failed to clean existing output dir '{output_base_dir}': {e}", "output_path": output_base_dir}
    
    _AGENT_TOOLS['create_directory'](path=output_base_dir)
    src_dir = os.path.join(output_base_dir, "src")
    _AGENT_TOOLS['create_directory'](path=src_dir)
    tool_definitions_dir = os.path.join(src_dir, "tool_definitions")
    _AGENT_TOOLS['create_directory'](path=tool_definitions_dir)

    # Core agent functions (not tools but part of framework)
    core_function_names_to_export = {
        "get_agent_version_info": globals().get("get_agent_version_info"),
        "add_tool": globals().get("add_tool"),
    }
    
    data_structures_to_export = {
        "_AGENT_VERSION": _AGENT_VERSION if "_AGENT_VERSION" in globals() else "0.0.0-unknown",
        "_AGENT_TOOL_CONCEPT_KEYWORDS": _AGENT_TOOL_CONCEPT_KEYWORDS if "_AGENT_TOOL_CONCEPT_KEYWORDS" in globals() else {},
    }

    agent_core_path = os.path.join(src_dir, "agent_core.py")
    core_content_parts = [
        "# Agent Core Infrastructure\n",
        "import pprint\nimport inspect\nimport re\nimport json\nimport traceback\nimport os\nimport time\nimport uuid\nimport shutil\nimport contextlib\nimport io\n",
        "# Global stores, populated by tool_registration.py or direct use\n",
        "_AGENT_TOOLS = {}",
        "_TOOL_DESCRIPTIONS = {}\n", # Populated by add_tool calls
    ]
    for name, data in data_structures_to_export.items():
        if isinstance(data, str): core_content_parts.append(f"{name} = \"{data}\"\n")
        else: core_content_parts.append(f"{name} = {pprint.pformat(data)}\n")
    
    for func_name, func_obj in core_function_names_to_export.items():
        if func_obj and callable(func_obj):
            core_content_parts.append(_get_function_source_safely(func_obj))
            core_content_parts.append("\n")
        else: core_content_parts.append(f"# Core function '{func_name}' not found.\n")

    _AGENT_TOOLS['write_file'](filepath=agent_core_path, content="".join(core_content_parts))
    print(f"{log_prefix}Written '{agent_core_path}'.")

    # Tool Definitions
    # Map tool *function object names* to their desired module file.
    # This assumes the function's __name__ is what we want to use for definition.
    # If aliases are complex, this mapping needs care.
    tool_func_to_module_file = {
        # File System
        globals().get('write_file_v2', '').__name__ if 'write_file_v2' in globals() else 'write_file_v2': "file_system.py",
        globals().get('read_file_v2', '').__name__ if 'read_file_v2' in globals() else 'read_file_v2': "file_system.py",
        globals().get('list_directory_v2', '').__name__ if 'list_directory_v2' in globals() else 'list_directory_v2': "file_system.py",
        globals().get('create_directory_tool', '').__name__ if 'create_directory_tool' in globals() else 'create_directory_tool': "file_system.py",
        globals().get('delete_file_tool', '').__name__ if 'delete_file_tool' in globals() else 'delete_file_tool': "file_system.py",
        globals().get('delete_directory_tool', '').__name__ if 'delete_directory_tool' in globals() else 'delete_directory_tool': "file_system.py",
        # HTTP
        globals().get('make_http_request_tool', '').__name__ if 'make_http_request_tool' in globals() else 'make_http_request_tool': "http_requests.py",
        # Knowledge
        globals().get('_load_knowledge_store', '').__name__ if '_load_knowledge_store' in globals() else '_load_knowledge_store': "knowledge_store.py",
        globals().get('_save_knowledge_store', '').__name__ if '_save_knowledge_store' in globals() else '_save_knowledge_store': "knowledge_store.py",
        globals().get('save_knowledge_tool', '').__name__ if 'save_knowledge_tool' in globals() else 'save_knowledge_tool': "knowledge_store.py",
        globals().get('retrieve_knowledge_tool', '').__name__ if 'retrieve_knowledge_tool' in globals() else 'retrieve_knowledge_tool': "knowledge_store.py",
        globals().get('list_knowledge_keys_tool', '').__name__ if 'list_knowledge_keys_tool' in globals() else 'list_knowledge_keys_tool': "knowledge_store.py",
        globals().get('delete_knowledge_tool', '').__name__ if 'delete_knowledge_tool' in globals() else 'delete_knowledge_tool': "knowledge_store.py",
        # Task Management
        globals().get('_load_tasks', '').__name__ if '_load_tasks' in globals() else '_load_tasks': "task_management.py",
        globals().get('_save_tasks', '').__name__ if '_save_tasks' in globals() else '_save_tasks': "task_management.py",
        globals().get('add_task_tool', '').__name__ if 'add_task_tool' in globals() else 'add_task_tool': "task_management.py",
        globals().get('get_task_tool', '').__name__ if 'get_task_tool' in globals() else 'get_task_tool': "task_management.py",
        globals().get('update_task_tool', '').__name__ if 'update_task_tool' in globals() else 'update_task_tool': "task_management.py",
        globals().get('list_tasks_tool', '').__name__ if 'list_tasks_tool' in globals() else 'list_tasks_tool': "task_management.py", # Also a tool
        globals().get('remove_task_tool', '').__name__ if 'remove_task_tool' in globals() else 'remove_task_tool': "task_management.py",
        # Code Execution
        globals().get('execute_python_code', '').__name__ if 'execute_python_code' in globals() else 'execute_python_code': "code_execution.py",
        # Goal Processing & Agent Utils (combining suggest, heuristic, autonomous, planner, list_tools)
        globals().get('list_tools', '').__name__ if 'list_tools' in globals() else 'list_tools': "agent_utils.py", # list_tools is a tool
        globals().get('get_agent_status', '').__name__ if 'get_agent_status' in globals() else 'get_agent_status': "agent_utils.py", # get_agent_status is a tool
        globals().get('suggest_tools_for_goal_v3', '').__name__ if 'suggest_tools_for_goal_v3' in globals() else 'suggest_tools_for_goal_v3': "goal_processing.py",
        globals().get('_extract_parameter_value_heuristically_v11_final_syntax_fix', '').__name__ if '_extract_parameter_value_heuristically_v11_final_syntax_fix' in globals() else '_extract_parameter_value_heuristically_v11_final_syntax_fix': "goal_processing.py",
        globals().get('attempt_goal_autonomously_v12_tool', '').__name__ if 'attempt_goal_autonomously_v12_tool' in globals() else 'attempt_goal_autonomously_v12_tool': "goal_processing.py",
        globals().get('execute_multi_step_goal_v1_tool', '').__name__ if 'execute_multi_step_goal_v1_tool' in globals() else 'execute_multi_step_goal_v1_tool': "goal_processing.py",
    }
    # Filter out entries where function name could not be determined (value is empty string)
    tool_func_to_module_file = {k:v for k,v in tool_func_to_module_file.items() if k}


    # Group functions by their target module file
    module_to_funcs = {}
    # Add helper functions explicitly
    for helper_name in helper_function_names: # Defined in thought process
        if helper_name in globals() and callable(globals()[helper_name]):
            module_file = tool_func_to_module_file.get(helper_name, "helpers.py") # Default module for unmapped helpers
            module_to_funcs.setdefault(module_file, []).append(globals()[helper_name])
    
    # Add tool callables
    for tool_callable_obj in _AGENT_TOOLS.values():
        func_name = tool_callable_obj.__name__
        module_file = tool_func_to_module_file.get(func_name)
        if module_file:
            module_to_funcs.setdefault(module_file, []).append(tool_callable_obj)
        else:
            print(f"{log_prefix}Warning: Callable '{func_name}' not mapped to a module file. Skipping direct export to tool_definitions.")


    standard_imports_str = "\n".join(standard_imports_list) + "\n\n" # Using standard_imports_list from thought process

    for module_file, func_obj_list in module_to_funcs.items():
        module_path = os.path.join(tool_definitions_dir, module_file)
        unique_funcs_in_module = list(dict.fromkeys(func_obj_list)) # Keep order, remove duplicates
        
        module_content = [standard_imports_str]
        for func_obj in unique_funcs_in_module:
            module_content.append(_get_function_source_safely(func_obj))
            module_content.append("\n\n")
        _AGENT_TOOLS['write_file'](filepath=module_path, content="".join(module_content))
        print(f"{log_prefix}Written '{module_path}'.")

    # Tool Registration File
    registration_path = os.path.join(src_dir, "tool_registration.py")
    reg_content_parts = [
        "from .agent_core import add_tool, _TOOL_DESCRIPTIONS\n",
        "# Dynamically generate imports based on where tools were written\n"
    ]
    # Build import statements for tool_registration.py
    # Map: module_filename (e.g. file_system.py) -> set of function_names
    module_exports = {} 
    for func_obj, (module_rel_path, func_name_in_module) in func_to_module_map.items():
        # module_rel_path is like ".tool_definitions.file_system"
        # We just need the last part for the from .tool_definitions import ...
        module_short_name = module_rel_path.split('.')[-1]
        module_exports.setdefault(module_short_name, set()).add(func_name_in_module)

    for module_short_name, func_names_set in module_exports.items():
        if func_names_set: # Only if there are functions to import
             reg_content_parts.append(f"from .tool_definitions.{module_short_name} import {', '.join(sorted(list(func_names_set)))}\n")
    
    reg_content_parts.append("\n\ndef register_all_tools():\n")
    reg_content_parts.append("    print(\"Registering all agent tools...\")\n")
    for tool_reg_name, tool_callable_obj in _AGENT_TOOLS.items():
        description = _TOOL_DESCRIPTIONS.get(tool_reg_name, "No description provided.")
        escaped_description = description.replace("\\", "\\\\").replace("\"", "\\\"").replace("\n", "\\n")
        
        # Use the actual name of the function as it was defined and imported
        callable_name_for_script = tool_callable_obj.__name__ 
        
        # Check if this callable_name_for_script is in our generated imports
        imported_check = any(callable_name_for_script in func_names for func_names in module_exports.values())
        if not imported_check and callable_name_for_script not in core_function_names_to_export and callable_name_for_script != "list_tools": # list_tools also in agent_utils
            print(f"{log_prefix}Warning: Callable '{callable_name_for_script}' for tool '{tool_reg_name}' might not be correctly imported in tool_registration.py.")
        
        reg_content_parts.append(f"    add_tool(name=\"{tool_reg_name}\", tool_callable={callable_name_for_script}, description=\"\"\"{escaped_description}\"\"\")\n")
    
    reg_content_parts.append("    print(\"Tool registration complete.\")\n")
    reg_content_parts.append("\nif __name__ == \"__main__\":\n    register_all_tools()\n")
    _AGENT_TOOLS['write_file'](filepath=registration_path, content="".join(reg_content_parts))
    print(f"{log_prefix}Written '{registration_path}'.")

    # README, requirements.txt, main.py, __init__.py files
    # (Using content from previous thought process for these)
    readme_content = f"# Agent Code Export\nVersion: {_AGENT_VERSION if '_AGENT_VERSION' in globals() else 'N/A'}\n..." # Truncated for brevity
    _AGENT_TOOLS['write_file'](filepath=os.path.join(output_base_dir, "README.md"), content=readme_content)
    _AGENT_TOOLS['write_file'](filepath=os.path.join(output_base_dir, "requirements.txt"), content="requests\n")
    
    latest_autonomous_tool_alias = 'attempt_goal_autonomously_v3' # This is the alias the planner uses
    main_py_content = f"""
# Main script to initialize and run the agent
from src.agent_core import _AGENT_TOOLS, get_agent_version_info
from src.tool_registration import register_all_tools

def run_agent_example():
    print(f"Initializing agent example, version: {{get_agent_version_info()}}")
    register_all_tools()
    print("\\n--- Agent Initialized ---")
    if 'list_tools' in _AGENT_TOOLS:
        _AGENT_TOOLS['list_tools']()
    else:
        print("list_tools not found.")
    print("\\n--- Example Autonomous Goal ---")
    if '{latest_autonomous_tool_alias}' in _AGENT_TOOLS:
        goal = "Write 'hello from exported agent' to 'exported_agent_test.txt'"
        result = _AGENT_TOOLS['{latest_autonomous_tool_alias}'](goal_description=goal)
        print(f"Goal attempt result: {{result}}")
        if result.get('success') and result.get('execution_result',{{}}).get('success'):
            print("Autonomous goal successful!")
            if 'delete_file' in _AGENT_TOOLS:
                 _AGENT_TOOLS['delete_file'](filepath='exported_agent_test.txt'); print("Cleaned up test file.")
    else:
        print("'{latest_autonomous_tool_alias}' not found.")
if __name__ == \"__main__\": run_agent_example()
"""
    _AGENT_TOOLS['write_file'](filepath=os.path.join(src_dir, "main.py"), content=main_py_content)
    _AGENT_TOOLS['write_file'](filepath=os.path.join(src_dir, "__init__.py"), content="# src init\n")
    _AGENT_TOOLS['write_file'](filepath=os.path.join(tool_definitions_dir, "__init__.py"), content="# tool_definitions init\n")

    print(f"{log_prefix}Agent code export completed to '{output_base_dir}'.")
    return {"success": True, "message": f"Agent code exported to '{output_base_dir}'.", "output_path": os.path.abspath(output_base_dir)}

# --- Attempt to register the export tool itself ---
print("\nAgent: Defining and attempting to register the self-export tool.")
if 'add_tool' in globals() and callable(add_tool) and '_AGENT_TOOLS' in globals() and '_TOOL_DESCRIPTIONS' in globals():
    # Check dependencies for the export tool itself
    required_globals_for_export = [
        "_AGENT_VERSION", "_AGENT_TOOL_CONCEPT_KEYWORDS", 
        "get_agent_version_info", "add_tool", # Core functions
        "write_file_v2", "read_file_v2", "list_directory_v2", "create_directory_tool", # File system tools (used by export)
        "make_http_request_tool", # HTTP
        "_load_knowledge_store", "_save_knowledge_store", "save_knowledge_tool", "retrieve_knowledge_tool", "list_knowledge_keys_tool", "delete_knowledge_tool", # Knowledge
        "_load_tasks", "_save_tasks", "add_task_tool", "get_task_tool", "update_task_tool", "list_tasks_tool", "remove_task_tool", # Task
        "execute_python_code", # Execution
        "list_tools", "get_agent_status", # Agent utils
        "suggest_tools_for_goal_v3", 
        "_extract_parameter_value_heuristically_v11_final_syntax_fix", # Latest heuristic
        "attempt_goal_autonomously_v12_tool", # Latest autonomous
        "execute_multi_step_goal_v1_tool" # Latest planner
    ]
    # Ensure all these functions (or their latest versions that are actually used by the agent) are in globals()
    # This is a simplification; a robust check would trace dependencies.
    # For now, the export tool definition assumes these are globally accessible as they were defined in previous turns.

    add_tool(
        name="export_agent_to_repository",
        tool_callable=export_agent_code_tool,
        description="Exports the agent's current code and configurations to a directory structure."
    )
    print("Tool 'export_agent_to_repository' registration attempted.")
else:
    print("Warning: Cannot register 'export_agent_to_repository' due to missing core components (add_tool, _AGENT_TOOLS, etc.).")

print("\nAgent: Self-export tool 'export_agent_code_tool' defined. Registration was attempted.")
print("To use it, call `_AGENT_TOOLS['export_agent_to_repository'](output_base_dir='your_chosen_directory_name')` if registration was successful.")
print("Waiting for your instruction (e.g., to call the export tool).")


Agent: Defining and attempting to register the self-export tool.
Tool 'export_agent_to_repository' added successfully. Description: Exports the agent's current code and configurations to a directory structure.
Tool 'export_agent_to_repository' registration attempted.

Agent: Self-export tool 'export_agent_code_tool' defined. Registration was attempted.
To use it, call `_AGENT_TOOLS['export_agent_to_repository'](output_base_dir='your_chosen_directory_name')` if registration was successful.
Waiting for your instruction (e.g., to call the export tool).


In [34]:
_AGENT_TOOLS['export_agent_to_repository'](output_base_dir='/Users/yiheinchai/Documents/Learn/auto-agent/v1-agent')

Agent log (export_agent_code_tool): Starting agent code export to '/Users/yiheinchai/Documents/Learn/auto-agent/v1-agent'.
Agent log (export_agent_code_tool): Output directory '/Users/yiheinchai/Documents/Learn/auto-agent/v1-agent' exists. Cleaning and recreating.
Agent log (create_directory_tool): Called with path='/Users/yiheinchai/Documents/Learn/auto-agent/v1-agent'.
Agent log (create_directory_tool): Directory '/Users/yiheinchai/Documents/Learn/auto-agent/v1-agent' created successfully (or already existed).
Agent log (create_directory_tool): Called with path='/Users/yiheinchai/Documents/Learn/auto-agent/v1-agent/src'.
Agent log (create_directory_tool): Directory '/Users/yiheinchai/Documents/Learn/auto-agent/v1-agent/src' created successfully (or already existed).
Agent log (create_directory_tool): Called with path='/Users/yiheinchai/Documents/Learn/auto-agent/v1-agent/src/tool_definitions'.
Agent log (create_directory_tool): Directory '/Users/yiheinchai/Documents/Learn/auto-agent/

NameError: name 'helper_function_names' is not defined

In [37]:
import os
import inspect
import json
import re
from collections import OrderedDict # To keep tools in README somewhat ordered

# --- Configuration ---
EXPORT_DIR = "/Users/yiheinchai/Documents/Learn/auto-agent/v1-agent"
TOOLS_SUBDIR = "tools"
CORE_SUBDIR = "core"

# --- Helper Functions for the Export Script ---
def ensure_dir(dir_path):
    """Creates a directory if it doesn't exist."""
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    print(f"Ensured directory: {dir_path}")

def save_text_content(content, filepath):
    """Saves text content to a file."""
    try:
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f"Saved: {filepath}")
    except Exception as e:
        print(f"Error saving {filepath}: {e}")

def save_source_code(func_callable, filepath, header=""):
    """Gets source code of a callable and saves it."""
    try:
        source = inspect.getsource(func_callable)
        full_content = header + source
        save_text_content(full_content, filepath)
    except TypeError:
        print(f"Warning: Could not get source for {func_callable.__name__} (likely a built-in or C extension). Skipping source export.")
    except OSError:
         print(f"Warning: Could not get source for {func_callable.__name__} (OSError, e.g. defined in REPL). Saving placeholder.")
         save_text_content(f"# Source code for '{func_callable.__name__}' could not be retrieved (likely defined in REPL or dynamically).\n# Description might be available in the corresponding .md file.", filepath)
    except Exception as e:
        print(f"Error getting source for {func_callable.__name__}: {e}. Saving placeholder.")
        save_text_content(f"# Error retrieving source for '{func_callable.__name__}': {e}", filepath)

# --- Main Export Logic ---
def export_agent_repository():
    print(f"Starting agent export to directory: {EXPORT_DIR}")
    
    # Create base directories
    ensure_dir(EXPORT_DIR)
    tools_dir_path = os.path.join(EXPORT_DIR, TOOLS_SUBDIR)
    ensure_dir(tools_dir_path)
    core_dir_path = os.path.join(EXPORT_DIR, CORE_SUBDIR)
    ensure_dir(core_dir_path)

    # --- Export Tools ---
    exported_tools_info = []
    if '_AGENT_TOOLS' in globals() and isinstance(_AGENT_TOOLS, dict):
        print(f"\nFound {len(_AGENT_TOOLS)} tools in _AGENT_TOOLS. Exporting...")
        # Sort tools by name for consistent output order
        sorted_tool_items = sorted(_AGENT_TOOLS.items())

        for tool_name, tool_callable in sorted_tool_items:
            print(f"  Exporting tool: {tool_name}")
            
            # Save tool source code
            tool_source_filename = f"{tool_name}.py"
            tool_source_filepath = os.path.join(tools_dir_path, tool_source_filename)
            header = f"# Tool: {tool_name}\n# Original callable name might differ if aliased (e.g. vN versions)\n\n"
            save_source_code(tool_callable, tool_source_filepath, header=header)

            # Save tool description
            description = "No description found in _TOOL_DESCRIPTIONS."
            if '_TOOL_DESCRIPTIONS' in globals() and tool_name in _TOOL_DESCRIPTIONS:
                description = _TOOL_DESCRIPTIONS[tool_name]
            
            tool_desc_filename = f"{tool_name}.md"
            tool_desc_filepath = os.path.join(tools_dir_path, tool_desc_filename)
            desc_content = f"# Tool: {tool_name}\n\n**Description:**\n{description}\n"
            # Try to add signature
            try:
                sig = inspect.signature(tool_callable)
                desc_content += f"\n**Signature:**\n```python\n{tool_name}{sig}\n```\n"
            except Exception:
                desc_content += "\n**Signature:**\n(Could not retrieve signature)\n"

            save_text_content(desc_content, tool_desc_filepath)
            exported_tools_info.append({
                "name": tool_name, 
                "source_file": os.path.join(TOOLS_SUBDIR, tool_source_filename),
                "description_file": os.path.join(TOOLS_SUBDIR, tool_desc_filename)
            })
    else:
        print("Warning: _AGENT_TOOLS dictionary not found or not a dictionary. Cannot export tools.")

    # --- Export Core Components and Helpers ---
    print("\nExporting core components and helper functions...")
    core_components_to_export = OrderedDict() # Using OrderedDict to control order in file

    # Attempt to find specific functions defined throughout the agent's development
    # This relies on them being in the global scope where this script runs.
    # List known core/helper function names (latest versions if versioned)
    # We'll try to get their source.
    
    # Initial core infrastructure (from first turns)
    # These might be harder to get if they were simple variable assignments,
    # but the functions should be gettable.
    if 'add_tool' in globals() and callable(add_tool): # The actual function from the environment
        core_components_to_export['add_tool_core_function'] = add_tool
    # The list_tools and get_agent_status are already exported as tools if they are the latest.
    # If there were initial, simpler versions not in _AGENT_TOOLS, they are harder to capture now.

    # Heuristic extractors
    # Find all versions by pattern
    heuristic_extractors_found = {}
    for name, obj in globals().items():
        if callable(obj) and name.startswith("_extract_parameter_value_heuristically_v"):
            heuristic_extractors_found[name] = obj
    
    # Sort them by version number found in name (simple sort for now)
    sorted_heuristic_extractors = sorted(heuristic_extractors_found.items(), key=lambda x: x[0])
    for name, func in sorted_heuristic_extractors:
        core_components_to_export[name] = func

    # Save collated core/helper functions
    if core_components_to_export:
        core_helpers_content = "# Agent Core Helper Functions\n\n"
        core_helpers_content += "'''\nThis file collates various helper functions and core components\n"
        core_helpers_content += "defined during the agent's development.\n'''\n\n"
        for name, func in core_components_to_export.items():
            core_helpers_content += f"\n# --- Source for: {name} ---\n"
            try:
                core_helpers_content += inspect.getsource(func) + "\n"
            except Exception as e:
                core_helpers_content += f"# Could not retrieve source for {name}: {e}\n"
        save_text_content(core_helpers_content, os.path.join(core_dir_path, "core_helper_functions.py"))

    # Export _AGENT_TOOL_CONCEPT_KEYWORDS
    if '_AGENT_TOOL_CONCEPT_KEYWORDS' in globals() and isinstance(_AGENT_TOOL_CONCEPT_KEYWORDS, dict):
        concepts_content = "# Agent Tool Concept Keywords\n\n"
        concepts_content += "_AGENT_TOOL_CONCEPT_KEYWORDS = {\n"
        for key, value_set in sorted(_AGENT_TOOL_CONCEPT_KEYWORDS.items()):
            # Convert set to sorted list for consistent output, then to string
            sorted_list_str = str(sorted(list(value_set)))
            concepts_content += f"    \"{key}\": {sorted_list_str},\n"
        concepts_content += "}\n"
        save_text_content(concepts_content, os.path.join(core_dir_path, "concept_keywords.py"))
    else:
        print("Warning: _AGENT_TOOL_CONCEPT_KEYWORDS not found. Cannot export.")
        
    # Initial global variable definitions (schematic)
    initial_globals_content = "# Initial Agent Global State (Schematic)\n\n"
    initial_globals_content += "_AGENT_TOOLS = {}\n"
    initial_globals_content += "_TOOL_DESCRIPTIONS = {}\n"
    if '_AGENT_VERSION' in globals():
        initial_globals_content += f"_AGENT_VERSION = \"{_AGENT_VERSION}\"\n"
    else:
        initial_globals_content += "_AGENT_VERSION = \"unknown\"\n"
    initial_globals_content += "\n# Note: The actual add_tool function's source can be found in core_helper_functions.py (if exported)\n"
    initial_globals_content += "# or as one of the tool .py files if it was overwritten and became a tool itself.\n"
    save_text_content(initial_globals_content, os.path.join(core_dir_path, "initial_globals_setup.py"))


    # --- Generate README.md ---
    readme_content = f"# Agent Export Repository\n\nThis repository contains the exported tools and core components of the AI agent.\n"
    readme_content += f"Agent Version at export (if available): {_AGENT_VERSION if '_AGENT_VERSION' in globals() else 'N/A'}\n\n"
    
    readme_content += "## Tools\n\n"
    readme_content += "The `tools/` directory contains the source code (`.py`) and description (`.md`) for each registered tool.\n\n"
    if exported_tools_info:
        readme_content += "| Tool Name | Source File | Description File |\n"
        readme_content += "|-----------|-------------|------------------|\n"
        for tool_info in exported_tools_info:
            readme_content += f"| {tool_info['name']} | [{tool_info['source_file']}]({tool_info['source_file']}) | [{tool_info['description_file']}]({tool_info['description_file']}) |\n"
    else:
        readme_content += "No tools were found or exported.\n"

    readme_content += "\n## Core Components\n\n"
    readme_content += "The `core/` directory contains other important parts of the agent's infrastructure:\n\n"
    readme_content += "- **core_helper_functions.py**: Contains source code for various helper functions used by the agent, including different versions of parameter extractors and potentially the `add_tool` function.\n"
    readme_content += "- **concept_keywords.py**: Contains the `_AGENT_TOOL_CONCEPT_KEYWORDS` dictionary used for tool suggestions.\n"
    readme_content += "- **initial_globals_setup.py**: A schematic representation of how core global variables like `_AGENT_TOOLS`, `_TOOL_DESCRIPTIONS`, and `_AGENT_VERSION` were initialized.\n"
    
    readme_content += "\n## Notes\n"
    readme_content += "- The source code for tools might represent the latest version of a function if it was updated/overwritten during development (e.g., `write_file` might point to `write_file_v2`).\n"
    readme_content += "- This export is a snapshot and relies on the functions and variables being present in the global scope at the time of export.\n"

    save_text_content(readme_content, os.path.join(EXPORT_DIR, "README.md"))

    print(f"\nAgent export complete. Files saved in ./{EXPORT_DIR}")
    print("Please check the contents, especially for any functions defined in the REPL/dynamically, as their source might not be fully captured.")

# --- Execute the Export ---
# This function call would be done by you in the environment where the agent's state exists.
# For this exercise, I'm just providing the script.
export_agent_repository() 
# print("\nTo run the export, uncomment the line 'export_agent_repository()' and execute this script in the agent's environment.")

Starting agent export to directory: /Users/yiheinchai/Documents/Learn/auto-agent/v1-agent
Ensured directory: /Users/yiheinchai/Documents/Learn/auto-agent/v1-agent
Ensured directory: /Users/yiheinchai/Documents/Learn/auto-agent/v1-agent/tools
Ensured directory: /Users/yiheinchai/Documents/Learn/auto-agent/v1-agent/core

Found 30 tools in _AGENT_TOOLS. Exporting...
  Exporting tool: add_task
Saved: /Users/yiheinchai/Documents/Learn/auto-agent/v1-agent/tools/add_task.py
Saved: /Users/yiheinchai/Documents/Learn/auto-agent/v1-agent/tools/add_task.md
  Exporting tool: attempt_goal_autonomously_v1
Saved: /Users/yiheinchai/Documents/Learn/auto-agent/v1-agent/tools/attempt_goal_autonomously_v1.py
Saved: /Users/yiheinchai/Documents/Learn/auto-agent/v1-agent/tools/attempt_goal_autonomously_v1.md
  Exporting tool: attempt_goal_autonomously_v10
Saved: /Users/yiheinchai/Documents/Learn/auto-agent/v1-agent/tools/attempt_goal_autonomously_v10.py
Saved: /Users/yiheinchai/Documents/Learn/auto-agent/v1-a